In [1]:
import datetime as dt
import re
import textwrap
from io import StringIO
from pathlib import Path

import requests
import pytz
import pandas as pd
from IPython.display import display, Markdown, HTML
from lxml import etree

In [2]:
league_ids = {
    'Americas': '102',
    'Americas Women': '125',
    'Europe': '100',
    'Europe Women': '124',
    'Pacific-Asia': '101'
}

In [3]:
# Fetch all the data from ZP.
# Note that ZP does provide individual rankings data via its API at:
# https://zwiftpower.com/api3.php?do=league_standings&id=XXX
# But where's the fun in that?

raw_events = {
    name: requests.get(f'https://zwiftpower.com/api3.php?do=league_event_results', params={'id': league_id}).json()['data']
    for name, league_id in league_ids.items()
}

In [4]:
raw_event_results = [
    requests.get('https://zwiftpower.com/api3.php?do=event_results', 
                             params={'zid': event['zid']}).json()['data']
    for events in raw_events.values() for event in events
]

In [5]:
events = pd.DataFrame.from_records(
    {'league': name,
     'event_id': event['zid'],
     'date': dt.datetime.fromtimestamp(event['tm'], pytz.utc)}
    for name, data in raw_events.items() for event in data
)
events = events.sort_values(by=['league', 'date']).set_index(['league', 'event_id'])
events['ordinal'] = events['date'].groupby('league').rank(method='dense').astype(int)
events.head()

date  ordinal
league   event_id                                   
Americas 152781   2019-02-15 01:00:00+00:00        1
         155997   2019-02-22 01:00:00+00:00        2
         159644   2019-03-01 01:00:00+00:00        3
         163028   2019-03-08 01:00:00+00:00        4
         167034   2019-03-15 00:00:00+00:00        5

In [6]:
raw_placings = (pd.DataFrame.from_records(row for event in raw_event_results for row in event)
                .rename(columns={'zid': 'event_id', 'zwid': 'rider_id', 'tid': 'team_id', 'tname': 'team_name'}))

# Get the league each result is from
raw_placings = pd.merge(raw_placings, events.index.to_frame()[['league']], left_on='event_id', right_on='event_id')

raw_placings.head()

,DT_RowId,act_id,act_name,age,aid,avg_hr,avg_power,avg_wkg,category,cp,...,wkg300,wkg5,wkg60,wkg_ftp,wkg_guess,zada,zeff,event_id,rider_id,league
0,,0,,35,50308,"[166, 0]","[354, 1]","[4.7, 1]",A,1,...,"[6.3, 1]","[14.9, 4]","[7.6, 1]","[4.6, 1]",0,0,1,176476,9044,Americas
1,,0,,41,406161,"[165, 0]","[314, 0]","[4.5, 0]",A,1,...,"[5.5, 0]","[12.7, 0]","[7.4, 0]","[4.4, 0]",0,1,1,176476,1412,Americas
2,,0,,34,2721607,"[156, 0]","[345, 0]","[4.5, 0]",A,1,...,"[5.4, 0]","[10.3, 0]","[7.3, 0]","[4.3, 0]",0,1,0,176476,15522,Americas
3,,0,,33,5562374,"[168, 0]","[347, 0]","[4.6, 0]",A,1,...,"[5.6, 0]","[12.8, 0]","[7.4, 0]","[4.4, 0]",0,0,1,176476,322602,Americas
4,,0,,40,187245,"[154, 0]","[283, 0]","[4.3, 0]",A,1,...,"[5.2, 0]","[12.6, 0]","[7.5, 0]","[4.2, 0]",0,0,1,176476,4641,Americas


In [7]:
def get_html_text(html_string):
    if not html_string:
        return ''
    tree = etree.parse(StringIO(html_string), etree.HTMLParser())
    return tree.xpath('normalize-space()')

def score_for_position(position_in_category):
    return (1001 - position_in_category * 2)

In [8]:
placings = (raw_placings[['event_id', 'rider_id', 'pos', 'position_in_cat', 
                          'name', 'category', 'team_name', 'team_id', 'league']]
            .copy()
            .set_index(['league', 'category', 'rider_id', 'event_id']))

# names are HTML encoded, (special chars are HTML entities), let's fix that
placings['name'] = placings['name'].map(get_html_text)
placings['team_name'] = placings['team_name'].map(get_html_text)

# People without teams get '0' as a team ID - drop such IDs
placings['team_id'] = placings.apply(lambda row: '' if row['team_name'] == '' and row['team_id'] == '0' else row['team_id'], axis=1)

# Calculate the points earned by the rider's result
placings['points'] = score_for_position(placings['position_in_cat'])

placings = placings.join(events['ordinal']).rename(columns={'ordinal': 'event_ordinal'})

placings.head()

pos  position_in_cat             name  \
league   event_id category rider_id                                          
Americas 152781   A        745         1                1    SCOTTIE WEISS   
                           81778       2                2   Eric Loiselle🔰   
                           9044        3                3  Tyler Waldron 🔰   
                           463284      4                4      Seth G ☁️ 🔰   
                           471022      5                5            键盘仁波切   

                                            team_name team_id  points  \
league   event_id category rider_id                                     
Americas 152781   A        745                                    999   
                           81778     Cannondale/4iiii    2105     997   
                           9044           ⚠️ Dropouts    3711     995   
                           463284                 AFC    1405     993   
                           471022            Team +1S    1414     991   

                                     event_ordinal  
league   event_id category rider_id                 
Americas 152781   A        745                   1  
                           81778                 1  
                           9044                  1  
                           463284                1  
                           471022                1

In [9]:
rider_ranks = placings[['name', 'team_name', 'team_id']].groupby(['league', 'category', 'rider_id']).first()

# Calculate each rider's total points
rider_ranks['total_points'] = placings['points'].groupby(['league', 'category', 'rider_id']).sum()

# Rank the riders (within their category) by score. Riders with the same score are given the same rank position
rider_ranks['rank'] = rider_ranks['total_points'].groupby(['league', 'category']).rank(method='dense', ascending=False).astype(int)

rider_ranks = rider_ranks.sort_values(['league', 'category', 'total_points'], ascending=[True, True, False])

rider_ranks.head()

name         team_name team_id  \
league   category rider_id                                               
Americas A        9044       Tyler Waldron 🔰       ⚠️ Dropouts    3711   
                  198103    Frédéric Meunier  Cannondale/4iiii    2105   
                  514852           Eric Urff           TeamODZ       7   
                  250              Mark Minn               HPP    1528   
                  341672       Daniel Riddle               HPP    1528   

                            total_points  rank  
league   category rider_id                      
Americas A        9044              7970     1  
                  198103            7712     2  
                  514852            7588     3  
                  250               7582     4  
                  341672            7582     4

Create markdown tables for publication

In [10]:
def build_markdown_escaper():
    reserved = '\`*_{}[]()#+-.!|'
    reserved_pattern = re.compile(f'[{re.escape(reserved)}]')

    def markdown_escape(text):
        return reserved_pattern.sub(lambda m: '\\' + m.group(0), text)
    return markdown_escape
markdown_escape = build_markdown_escaper()

def markdown_link(text, url):
    return f'[{markdown_escape(text)}]({markdown_escape(url)})'

def rider_name_link(row):
    return markdown_link(row["name"], f'https://zwiftpower.com/profile.php?z={row["rider_id"]}')

def rider_team_name_link(row):
    if not row["team_id"]:
        return None
    return markdown_link(row["team_name"], f'https://zwiftpower.com/team.php?id={row["team_id"]}')

# https://codegolf.stackexchange.com/questions/4707/outputting-ordinal-numbers-1st-2nd-3rd#answer-4712
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n//10%10!=1)*(n%10<4)*n%10::4])

def position_label(position):
    return {1: '🥇', 2: '🥈', 3: '🥉'}.get(position) or ordinal(position)
                         
def position_summary(placings_row):
    pos_label = position_label(placings_row['position_in_cat'])    
    return markdown_link(pos_label, f'https://zwiftpower.com/events.php?zid={placings_row["event_id"]}')

def positions_summary(rider_cat_results):
    return rider_cat_results.apply(position_summary, axis=1).str.cat(sep=' ')

In [11]:
display_positions = placings.reset_index().set_index(['league', 'category', 'rider_id', 'event_ordinal'])[['position_in_cat', 'event_id']].apply(position_summary, axis=1)
display_positions = display_positions.unstack()
display_positions.head()

event_ordinal                                                               1  \
league   category rider_id                                                      
Americas A        250       [17th](https://zwiftpower\.com/events\.php?zid...   
                  745       [🥇](https://zwiftpower\.com/events\.php?zid=15...   
                  1412                                                    NaN   
                  3228      [30th](https://zwiftpower\.com/events\.php?zid...   
                  3313                                                    NaN   

event_ordinal                                                               2  \
league   category rider_id                                                      
Americas A        250       [25th](https://zwiftpower\.com/events\.php?zid...   
                  745       [🥇](https://zwiftpower\.com/events\.php?zid=15...   
                  1412                                                    NaN   
                  3228      [30th](https://zwiftpower\.com/events\.php?zid...   
                  3313                                                    NaN   

event_ordinal                                                               3  \
league   category rider_id                                                      
Americas A        250       [34th](https://zwiftpower\.com/events\.php?zid...   
                  745       [🥇](https://zwiftpower\.com/events\.php?zid=15...   
                  1412                                                    NaN   
                  3228      [38th](https://zwiftpower\.com/events\.php?zid...   
                  3313                                                    NaN   

event_ordinal                                                               4  \
league   category rider_id                                                      
Americas A        250       [35th](https://zwiftpower\.com/events\.php?zid...   
                  745       [4th](https://zwiftpower\.com/events\.php?zid=...   
                  1412                                                    NaN   
                  3228      [39th](https://zwiftpower\.com/events\.php?zid...   
                  3313                                                    NaN   

event_ordinal                                                               5  \
league   category rider_id                                                      
Americas A        250       [37th](https://zwiftpower\.com/events\.php?zid...   
                  745       [🥇](https://zwiftpower\.com/events\.php?zid=16...   
                  1412      [5th](https://zwiftpower\.com/events\.php?zid=...   
                  3228      [34th](https://zwiftpower\.com/events\.php?zid...   
                  3313      [13th](https://zwiftpower\.com/events\.php?zid...   

event_ordinal                                                               6  \
league   category rider_id                                                      
Americas A        250       [23rd](https://zwiftpower\.com/events\.php?zid...   
                  745                                                     NaN   
                  1412      [7th](https://zwiftpower\.com/events\.php?zid=...   
                  3228                                                    NaN   
                  3313      [30th](https://zwiftpower\.com/events\.php?zid...   

event_ordinal                                                               7  \
league   category rider_id                                                      
Americas A        250       [22nd](https://zwiftpower\.com/events\.php?zid...   
                  745       [🥇](https://zwiftpower\.com/events\.php?zid=17...   
                  1412                                                    NaN   
                  3228                                                    NaN   
                  3313      [10th](https://zwiftpower\.com/events\.php?zid...   

event_ordinal           

In [12]:
display_rider_ranks = rider_ranks[['rank', 'total_points']].copy()
display_rider_ranks['name'] = pd.merge(rider_ranks.index.to_frame(), rider_ranks, 
                                        left_index=True, right_index=True).apply(rider_name_link, axis=1)
display_rider_ranks['team'] = rider_ranks.apply(rider_team_name_link, axis=1)
display_rider_ranks.head()

rank  total_points  \
league   category rider_id                       
Americas A        9044         1          7970   
                  198103       2          7712   
                  514852       3          7588   
                  250          4          7582   
                  341672       4          7582   

                                                                         name  \
league   category rider_id                                                      
Americas A        9044      [Tyler Waldron 🔰](https://zwiftpower\.com/prof...   
                  198103    [Frédéric Meunier](https://zwiftpower\.com/pro...   
                  514852    [Eric Urff](https://zwiftpower\.com/profile\.p...   
                  250       [Mark Minn](https://zwiftpower\.com/profile\.p...   
                  341672    [Daniel Riddle](https://zwiftpower\.com/profil...   

                                                                         team  
league   category rider_id                                                     
Americas A        9044      [⚠️ Dropouts](https://zwiftpower\.com/team\.ph...  
                  198103    [Cannondale/4iiii](https://zwiftpower\.com/tea...  
                  514852    [TeamODZ](https://zwiftpower\.com/team\.php?id=7)  
                  250        [HPP](https://zwiftpower\.com/team\.php?id=1528)  
                  341672     [HPP](https://zwiftpower\.com/team\.php?id=1528)

In [13]:
full_display = pd.merge(display_rider_ranks, display_positions, left_index=True, right_index=True, how='left')
full_display = full_display.fillna('—')

In [14]:
def markdown_table(df):
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    return df_formatted.to_csv(sep="|", index=False)

In [15]:
def get_league_rank_table(league, full_display):
    league_table = full_display.loc[league, :, :]
    league_table = league_table.reset_index().drop(columns=['league', 'rider_id'])
    columns = dict(zip(
        list(league_table.columns),
        ['Category', 'Rank', 'Total Points', 'Name', 'Team']
    ))
    return league_table.rename(columns=columns)

def render_league_table(league_table):
    return markdown_table(league_table)

def league_filename(league):
    return league.lower().replace(' ', '-')

In [16]:
league_names = list(full_display.index.levels[0])

league_tables = {
    league: render_league_table(get_league_rank_table(league, full_display))
    for league in league_names
}

In [17]:
league_markdown = {
    league: (f'''\
## KISS Community League — {league}

{league_tables[league]}
''')
    for league in league_names
}

all_tables_markdown = '# KISS Community League Rider Rank Tables\n\n' + '\n\n'.join(league_markdown.values())

In [18]:
# Create league result files - but not by default
WRITE_MARKDOWN_FILES = True

if WRITE_MARKDOWN_FILES:
    dest_dir = Path('table-markdown')
    dest_dir.mkdir(exist_ok=True)

    for league in league_names:
        with open(dest_dir / f'league-table-{league_filename(league)}.md', 'w') as f:
            f.write(league_markdown[league])

    with open(dest_dir / 'all-tables.md', 'w') as f:
        f.write(all_tables_markdown)

In [19]:
# Render the results inline
display(Markdown(all_tables_markdown))

# KISS Community League Rider Rank Tables

## KISS Community League — Americas

Category|Rank|Total Points|Name|Team|1|2|3|4|5|6|7|8
---|---|---|---|---|---|---|---|---|---|---|---|---
A|1|7970|[Tyler Waldron 🔰](https://zwiftpower\.com/profile\.php?z=9044)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[🥉](https://zwiftpower\.com/events\.php?zid=152781)|[🥈](https://zwiftpower\.com/events\.php?zid=155997)|[6th](https://zwiftpower\.com/events\.php?zid=159644)|[🥈](https://zwiftpower\.com/events\.php?zid=163028)|[🥈](https://zwiftpower\.com/events\.php?zid=167034)|[🥇](https://zwiftpower\.com/events\.php?zid=170592)|[🥈](https://zwiftpower\.com/events\.php?zid=173965)|[🥇](https://zwiftpower\.com/events\.php?zid=176476)
A|2|7712|[Frédéric Meunier](https://zwiftpower\.com/profile\.php?z=198103)|[Cannondale/4iiii](https://zwiftpower\.com/team\.php?id=2105)|[20th](https://zwiftpower\.com/events\.php?zid=152781)|[16th](https://zwiftpower\.com/events\.php?zid=155997)|[22nd](https://zwiftpower\.com/events\.php?zid=159644)|[38th](https://zwiftpower\.com/events\.php?zid=163028)|[15th](https://zwiftpower\.com/events\.php?zid=167034)|[12th](https://zwiftpower\.com/events\.php?zid=170592)|[17th](https://zwiftpower\.com/events\.php?zid=173965)|[8th](https://zwiftpower\.com/events\.php?zid=176476)
A|3|7588|[Eric Urff](https://zwiftpower\.com/profile\.php?z=514852)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[34th](https://zwiftpower\.com/events\.php?zid=152781)|[33rd](https://zwiftpower\.com/events\.php?zid=155997)|[36th](https://zwiftpower\.com/events\.php?zid=159644)|[15th](https://zwiftpower\.com/events\.php?zid=163028)|[29th](https://zwiftpower\.com/events\.php?zid=167034)|[21st](https://zwiftpower\.com/events\.php?zid=170592)|[20th](https://zwiftpower\.com/events\.php?zid=173965)|[22nd](https://zwiftpower\.com/events\.php?zid=176476)
A|4|7582|[Mark Minn](https://zwiftpower\.com/profile\.php?z=250)|[HPP](https://zwiftpower\.com/team\.php?id=1528)|[17th](https://zwiftpower\.com/events\.php?zid=152781)|[25th](https://zwiftpower\.com/events\.php?zid=155997)|[34th](https://zwiftpower\.com/events\.php?zid=159644)|[35th](https://zwiftpower\.com/events\.php?zid=163028)|[37th](https://zwiftpower\.com/events\.php?zid=167034)|[23rd](https://zwiftpower\.com/events\.php?zid=170592)|[22nd](https://zwiftpower\.com/events\.php?zid=173965)|[20th](https://zwiftpower\.com/events\.php?zid=176476)
A|4|7582|[Daniel Riddle](https://zwiftpower\.com/profile\.php?z=341672)|[HPP](https://zwiftpower\.com/team\.php?id=1528)|[25th](https://zwiftpower\.com/events\.php?zid=152781)|[23rd](https://zwiftpower\.com/events\.php?zid=155997)|[28th](https://zwiftpower\.com/events\.php?zid=159644)|[33rd](https://zwiftpower\.com/events\.php?zid=163028)|[33rd](https://zwiftpower\.com/events\.php?zid=167034)|[34th](https://zwiftpower\.com/events\.php?zid=170592)|[18th](https://zwiftpower\.com/events\.php?zid=173965)|[19th](https://zwiftpower\.com/events\.php?zid=176476)
A|5|7576|[Frank Deal](https://zwiftpower\.com/profile\.php?z=90541)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[29th](https://zwiftpower\.com/events\.php?zid=152781)|[29th](https://zwiftpower\.com/events\.php?zid=155997)|[35th](https://zwiftpower\.com/events\.php?zid=159644)|[25th](https://zwiftpower\.com/events\.php?zid=163028)|[36th](https://zwiftpower\.com/events\.php?zid=167034)|[35th](https://zwiftpower\.com/events\.php?zid=170592)|[12th](https://zwiftpower\.com/events\.php?zid=173965)|[15th](https://zwiftpower\.com/events\.php?zid=176476)
A|6|6849|[Alex Dion 🔰](https://zwiftpower\.com/profile\.php?z=92891)|[Cannondale/4iiii](https://zwiftpower\.com/team\.php?id=2105)|[7th](https://zwiftpower\.com/events\.php?zid=152781)|[8th](https://zwiftpower\.com/events\.php?zid=155997)|[9th](https://zwiftpower\.com/events\.php?zid=159644)|[8th](https://zwiftpower\.com/events\.php?zid=163028)|[26th](https://zwiftpower\.com/events\.php?zid=167034)|[15th](https://zwiftpower\.com/events\.php?zid=170592)|—|[6th](https://zwiftpower\.com/events\.php?zid=176476)
A|7|6773|[Eugene Chan 🍌](https://zwiftpower\.com/profile\.php?z=140937)|[HPP](https://zwiftpower\.com/team\.php?id=1528)|—|[13th](https://zwiftpower\.com/events\.php?zid=155997)|[17th](https://zwiftpower\.com/events\.php?zid=159644)|[20th](https://zwiftpower\.com/events\.php?zid=163028)|[16th](https://zwiftpower\.com/events\.php?zid=167034)|[26th](https://zwiftpower\.com/events\.php?zid=170592)|[15th](https://zwiftpower\.com/events\.php?zid=173965)|[10th](https://zwiftpower\.com/events\.php?zid=176476)
A|8|5988|[SCOTTIE WEISS](https://zwiftpower\.com/profile\.php?z=745)|—|[🥇](https://zwiftpower\.com/events\.php?zid=152781)|[🥇](https://zwiftpower\.com/events\.php?zid=155997)|[🥇](https://zwiftpower\.com/events\.php?zid=159644)|[4th](https://zwiftpower\.com/events\.php?zid=163028)|[🥇](https://zwiftpower\.com/events\.php?zid=167034)|—|[🥇](https://zwiftpower\.com/events\.php?zid=173965)|—
A|9|5774|[David Galati](https://zwiftpower\.com/profile\.php?z=210781)|[LaPrima](https://zwiftpower\.com/team\.php?id=2586)|[24th](https://zwiftpower\.com/events\.php?zid=152781)|[22nd](https://zwiftpower\.com/events\.php?zid=155997)|[20th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=170592)|[14th](https://zwiftpower\.com/events\.php?zid=173965)|[14th](https://zwiftpower\.com/events\.php?zid=176476)
A|10|5764|[Brian Gefroh](https://zwiftpower\.com/profile\.php?z=414438)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[11th](https://zwiftpower\.com/events\.php?zid=155997)|[30th](https://zwiftpower\.com/events\.php?zid=159644)|[18th](https://zwiftpower\.com/events\.php?zid=163028)|—|[17th](https://zwiftpower\.com/events\.php?zid=170592)|[27th](https://zwiftpower\.com/events\.php?zid=173965)|[18th](https://zwiftpower\.com/events\.php?zid=176476)
A|11|5664|[Marc Pensyl \(RCC\)](https://zwiftpower\.com/profile\.php?z=218740)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[28th](https://zwiftpower\.com/events\.php?zid=152781)|[31st](https://zwiftpower\.com/events\.php?zid=155997)|[39th](https://zwiftpower\.com/events\.php?zid=159644)|—|[18th](https://zwiftpower\.com/events\.php?zid=167034)|[27th](https://zwiftpower\.com/events\.php?zid=170592)|[28th](https://zwiftpower\.com/events\.php?zid=173965)|—
A|12|4959|[Eric Loiselle🔰](https://zwiftpower\.com/profile\.php?z=81778)|[Cannondale/4iiii](https://zwiftpower\.com/team\.php?id=2105)|[🥈](https://zwiftpower\.com/events\.php?zid=152781)|—|[🥉](https://zwiftpower\.com/events\.php?zid=159644)|[6th](https://zwiftpower\.com/events\.php?zid=163028)|[7th](https://zwiftpower\.com/events\.php?zid=167034)|[5th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
A|13|4905|[Mat Wilson](https://zwiftpower\.com/profile\.php?z=322602)|[POPTARTZ](https://zwiftpower\.com/team\.php?id=2880)|—|—|[19th](https://zwiftpower\.com/events\.php?zid=159644)|—|[9th](https://zwiftpower\.com/events\.php?zid=167034)|[11th](https://zwiftpower\.com/events\.php?zid=170592)|[7th](https://zwiftpower\.com/events\.php?zid=173965)|[4th](https://zwiftpower\.com/events\.php?zid=176476)
A|14|4845|[Peter Mogg 🍻🚲](https://zwiftpower\.com/profile\.php?z=467127)|[PTz](https://zwiftpower\.com/team\.php?id=1)|—|[20th](https://zwiftpower\.com/events\.php?zid=155997)|[18th](https://zwiftpower\.com/events\.php?zid=159644)|[14th](https://zwiftpower\.com/events\.php?zid=163028)|[12th](https://zwiftpower\.com/events\.php?zid=167034)|[16th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
A|15|4837|[David Charles](https://zwiftpower\.com/profile\.php?z=4568)|[HPP](https://zwiftpower\.com/team\.php?id=1528)|[23rd](https://zwiftpower\.com/events\.php?zid=152781)|—|—|[19th](https://zwiftpower\.com/events\.php?zid=163028)|[24th](https://zwiftpower\.com/events\.php?zid=167034)|—|[11th](https://zwiftpower\.com/events\.php?zid=173965)|[7th](https://zwiftpower\.com/events\.php?zid=176476)
A|16|4821|[Max Sibum](https://zwiftpower\.com/profile\.php?z=156551)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|[12th](https://zwiftpower\.com/events\.php?zid=155997)|—|[26th](https://zwiftpower\.com/events\.php?zid=163028)|—|[33rd](https://zwiftpower\.com/events\.php?zid=170592)|[9th](https://zwiftpower\.com/events\.php?zid=173965)|[12th](https://zwiftpower\.com/events\.php?zid=176476)
A|16|4821|[Alex Side](https://zwiftpower\.com/profile\.php?z=214679)|—|—|[28th](https://zwiftpower\.com/events\.php?zid=155997)|[24th](https://zwiftpower\.com/events\.php?zid=159644)|[11th](https://zwiftpower\.com/events\.php?zid=163028)|[20th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|[9th](https://zwiftpower\.com/events\.php?zid=176476)
A|17|4773|[Adrian Amos](https://zwiftpower\.com/profile\.php?z=73357)|[HPP](https://zwiftpower\.com/team\.php?id=1528)|—|[19th](https://zwiftpower\.com/events\.php?zid=155997)|[16th](https://zwiftpower\.com/events\.php?zid=159644)|[27th](https://zwiftpower\.com/events\.php?zid=163028)|[35th](https://zwiftpower\.com/events\.php?zid=167034)|—|[19th](https://zwiftpower\.com/events\.php?zid=173965)|—
A|18|4763|[Jean\-Yan Perron](https://zwiftpower\.com/profile\.php?z=390308)|[Laferte bicycle](https://zwiftpower\.com/team\.php?id=4054)|—|[26th](https://zwiftpower\.com/events\.php?zid=155997)|[23rd](https://zwiftpower\.com/events\.php?zid=159644)|[28th](https://zwiftpower\.com/events\.php?zid=163028)|[19th](https://zwiftpower\.com/events\.php?zid=167034)|[25th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
A|19|4663|[John Cservek \[AFC\] 🔰](https://zwiftpower\.com/profile\.php?z=3228)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|[30th](https://zwiftpower\.com/events\.php?zid=152781)|[30th](https://zwiftpower\.com/events\.php?zid=155997)|[38th](https://zwiftpower\.com/events\.php?zid=159644)|[39th](https://zwiftpower\.com/events\.php?zid=163028)|[34th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
A|20|3962|[Maverick Millar](https://zwiftpower\.com/profile\.php?z=165166)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[12th](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=167034)|[🥈](https://zwiftpower\.com/events\.php?zid=170592)|[🥉](https://zwiftpower\.com/events\.php?zid=173965)|—
A|21|3952|[M O 🔰](https://zwiftpower\.com/profile\.php?z=45460)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|[11th](https://zwiftpower\.com/events\.php?zid=152781)|[7th](https://zwiftpower\.com/events\.php?zid=155997)|[5th](https://zwiftpower\.com/events\.php?zid=159644)|[🥉](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
A|21|3952|[Warren Muir 🔰](https://zwiftpower\.com/profile\.php?z=68432)|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=155997)|[10th](https://zwiftpower\.com/events\.php?zid=159644)|[7th](https://zwiftpower\.com/events\.php?zid=163028)|[6th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
A|22|3816|[Stéphane Sévigny](https://zwiftpower\.com/profile\.php?z=161752)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[21st](https://zwiftpower\.com/events\.php?zid=152781)|[27th](https://zwiftpower\.com/events\.php?zid=155997)|—|[29th](https://zwiftpower\.com/events\.php?zid=163028)|[17th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
A|23|3808|[Tofu Anarchy](https://zwiftpower\.com/profile\.php?z=140251)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[27th](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|[31st](https://zwiftpower\.com/events\.php?zid=167034)|—|[24th](https://zwiftpower\.com/events\.php?zid=173965)|[16th](https://zwiftpower\.com/events\.php?zid=176476)
A|24|3774|[Guy Bergeron \(JZQc\)](https://zwiftpower\.com/profile\.php?z=433249)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[33rd](https://zwiftpower\.com/events\.php?zid=152781)|[34th](https://zwiftpower\.com/events\.php?zid=155997)|—|[22nd](https://zwiftpower\.com/events\.php?zid=163028)|—|—|[26th](https://zwiftpower\.com/events\.php?zid=173965)|—
A|25|3742|[Sébastien Desgagné](https://zwiftpower\.com/profile\.php?z=189612)|—|—|—|[41st](https://zwiftpower\.com/events\.php?zid=159644)|—|—|[40th](https://zwiftpower\.com/events\.php?zid=170592)|[29th](https://zwiftpower\.com/events\.php?zid=173965)|[21st](https://zwiftpower\.com/events\.php?zid=176476)
A|26|2981|[Ryan Larson VR2209 🔰](https://zwiftpower\.com/profile\.php?z=263018)|[VR2209](https://zwiftpower\.com/team\.php?id=3121)|—|[4th](https://zwiftpower\.com/events\.php?zid=155997)|[4th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=170592)|—|—
A|27|2975|[Casey Schumm 🍕🔰](https://zwiftpower\.com/profile\.php?z=1412)|[Team X](https://zwiftpower\.com/team\.php?id=2)|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=167034)|[7th](https://zwiftpower\.com/events\.php?zid=170592)|—|[🥈](https://zwiftpower\.com/events\.php?zid=176476)
A|28|2959|[Brian Hodges](https://zwiftpower\.com/profile\.php?z=399270)|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=159644)|[🥇](https://zwiftpower\.com/events\.php?zid=163028)|—|[8th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
A|28|2959|[Danny Mossman](https://zwiftpower\.com/profile\.php?z=468401)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[5th](https://zwiftpower\.com/events\.php?zid=155997)|[7th](https://zwiftpower\.com/events\.php?zid=159644)|[10th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
A|29|2951|[Frederic Beaulieu](https://zwiftpower\.com/profile\.php?z=189463)|—|[9th](https://zwiftpower\.com/events\.php?zid=152781)|[9th](https://zwiftpower\.com/events\.php?zid=155997)|[8th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
A|30|2949|[Eric Sorenson 🦔🔰](https://zwiftpower\.com/profile\.php?z=50641)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|[13th](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|[10th](https://zwiftpower\.com/events\.php?zid=170592)|[4th](https://zwiftpower\.com/events\.php?zid=173965)|—
A|31|2931|[JP Venne](https://zwiftpower\.com/profile\.php?z=50835)|[CLAN KNOX](https://zwiftpower\.com/team\.php?id=3073)|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=163028)|[🥉](https://zwiftpower\.com/events\.php?zid=167034)|[28th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
A|32|2913|[Israël Mercier](https://zwiftpower\.com/profile\.php?z=258075)|—|—|—|[25th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|[14th](https://zwiftpower\.com/events\.php?zid=170592)|[6th](https://zwiftpower\.com/events\.php?zid=173965)|—
A|33|2909|[Benedict Mercier](https://zwiftpower\.com/profile\.php?z=429891)|—|—|—|[29th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|[13th](https://zwiftpower\.com/events\.php?zid=170592)|[5th](https://zwiftpower\.com/events\.php?zid=173965)|—
A|34|2897|[Mike Baird \- 1R](https://zwiftpower\.com/profile\.php?z=3313)|[POPTARTZ](https://zwiftpower\.com/team\.php?id=2880)|—|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=167034)|[30th](https://zwiftpower\.com/events\.php?zid=170592)|[10th](https://zwiftpower\.com/events\.php?zid=173965)|—
A|35|2891|[Edmund Leduc ⓥ](https://zwiftpower\.com/profile\.php?z=611056)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|[21st](https://zwiftpower\.com/events\.php?zid=159644)|—|—|[19th](https://zwiftpower\.com/events\.php?zid=170592)|[16th](https://zwiftpower\.com/events\.php?zid=173965)|—
A|36|2879|[Mathieu Hould](https://zwiftpower\.com/profile\.php?z=71850)|—|—|[18th](https://zwiftpower\.com/events\.php?zid=155997)|[26th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|[18th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
A|37|2877|[Andrew Walter 🐿️🔰](https://zwiftpower\.com/profile\.php?z=478707)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|[18th](https://zwiftpower\.com/events\.php?zid=152781)|[21st](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|[24th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
A|38|2873|[Daniel Lucas](https://zwiftpower\.com/profile\.php?z=41577)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|—|[14th](https://zwiftpower\.com/events\.php?zid=155997)|—|[24th](https://zwiftpower\.com/events\.php?zid=163028)|[27th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
A|39|2867|[Andrew Sargent](https://zwiftpower\.com/profile\.php?z=46394)|—|[22nd](https://zwiftpower\.com/events\.php?zid=152781)|—|[33rd](https://zwiftpower\.com/events\.php?zid=159644)|[13th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
A|40|2859|[Adam Schwengler](https://zwiftpower\.com/profile\.php?z=177457)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[26th](https://zwiftpower\.com/events\.php?zid=152781)|[17th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|[29th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
A|41|2841|[Guidel'Esse\#JZQC](https://zwiftpower\.com/profile\.php?z=455587)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[15th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|[28th](https://zwiftpower\.com/events\.php?zid=167034)|[38th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
A|42|2827|[Yvan Bérubé](https://zwiftpower\.com/profile\.php?z=182836)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[32nd](https://zwiftpower\.com/events\.php?zid=155997)|—|[31st](https://zwiftpower\.com/events\.php?zid=163028)|[25th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
A|43|1988|[键盘仁波切](https://zwiftpower\.com/profile\.php?z=471022)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|[5th](https://zwiftpower\.com/events\.php?zid=152781)|—|[🥈](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
A|44|1974|[Nick Proulx](https://zwiftpower\.com/profile\.php?z=226916)|[Cannondale/4iiii](https://zwiftpower\.com/team\.php?id=2105)|[8th](https://zwiftpower\.com/events\.php?zid=152781)|[6th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
A|45|1968|[jesse riley1488](https://zwiftpower\.com/profile\.php?z=149529)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[6th](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
A|46|1960|[Luc Guilbeault](https://zwiftpower\.com/profile\.php?z=212990)|[Laferte bicycle](https://zwiftpower\.com/team\.php?id=4054)|—|—|[12th](https://zwiftpower\.com/events\.php?zid=159644)|[9th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
A|47|1932|[Fred Goulet](https://zwiftpower\.com/profile\.php?z=350021)|[VelociT](https://zwiftpower\.com/team\.php?id=1071)|—|[24th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=176476)
A|48|1918|[Marc\-Antoine Pouliot](https://zwiftpower\.com/profile\.php?z=171939)|[FINESSE](https://zwiftpower\.com/team\.php?id=2779)|—|—|[11th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|[31st](https://zwiftpower\.com/events\.php?zid=170592)|—|—
A|49|1916|[Jake Campbell](https://zwiftpower\.com/profile\.php?z=444969)|[BUB](https://zwiftpower\.com/team\.php?id=2059)|—|—|[27th](https://zwiftpower\.com/events\.php?zid=159644)|[16th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
A|50|1914|[Matt Wheeler 🦊 \[VFox\] 🔰](https://zwiftpower\.com/profile\.php?z=452777)|[Vixen/Fox](https://zwiftpower\.com/team\.php?id=1940)|—|—|—|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=173965)|[23rd](https://zwiftpower\.com/events\.php?zid=176476)
A|51|1902|[Olivier Hebert Cannondale 4IIII](https://zwiftpower\.com/profile\.php?z=939073)|—|—|—|[14th](https://zwiftpower\.com/events\.php?zid=159644)|[36th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
A|52|1898|[Blair Drader ZHR B TeamWKG](https://zwiftpower\.com/profile\.php?z=520895)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|—|—|[31st](https://zwiftpower\.com/events\.php?zid=159644)|[21st](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
A|53|1894|[Marc Fourcaudot](https://zwiftpower\.com/profile\.php?z=330044)|—|[19th](https://zwiftpower\.com/events\.php?zid=152781)|[35th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
A|54|1890|[Chris Hardee](https://zwiftpower\.com/profile\.php?z=569380)|[DCVELO](https://zwiftpower\.com/team\.php?id=3894)|[32nd](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|[24th](https://zwiftpower\.com/events\.php?zid=176476)
A|55|1882|[Serge Bisson](https://zwiftpower\.com/profile\.php?z=66189)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|[37th](https://zwiftpower\.com/events\.php?zid=159644)|—|[23rd](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
A|56|1876|[Robert Moore](https://zwiftpower\.com/profile\.php?z=98365)|—|—|—|—|—|—|[32nd](https://zwiftpower\.com/events\.php?zid=170592)|[31st](https://zwiftpower\.com/events\.php?zid=173965)|—
A|57|1868|[Tom21 Turner3998](https://zwiftpower\.com/profile\.php?z=453469)|—|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|[25th](https://zwiftpower\.com/events\.php?zid=173965)|—
A|58|1866|[kevin thompson](https://zwiftpower\.com/profile\.php?z=92113)|—|—|—|—|—|[32nd](https://zwiftpower\.com/events\.php?zid=167034)|[36th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
A|58|1866|[L\.Blueberry](https://zwiftpower\.com/profile\.php?z=102968)|—|—|[36th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|[32nd](https://zwiftpower\.com/events\.php?zid=173965)|—
A|59|1862|[M Ssmith \(V\)](https://zwiftpower\.com/profile\.php?z=321324)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|[32nd](https://zwiftpower\.com/events\.php?zid=163028)|[38th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
A|60|1860|[Paul Nederveld](https://zwiftpower\.com/profile\.php?z=151748)|—|[31st](https://zwiftpower\.com/events\.php?zid=152781)|—|[40th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
A|61|1848|[John Morris](https://zwiftpower\.com/profile\.php?z=116770)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|[40th](https://zwiftpower\.com/events\.php?zid=167034)|[37th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
A|62|995|[Michael Podzaline](https://zwiftpower\.com/profile\.php?z=15522)|[Team X](https://zwiftpower\.com/team\.php?id=2)|—|—|—|—|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=176476)
A|63|993|[Seth G ☁️ 🔰](https://zwiftpower\.com/profile\.php?z=463284)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|[4th](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|—
A|63|993|[K KONNO](https://zwiftpower\.com/profile\.php?z=563135)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|—|—|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
A|64|991|[Tom Burke](https://zwiftpower\.com/profile\.php?z=4641)|[Team AM](https://zwiftpower\.com/team\.php?id=2065)|—|—|—|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=176476)
A|65|989|[Julian White](https://zwiftpower\.com/profile\.php?z=563359)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
A|66|985|[Jero Nefkens](https://zwiftpower\.com/profile\.php?z=78992)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|—|—|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=173965)|—
A|66|985|[Ryan Mckay](https://zwiftpower\.com/profile\.php?z=245426)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
A|67|983|[PJ Kings](https://zwiftpower\.com/profile\.php?z=13546)|[SWOZR](https://zwiftpower\.com/team\.php?id=2971)|—|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
A|68|981|[Eric Gauvin](https://zwiftpower\.com/profile\.php?z=160112)|—|[10th](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|—
A|68|981|[Max TT 🔰](https://zwiftpower\.com/profile\.php?z=559553)|—|—|[10th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
A|68|981|[Changho Lee\-TDM](https://zwiftpower\.com/profile\.php?z=1016795)|—|—|—|—|—|[10th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
A|69|977|[John Moore](https://zwiftpower\.com/profile\.php?z=306865)|[BTX](https://zwiftpower\.com/team\.php?id=2131)|—|—|—|[12th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
A|70|975|[Mathieu Thibault](https://zwiftpower\.com/profile\.php?z=318656)|—|—|—|—|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=173965)|—
A|70|975|[Glenn Kempton 🙈](https://zwiftpower\.com/profile\.php?z=725699)|[SAS Cycle Club](https://zwiftpower\.com/team\.php?id=1348)|—|—|—|—|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=176476)
A|71|973|[Paul Lavoie](https://zwiftpower\.com/profile\.php?z=13944)|—|[14th](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|—
A|71|973|[Scott McLean](https://zwiftpower\.com/profile\.php?z=180616)|[Cannondale\-USA](https://zwiftpower\.com/team\.php?id=4103)|—|—|—|—|[14th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
A|72|971|[蔡 蔡](https://zwiftpower\.com/profile\.php?z=49402)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|—|—|[15th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
A|72|971|[David Kim Amado 🥐 🍰 🥖](https://zwiftpower\.com/profile\.php?z=690464)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[15th](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|—
A|73|969|"[Mark ""Dallas"" Henry](https://zwiftpower\.com/profile\.php?z=332807)"|—|[16th](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|—
A|74|967|[Brayden Halvorson](https://zwiftpower\.com/profile\.php?z=196936)|—|—|—|—|[17th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
A|74|967|[N 8](https://zwiftpower\.com/profile\.php?z=1158798)|—|—|—|—|—|—|—|—|[17th](https://zwiftpower\.com/events\.php?zid=176476)
A|75|961|[Eric Ellis](https://zwiftpower\.com/profile\.php?z=59459)|—|—|—|—|—|—|[20th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
A|76|959|[Andrés Boza](https://zwiftpower\.com/profile\.php?z=460661)|—|—|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
A|77|957|[Dean Schives](https://zwiftpower\.com/profile\.php?z=65698)|[SWOZR](https://zwiftpower\.com/team\.php?id=2971)|—|—|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
A|78|955|[Chris Gregory](https://zwiftpower\.com/profile\.php?z=149534)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|[23rd](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
A|78|955|[Tomo Ishii \(ISHITOMO\)](https://zwiftpower\.com/profile\.php?z=707432)|—|—|—|—|—|—|—|[23rd](https://zwiftpower\.com/events\.php?zid=173965)|—
A|79|941|[Mark Craig](https://zwiftpower\.com/profile\.php?z=266006)|—|—|—|—|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=173965)|—
A|79|941|[Pierre\-Luc Samson](https://zwiftpower\.com/profile\.php?z=652828)|—|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
A|79|941|[Jeffrey Werner](https://zwiftpower\.com/profile\.php?z=1037294)|—|—|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
A|80|937|[Dylan Ogle](https://zwiftpower\.com/profile\.php?z=348233)|[Storage Squad](https://zwiftpower\.com/team\.php?id=4061)|—|—|[32nd](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
A|81|933|[Justin Mayfield](https://zwiftpower\.com/profile\.php?z=948550)|—|—|—|—|[34th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
A|82|931|[Simon Beaulieu](https://zwiftpower\.com/profile\.php?z=530995)|—|[35th](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|—
A|83|927|[Joseph Bank 🔰](https://zwiftpower\.com/profile\.php?z=11837)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|—|—|—|[37th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
A|84|923|[Christian Clarkson \(Tower Cycling\)](https://zwiftpower\.com/profile\.php?z=74394)|—|—|—|—|—|[39th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
A|84|923|[Mustafa Branche](https://zwiftpower\.com/profile\.php?z=561065)|[DUCE](https://zwiftpower\.com/team\.php?id=3830)|—|—|—|—|—|[39th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
A|85|919|[Brent Krause](https://zwiftpower\.com/profile\.php?z=475280)|—|—|—|—|—|[41st](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
A|85|919|[Nicolas Fortin](https://zwiftpower\.com/profile\.php?z=490825)|—|—|—|—|—|—|[41st](https://zwiftpower\.com/events\.php?zid=170592)|—|—
A|86|917|[Tyson Wieland](https://zwiftpower\.com/profile\.php?z=430793)|[SAKARI](https://zwiftpower\.com/team\.php?id=1187)|—|—|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
A|86|917|[Lucas Versluys](https://zwiftpower\.com/profile\.php?z=815941)|—|—|—|—|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|1|7932|[Matthew Martin](https://zwiftpower\.com/profile\.php?z=7515)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|[🥈](https://zwiftpower\.com/events\.php?zid=152781)|[8th](https://zwiftpower\.com/events\.php?zid=155997)|[7th](https://zwiftpower\.com/events\.php?zid=159644)|[13th](https://zwiftpower\.com/events\.php?zid=163028)|[🥈](https://zwiftpower\.com/events\.php?zid=167034)|[4th](https://zwiftpower\.com/events\.php?zid=170592)|[🥇](https://zwiftpower\.com/events\.php?zid=173965)|[🥇](https://zwiftpower\.com/events\.php?zid=176476)
B|2|7914|[scott carroll oregon](https://zwiftpower\.com/profile\.php?z=17356)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[7th](https://zwiftpower\.com/events\.php?zid=152781)|[13th](https://zwiftpower\.com/events\.php?zid=155997)|[🥉](https://zwiftpower\.com/events\.php?zid=159644)|[6th](https://zwiftpower\.com/events\.php?zid=163028)|[🥉](https://zwiftpower\.com/events\.php?zid=167034)|[7th](https://zwiftpower\.com/events\.php?zid=170592)|[🥉](https://zwiftpower\.com/events\.php?zid=173965)|[5th](https://zwiftpower\.com/events\.php?zid=176476)
B|3|6877|[Brent Dickie](https://zwiftpower\.com/profile\.php?z=210021)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[12th](https://zwiftpower\.com/events\.php?zid=152781)|—|[13th](https://zwiftpower\.com/events\.php?zid=159644)|[9th](https://zwiftpower\.com/events\.php?zid=163028)|[15th](https://zwiftpower\.com/events\.php?zid=167034)|[5th](https://zwiftpower\.com/events\.php?zid=170592)|[5th](https://zwiftpower\.com/events\.php?zid=173965)|[6th](https://zwiftpower\.com/events\.php?zid=176476)
B|4|6739|[Jim Horstmann Ⓥ](https://zwiftpower\.com/profile\.php?z=182374)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[22nd](https://zwiftpower\.com/events\.php?zid=152781)|[25th](https://zwiftpower\.com/events\.php?zid=155997)|[21st](https://zwiftpower\.com/events\.php?zid=159644)|[20th](https://zwiftpower\.com/events\.php?zid=163028)|[23rd](https://zwiftpower\.com/events\.php?zid=167034)|—|[12th](https://zwiftpower\.com/events\.php?zid=173965)|[11th](https://zwiftpower\.com/events\.php?zid=176476)
B|5|6703|[Florian Lungu](https://zwiftpower\.com/profile\.php?z=225897)|[LaPrima](https://zwiftpower\.com/team\.php?id=2586)|[11th](https://zwiftpower\.com/events\.php?zid=152781)|[26th](https://zwiftpower\.com/events\.php?zid=155997)|[25th](https://zwiftpower\.com/events\.php?zid=159644)|[34th](https://zwiftpower\.com/events\.php?zid=163028)|—|[20th](https://zwiftpower\.com/events\.php?zid=170592)|[17th](https://zwiftpower\.com/events\.php?zid=173965)|[19th](https://zwiftpower\.com/events\.php?zid=176476)
B|6|6427|[LOUIS\-PIERRE DUPUIS\(pjcgrch\)\(JZQc\) B 62 years](https://zwiftpower\.com/profile\.php?z=261617)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[60th](https://zwiftpower\.com/events\.php?zid=155997)|[53rd](https://zwiftpower\.com/events\.php?zid=159644)|[58th](https://zwiftpower\.com/events\.php?zid=163028)|[32nd](https://zwiftpower\.com/events\.php?zid=167034)|[29th](https://zwiftpower\.com/events\.php?zid=170592)|[32nd](https://zwiftpower\.com/events\.php?zid=173965)|[26th](https://zwiftpower\.com/events\.php?zid=176476)
B|7|5894|[tom wilson \[X\]](https://zwiftpower\.com/profile\.php?z=380248)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[6th](https://zwiftpower\.com/events\.php?zid=152781)|[9th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|[14th](https://zwiftpower\.com/events\.php?zid=167034)|[9th](https://zwiftpower\.com/events\.php?zid=170592)|[9th](https://zwiftpower\.com/events\.php?zid=173965)|[9th](https://zwiftpower\.com/events\.php?zid=176476)
B|8|5874|[Tommy Belanger](https://zwiftpower\.com/profile\.php?z=444759)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[5th](https://zwiftpower\.com/events\.php?zid=152781)|[4th](https://zwiftpower\.com/events\.php?zid=155997)|—|[33rd](https://zwiftpower\.com/events\.php?zid=163028)|[4th](https://zwiftpower\.com/events\.php?zid=167034)|—|[7th](https://zwiftpower\.com/events\.php?zid=173965)|[13th](https://zwiftpower\.com/events\.php?zid=176476)
B|9|5830|[Silas Swaim \[CRYO\-GEN\] B](https://zwiftpower\.com/profile\.php?z=95092)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[27th](https://zwiftpower\.com/events\.php?zid=155997)|[14th](https://zwiftpower\.com/events\.php?zid=159644)|[14th](https://zwiftpower\.com/events\.php?zid=163028)|[22nd](https://zwiftpower\.com/events\.php?zid=167034)|—|[4th](https://zwiftpower\.com/events\.php?zid=173965)|[7th](https://zwiftpower\.com/events\.php?zid=176476)
B|10|5826|[Nathan Kershner 🅥](https://zwiftpower\.com/profile\.php?z=920279)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[9th](https://zwiftpower\.com/events\.php?zid=152781)|[12th](https://zwiftpower\.com/events\.php?zid=155997)|—|[10th](https://zwiftpower\.com/events\.php?zid=163028)|—|[35th](https://zwiftpower\.com/events\.php?zid=170592)|[10th](https://zwiftpower\.com/events\.php?zid=173965)|[14th](https://zwiftpower\.com/events\.php?zid=176476)
B|11|5770|[David MacKusick](https://zwiftpower\.com/profile\.php?z=364871)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[20th](https://zwiftpower\.com/events\.php?zid=152781)|[18th](https://zwiftpower\.com/events\.php?zid=155997)|[24th](https://zwiftpower\.com/events\.php?zid=159644)|[23rd](https://zwiftpower\.com/events\.php?zid=163028)|[12th](https://zwiftpower\.com/events\.php?zid=167034)|[21st](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|12|5738|[Carl Desrosiers](https://zwiftpower\.com/profile\.php?z=413249)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[39th](https://zwiftpower\.com/events\.php?zid=155997)|[29th](https://zwiftpower\.com/events\.php?zid=159644)|[31st](https://zwiftpower\.com/events\.php?zid=163028)|[9th](https://zwiftpower\.com/events\.php?zid=167034)|—|[14th](https://zwiftpower\.com/events\.php?zid=173965)|[12th](https://zwiftpower\.com/events\.php?zid=176476)
B|13|5710|[Justin Terry Ⓥ](https://zwiftpower\.com/profile\.php?z=85080)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[19th](https://zwiftpower\.com/events\.php?zid=152781)|[34th](https://zwiftpower\.com/events\.php?zid=155997)|—|[24th](https://zwiftpower\.com/events\.php?zid=163028)|[26th](https://zwiftpower\.com/events\.php?zid=167034)|[25th](https://zwiftpower\.com/events\.php?zid=170592)|[20th](https://zwiftpower\.com/events\.php?zid=173965)|—
B|14|5706|[Eric Guimond](https://zwiftpower\.com/profile\.php?z=931823)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[29th](https://zwiftpower\.com/events\.php?zid=155997)|[19th](https://zwiftpower\.com/events\.php?zid=159644)|[43rd](https://zwiftpower\.com/events\.php?zid=163028)|—|[23rd](https://zwiftpower\.com/events\.php?zid=170592)|[15th](https://zwiftpower\.com/events\.php?zid=173965)|[21st](https://zwiftpower\.com/events\.php?zid=176476)
B|15|5666|[Ben Nick\[CRYOGEN\]B](https://zwiftpower\.com/profile\.php?z=751441)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[43rd](https://zwiftpower\.com/events\.php?zid=155997)|[33rd](https://zwiftpower\.com/events\.php?zid=159644)|[42nd](https://zwiftpower\.com/events\.php?zid=163028)|[20th](https://zwiftpower\.com/events\.php?zid=167034)|[15th](https://zwiftpower\.com/events\.php?zid=170592)|—|[17th](https://zwiftpower\.com/events\.php?zid=176476)
B|16|4955|[David Partridge](https://zwiftpower\.com/profile\.php?z=238940)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[8th](https://zwiftpower\.com/events\.php?zid=152781)|[10th](https://zwiftpower\.com/events\.php?zid=155997)|—|[4th](https://zwiftpower\.com/events\.php?zid=163028)|[🥇](https://zwiftpower\.com/events\.php?zid=167034)|[🥈](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|17|4933|[Tom Schultz](https://zwiftpower\.com/profile\.php?z=214796)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[4th](https://zwiftpower\.com/events\.php?zid=152781)|[🥉](https://zwiftpower\.com/events\.php?zid=155997)|—|[16th](https://zwiftpower\.com/events\.php?zid=163028)|[10th](https://zwiftpower\.com/events\.php?zid=167034)|[🥉](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|18|4823|[Chad Skinner](https://zwiftpower\.com/profile\.php?z=83631)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[19th](https://zwiftpower\.com/events\.php?zid=155997)|[18th](https://zwiftpower\.com/events\.php?zid=159644)|[19th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|[19th](https://zwiftpower\.com/events\.php?zid=173965)|[16th](https://zwiftpower\.com/events\.php?zid=176476)
B|19|4803|[Simon Piniel](https://zwiftpower\.com/profile\.php?z=202301)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|[30th](https://zwiftpower\.com/events\.php?zid=155997)|[11th](https://zwiftpower\.com/events\.php?zid=159644)|[18th](https://zwiftpower\.com/events\.php?zid=163028)|—|[27th](https://zwiftpower\.com/events\.php?zid=170592)|—|[15th](https://zwiftpower\.com/events\.php?zid=176476)
B|20|4691|[Jeff Arnier, Sr\. CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=322928)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[32nd](https://zwiftpower\.com/events\.php?zid=155997)|[32nd](https://zwiftpower\.com/events\.php?zid=159644)|[44th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|[25th](https://zwiftpower\.com/events\.php?zid=173965)|[24th](https://zwiftpower\.com/events\.php?zid=176476)
B|21|4645|[Bruce MacDonald Cryo\-Gen](https://zwiftpower\.com/profile\.php?z=76296)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[56th](https://zwiftpower\.com/events\.php?zid=155997)|[48th](https://zwiftpower\.com/events\.php?zid=159644)|—|[34th](https://zwiftpower\.com/events\.php?zid=167034)|—|[24th](https://zwiftpower\.com/events\.php?zid=173965)|[18th](https://zwiftpower\.com/events\.php?zid=176476)
B|22|4591|[Jimmy Merckx](https://zwiftpower\.com/profile\.php?z=326614)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[38th](https://zwiftpower\.com/events\.php?zid=155997)|[45th](https://zwiftpower\.com/events\.php?zid=159644)|[52nd](https://zwiftpower\.com/events\.php?zid=163028)|[41st](https://zwiftpower\.com/events\.php?zid=167034)|[31st](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|23|3970|[Steven Drost](https://zwiftpower\.com/profile\.php?z=115162)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|—|[5th](https://zwiftpower\.com/events\.php?zid=155997)|[4th](https://zwiftpower\.com/events\.php?zid=159644)|[🥉](https://zwiftpower\.com/events\.php?zid=163028)|[5th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
B|24|3938|[Jr Caron](https://zwiftpower\.com/profile\.php?z=489115)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[6th](https://zwiftpower\.com/events\.php?zid=155997)|[9th](https://zwiftpower\.com/events\.php?zid=159644)|[12th](https://zwiftpower\.com/events\.php?zid=163028)|—|[6th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|25|3808|[Mike Dooley](https://zwiftpower\.com/profile\.php?z=535701)|[BCA](https://zwiftpower\.com/team\.php?id=3668)|—|[31st](https://zwiftpower\.com/events\.php?zid=155997)|[23rd](https://zwiftpower\.com/events\.php?zid=159644)|[25th](https://zwiftpower\.com/events\.php?zid=163028)|—|[19th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|26|3804|[Jeff Allen](https://zwiftpower\.com/profile\.php?z=225142)|—|[10th](https://zwiftpower\.com/events\.php?zid=152781)|—|[55th](https://zwiftpower\.com/events\.php?zid=159644)|[27th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|[8th](https://zwiftpower\.com/events\.php?zid=173965)|—
B|27|3798|[Francois Caron JZQc \(B\)](https://zwiftpower\.com/profile\.php?z=437205)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[37th](https://zwiftpower\.com/events\.php?zid=155997)|[28th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|[16th](https://zwiftpower\.com/events\.php?zid=170592)|[22nd](https://zwiftpower\.com/events\.php?zid=173965)|—
B|28|3758|[Russell S](https://zwiftpower\.com/profile\.php?z=11426)|[ZHR](https://zwiftpower\.com/team\.php?id=1979)|[24th](https://zwiftpower\.com/events\.php?zid=152781)|—|[36th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|[28th](https://zwiftpower\.com/events\.php?zid=170592)|[35th](https://zwiftpower\.com/events\.php?zid=173965)|—
B|29|3728|[Don Morgan](https://zwiftpower\.com/profile\.php?z=16961)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[31st](https://zwiftpower\.com/events\.php?zid=152781)|[50th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|[35th](https://zwiftpower\.com/events\.php?zid=167034)|[22nd](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|30|3682|[Jonathan Bergeron\#JZQc](https://zwiftpower\.com/profile\.php?z=418430)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[29th](https://zwiftpower\.com/events\.php?zid=152781)|[58th](https://zwiftpower\.com/events\.php?zid=155997)|—|[41st](https://zwiftpower\.com/events\.php?zid=163028)|[33rd](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
B|31|3662|[Keven Cormier \(JZQc\)](https://zwiftpower\.com/profile\.php?z=892751)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|[46th](https://zwiftpower\.com/events\.php?zid=159644)|[54th](https://zwiftpower\.com/events\.php?zid=163028)|[39th](https://zwiftpower\.com/events\.php?zid=167034)|[32nd](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|32|2975|[John Morris](https://zwiftpower\.com/profile\.php?z=116770)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[7th](https://zwiftpower\.com/events\.php?zid=155997)|[5th](https://zwiftpower\.com/events\.php?zid=159644)|[🥈](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|33|2951|[Bartek Ksok](https://zwiftpower\.com/profile\.php?z=226723)|—|[🥉](https://zwiftpower\.com/events\.php?zid=152781)|[21st](https://zwiftpower\.com/events\.php?zid=155997)|[🥈](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|34|2917|[Steve Klins \[JZQC\]](https://zwiftpower\.com/profile\.php?z=166167)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[16th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|[13th](https://zwiftpower\.com/events\.php?zid=167034)|[14th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|35|2905|[JP Provost1936](https://zwiftpower\.com/profile\.php?z=846208)|—|[15th](https://zwiftpower\.com/events\.php?zid=152781)|—|[22nd](https://zwiftpower\.com/events\.php?zid=159644)|—|—|[12th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|36|2895|[James Kupernik](https://zwiftpower\.com/profile\.php?z=166558)|[BCA](https://zwiftpower\.com/team\.php?id=3668)|—|[20th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|[24th](https://zwiftpower\.com/events\.php?zid=167034)|[10th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|37|2891|[Adam Weddington 🔰](https://zwiftpower\.com/profile\.php?z=206248)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|[14th](https://zwiftpower\.com/events\.php?zid=152781)|[14th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|[28th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
B|38|2887|[Guy M](https://zwiftpower\.com/profile\.php?z=161461)|—|[13th](https://zwiftpower\.com/events\.php?zid=152781)|[23rd](https://zwiftpower\.com/events\.php?zid=155997)|—|[22nd](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|39|2841|[Jason Funderburk](https://zwiftpower\.com/profile\.php?z=114715)|—|—|—|[44th](https://zwiftpower\.com/events\.php?zid=159644)|[29th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=176476)
B|39|2841|[Sylvain Roux](https://zwiftpower\.com/profile\.php?z=167374)|[JFZ](https://zwiftpower\.com/team\.php?id=3974)|—|—|[35th](https://zwiftpower\.com/events\.php?zid=159644)|[17th](https://zwiftpower\.com/events\.php?zid=163028)|[29th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
B|40|2833|[Demian Bailey](https://zwiftpower\.com/profile\.php?z=413276)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|—|[40th](https://zwiftpower\.com/events\.php?zid=163028)|[27th](https://zwiftpower\.com/events\.php?zid=167034)|[18th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|41|2823|[J\-P Mercier](https://zwiftpower\.com/profile\.php?z=244856)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[22nd](https://zwiftpower\.com/events\.php?zid=155997)|—|[47th](https://zwiftpower\.com/events\.php?zid=163028)|[21st](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
B|42|2745|[Pascal Gauthier](https://zwiftpower\.com/profile\.php?z=995428)|—|—|—|[47th](https://zwiftpower\.com/events\.php?zid=159644)|—|[42nd](https://zwiftpower\.com/events\.php?zid=167034)|[40th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|43|1972|[Jeremie Fontanaud Scott/RackUltra](https://zwiftpower\.com/profile\.php?z=127235)|[Scott\-RackUltra](https://zwiftpower\.com/team\.php?id=3562)|—|—|[8th](https://zwiftpower\.com/events\.php?zid=159644)|[7th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|44|1968|[Enrico Magnani](https://zwiftpower\.com/profile\.php?z=492758)|—|—|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=167034)|[11th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|45|1950|[Ryan McGee \[DDC\]](https://zwiftpower\.com/profile\.php?z=545017)|[Dutch Diesel Cyc](https://zwiftpower\.com/team\.php?id=2649)|—|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|[10th](https://zwiftpower\.com/events\.php?zid=176476)
B|46|1926|[Nicolas Fortin](https://zwiftpower\.com/profile\.php?z=490825)|—|—|—|[27th](https://zwiftpower\.com/events\.php?zid=159644)|—|[11th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
B|47|1912|[Ryosuke Baba](https://zwiftpower\.com/profile\.php?z=585484)|—|—|[35th](https://zwiftpower\.com/events\.php?zid=155997)|[10th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|48|1904|[Michael Rex Schumacher](https://zwiftpower\.com/profile\.php?z=381825)|—|—|—|—|—|[25th](https://zwiftpower\.com/events\.php?zid=167034)|[24th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|49|1896|[David Navarrete \[ODZ\]](https://zwiftpower\.com/profile\.php?z=194375)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|[41st](https://zwiftpower\.com/events\.php?zid=155997)|[12th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|50|1884|[Ryan Davies](https://zwiftpower\.com/profile\.php?z=412618)|—|—|—|[39th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|[20th](https://zwiftpower\.com/events\.php?zid=176476)
B|51|1882|[Michael Placidi Vegan Italiano/Australiano7558](https://zwiftpower\.com/profile\.php?z=82858)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|—|[37th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|[23rd](https://zwiftpower\.com/events\.php?zid=176476)
B|52|1880|[PP V JZQC](https://zwiftpower\.com/profile\.php?z=788575)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[17th](https://zwiftpower\.com/events\.php?zid=152781)|[44th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|53|1866|[S\. Deslauriers Velozone JZQC](https://zwiftpower\.com/profile\.php?z=182546)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|[20th](https://zwiftpower\.com/events\.php?zid=159644)|[48th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|53|1866|[Tim Duivesteyn](https://zwiftpower\.com/profile\.php?z=510856)|—|—|—|[31st](https://zwiftpower\.com/events\.php?zid=159644)|—|—|[37th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|54|1864|[Gene Fowler \(BCC\)](https://zwiftpower\.com/profile\.php?z=186111)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|[38th](https://zwiftpower\.com/events\.php?zid=163028)|[31st](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
B|55|1860|[Scott Farrell ZHR \(E\)](https://zwiftpower\.com/profile\.php?z=581147)|—|—|—|—|[35th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|[36th](https://zwiftpower\.com/events\.php?zid=173965)|—
B|56|1858|[O prah Spinfrey](https://zwiftpower\.com/profile\.php?z=838061)|—|—|—|—|—|[38th](https://zwiftpower\.com/events\.php?zid=167034)|[34th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|57|1842|[Brian Griff Dirt](https://zwiftpower\.com/profile\.php?z=423430)|—|[30th](https://zwiftpower\.com/events\.php?zid=152781)|—|[50th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|58|1838|[Robert Jr Gagné \{DiRT\} \(B\)](https://zwiftpower\.com/profile\.php?z=678429)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|[43rd](https://zwiftpower\.com/events\.php?zid=159644)|[39th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|59|1828|[Pierre Maurice Dufresne](https://zwiftpower\.com/profile\.php?z=431492)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[49th](https://zwiftpower\.com/events\.php?zid=155997)|[38th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|60|1822|[Jeff Grabinski \(BSR\)](https://zwiftpower\.com/profile\.php?z=59133)|—|—|[53rd](https://zwiftpower\.com/events\.php?zid=155997)|[37th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|60|1822|[Jason Wright Ⓥ](https://zwiftpower\.com/profile\.php?z=71194)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|[48th](https://zwiftpower\.com/events\.php?zid=155997)|[42nd](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|61|1800|[Ryan La](https://zwiftpower\.com/profile\.php?z=63118)|[WUMS](https://zwiftpower\.com/team\.php?id=4067)|—|—|[51st](https://zwiftpower\.com/events\.php?zid=159644)|[50th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|61|1800|[Chris Tommila](https://zwiftpower\.com/profile\.php?z=984241)|[WUMS](https://zwiftpower\.com/team\.php?id=4067)|—|—|[52nd](https://zwiftpower\.com/events\.php?zid=159644)|[49th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|62|999|[Ben Beger](https://zwiftpower\.com/profile\.php?z=163892)|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|62|999|[Ryan Kettleson](https://zwiftpower\.com/profile\.php?z=188883)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|62|999|[Jonathan Dee](https://zwiftpower\.com/profile\.php?z=449521)|—|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|62|999|[Blair Drader ZHR B TeamWKG](https://zwiftpower\.com/profile\.php?z=520895)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|—|[🥇](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|62|999|[Jeffrey Werner](https://zwiftpower\.com/profile\.php?z=1037294)|—|[🥇](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|—
B|63|997|[Gregg Magnus](https://zwiftpower\.com/profile\.php?z=60786)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|—|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=176476)
B|63|997|[kevin thompson](https://zwiftpower\.com/profile\.php?z=92113)|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|63|997|[Randy Reichardt \(Above & Beyond Cancer\)](https://zwiftpower\.com/profile\.php?z=235686)|—|—|—|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=173965)|—
B|64|995|[MarcAndre Doucet](https://zwiftpower\.com/profile\.php?z=703114)|[Stallion](https://zwiftpower\.com/team\.php?id=4154)|—|—|—|—|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=176476)
B|65|993|[Bryan Wenzel](https://zwiftpower\.com/profile\.php?z=90771)|[IOWA](https://zwiftpower\.com/team\.php?id=3870)|—|—|—|—|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=176476)
B|66|991|[David Anderson](https://zwiftpower\.com/profile\.php?z=385760)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|67|989|[Neil Ragan 🐘](https://zwiftpower\.com/profile\.php?z=41844)|[FINESSE](https://zwiftpower\.com/team\.php?id=2779)|—|—|[6th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|67|989|[Sergio Estrada](https://zwiftpower\.com/profile\.php?z=882916)|—|—|—|—|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=173965)|—
B|68|987|[Pete Watson \- 1R](https://zwiftpower\.com/profile\.php?z=106825)|[1R](https://zwiftpower\.com/team\.php?id=387)|—|—|—|—|[7th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
B|69|985|[Jean\-François Lorange CCC](https://zwiftpower\.com/profile\.php?z=383096)|[CCC](https://zwiftpower\.com/team\.php?id=2975)|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
B|69|985|[Rob Peters \(Team EN\)](https://zwiftpower\.com/profile\.php?z=536679)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|69|985|[Jeremy Pletka](https://zwiftpower\.com/profile\.php?z=1186202)|—|—|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|70|979|[Jon Roobol](https://zwiftpower\.com/profile\.php?z=12131)|—|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|70|979|[Derek D'Angelo 🐇](https://zwiftpower\.com/profile\.php?z=121836)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|—|—|—|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=173965)|—
B|70|979|[Chilam Balam](https://zwiftpower\.com/profile\.php?z=923642)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|[11th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|71|975|[Alexandre Filiatrault](https://zwiftpower\.com/profile\.php?z=302498)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=173965)|—
B|71|975|[Ed Braz](https://zwiftpower\.com/profile\.php?z=1179624)|—|—|—|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|72|971|[Nicolas Berger](https://zwiftpower\.com/profile\.php?z=193109)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|[15th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|72|971|[Josh McBee \(Seattle\!\) 4751](https://zwiftpower\.com/profile\.php?z=258134)|—|—|[15th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|72|971|[Simon Emery3823](https://zwiftpower\.com/profile\.php?z=865804)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|[15th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|73|969|[Tom Box](https://zwiftpower\.com/profile\.php?z=84)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[16th](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|—
B|73|969|[Jason Johnson](https://zwiftpower\.com/profile\.php?z=129871)|—|—|—|—|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=173965)|—
B|73|969|[Martin Faucher](https://zwiftpower\.com/profile\.php?z=762011)|[IRIS\-Bob Cyclo](https://zwiftpower\.com/team\.php?id=3601)|—|—|[16th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|74|967|[Frederic Wallman JZQc](https://zwiftpower\.com/profile\.php?z=8531)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[17th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|74|967|[John Stoker](https://zwiftpower\.com/profile\.php?z=99918)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|—|—|[17th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
B|74|967|[Poochiemoomoos 🐌](https://zwiftpower\.com/profile\.php?z=426785)|—|—|—|—|—|—|[17th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|74|967|[Mason Butler](https://zwiftpower\.com/profile\.php?z=481523)|[FlatLine](https://zwiftpower\.com/team\.php?id=2471)|—|—|[17th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|75|965|[Martin Roy](https://zwiftpower\.com/profile\.php?z=69421)|—|—|—|—|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=173965)|—
B|75|965|[Anson Michael Brooks](https://zwiftpower\.com/profile\.php?z=155766)|—|—|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
B|75|965|[Nam Lam TeamEN](https://zwiftpower\.com/profile\.php?z=594184)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[18th](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|—
B|76|963|[Alain St\-Louis](https://zwiftpower\.com/profile\.php?z=449185)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|—|[19th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
B|77|959|[Francois Menard\-Kilrane](https://zwiftpower\.com/profile\.php?z=195299)|[Cannondale/4iiii](https://zwiftpower\.com/team\.php?id=2105)|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|77|959|[Joshua Ricks](https://zwiftpower\.com/profile\.php?z=823201)|—|[21st](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|—
B|77|959|[Andrew Zurn](https://zwiftpower\.com/profile\.php?z=985501)|—|—|—|—|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=173965)|—
B|78|957|[Cory Dams](https://zwiftpower\.com/profile\.php?z=460663)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|—|—|—|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=176476)
B|79|955|[Brian McInnes](https://zwiftpower\.com/profile\.php?z=171139)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[23rd](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|—
B|79|955|[Dieter Sticht](https://zwiftpower\.com/profile\.php?z=404754)|—|—|—|—|—|—|—|[23rd](https://zwiftpower\.com/events\.php?zid=173965)|—
B|80|953|[P\. Schaefer](https://zwiftpower\.com/profile\.php?z=1125195)|—|—|[24th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|81|951|[Jeff Geisler 4278](https://zwiftpower\.com/profile\.php?z=709348)|[SAS Cycle Club](https://zwiftpower\.com/team\.php?id=1348)|[25th](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|—
B|81|951|[Franck FLP 6622](https://zwiftpower\.com/profile\.php?z=1051223)|—|—|—|—|—|—|—|—|[25th](https://zwiftpower\.com/events\.php?zid=176476)
B|82|949|[Bjortmund Malmhagen](https://zwiftpower\.com/profile\.php?z=58622)|[HBC](https://zwiftpower\.com/team\.php?id=1819)|—|—|—|—|—|[26th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|82|949|[George York](https://zwiftpower\.com/profile\.php?z=98092)|[COS Racing](https://zwiftpower\.com/team\.php?id=86)|[26th](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|—
B|82|949|[Nicolas Genest](https://zwiftpower\.com/profile\.php?z=182597)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|—|—|—|[26th](https://zwiftpower\.com/events\.php?zid=173965)|—
B|82|949|[Christof F](https://zwiftpower\.com/profile\.php?z=538691)|—|—|—|—|[26th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|82|949|[Hugo Gosselin](https://zwiftpower\.com/profile\.php?z=904218)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|[26th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|83|947|[James Easden](https://zwiftpower\.com/profile\.php?z=45531)|—|—|—|—|—|—|—|—|[27th](https://zwiftpower\.com/events\.php?zid=176476)
B|83|947|[Carlos Cancio](https://zwiftpower\.com/profile\.php?z=211428)|—|[27th](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|—
B|83|947|[Jason Blanchette](https://zwiftpower\.com/profile\.php?z=1153335)|—|—|—|—|—|—|—|[27th](https://zwiftpower\.com/events\.php?zid=173965)|—
B|84|945|[Will Peterson\- 9072](https://zwiftpower\.com/profile\.php?z=232287)|[GPVELOTEK](https://zwiftpower\.com/team\.php?id=3056)|—|—|—|[28th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|84|945|[Paul St\-Arneault JZQc](https://zwiftpower\.com/profile\.php?z=635764)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[28th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|84|945|[Myles Fennell](https://zwiftpower\.com/profile\.php?z=928234)|—|[28th](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|—
B|84|945|[Addison Simis](https://zwiftpower\.com/profile\.php?z=1208309)|—|—|—|—|—|—|—|[28th](https://zwiftpower\.com/events\.php?zid=173965)|—
B|85|943|[Lance Mitamura 5699](https://zwiftpower\.com/profile\.php?z=631643)|—|—|—|—|—|—|—|[29th](https://zwiftpower\.com/events\.php?zid=173965)|—
B|86|941|[AJ Jessee](https://zwiftpower\.com/profile\.php?z=261444)|—|—|—|—|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=173965)|—
B|86|941|[Josh Mahlandt BSC](https://zwiftpower\.com/profile\.php?z=324219)|[BikeSurgeon](https://zwiftpower\.com/team\.php?id=2179)|—|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
B|86|941|[Francis Bissonnette3926](https://zwiftpower\.com/profile\.php?z=452415)|—|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|86|941|[David Torrelio🐊🔰](https://zwiftpower\.com/profile\.php?z=490665)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|—|—|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|86|941|[\# OneKzBiker6813](https://zwiftpower\.com/profile\.php?z=523588)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|[30th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|87|939|[Matias Oyola](https://zwiftpower\.com/profile\.php?z=242270)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|—|—|—|—|—|—|[31st](https://zwiftpower\.com/events\.php?zid=173965)|—
B|88|937|[Randy Gemenden](https://zwiftpower\.com/profile\.php?z=780529)|—|—|—|—|[32nd](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|89|935|[Robert Dodds CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=378471)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|—|[33rd](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|89|935|[Sarah\-anne Beaupre](https://zwiftpower\.com/profile\.php?z=613766)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|—|—|—|[33rd](https://zwiftpower\.com/events\.php?zid=173965)|—
B|89|935|[Adam Susong](https://zwiftpower\.com/profile\.php?z=978092)|—|—|[33rd](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|90|933|[Erick Kercheval](https://zwiftpower\.com/profile\.php?z=234962)|[NLC](https://zwiftpower\.com/team\.php?id=1957)|—|—|[34th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|90|933|[Samuel Moore BSC B1](https://zwiftpower\.com/profile\.php?z=290222)|[BikeSurgeon](https://zwiftpower\.com/team\.php?id=2179)|—|—|—|—|—|—|[34th](https://zwiftpower\.com/events\.php?zid=173965)|—
B|91|929|[Cody McCullough](https://zwiftpower\.com/profile\.php?z=6044)|—|—|[36th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|91|929|[Nathan Tamayo 🐱](https://zwiftpower\.com/profile\.php?z=11921)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|—|—|—|[36th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|91|929|[Y ISSY](https://zwiftpower\.com/profile\.php?z=208937)|—|—|—|—|—|[36th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
B|91|929|[J\. Caselli](https://zwiftpower\.com/profile\.php?z=934540)|—|—|—|—|—|—|[36th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|92|927|[Grant W](https://zwiftpower\.com/profile\.php?z=6014)|[Bat City Cyc](https://zwiftpower\.com/team\.php?id=171)|—|—|—|[37th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|93|925|[Dan Glowitz](https://zwiftpower\.com/profile\.php?z=612100)|—|—|—|—|—|—|[38th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|94|923|[Martin Levasseur](https://zwiftpower\.com/profile\.php?z=1009997)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|—|—|[39th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
B|95|921|[Sylvain Roy](https://zwiftpower\.com/profile\.php?z=162971)|—|—|—|[40th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|95|921|[Mack Levine2951](https://zwiftpower\.com/profile\.php?z=726976)|—|—|[40th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|95|921|[Scott Izutsu3697](https://zwiftpower\.com/profile\.php?z=1074538)|—|—|—|—|—|[40th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
B|96|919|[David Coupland HERD](https://zwiftpower\.com/profile\.php?z=429495)|[HERD](https://zwiftpower\.com/team\.php?id=2639)|—|—|[41st](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|97|917|[Sébastien Hue](https://zwiftpower\.com/profile\.php?z=174461)|[CXS](https://zwiftpower\.com/team\.php?id=2321)|—|[42nd](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|98|915|[Stephane BELLIVIER](https://zwiftpower\.com/profile\.php?z=568538)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|—|[43rd](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
B|99|911|[Peter Walker](https://zwiftpower\.com/profile\.php?z=115121)|—|—|[45th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|99|911|[Derek McIntire Ⓥ](https://zwiftpower\.com/profile\.php?z=955953)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|[45th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|100|909|[Cycling Jobin](https://zwiftpower\.com/profile\.php?z=18671)|—|—|[46th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|100|909|[Brian Kingston](https://zwiftpower\.com/profile\.php?z=651269)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|—|[46th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|101|907|[Michelle Patton](https://zwiftpower\.com/profile\.php?z=98297)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|[47th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|102|903|[Max Rapoff](https://zwiftpower\.com/profile\.php?z=179823)|[Warriors](https://zwiftpower\.com/team\.php?id=3941)|—|—|[49th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|103|899|[Alan Dempsey](https://zwiftpower\.com/profile\.php?z=6999)|[HPP](https://zwiftpower\.com/team\.php?id=1528)|—|[51st](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|103|899|[Mark Springer](https://zwiftpower\.com/profile\.php?z=513187)|—|—|—|—|[51st](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|104|897|[D Zeggelaar](https://zwiftpower\.com/profile\.php?z=812036)|—|—|[52nd](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|105|895|[Chris Herlin](https://zwiftpower\.com/profile\.php?z=167348)|—|—|—|—|[53rd](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|106|893|[JR Ritchie](https://zwiftpower\.com/profile\.php?z=877541)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|[54th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|106|893|[Robert Jackson](https://zwiftpower\.com/profile\.php?z=1065723)|—|—|—|[54th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|107|891|[Mike Platt](https://zwiftpower\.com/profile\.php?z=307681)|—|—|—|—|[55th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|107|891|[David Calfat JZQC](https://zwiftpower\.com/profile\.php?z=1074349)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[55th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|108|889|[Michael Latsch](https://zwiftpower\.com/profile\.php?z=570992)|—|—|—|[56th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|108|889|[R oadieJodi \(Vegan\)](https://zwiftpower\.com/profile\.php?z=1004115)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|[56th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|109|887|[Rick Borns \(WMCR\)](https://zwiftpower\.com/profile\.php?z=69308)|—|—|—|—|[57th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|109|887|[Terry Patton](https://zwiftpower\.com/profile\.php?z=96440)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|[57th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|109|887|[Norman Romero](https://zwiftpower\.com/profile\.php?z=980877)|—|—|—|[57th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
B|110|883|[Doug Descheneaux](https://zwiftpower\.com/profile\.php?z=48160)|—|—|[59th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|110|883|[Michael Steinberg](https://zwiftpower\.com/profile\.php?z=238678)|—|—|—|—|[59th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
B|111|879|[Steve Laurin](https://zwiftpower\.com/profile\.php?z=172582)|[BCA](https://zwiftpower\.com/team\.php?id=3668)|—|[61st](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|112|877|[Jorge Martinez \#E3TriCoaching7100](https://zwiftpower\.com/profile\.php?z=15031)|—|—|[62nd](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|113|875|[Stu Schopp \(B\-\)](https://zwiftpower\.com/profile\.php?z=468681)|—|—|[63rd](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
B|114|873|[Juan Ascoli](https://zwiftpower\.com/profile\.php?z=46115)|—|—|[64th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
C|1|7896|[Mick Elliott](https://zwiftpower\.com/profile\.php?z=131471)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[🥇](https://zwiftpower\.com/events\.php?zid=152781)|[14th](https://zwiftpower\.com/events\.php?zid=155997)|[6th](https://zwiftpower\.com/events\.php?zid=159644)|[9th](https://zwiftpower\.com/events\.php?zid=163028)|[17th](https://zwiftpower\.com/events\.php?zid=167034)|[🥇](https://zwiftpower\.com/events\.php?zid=170592)|[6th](https://zwiftpower\.com/events\.php?zid=173965)|[🥈](https://zwiftpower\.com/events\.php?zid=176476)
C|2|7874|[Pete Greges](https://zwiftpower\.com/profile\.php?z=11230)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[4th](https://zwiftpower\.com/events\.php?zid=152781)|[9th](https://zwiftpower\.com/events\.php?zid=155997)|[5th](https://zwiftpower\.com/events\.php?zid=159644)|[8th](https://zwiftpower\.com/events\.php?zid=163028)|[19th](https://zwiftpower\.com/events\.php?zid=167034)|[17th](https://zwiftpower\.com/events\.php?zid=170592)|[4th](https://zwiftpower\.com/events\.php?zid=173965)|[🥇](https://zwiftpower\.com/events\.php?zid=176476)
C|3|6877|[Marko Hlibchuk Ⓥ](https://zwiftpower\.com/profile\.php?z=426765)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[🥉](https://zwiftpower\.com/events\.php?zid=152781)|[12th](https://zwiftpower\.com/events\.php?zid=155997)|[11th](https://zwiftpower\.com/events\.php?zid=159644)|[10th](https://zwiftpower\.com/events\.php?zid=163028)|[10th](https://zwiftpower\.com/events\.php?zid=167034)|[9th](https://zwiftpower\.com/events\.php?zid=170592)|—|[10th](https://zwiftpower\.com/events\.php?zid=176476)
C|4|6777|[Mike Naessens Cryo\-Gen](https://zwiftpower\.com/profile\.php?z=132112)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[16th](https://zwiftpower\.com/events\.php?zid=155997)|[13th](https://zwiftpower\.com/events\.php?zid=159644)|[17th](https://zwiftpower\.com/events\.php?zid=163028)|[28th](https://zwiftpower\.com/events\.php?zid=167034)|[8th](https://zwiftpower\.com/events\.php?zid=170592)|[12th](https://zwiftpower\.com/events\.php?zid=173965)|[21st](https://zwiftpower\.com/events\.php?zid=176476)
C|5|6667|[Manuel Rocha](https://zwiftpower\.com/profile\.php?z=70647)|[eRoberto](https://zwiftpower\.com/team\.php?id=3470)|—|[28th](https://zwiftpower\.com/events\.php?zid=155997)|[37th](https://zwiftpower\.com/events\.php?zid=159644)|[25th](https://zwiftpower\.com/events\.php?zid=163028)|[22nd](https://zwiftpower\.com/events\.php?zid=167034)|[23rd](https://zwiftpower\.com/events\.php?zid=170592)|[19th](https://zwiftpower\.com/events\.php?zid=173965)|[16th](https://zwiftpower\.com/events\.php?zid=176476)
C|6|6657|[Michael Siefritz Ⓥ](https://zwiftpower\.com/profile\.php?z=514647)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|[37th](https://zwiftpower\.com/events\.php?zid=155997)|[35th](https://zwiftpower\.com/events\.php?zid=159644)|[21st](https://zwiftpower\.com/events\.php?zid=163028)|[21st](https://zwiftpower\.com/events\.php?zid=167034)|[24th](https://zwiftpower\.com/events\.php?zid=170592)|[13th](https://zwiftpower\.com/events\.php?zid=173965)|[24th](https://zwiftpower\.com/events\.php?zid=176476)
C|7|5946|[Glynn Broner \(BCC\)KY](https://zwiftpower\.com/profile\.php?z=77670)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[5th](https://zwiftpower\.com/events\.php?zid=155997)|[9th](https://zwiftpower\.com/events\.php?zid=159644)|[🥈](https://zwiftpower\.com/events\.php?zid=163028)|[9th](https://zwiftpower\.com/events\.php?zid=167034)|[🥈](https://zwiftpower\.com/events\.php?zid=170592)|—|[🥉](https://zwiftpower\.com/events\.php?zid=176476)
C|8|5910|[Jason Economou](https://zwiftpower\.com/profile\.php?z=290132)|[Team X](https://zwiftpower\.com/team\.php?id=2)|—|—|[8th](https://zwiftpower\.com/events\.php?zid=159644)|[13th](https://zwiftpower\.com/events\.php?zid=163028)|[7th](https://zwiftpower\.com/events\.php?zid=167034)|[6th](https://zwiftpower\.com/events\.php?zid=170592)|[🥉](https://zwiftpower\.com/events\.php?zid=173965)|[11th](https://zwiftpower\.com/events\.php?zid=176476)
C|9|5856|[Kevin Mabey 🔰](https://zwiftpower\.com/profile\.php?z=835773)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|[29th](https://zwiftpower\.com/events\.php?zid=159644)|[5th](https://zwiftpower\.com/events\.php?zid=163028)|[5th](https://zwiftpower\.com/events\.php?zid=167034)|[21st](https://zwiftpower\.com/events\.php?zid=170592)|[7th](https://zwiftpower\.com/events\.php?zid=173965)|[8th](https://zwiftpower\.com/events\.php?zid=176476)
C|10|5848|[Jean Grenier JZQC \(C\)](https://zwiftpower\.com/profile\.php?z=421295)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[11th](https://zwiftpower\.com/events\.php?zid=155997)|[15th](https://zwiftpower\.com/events\.php?zid=159644)|[19th](https://zwiftpower\.com/events\.php?zid=163028)|—|[16th](https://zwiftpower\.com/events\.php?zid=170592)|[11th](https://zwiftpower\.com/events\.php?zid=173965)|[7th](https://zwiftpower\.com/events\.php?zid=176476)
C|11|5818|[Sebastien Bergeron](https://zwiftpower\.com/profile\.php?z=938286)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[5th](https://zwiftpower\.com/events\.php?zid=152781)|—|[22nd](https://zwiftpower\.com/events\.php?zid=159644)|—|[15th](https://zwiftpower\.com/events\.php?zid=167034)|[19th](https://zwiftpower\.com/events\.php?zid=170592)|[14th](https://zwiftpower\.com/events\.php?zid=173965)|[19th](https://zwiftpower\.com/events\.php?zid=176476)
C|12|5758|[Ayaz Asif Ⓥ](https://zwiftpower\.com/profile\.php?z=7759)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|[24th](https://zwiftpower\.com/events\.php?zid=155997)|[25th](https://zwiftpower\.com/events\.php?zid=159644)|[18th](https://zwiftpower\.com/events\.php?zid=163028)|[27th](https://zwiftpower\.com/events\.php?zid=167034)|[18th](https://zwiftpower\.com/events\.php?zid=170592)|—|[12th](https://zwiftpower\.com/events\.php?zid=176476)
C|13|5702|[Jeff Cockerill CryoGen](https://zwiftpower\.com/profile\.php?z=204152)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[32nd](https://zwiftpower\.com/events\.php?zid=155997)|[19th](https://zwiftpower\.com/events\.php?zid=159644)|[23rd](https://zwiftpower\.com/events\.php?zid=163028)|[33rd](https://zwiftpower\.com/events\.php?zid=167034)|[31st](https://zwiftpower\.com/events\.php?zid=170592)|—|[14th](https://zwiftpower\.com/events\.php?zid=176476)
C|14|4857|[Percy S](https://zwiftpower\.com/profile\.php?z=166608)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|—|—|[17th](https://zwiftpower\.com/events\.php?zid=159644)|[27th](https://zwiftpower\.com/events\.php?zid=163028)|[16th](https://zwiftpower\.com/events\.php?zid=167034)|[5th](https://zwiftpower\.com/events\.php?zid=170592)|—|[9th](https://zwiftpower\.com/events\.php?zid=176476)
C|15|4777|[Don Lycette CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=244209)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[7th](https://zwiftpower\.com/events\.php?zid=152781)|—|[23rd](https://zwiftpower\.com/events\.php?zid=159644)|[34th](https://zwiftpower\.com/events\.php?zid=163028)|—|[32nd](https://zwiftpower\.com/events\.php?zid=170592)|—|[18th](https://zwiftpower\.com/events\.php?zid=176476)
C|16|4759|[Adam Larrea \(5170\)](https://zwiftpower\.com/profile\.php?z=925229)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|[34th](https://zwiftpower\.com/events\.php?zid=155997)|[16th](https://zwiftpower\.com/events\.php?zid=159644)|[26th](https://zwiftpower\.com/events\.php?zid=163028)|—|[25th](https://zwiftpower\.com/events\.php?zid=170592)|—|[22nd](https://zwiftpower\.com/events\.php?zid=176476)
C|17|4703|[P\. Routledge CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=526163)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[31st](https://zwiftpower\.com/events\.php?zid=155997)|[40th](https://zwiftpower\.com/events\.php?zid=159644)|[24th](https://zwiftpower\.com/events\.php?zid=163028)|[30th](https://zwiftpower\.com/events\.php?zid=167034)|[26th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
C|18|3910|[Vince Kim🤖TFC](https://zwiftpower\.com/profile\.php?z=261873)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[10th](https://zwiftpower\.com/events\.php?zid=152781)|—|[21st](https://zwiftpower\.com/events\.php?zid=159644)|[4th](https://zwiftpower\.com/events\.php?zid=163028)|—|[12th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
C|19|3860|[John P](https://zwiftpower\.com/profile\.php?z=164752)|[Team X](https://zwiftpower\.com/team\.php?id=2)|—|—|[31st](https://zwiftpower\.com/events\.php?zid=159644)|[7th](https://zwiftpower\.com/events\.php?zid=163028)|[24th](https://zwiftpower\.com/events\.php?zid=167034)|[10th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
C|20|3838|[Steve Thibodeau](https://zwiftpower\.com/profile\.php?z=897282)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[19th](https://zwiftpower\.com/events\.php?zid=155997)|[41st](https://zwiftpower\.com/events\.php?zid=159644)|—|—|[15th](https://zwiftpower\.com/events\.php?zid=170592)|[8th](https://zwiftpower\.com/events\.php?zid=173965)|—
C|21|3812|[Martin Brousseau \(JZQc\)](https://zwiftpower\.com/profile\.php?z=886279)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[20th](https://zwiftpower\.com/events\.php?zid=155997)|—|[33rd](https://zwiftpower\.com/events\.php?zid=163028)|—|[28th](https://zwiftpower\.com/events\.php?zid=170592)|—|[15th](https://zwiftpower\.com/events\.php?zid=176476)
C|22|3786|[Michel Descoteaux JZQC](https://zwiftpower\.com/profile\.php?z=193845)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[6th](https://zwiftpower\.com/events\.php?zid=152781)|—|[42nd](https://zwiftpower\.com/events\.php?zid=159644)|—|[32nd](https://zwiftpower\.com/events\.php?zid=167034)|[29th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
C|23|2975|[Justin Kitney](https://zwiftpower\.com/profile\.php?z=123889)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|[7th](https://zwiftpower\.com/events\.php?zid=159644)|[6th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=173965)|—
C|24|2965|[John Culhane](https://zwiftpower\.com/profile\.php?z=156937)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[🥉](https://zwiftpower\.com/events\.php?zid=155997)|—|[12th](https://zwiftpower\.com/events\.php?zid=163028)|[4th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|25|2879|[Dale Bradbury](https://zwiftpower\.com/profile\.php?z=817234)|—|—|—|[20th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=170592)|—|[20th](https://zwiftpower\.com/events\.php?zid=176476)
C|26|2797|[Danny Arana](https://zwiftpower\.com/profile\.php?z=204059)|—|—|—|[44th](https://zwiftpower\.com/events\.php?zid=159644)|[36th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|[23rd](https://zwiftpower\.com/events\.php?zid=176476)
C|27|1992|[Stefanie Hollmichel](https://zwiftpower\.com/profile\.php?z=51322)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|[4th](https://zwiftpower\.com/events\.php?zid=155997)|[🥇](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
C|27|1992|[Ryan DeadPool Miller TFC \(C\)](https://zwiftpower\.com/profile\.php?z=331384)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=167034)|[4th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
C|28|1986|[Aaron Fields](https://zwiftpower\.com/profile\.php?z=640932)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|—|[6th](https://zwiftpower\.com/events\.php?zid=155997)|[🥈](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
C|29|1984|[Jim Coker](https://zwiftpower\.com/profile\.php?z=589891)|—|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=176476)
C|30|1980|[Dany Bouchard4414](https://zwiftpower\.com/profile\.php?z=434253)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|—|—|[7th](https://zwiftpower\.com/events\.php?zid=170592)|—|[4th](https://zwiftpower\.com/events\.php?zid=176476)
C|31|1978|[Rudolfo Poomp](https://zwiftpower\.com/profile\.php?z=64796)|[HBC](https://zwiftpower\.com/team\.php?id=1819)|—|—|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=170592)|[9th](https://zwiftpower\.com/events\.php?zid=173965)|—
C|32|1962|[Jeff Skinner CRYO\-GEN \(C\)](https://zwiftpower\.com/profile\.php?z=96418)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|—|—|[15th](https://zwiftpower\.com/events\.php?zid=173965)|[5th](https://zwiftpower\.com/events\.php?zid=176476)
C|33|1960|[Keevin Griffith](https://zwiftpower\.com/profile\.php?z=538740)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=170592)|[10th](https://zwiftpower\.com/events\.php?zid=173965)|—
C|34|1950|[Jeremy hopwood](https://zwiftpower\.com/profile\.php?z=36654)|—|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=163028)|[25th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|35|1944|[Nicolas Trépanier](https://zwiftpower\.com/profile\.php?z=183617)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|[14th](https://zwiftpower\.com/events\.php?zid=159644)|[15th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
C|35|1944|[Wynn Legon](https://zwiftpower\.com/profile\.php?z=560232)|—|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=163028)|—|[13th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
C|36|1942|[Ed Phinney <HERD>](https://zwiftpower\.com/profile\.php?z=769917)|[HERD](https://zwiftpower\.com/team\.php?id=2639)|—|—|[10th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|[20th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
C|37|1932|[shii tod](https://zwiftpower\.com/profile\.php?z=365733)|—|—|—|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=170592)|[5th](https://zwiftpower\.com/events\.php?zid=173965)|—
C|38|1920|[JF Marsan](https://zwiftpower\.com/profile\.php?z=411968)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[9th](https://zwiftpower\.com/events\.php?zid=152781)|—|[32nd](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
C|39|1918|[Jason Thompson](https://zwiftpower\.com/profile\.php?z=838875)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|[22nd](https://zwiftpower\.com/events\.php?zid=155997)|—|—|[20th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|40|1910|[Francois Pelletier JZQc](https://zwiftpower\.com/profile\.php?z=943511)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[33rd](https://zwiftpower\.com/events\.php?zid=155997)|—|—|[13th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|41|1900|[Brian Reimer \[HERD\] 🍺🐝](https://zwiftpower\.com/profile\.php?z=147510)|[HERD](https://zwiftpower\.com/team\.php?id=2639)|—|—|—|[31st](https://zwiftpower\.com/events\.php?zid=163028)|—|—|[20th](https://zwiftpower\.com/events\.php?zid=173965)|—
C|42|1898|[Carlos Roldán](https://zwiftpower\.com/profile\.php?z=396592)|—|—|—|—|—|[36th](https://zwiftpower\.com/events\.php?zid=167034)|—|[16th](https://zwiftpower\.com/events\.php?zid=173965)|—
C|43|1888|[Terio Sutherland \(BCC\)](https://zwiftpower\.com/profile\.php?z=912201)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|[20th](https://zwiftpower\.com/events\.php?zid=163028)|—|[37th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
C|44|1880|[Dave Pyle](https://zwiftpower\.com/profile\.php?z=430828)|[Team X](https://zwiftpower\.com/team\.php?id=2)|—|—|[34th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|[27th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
C|45|1876|[Martin Levasseur](https://zwiftpower\.com/profile\.php?z=1009997)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|[45th](https://zwiftpower\.com/events\.php?zid=159644)|—|[18th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|46|1868|[Marc Sharer](https://zwiftpower\.com/profile\.php?z=901750)|—|—|—|[39th](https://zwiftpower\.com/events\.php?zid=159644)|[28th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
C|47|1858|[Darrell Kincer \(BCC\)](https://zwiftpower\.com/profile\.php?z=27344)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|[38th](https://zwiftpower\.com/events\.php?zid=167034)|[34th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
C|48|1854|[Bryan Pearson \[CRYO\-GEN\]](https://zwiftpower\.com/profile\.php?z=649507)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|[40th](https://zwiftpower\.com/events\.php?zid=163028)|[34th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|49|999|[S\. Deslauriers Velozone JZQC](https://zwiftpower\.com/profile\.php?z=182546)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[🥇](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
C|50|997|[David M Renton III](https://zwiftpower\.com/profile\.php?z=83511)|[Zwift HQ](https://zwiftpower\.com/team\.php?id=1977)|[🥈](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|—
C|50|997|[Gene Fowler \(BCC\)](https://zwiftpower\.com/profile\.php?z=186111)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[🥈](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
C|50|997|"[Charlie ""Dactah"" Miller TFC](https://zwiftpower\.com/profile\.php?z=276186)"|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|50|997|[Marcus Fellowes](https://zwiftpower\.com/profile\.php?z=754114)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=173965)|—
C|51|995|[Martin Savard](https://zwiftpower\.com/profile\.php?z=235515)|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
C|51|995|[Robert Dodds CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=378471)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|52|993|[Bailey Hamilton](https://zwiftpower\.com/profile\.php?z=833877)|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
C|53|989|[Pierre Arbez2134JZQc](https://zwiftpower\.com/profile\.php?z=568382)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|54|987|[Mike Dent](https://zwiftpower\.com/profile\.php?z=955715)|—|—|[7th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
C|55|985|[Will Botti \(C\)](https://zwiftpower\.com/profile\.php?z=211237)|—|[8th](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|—
C|55|985|[David Sorg](https://zwiftpower\.com/profile\.php?z=243886)|[Team CLS](https://zwiftpower\.com/team\.php?id=1426)|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|55|985|[Zach Bennett \#EN](https://zwiftpower\.com/profile\.php?z=968668)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|—|[8th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
C|56|981|[Olivier De Ladurantaye6755](https://zwiftpower\.com/profile\.php?z=840643)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[10th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
C|57|979|[Dennis Porter1754](https://zwiftpower\.com/profile\.php?z=68968)|—|—|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|57|979|[Chris McCormick](https://zwiftpower\.com/profile\.php?z=380762)|—|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
C|58|977|[Daniel Roy \- CCC 🚴](https://zwiftpower\.com/profile\.php?z=720162)|[CCC](https://zwiftpower\.com/team\.php?id=2975)|—|—|[12th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
C|58|977|[Luke Austen 7802](https://zwiftpower\.com/profile\.php?z=817734)|—|—|—|—|—|[12th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|59|975|[Doug Pape](https://zwiftpower\.com/profile\.php?z=5425)|—|—|[13th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
C|59|975|[Brian Kristof](https://zwiftpower\.com/profile\.php?z=878403)|—|—|—|—|—|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=176476)
C|60|973|[Ray J4124](https://zwiftpower\.com/profile\.php?z=319615)|—|—|—|—|—|—|[14th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
C|60|973|[Laurent Huot JZQc](https://zwiftpower\.com/profile\.php?z=669891)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|[14th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
C|60|973|[Kim Schaff](https://zwiftpower\.com/profile\.php?z=1053714)|—|—|—|—|—|[14th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|61|971|[Martin Wright Velozone\(JZQC\)](https://zwiftpower\.com/profile\.php?z=76715)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[15th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
C|62|967|[Ryan Johs](https://zwiftpower\.com/profile\.php?z=1953)|—|—|—|—|—|—|—|—|[17th](https://zwiftpower\.com/events\.php?zid=176476)
C|62|967|[Pierre Higgins](https://zwiftpower\.com/profile\.php?z=114363)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[17th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
C|62|967|[Craig Gritzen ⓥ](https://zwiftpower\.com/profile\.php?z=527678)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|—|—|—|[17th](https://zwiftpower\.com/events\.php?zid=173965)|—
C|63|965|[Bob Friesen](https://zwiftpower\.com/profile\.php?z=110692)|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
C|63|965|[Eric Olds \[BIKEMAN\.COM\]](https://zwiftpower\.com/profile\.php?z=873324)|[Bikeman\.com](https://zwiftpower\.com/team\.php?id=3969)|—|[18th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
C|63|965|[Christopher Mc Mahon JZQc](https://zwiftpower\.com/profile\.php?z=966195)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=173965)|—
C|64|959|[Steve Lidston Ⓥ](https://zwiftpower\.com/profile\.php?z=180464)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|[21st](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
C|64|959|[R NII](https://zwiftpower\.com/profile\.php?z=516693)|—|—|—|—|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=173965)|—
C|65|957|[Josh Becker](https://zwiftpower\.com/profile\.php?z=203930)|[HPP](https://zwiftpower\.com/team\.php?id=1528)|—|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
C|65|957|[Me Bonkers](https://zwiftpower\.com/profile\.php?z=246764)|—|—|—|—|—|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=173965)|—
C|66|955|[Paul Kelliher \(HERD\)](https://zwiftpower\.com/profile\.php?z=657014)|[HERD](https://zwiftpower\.com/team\.php?id=2639)|—|—|—|—|—|—|[23rd](https://zwiftpower\.com/events\.php?zid=173965)|—
C|66|955|[Jeff Brand](https://zwiftpower\.com/profile\.php?z=938363)|—|—|—|—|—|[23rd](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|66|955|[Doug Streeter](https://zwiftpower\.com/profile\.php?z=1032450)|—|—|[23rd](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
C|67|953|[Hugo Walker](https://zwiftpower\.com/profile\.php?z=113284)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|[24th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
C|67|953|[William Hunt](https://zwiftpower\.com/profile\.php?z=724628)|—|—|—|—|—|—|—|[24th](https://zwiftpower\.com/events\.php?zid=173965)|—
C|68|951|[AJ Carr CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=744194)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|—|—|—|[25th](https://zwiftpower\.com/events\.php?zid=176476)
C|68|951|[Joel Davidson](https://zwiftpower\.com/profile\.php?z=1103458)|—|—|[25th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
C|69|949|[Mike Louie](https://zwiftpower\.com/profile\.php?z=261230)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|—|—|—|—|—|—|—|[26th](https://zwiftpower\.com/events\.php?zid=176476)
C|69|949|[Christian Dumaine \(C\)](https://zwiftpower\.com/profile\.php?z=340884)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[26th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
C|69|949|[Ed Lai \[DIRT\]](https://zwiftpower\.com/profile\.php?z=795742)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|—|—|[26th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|69|949|[Winston Young](https://zwiftpower\.com/profile\.php?z=855584)|—|—|—|[26th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
C|70|947|[James Hall](https://zwiftpower\.com/profile\.php?z=382471)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|[27th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
C|70|947|[Scott Haller \(HERD\)](https://zwiftpower\.com/profile\.php?z=1122925)|[HERD](https://zwiftpower\.com/team\.php?id=2639)|—|—|[27th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
C|71|945|[Scott Sugimoto](https://zwiftpower\.com/profile\.php?z=862965)|—|—|—|[28th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
C|72|943|[Chris Vachon](https://zwiftpower\.com/profile\.php?z=2190)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|—|—|[29th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|72|943|[Tom Lokar 🐺](https://zwiftpower\.com/profile\.php?z=88976)|—|—|[29th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
C|72|943|[PC Mountain](https://zwiftpower\.com/profile\.php?z=150525)|—|—|—|—|[29th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
C|73|941|[Paul Wright](https://zwiftpower\.com/profile\.php?z=303501)|—|—|[30th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
C|73|941|[Burt Ayers](https://zwiftpower\.com/profile\.php?z=708161)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
C|73|941|[Francois Courteau 4770](https://zwiftpower\.com/profile\.php?z=788394)|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
C|74|939|[Henry Holmes](https://zwiftpower\.com/profile\.php?z=8494)|[Team BMTR](https://zwiftpower\.com/team\.php?id=2873)|—|—|—|—|[31st](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|75|937|[joe payd](https://zwiftpower\.com/profile\.php?z=654223)|—|—|—|—|[32nd](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
C|76|935|[Benoit Auger Forest KISS\(D\)](https://zwiftpower\.com/profile\.php?z=222531)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|[33rd](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
C|76|935|[Martin Ross](https://zwiftpower\.com/profile\.php?z=388088)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|—|—|[33rd](https://zwiftpower\.com/events\.php?zid=170592)|—|—
C|77|931|[J Hysler \(HERD\)](https://zwiftpower\.com/profile\.php?z=313837)|[HERD](https://zwiftpower\.com/team\.php?id=2639)|—|—|—|—|—|[35th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
C|77|931|[Ghislain Cormier 7660](https://zwiftpower\.com/profile\.php?z=460728)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[35th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
C|77|931|[Shug Wilson Ⓥ\[VEGAN\]](https://zwiftpower\.com/profile\.php?z=541203)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|[35th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
C|77|931|[JC Smith](https://zwiftpower\.com/profile\.php?z=811004)|—|—|—|—|—|[35th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|78|929|[Brian Mull CRYO\-GEN \(C\)](https://zwiftpower\.com/profile\.php?z=671125)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[36th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
C|78|929|[Garron Amos](https://zwiftpower\.com/profile\.php?z=870571)|[ExpFay](https://zwiftpower\.com/team\.php?id=2260)|—|—|[36th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
C|78|929|[Stefan Van Kessel](https://zwiftpower\.com/profile\.php?z=901165)|—|—|—|—|—|—|[36th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
C|79|927|[Mark Jeans](https://zwiftpower\.com/profile\.php?z=428951)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|—|[37th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
C|79|927|[Kevin Hyre CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=557094)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|[37th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|80|925|[marian haiducJZQC](https://zwiftpower\.com/profile\.php?z=483193)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|[38th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
C|80|925|[Chris Sweigard](https://zwiftpower\.com/profile\.php?z=951710)|—|—|—|—|[38th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
C|81|923|[Paige Haydon Lutz \(CRYO\-GEN\)](https://zwiftpower\.com/profile\.php?z=311565)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|[39th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
C|81|923|[Tim Norejko](https://zwiftpower\.com/profile\.php?z=1092297)|—|—|—|—|—|[39th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
C|82|919|[Brian Hester](https://zwiftpower\.com/profile\.php?z=33436)|—|—|—|—|[41st](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
C|83|917|[David KHUONG VietVelo](https://zwiftpower\.com/profile\.php?z=438132)|[V I E T V E L O](https://zwiftpower\.com/team\.php?id=2231)|—|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
C|84|915|[John Walker](https://zwiftpower\.com/profile\.php?z=411864)|—|—|—|[43rd](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
C|85|909|[David LeBaron](https://zwiftpower\.com/profile\.php?z=414671)|—|—|—|[46th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
C|86|907|[Tee Rex](https://zwiftpower\.com/profile\.php?z=246952)|—|—|—|[47th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
C|87|905|[Wenzel Ablola](https://zwiftpower\.com/profile\.php?z=302851)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|[48th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
D|1|5974|[Alain Lejeune](https://zwiftpower\.com/profile\.php?z=36413)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥉](https://zwiftpower\.com/events\.php?zid=152781)|[🥇](https://zwiftpower\.com/events\.php?zid=155997)|[🥇](https://zwiftpower\.com/events\.php?zid=159644)|—|[5th](https://zwiftpower\.com/events\.php?zid=167034)|—|[5th](https://zwiftpower\.com/events\.php?zid=173965)|[🥇](https://zwiftpower\.com/events\.php?zid=176476)
D|2|5946|[Sylvain Normand JZQc](https://zwiftpower\.com/profile\.php?z=357055)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[5th](https://zwiftpower\.com/events\.php?zid=152781)|[9th](https://zwiftpower\.com/events\.php?zid=155997)|[5th](https://zwiftpower\.com/events\.php?zid=159644)|[🥉](https://zwiftpower\.com/events\.php?zid=163028)|—|[4th](https://zwiftpower\.com/events\.php?zid=170592)|—|[4th](https://zwiftpower\.com/events\.php?zid=176476)
D|3|5908|[Chuck Kozlowski](https://zwiftpower\.com/profile\.php?z=998)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|[10th](https://zwiftpower\.com/events\.php?zid=159644)|[12th](https://zwiftpower\.com/events\.php?zid=163028)|[11th](https://zwiftpower\.com/events\.php?zid=167034)|[5th](https://zwiftpower\.com/events\.php?zid=170592)|[9th](https://zwiftpower\.com/events\.php?zid=173965)|[🥈](https://zwiftpower\.com/events\.php?zid=176476)
D|4|4973|[Xavier Laroche](https://zwiftpower\.com/profile\.php?z=427658)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥈](https://zwiftpower\.com/events\.php?zid=152781)|[🥉](https://zwiftpower\.com/events\.php?zid=155997)|[7th](https://zwiftpower\.com/events\.php?zid=159644)|—|[🥈](https://zwiftpower\.com/events\.php?zid=167034)|—|[🥈](https://zwiftpower\.com/events\.php?zid=173965)|—
D|5|4953|[Christopher Mc Mahon JZQc](https://zwiftpower\.com/profile\.php?z=966195)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[5th](https://zwiftpower\.com/events\.php?zid=155997)|[12th](https://zwiftpower\.com/events\.php?zid=159644)|[🥇](https://zwiftpower\.com/events\.php?zid=163028)|[7th](https://zwiftpower\.com/events\.php?zid=167034)|[🥇](https://zwiftpower\.com/events\.php?zid=170592)|—|—
D|6|3972|[Ranji Carluen](https://zwiftpower\.com/profile\.php?z=552690)|[MGCC](https://zwiftpower\.com/team\.php?id=114)|[4th](https://zwiftpower\.com/events\.php?zid=152781)|—|[6th](https://zwiftpower\.com/events\.php?zid=159644)|[🥈](https://zwiftpower\.com/events\.php?zid=163028)|[4th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
D|7|3960|[AJ Carr CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=744194)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=163028)|[🥉](https://zwiftpower\.com/events\.php?zid=167034)|[🥈](https://zwiftpower\.com/events\.php?zid=170592)|[6th](https://zwiftpower\.com/events\.php?zid=173965)|—
D|8|3940|[Jacques Wiseman JZQC \(D\)](https://zwiftpower\.com/profile\.php?z=327823)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[6th](https://zwiftpower\.com/events\.php?zid=152781)|[7th](https://zwiftpower\.com/events\.php?zid=155997)|[13th](https://zwiftpower\.com/events\.php?zid=159644)|[6th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
D|9|2973|[Allen Madore1830](https://zwiftpower\.com/profile\.php?z=4134)|[Team X](https://zwiftpower\.com/team\.php?id=2)|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=167034)|[🥉](https://zwiftpower\.com/events\.php?zid=170592)|—|[🥉](https://zwiftpower\.com/events\.php?zid=176476)
D|10|2955|[Joseph Padilla \[HERD\]](https://zwiftpower\.com/profile\.php?z=923771)|[HERD](https://zwiftpower\.com/team\.php?id=2639)|—|—|[11th](https://zwiftpower\.com/events\.php?zid=159644)|[8th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=176476)
D|11|2943|[Didier Bonneville JZQC](https://zwiftpower\.com/profile\.php?z=824267)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[8th](https://zwiftpower\.com/events\.php?zid=152781)|[10th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|[12th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
D|12|1982|[Brian Mackinnon](https://zwiftpower\.com/profile\.php?z=39554)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[8th](https://zwiftpower\.com/events\.php?zid=155997)|[🥈](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
D|12|1982|[Marc Boily](https://zwiftpower\.com/profile\.php?z=175371)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[4th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|[6th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
D|12|1982|[Monty McNeil](https://zwiftpower\.com/profile\.php?z=966252)|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|[7th](https://zwiftpower\.com/events\.php?zid=173965)|—
D|12|1982|[Doug Moore](https://zwiftpower\.com/profile\.php?z=972377)|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=159644)|—|[🥇](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
D|13|999|[Benoit Auger Forest KISS\(D\)](https://zwiftpower\.com/profile\.php?z=222531)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=173965)|—
D|13|999|[JAvello Temukano](https://zwiftpower\.com/profile\.php?z=479841)|—|[🥇](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|—
D|14|997|[Jonathan Rivest](https://zwiftpower\.com/profile\.php?z=48031)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[🥈](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
D|15|995|[Genevieve Dumais](https://zwiftpower\.com/profile\.php?z=955847)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=173965)|—
D|16|993|[Jeff May CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=194178)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|[4th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
D|16|993|[Ian Tuck](https://zwiftpower\.com/profile\.php?z=622819)|—|—|—|—|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=173965)|—
D|16|993|[Xenoatic K](https://zwiftpower\.com/profile\.php?z=920072)|—|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
D|17|991|[Jeff Patterson](https://zwiftpower\.com/profile\.php?z=1151609)|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
D|18|989|[Mark Mamrega](https://zwiftpower\.com/profile\.php?z=766185)|—|—|—|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
D|18|989|[Jamie Overton](https://zwiftpower\.com/profile\.php?z=774249)|—|—|[6th](https://zwiftpower\.com/events\.php?zid=155997)|—|—|—|—|—|—
D|19|987|[Keri Hindman](https://zwiftpower\.com/profile\.php?z=429197)|—|—|—|—|[7th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
D|19|987|[David Hunt](https://zwiftpower\.com/profile\.php?z=777791)|—|[7th](https://zwiftpower\.com/events\.php?zid=152781)|—|—|—|—|—|—|—
D|19|987|[Anthony Gregoire](https://zwiftpower\.com/profile\.php?z=1024640)|—|—|—|—|—|—|[7th](https://zwiftpower\.com/events\.php?zid=170592)|—|—
D|20|985|[Brandon OHern](https://zwiftpower\.com/profile\.php?z=312554)|—|—|—|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=173965)|—
D|20|985|[Terio Sutherland \(BCC\)](https://zwiftpower\.com/profile\.php?z=912201)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|[8th](https://zwiftpower\.com/events\.php?zid=159644)|—|—|—|—|—
D|20|985|[Brad Bunde](https://zwiftpower\.com/profile\.php?z=1151067)|—|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
D|21|983|[Jeffry Archer \(Montreal\)](https://zwiftpower\.com/profile\.php?z=1005681)|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—
D|22|981|[Richard Lanoue](https://zwiftpower\.com/profile\.php?z=178958)|—|—|—|—|—|[10th](https://zwiftpower\.com/events\.php?zid=167034)|—|—|—
D|22|981|[Jason Mcdonald \(anderson sc\)](https://zwiftpower\.com/profile\.php?z=227384)|—|—|—|—|[10th](https://zwiftpower\.com/events\.php?zid=163028)|—|—|—|—



## KISS Community League — Americas Women

Category|Rank|Total Points|Name|Team|1|2|3|4|5|6|7|8
---|---|---|---|---|---|---|---|---|---|---|---|---
A|1|7954|[Catrina Weiss \(CIS/Ladies\)](https://zwiftpower\.com/profile\.php?z=4727)|[CIS/Ladies](https://zwiftpower\.com/team\.php?id=3773)|[4th](https://zwiftpower\.com/events\.php?zid=154311)|[5th](https://zwiftpower\.com/events\.php?zid=155996)|[🥈](https://zwiftpower\.com/events\.php?zid=159645)|[5th](https://zwiftpower\.com/events\.php?zid=163029)|[🥉](https://zwiftpower\.com/events\.php?zid=167035)|[4th](https://zwiftpower\.com/events\.php?zid=170591)|[🥇](https://zwiftpower\.com/events\.php?zid=173966)|[🥉](https://zwiftpower\.com/events\.php?zid=176369)
A|2|4991|[Ronna Akins](https://zwiftpower\.com/profile\.php?z=72495)|[We Showed Up](https://zwiftpower\.com/team\.php?id=2216)|—|[🥈](https://zwiftpower\.com/events\.php?zid=155996)|—|[🥇](https://zwiftpower\.com/events\.php?zid=163029)|[🥇](https://zwiftpower\.com/events\.php?zid=167035)|[🥇](https://zwiftpower\.com/events\.php?zid=170591)|—|[🥈](https://zwiftpower\.com/events\.php?zid=176369)
A|3|4985|[Stephanie Ossenbrink](https://zwiftpower\.com/profile\.php?z=479580)|[We Showed Up](https://zwiftpower\.com/team\.php?id=2216)|[🥇](https://zwiftpower\.com/events\.php?zid=154311)|—|—|[4th](https://zwiftpower\.com/events\.php?zid=163029)|[🥈](https://zwiftpower\.com/events\.php?zid=167035)|[🥈](https://zwiftpower\.com/events\.php?zid=170591)|—|[🥇](https://zwiftpower\.com/events\.php?zid=176369)
A|4|2985|[Angela Webster](https://zwiftpower\.com/profile\.php?z=1366)|[Vixen/Fox](https://zwiftpower\.com/team\.php?id=1940)|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=159645)|[🥉](https://zwiftpower\.com/events\.php?zid=163029)|—|[🥉](https://zwiftpower\.com/events\.php?zid=170591)|—|—
A|4|2985|[Jill Seeman](https://zwiftpower\.com/profile\.php?z=15327)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥉](https://zwiftpower\.com/events\.php?zid=154311)|[4th](https://zwiftpower\.com/events\.php?zid=155996)|—|[🥈](https://zwiftpower\.com/events\.php?zid=163029)|—|—|—|—
A|5|1998|[Janna Gillick](https://zwiftpower\.com/profile\.php?z=460700)|[GSZ](https://zwiftpower\.com/team\.php?id=3992)|—|[🥇](https://zwiftpower\.com/events\.php?zid=155996)|[🥇](https://zwiftpower\.com/events\.php?zid=159645)|—|—|—|—|—
A|6|1994|[Emily Joyner](https://zwiftpower\.com/profile\.php?z=394039)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|[🥈](https://zwiftpower\.com/events\.php?zid=154311)|—|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=173966)|—
A|7|995|[Isabelle Veilleux JZQC](https://zwiftpower\.com/profile\.php?z=932401)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[🥉](https://zwiftpower\.com/events\.php?zid=155996)|—|—|—|—|—|—
B|1|6985|[Debra Sluz Parker](https://zwiftpower\.com/profile\.php?z=260577)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥈](https://zwiftpower\.com/events\.php?zid=154311)|[🥈](https://zwiftpower\.com/events\.php?zid=155996)|[🥈](https://zwiftpower\.com/events\.php?zid=159645)|[🥈](https://zwiftpower\.com/events\.php?zid=163029)|—|[🥇](https://zwiftpower\.com/events\.php?zid=170591)|[🥇](https://zwiftpower\.com/events\.php?zid=173966)|[🥇](https://zwiftpower\.com/events\.php?zid=176369)
B|2|5968|[Kristina Pearson](https://zwiftpower\.com/profile\.php?z=43324)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[4th](https://zwiftpower\.com/events\.php?zid=154311)|[🥉](https://zwiftpower\.com/events\.php?zid=155996)|[🥇](https://zwiftpower\.com/events\.php?zid=159645)|[5th](https://zwiftpower\.com/events\.php?zid=163029)|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=173966)|[4th](https://zwiftpower\.com/events\.php?zid=176369)
B|3|3966|[Annie Dulude JZQC](https://zwiftpower\.com/profile\.php?z=558272)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[10th](https://zwiftpower\.com/events\.php?zid=155996)|—|[4th](https://zwiftpower\.com/events\.php?zid=163029)|[🥇](https://zwiftpower\.com/events\.php?zid=167035)|[4th](https://zwiftpower\.com/events\.php?zid=170591)|—|—
B|4|3952|[Lorena Sandoval](https://zwiftpower\.com/profile\.php?z=328805)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|—|[9th](https://zwiftpower\.com/events\.php?zid=155996)|[7th](https://zwiftpower\.com/events\.php?zid=159645)|[7th](https://zwiftpower\.com/events\.php?zid=163029)|[🥉](https://zwiftpower\.com/events\.php?zid=167035)|—|—|—
B|5|2989|[tracy moore](https://zwiftpower\.com/profile\.php?z=654026)|[We Showed Up](https://zwiftpower\.com/team\.php?id=2216)|—|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=170591)|[🥉](https://zwiftpower\.com/events\.php?zid=173966)|[🥈](https://zwiftpower\.com/events\.php?zid=176369)
B|6|2981|[Lisa Hynes](https://zwiftpower\.com/profile\.php?z=405131)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥉](https://zwiftpower\.com/events\.php?zid=154311)|[4th](https://zwiftpower\.com/events\.php?zid=155996)|[4th](https://zwiftpower\.com/events\.php?zid=159645)|—|—|—|—|—
B|7|2979|[Ms Sakushi](https://zwiftpower\.com/profile\.php?z=179751)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[🥇](https://zwiftpower\.com/events\.php?zid=154311)|[5th](https://zwiftpower\.com/events\.php?zid=155996)|—|[6th](https://zwiftpower\.com/events\.php?zid=163029)|—|—|—|—
B|7|2979|[Elise Rowe](https://zwiftpower\.com/profile\.php?z=213683)|[LaPrima](https://zwiftpower\.com/team\.php?id=2586)|—|[6th](https://zwiftpower\.com/events\.php?zid=155996)|[🥉](https://zwiftpower\.com/events\.php?zid=159645)|[🥉](https://zwiftpower\.com/events\.php?zid=163029)|—|—|—|—
B|8|2969|[Sheryl Mohwinkel](https://zwiftpower\.com/profile\.php?z=586407)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[5th](https://zwiftpower\.com/events\.php?zid=154311)|[7th](https://zwiftpower\.com/events\.php?zid=155996)|[5th](https://zwiftpower\.com/events\.php?zid=159645)|—|—|—|—|—
B|9|1998|[Morgan Cabot](https://zwiftpower\.com/profile\.php?z=967586)|[GSZ](https://zwiftpower\.com/team\.php?id=3992)|—|[🥇](https://zwiftpower\.com/events\.php?zid=155996)|—|[🥇](https://zwiftpower\.com/events\.php?zid=163029)|—|—|—|—
B|10|1990|[Anna Rogers \[REVO\]](https://zwiftpower\.com/profile\.php?z=265859)|[REVO](https://zwiftpower\.com/team\.php?id=2672)|—|—|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=170591)|—|[🥉](https://zwiftpower\.com/events\.php?zid=176369)
B|11|1982|[Sarah\-anne Beaupre](https://zwiftpower\.com/profile\.php?z=613766)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[8th](https://zwiftpower\.com/events\.php?zid=155996)|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=167035)|—|—|—
B|12|1978|[Kristi Mccracken](https://zwiftpower\.com/profile\.php?z=195887)|[LPC](https://zwiftpower\.com/team\.php?id=2089)|[6th](https://zwiftpower\.com/events\.php?zid=154311)|—|[6th](https://zwiftpower\.com/events\.php?zid=159645)|—|—|—|—|—
B|13|991|[Krista Kadatz](https://zwiftpower\.com/profile\.php?z=928030)|—|—|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=170591)|—|—
B|14|979|[Elizabeth Lawless8309](https://zwiftpower\.com/profile\.php?z=567443)|—|—|[11th](https://zwiftpower\.com/events\.php?zid=155996)|—|—|—|—|—|—
B|15|977|[AM FANNING](https://zwiftpower\.com/profile\.php?z=1134093)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|[12th](https://zwiftpower\.com/events\.php?zid=155996)|—|—|—|—|—|—
C|1|6975|[Pamela Abbott \- CISCYCLING](https://zwiftpower\.com/profile\.php?z=146648)|[CIS/Ladies](https://zwiftpower\.com/team\.php?id=3773)|—|[🥉](https://zwiftpower\.com/events\.php?zid=155996)|[🥉](https://zwiftpower\.com/events\.php?zid=159645)|[4th](https://zwiftpower\.com/events\.php?zid=163029)|[🥇](https://zwiftpower\.com/events\.php?zid=167035)|[🥉](https://zwiftpower\.com/events\.php?zid=170591)|[🥇](https://zwiftpower\.com/events\.php?zid=173966)|[🥇](https://zwiftpower\.com/events\.php?zid=176369)
C|2|4987|[Sherie Masse](https://zwiftpower\.com/profile\.php?z=177731)|[CIS/Ladies](https://zwiftpower\.com/team\.php?id=3773)|[🥈](https://zwiftpower\.com/events\.php?zid=154311)|—|[🥇](https://zwiftpower\.com/events\.php?zid=159645)|[🥈](https://zwiftpower\.com/events\.php?zid=163029)|—|[🥈](https://zwiftpower\.com/events\.php?zid=170591)|—|[🥈](https://zwiftpower\.com/events\.php?zid=176369)
C|3|2997|[Kelsey Tranel](https://zwiftpower\.com/profile\.php?z=214587)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|[🥇](https://zwiftpower\.com/events\.php?zid=155996)|—|[🥇](https://zwiftpower\.com/events\.php?zid=163029)|—|[🥇](https://zwiftpower\.com/events\.php?zid=170591)|—|—
C|4|1996|[Janie Lapointe Vélozone JZQC](https://zwiftpower\.com/profile\.php?z=462469)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥇](https://zwiftpower\.com/events\.php?zid=154311)|—|[🥈](https://zwiftpower\.com/events\.php?zid=159645)|—|—|—|—|—
C|5|1982|[Nathalie Van Acker JZQC 6326](https://zwiftpower\.com/profile\.php?z=329231)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[5th](https://zwiftpower\.com/events\.php?zid=155996)|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=170591)|—|—
C|5|1982|[Martin Ross](https://zwiftpower\.com/profile\.php?z=388088)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[6th](https://zwiftpower\.com/events\.php?zid=155996)|[4th](https://zwiftpower\.com/events\.php?zid=159645)|—|—|—|—|—
C|6|1978|[Manon Lafrance JZQc](https://zwiftpower\.com/profile\.php?z=224868)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=163029)|—|[4th](https://zwiftpower\.com/events\.php?zid=170591)|—|—
C|7|997|[Kristi Mccracken](https://zwiftpower\.com/profile\.php?z=195887)|[LPC](https://zwiftpower\.com/team\.php?id=2089)|—|[🥈](https://zwiftpower\.com/events\.php?zid=155996)|—|—|—|—|—|—
C|8|995|[Breanna Cahoon](https://zwiftpower\.com/profile\.php?z=726270)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=163029)|—|—|—|—
C|8|995|[YC\-Meao喵~ Tseng\[FEAR<\]](https://zwiftpower\.com/profile\.php?z=811326)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[🥉](https://zwiftpower\.com/events\.php?zid=154311)|—|—|—|—|—|—|—
C|9|993|[Sheila Leard TeamEN KISS \(C\)](https://zwiftpower\.com/profile\.php?z=276466)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[4th](https://zwiftpower\.com/events\.php?zid=154311)|—|—|—|—|—|—|—
C|9|993|[Kath Ferland\[Laferté Bicyles\]](https://zwiftpower\.com/profile\.php?z=487646)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[4th](https://zwiftpower\.com/events\.php?zid=155996)|—|—|—|—|—|—
C|10|991|[Kati Petry](https://zwiftpower\.com/profile\.php?z=698274)|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=163029)|—|—|—|—
C|11|989|[Carole Sinou](https://zwiftpower\.com/profile\.php?z=97277)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=163029)|—|—|—|—
C|11|989|[Isabelle\. Brault 9211](https://zwiftpower\.com/profile\.php?z=322684)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=170591)|—|—
C|12|987|[Genevieve Dumais](https://zwiftpower\.com/profile\.php?z=955847)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|[7th](https://zwiftpower\.com/events\.php?zid=163029)|—|—|—|—
D|1|4967|[Isabelle Picard JZQC7179](https://zwiftpower\.com/profile\.php?z=869091)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[6th](https://zwiftpower\.com/events\.php?zid=155996)|[🥉](https://zwiftpower\.com/events\.php?zid=159645)|[5th](https://zwiftpower\.com/events\.php?zid=163029)|[🥈](https://zwiftpower\.com/events\.php?zid=167035)|[🥉](https://zwiftpower\.com/events\.php?zid=170591)|—|—
D|2|3992|[J Langlois](https://zwiftpower\.com/profile\.php?z=430523)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=159645)|[🥇](https://zwiftpower\.com/events\.php?zid=163029)|[🥇](https://zwiftpower\.com/events\.php?zid=167035)|[🥈](https://zwiftpower\.com/events\.php?zid=170591)|—|—
D|3|3990|[Isabelle\. Brault 9211](https://zwiftpower\.com/profile\.php?z=322684)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[🥇](https://zwiftpower\.com/events\.php?zid=154311)|[🥇](https://zwiftpower\.com/events\.php?zid=155996)|[🥇](https://zwiftpower\.com/events\.php?zid=159645)|[4th](https://zwiftpower\.com/events\.php?zid=163029)|—|—|—|—
D|4|3986|[L Conrad 6533](https://zwiftpower\.com/profile\.php?z=283014)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[🥈](https://zwiftpower\.com/events\.php?zid=154311)|[4th](https://zwiftpower\.com/events\.php?zid=155996)|—|[🥈](https://zwiftpower\.com/events\.php?zid=163029)|—|[🥇](https://zwiftpower\.com/events\.php?zid=170591)|—|—
D|5|1982|[Lucie Cauchon](https://zwiftpower\.com/profile\.php?z=472347)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[7th](https://zwiftpower\.com/events\.php?zid=155996)|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=167035)|—|—|—
D|6|997|[Andrea Thackeray](https://zwiftpower\.com/profile\.php?z=322738)|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=155996)|—|—|—|—|—|—
D|7|995|[Manon Lafrance JZQc](https://zwiftpower\.com/profile\.php?z=224868)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[🥉](https://zwiftpower\.com/events\.php?zid=155996)|—|—|—|—|—|—
D|7|995|[Wendy Chow](https://zwiftpower\.com/profile\.php?z=379916)|[Team AHDR](https://zwiftpower\.com/team\.php?id=629)|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=163029)|—|—|—|—
D|8|991|[Julie Arcand \[JZQC\] 4916](https://zwiftpower\.com/profile\.php?z=272362)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|[5th](https://zwiftpower\.com/events\.php?zid=155996)|—|—|—|—|—|—



## KISS Community League — Europe

Category|Rank|Total Points|Name|Team|1|2|3|4|5|6|7|8
---|---|---|---|---|---|---|---|---|---|---|---|---
A|1|7918|[Alex West 🔰🦏](https://zwiftpower\.com/profile\.php?z=401735)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|[11th](https://zwiftpower\.com/events\.php?zid=152778)|[🥇](https://zwiftpower\.com/events\.php?zid=155628)|[🥈](https://zwiftpower\.com/events\.php?zid=159334)|[🥈](https://zwiftpower\.com/events\.php?zid=162691)|[4th](https://zwiftpower\.com/events\.php?zid=166630)|[19th](https://zwiftpower\.com/events\.php?zid=171935)|[🥈](https://zwiftpower\.com/events\.php?zid=173451)|[4th](https://zwiftpower\.com/events\.php?zid=176148)
A|2|7570|[Lionel Vujasin](https://zwiftpower\.com/profile\.php?z=311597)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|[6th](https://zwiftpower\.com/events\.php?zid=152778)|[142nd](https://zwiftpower\.com/events\.php?zid=155628)|[34th](https://zwiftpower\.com/events\.php?zid=159334)|[8th](https://zwiftpower\.com/events\.php?zid=162691)|[7th](https://zwiftpower\.com/events\.php?zid=166630)|[5th](https://zwiftpower\.com/events\.php?zid=171935)|[6th](https://zwiftpower\.com/events\.php?zid=173451)|[11th](https://zwiftpower\.com/events\.php?zid=176148)
A|3|7550|[Marc Waldemar \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=448426)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[57th](https://zwiftpower\.com/events\.php?zid=152778)|[20th](https://zwiftpower\.com/events\.php?zid=155628)|[45th](https://zwiftpower\.com/events\.php?zid=159334)|[19th](https://zwiftpower\.com/events\.php?zid=162691)|[29th](https://zwiftpower\.com/events\.php?zid=166630)|[21st](https://zwiftpower\.com/events\.php?zid=171935)|[23rd](https://zwiftpower\.com/events\.php?zid=173451)|[15th](https://zwiftpower\.com/events\.php?zid=176148)
A|4|7522|[Stefan Kirchmair \(KC\)](https://zwiftpower\.com/profile\.php?z=595291)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[13th](https://zwiftpower\.com/events\.php?zid=152778)|[144th](https://zwiftpower\.com/events\.php?zid=155628)|[14th](https://zwiftpower\.com/events\.php?zid=159334)|[4th](https://zwiftpower\.com/events\.php?zid=162691)|[9th](https://zwiftpower\.com/events\.php?zid=166630)|[4th](https://zwiftpower\.com/events\.php?zid=171935)|[🥉](https://zwiftpower\.com/events\.php?zid=173451)|[52nd](https://zwiftpower\.com/events\.php?zid=176148)
A|5|7390|[Mike Swart](https://zwiftpower\.com/profile\.php?z=503)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[59th](https://zwiftpower\.com/events\.php?zid=152778)|[25th](https://zwiftpower\.com/events\.php?zid=155628)|[38th](https://zwiftpower\.com/events\.php?zid=159334)|[28th](https://zwiftpower\.com/events\.php?zid=162691)|[57th](https://zwiftpower\.com/events\.php?zid=166630)|[39th](https://zwiftpower\.com/events\.php?zid=171935)|[35th](https://zwiftpower\.com/events\.php?zid=173451)|[28th](https://zwiftpower\.com/events\.php?zid=176148)
A|6|7244|[Klaus Näher \(KC\)](https://zwiftpower\.com/profile\.php?z=518275)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[82nd](https://zwiftpower\.com/events\.php?zid=152778)|[23rd](https://zwiftpower\.com/events\.php?zid=155628)|[58th](https://zwiftpower\.com/events\.php?zid=159334)|[39th](https://zwiftpower\.com/events\.php?zid=162691)|[74th](https://zwiftpower\.com/events\.php?zid=166630)|[36th](https://zwiftpower\.com/events\.php?zid=171935)|[48th](https://zwiftpower\.com/events\.php?zid=173451)|[22nd](https://zwiftpower\.com/events\.php?zid=176148)
A|7|6958|[Dafydd Williams](https://zwiftpower\.com/profile\.php?z=123836)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[86th](https://zwiftpower\.com/events\.php?zid=152778)|[35th](https://zwiftpower\.com/events\.php?zid=155628)|[59th](https://zwiftpower\.com/events\.php?zid=159334)|[85th](https://zwiftpower\.com/events\.php?zid=162691)|[62nd](https://zwiftpower\.com/events\.php?zid=166630)|[90th](https://zwiftpower\.com/events\.php?zid=171935)|[50th](https://zwiftpower\.com/events\.php?zid=173451)|[58th](https://zwiftpower\.com/events\.php?zid=176148)
A|8|6894|[Marcin Krzywonos](https://zwiftpower\.com/profile\.php?z=308424)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[78th](https://zwiftpower\.com/events\.php?zid=152778)|[52nd](https://zwiftpower\.com/events\.php?zid=155628)|[46th](https://zwiftpower\.com/events\.php?zid=159334)|[82nd](https://zwiftpower\.com/events\.php?zid=162691)|[119th](https://zwiftpower\.com/events\.php?zid=166630)|[67th](https://zwiftpower\.com/events\.php?zid=171935)|[60th](https://zwiftpower\.com/events\.php?zid=173451)|[53rd](https://zwiftpower\.com/events\.php?zid=176148)
A|9|6735|[Frank Sorbara 🔰](https://zwiftpower\.com/profile\.php?z=110603)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|[16th](https://zwiftpower\.com/events\.php?zid=152778)|—|[16th](https://zwiftpower\.com/events\.php?zid=159334)|[35th](https://zwiftpower\.com/events\.php?zid=162691)|[15th](https://zwiftpower\.com/events\.php?zid=166630)|[14th](https://zwiftpower\.com/events\.php?zid=171935)|[11th](https://zwiftpower\.com/events\.php?zid=173451)|[29th](https://zwiftpower\.com/events\.php?zid=176148)
A|10|6707|[Fabian Dé 🦍](https://zwiftpower\.com/profile\.php?z=4635)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|[19th](https://zwiftpower\.com/events\.php?zid=152778)|[6th](https://zwiftpower\.com/events\.php?zid=155628)|[35th](https://zwiftpower\.com/events\.php?zid=159334)|—|[32nd](https://zwiftpower\.com/events\.php?zid=166630)|[12th](https://zwiftpower\.com/events\.php?zid=171935)|[20th](https://zwiftpower\.com/events\.php?zid=173451)|[26th](https://zwiftpower\.com/events\.php?zid=176148)
A|11|6550|[Michael Durr \[BRT\]](https://zwiftpower\.com/profile\.php?z=74813)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[99th](https://zwiftpower\.com/events\.php?zid=152778)|[110th](https://zwiftpower\.com/events\.php?zid=155628)|[105th](https://zwiftpower\.com/events\.php?zid=159334)|[128th](https://zwiftpower\.com/events\.php?zid=162691)|[91st](https://zwiftpower\.com/events\.php?zid=166630)|[59th](https://zwiftpower\.com/events\.php?zid=171935)|[44th](https://zwiftpower\.com/events\.php?zid=173451)|[93rd](https://zwiftpower\.com/events\.php?zid=176148)
A|11|6550|[Vytautas Jokubonis](https://zwiftpower\.com/profile\.php?z=199043)|[LTZ Racing Team](https://zwiftpower\.com/team\.php?id=3734)|[118th](https://zwiftpower\.com/events\.php?zid=152778)|[67th](https://zwiftpower\.com/events\.php?zid=155628)|[81st](https://zwiftpower\.com/events\.php?zid=159334)|[104th](https://zwiftpower\.com/events\.php?zid=162691)|[133rd](https://zwiftpower\.com/events\.php?zid=166630)|[78th](https://zwiftpower\.com/events\.php?zid=171935)|[65th](https://zwiftpower\.com/events\.php?zid=173451)|[83rd](https://zwiftpower\.com/events\.php?zid=176148)
A|12|6521|[James Phillips](https://zwiftpower\.com/profile\.php?z=280522)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|[20th](https://zwiftpower\.com/events\.php?zid=152778)|[138th](https://zwiftpower\.com/events\.php?zid=155628)|[20th](https://zwiftpower\.com/events\.php?zid=159334)|[10th](https://zwiftpower\.com/events\.php?zid=162691)|[25th](https://zwiftpower\.com/events\.php?zid=166630)|[28th](https://zwiftpower\.com/events\.php?zid=171935)|—|[🥈](https://zwiftpower\.com/events\.php?zid=176148)
A|13|6461|[Ingo Reichart ✪](https://zwiftpower\.com/profile\.php?z=94244)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[23rd](https://zwiftpower\.com/events\.php?zid=152778)|[136th](https://zwiftpower\.com/events\.php?zid=155628)|[13th](https://zwiftpower\.com/events\.php?zid=159334)|[5th](https://zwiftpower\.com/events\.php?zid=162691)|[83rd](https://zwiftpower\.com/events\.php?zid=166630)|[🥉](https://zwiftpower\.com/events\.php?zid=171935)|[10th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|14|6333|[Morten Vaeng](https://zwiftpower\.com/profile\.php?z=259131)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[62nd](https://zwiftpower\.com/events\.php?zid=152778)|[143rd](https://zwiftpower\.com/events\.php?zid=155628)|[30th](https://zwiftpower\.com/events\.php?zid=159334)|—|[23rd](https://zwiftpower\.com/events\.php?zid=166630)|[20th](https://zwiftpower\.com/events\.php?zid=171935)|[19th](https://zwiftpower\.com/events\.php?zid=173451)|[40th](https://zwiftpower\.com/events\.php?zid=176148)
A|15|6320|[Mitchell Bell](https://zwiftpower\.com/profile\.php?z=445221)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[117th](https://zwiftpower\.com/events\.php?zid=152778)|[83rd](https://zwiftpower\.com/events\.php?zid=155628)|[124th](https://zwiftpower\.com/events\.php?zid=159334)|[136th](https://zwiftpower\.com/events\.php?zid=162691)|[112th](https://zwiftpower\.com/events\.php?zid=166630)|[93rd](https://zwiftpower\.com/events\.php?zid=171935)|[85th](https://zwiftpower\.com/events\.php?zid=173451)|[94th](https://zwiftpower\.com/events\.php?zid=176148)
A|16|6314|[Kjetil Berntzen🔰 🐥](https://zwiftpower\.com/profile\.php?z=38911)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[137th](https://zwiftpower\.com/events\.php?zid=152778)|[72nd](https://zwiftpower\.com/events\.php?zid=155628)|[107th](https://zwiftpower\.com/events\.php?zid=159334)|[92nd](https://zwiftpower\.com/events\.php?zid=162691)|[147th](https://zwiftpower\.com/events\.php?zid=166630)|[91st](https://zwiftpower\.com/events\.php?zid=171935)|[89th](https://zwiftpower\.com/events\.php?zid=173451)|[112th](https://zwiftpower\.com/events\.php?zid=176148)
A|17|6297|[Dean Cunningham](https://zwiftpower\.com/profile\.php?z=78803)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[100th](https://zwiftpower\.com/events\.php?zid=152778)|[37th](https://zwiftpower\.com/events\.php?zid=155628)|[73rd](https://zwiftpower\.com/events\.php?zid=159334)|—|[36th](https://zwiftpower\.com/events\.php?zid=166630)|[23rd](https://zwiftpower\.com/events\.php?zid=171935)|[27th](https://zwiftpower\.com/events\.php?zid=173451)|[59th](https://zwiftpower\.com/events\.php?zid=176148)
A|18|6249|[Vinz van Teeseling 🦈](https://zwiftpower\.com/profile\.php?z=807046)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|[64th](https://zwiftpower\.com/events\.php?zid=152778)|[13th](https://zwiftpower\.com/events\.php?zid=155628)|[100th](https://zwiftpower\.com/events\.php?zid=159334)|—|[70th](https://zwiftpower\.com/events\.php?zid=166630)|[22nd](https://zwiftpower\.com/events\.php?zid=171935)|[75th](https://zwiftpower\.com/events\.php?zid=173451)|[35th](https://zwiftpower\.com/events\.php?zid=176148)
A|19|6024|[Matthias Blümel \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=855399)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[151st](https://zwiftpower\.com/events\.php?zid=152778)|[126th](https://zwiftpower\.com/events\.php?zid=155628)|[127th](https://zwiftpower\.com/events\.php?zid=159334)|[123rd](https://zwiftpower\.com/events\.php?zid=162691)|[160th](https://zwiftpower\.com/events\.php?zid=166630)|[105th](https://zwiftpower\.com/events\.php?zid=171935)|[100th](https://zwiftpower\.com/events\.php?zid=173451)|[100th](https://zwiftpower\.com/events\.php?zid=176148)
A|20|5980|[Samuel Brännlund INC🔰](https://zwiftpower\.com/profile\.php?z=660836)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=159334)|[🥉](https://zwiftpower\.com/events\.php?zid=162691)|[🥇](https://zwiftpower\.com/events\.php?zid=166630)|[🥈](https://zwiftpower\.com/events\.php?zid=171935)|[🥇](https://zwiftpower\.com/events\.php?zid=173451)|[5th](https://zwiftpower\.com/events\.php?zid=176148)
A|21|5966|[Leandro Messineo 🐸](https://zwiftpower\.com/profile\.php?z=688785)|—|[🥇](https://zwiftpower\.com/events\.php?zid=152778)|—|[4th](https://zwiftpower\.com/events\.php?zid=159334)|[🥇](https://zwiftpower\.com/events\.php?zid=162691)|—|[🥇](https://zwiftpower\.com/events\.php?zid=171935)|[4th](https://zwiftpower\.com/events\.php?zid=173451)|[9th](https://zwiftpower\.com/events\.php?zid=176148)
A|22|5909|[Philippe Van Laethem](https://zwiftpower\.com/profile\.php?z=688960)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|[113th](https://zwiftpower\.com/events\.php?zid=152778)|[71st](https://zwiftpower\.com/events\.php?zid=155628)|[72nd](https://zwiftpower\.com/events\.php?zid=159334)|—|[68th](https://zwiftpower\.com/events\.php?zid=166630)|[74th](https://zwiftpower\.com/events\.php?zid=171935)|[110th](https://zwiftpower\.com/events\.php?zid=173451)|[41st](https://zwiftpower\.com/events\.php?zid=176148)
A|23|5899|[Wesley Ooms🔰Tacx Neo](https://zwiftpower\.com/profile\.php?z=561362)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|[69th](https://zwiftpower\.com/events\.php?zid=152778)|[84th](https://zwiftpower\.com/events\.php?zid=155628)|—|[135th](https://zwiftpower\.com/events\.php?zid=162691)|[76th](https://zwiftpower\.com/events\.php?zid=166630)|[72nd](https://zwiftpower\.com/events\.php?zid=171935)|[55th](https://zwiftpower\.com/events\.php?zid=173451)|[63rd](https://zwiftpower\.com/events\.php?zid=176148)
A|24|5835|[Hal Blackburn](https://zwiftpower\.com/profile\.php?z=904023)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[94th](https://zwiftpower\.com/events\.php?zid=152778)|[54th](https://zwiftpower\.com/events\.php?zid=155628)|[112th](https://zwiftpower\.com/events\.php?zid=159334)|[118th](https://zwiftpower\.com/events\.php?zid=162691)|—|[77th](https://zwiftpower\.com/events\.php?zid=171935)|[52nd](https://zwiftpower\.com/events\.php?zid=173451)|[79th](https://zwiftpower\.com/events\.php?zid=176148)
A|25|5793|[Balazs Galambos \(IRL\)](https://zwiftpower\.com/profile\.php?z=146672)|[IRL](https://zwiftpower\.com/team\.php?id=3369)|[90th](https://zwiftpower\.com/events\.php?zid=152778)|[109th](https://zwiftpower\.com/events\.php?zid=155628)|[119th](https://zwiftpower\.com/events\.php?zid=159334)|[73rd](https://zwiftpower\.com/events\.php?zid=162691)|[84th](https://zwiftpower\.com/events\.php?zid=166630)|[65th](https://zwiftpower\.com/events\.php?zid=171935)|[67th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|26|5782|[Stevie Young 🔰🐍](https://zwiftpower\.com/profile\.php?z=482899)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|[9th](https://zwiftpower\.com/events\.php?zid=152778)|—|[8th](https://zwiftpower\.com/events\.php?zid=159334)|[11th](https://zwiftpower\.com/events\.php?zid=162691)|[64th](https://zwiftpower\.com/events\.php?zid=166630)|[17th](https://zwiftpower\.com/events\.php?zid=171935)|—|[🥉](https://zwiftpower\.com/events\.php?zid=176148)
A|27|5682|[Birk Hansen 🔰](https://zwiftpower\.com/profile\.php?z=550613)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|—|[🥈](https://zwiftpower\.com/events\.php?zid=155628)|[24th](https://zwiftpower\.com/events\.php?zid=159334)|[7th](https://zwiftpower\.com/events\.php?zid=162691)|[60th](https://zwiftpower\.com/events\.php?zid=166630)|[33rd](https://zwiftpower\.com/events\.php?zid=171935)|[36th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|28|5596|[Stian Lersveen 🔰](https://zwiftpower\.com/profile\.php?z=381136)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[39th](https://zwiftpower\.com/events\.php?zid=152778)|[17th](https://zwiftpower\.com/events\.php?zid=155628)|—|[53rd](https://zwiftpower\.com/events\.php?zid=162691)|—|[34th](https://zwiftpower\.com/events\.php?zid=171935)|[25th](https://zwiftpower\.com/events\.php?zid=173451)|[37th](https://zwiftpower\.com/events\.php?zid=176148)
A|29|5588|"[Paul de Haan \| Zwift Dutch Racing](https://zwiftpower\.com/profile\.php?z=525478)"|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|[144th](https://zwiftpower\.com/events\.php?zid=152778)|[5th](https://zwiftpower\.com/events\.php?zid=155628)|[23rd](https://zwiftpower\.com/events\.php?zid=159334)|[17th](https://zwiftpower\.com/events\.php?zid=162691)|—|[6th](https://zwiftpower\.com/events\.php?zid=171935)|[14th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|30|5527|[M Ssmith \(V\)](https://zwiftpower\.com/profile\.php?z=321324)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[121st](https://zwiftpower\.com/events\.php?zid=152778)|[97th](https://zwiftpower\.com/events\.php?zid=155628)|[126th](https://zwiftpower\.com/events\.php?zid=159334)|[98th](https://zwiftpower\.com/events\.php?zid=162691)|[125th](https://zwiftpower\.com/events\.php?zid=166630)|—|[86th](https://zwiftpower\.com/events\.php?zid=173451)|[87th](https://zwiftpower\.com/events\.php?zid=176148)
A|31|5502|[Maxim Golev \[V\]](https://zwiftpower\.com/profile\.php?z=811341)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[22nd](https://zwiftpower\.com/events\.php?zid=152778)|—|[27th](https://zwiftpower\.com/events\.php?zid=159334)|[18th](https://zwiftpower\.com/events\.php?zid=162691)|[45th](https://zwiftpower\.com/events\.php?zid=166630)|[107th](https://zwiftpower\.com/events\.php?zid=171935)|—|[33rd](https://zwiftpower\.com/events\.php?zid=176148)
A|32|5479|[Nik Hall](https://zwiftpower\.com/profile\.php?z=297624)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[128th](https://zwiftpower\.com/events\.php?zid=152778)|[99th](https://zwiftpower\.com/events\.php?zid=155628)|—|[144th](https://zwiftpower\.com/events\.php?zid=162691)|[128th](https://zwiftpower\.com/events\.php?zid=166630)|[92nd](https://zwiftpower\.com/events\.php?zid=171935)|[76th](https://zwiftpower\.com/events\.php?zid=173451)|[97th](https://zwiftpower\.com/events\.php?zid=176148)
A|33|5460|[Joshua Modert](https://zwiftpower\.com/profile\.php?z=584631)|—|—|[34th](https://zwiftpower\.com/events\.php?zid=155628)|[95th](https://zwiftpower\.com/events\.php?zid=159334)|[50th](https://zwiftpower\.com/events\.php?zid=162691)|—|[27th](https://zwiftpower\.com/events\.php?zid=171935)|[29th](https://zwiftpower\.com/events\.php?zid=173451)|[38th](https://zwiftpower\.com/events\.php?zid=176148)
A|34|5427|[Anders Broberg](https://zwiftpower\.com/profile\.php?z=407765)|[Team 63\.8N](https://zwiftpower\.com/team\.php?id=2936)|[140th](https://zwiftpower\.com/events\.php?zid=152778)|[95th](https://zwiftpower\.com/events\.php?zid=155628)|[132nd](https://zwiftpower\.com/events\.php?zid=159334)|—|[148th](https://zwiftpower\.com/events\.php?zid=166630)|[100th](https://zwiftpower\.com/events\.php?zid=171935)|[84th](https://zwiftpower\.com/events\.php?zid=173451)|[91st](https://zwiftpower\.com/events\.php?zid=176148)
A|35|5424|[Matthew Collins U17](https://zwiftpower\.com/profile\.php?z=541352)|—|—|—|[37th](https://zwiftpower\.com/events\.php?zid=159334)|[51st](https://zwiftpower\.com/events\.php?zid=162691)|[77th](https://zwiftpower\.com/events\.php?zid=166630)|[37th](https://zwiftpower\.com/events\.php?zid=171935)|[28th](https://zwiftpower\.com/events\.php?zid=173451)|[61st](https://zwiftpower\.com/events\.php?zid=176148)
A|36|5406|[Tom Ove Kaland](https://zwiftpower\.com/profile\.php?z=416)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[58th](https://zwiftpower\.com/events\.php?zid=152778)|[28th](https://zwiftpower\.com/events\.php?zid=155628)|[64th](https://zwiftpower\.com/events\.php?zid=159334)|[71st](https://zwiftpower\.com/events\.php?zid=162691)|—|[45th](https://zwiftpower\.com/events\.php?zid=171935)|[34th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|37|5344|[D\. Sputnik 🚀](https://zwiftpower\.com/profile\.php?z=579101)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[72nd](https://zwiftpower\.com/events\.php?zid=152778)|[39th](https://zwiftpower\.com/events\.php?zid=155628)|—|[78th](https://zwiftpower\.com/events\.php?zid=162691)|[58th](https://zwiftpower\.com/events\.php?zid=166630)|[51st](https://zwiftpower\.com/events\.php?zid=171935)|[33rd](https://zwiftpower\.com/events\.php?zid=173451)|—
A|38|5264|[atsushi yamaguchi](https://zwiftpower\.com/profile\.php?z=870907)|[ZJRT](https://zwiftpower\.com/team\.php?id=2054)|[77th](https://zwiftpower\.com/events\.php?zid=152778)|[43rd](https://zwiftpower\.com/events\.php?zid=155628)|[118th](https://zwiftpower\.com/events\.php?zid=159334)|[52nd](https://zwiftpower\.com/events\.php?zid=162691)|[40th](https://zwiftpower\.com/events\.php?zid=166630)|—|[41st](https://zwiftpower\.com/events\.php?zid=173451)|—
A|39|5262|[Josh Harrison](https://zwiftpower\.com/profile\.php?z=700044)|—|[96th](https://zwiftpower\.com/events\.php?zid=152778)|—|[87th](https://zwiftpower\.com/events\.php?zid=159334)|[65th](https://zwiftpower\.com/events\.php?zid=162691)|[46th](https://zwiftpower\.com/events\.php?zid=166630)|[41st](https://zwiftpower\.com/events\.php?zid=171935)|[37th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|40|5252|[Adam Chamberlin](https://zwiftpower\.com/profile\.php?z=9087)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|[74th](https://zwiftpower\.com/events\.php?zid=152778)|[59th](https://zwiftpower\.com/events\.php?zid=155628)|[98th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=171935)|[42nd](https://zwiftpower\.com/events\.php?zid=173451)|[62nd](https://zwiftpower\.com/events\.php?zid=176148)
A|41|5244|[Freddy van der Ploeg 💣](https://zwiftpower\.com/profile\.php?z=95453)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|[101st](https://zwiftpower\.com/events\.php?zid=152778)|—|—|[88th](https://zwiftpower\.com/events\.php?zid=162691)|[66th](https://zwiftpower\.com/events\.php?zid=166630)|[53rd](https://zwiftpower\.com/events\.php?zid=171935)|[53rd](https://zwiftpower\.com/events\.php?zid=173451)|[20th](https://zwiftpower\.com/events\.php?zid=176148)
A|42|5222|[Justinas Leveika](https://zwiftpower\.com/profile\.php?z=423707)|[LTZ Racing Team](https://zwiftpower\.com/team\.php?id=3734)|—|[26th](https://zwiftpower\.com/events\.php?zid=155628)|[71st](https://zwiftpower\.com/events\.php?zid=159334)|[56th](https://zwiftpower\.com/events\.php?zid=162691)|[106th](https://zwiftpower\.com/events\.php?zid=166630)|[69th](https://zwiftpower\.com/events\.php?zid=171935)|[64th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|43|5166|[Keith Robertson <BRT> ⚡](https://zwiftpower\.com/profile\.php?z=894741)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|[83rd](https://zwiftpower\.com/events\.php?zid=159334)|[107th](https://zwiftpower\.com/events\.php?zid=162691)|[59th](https://zwiftpower\.com/events\.php?zid=166630)|[58th](https://zwiftpower\.com/events\.php?zid=171935)|[49th](https://zwiftpower\.com/events\.php?zid=173451)|[64th](https://zwiftpower\.com/events\.php?zid=176148)
A|44|4998|[Erling Strømsholm \[V\]](https://zwiftpower\.com/profile\.php?z=44022)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|—|[86th](https://zwiftpower\.com/events\.php?zid=155628)|—|[86th](https://zwiftpower\.com/events\.php?zid=162691)|[110th](https://zwiftpower\.com/events\.php?zid=166630)|[81st](https://zwiftpower\.com/events\.php?zid=171935)|[70th](https://zwiftpower\.com/events\.php?zid=173451)|[71st](https://zwiftpower\.com/events\.php?zid=176148)
A|45|4944|[Karl W ⚡️](https://zwiftpower\.com/profile\.php?z=77662)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[80th](https://zwiftpower\.com/events\.php?zid=152778)|[87th](https://zwiftpower\.com/events\.php?zid=155628)|[94th](https://zwiftpower\.com/events\.php?zid=159334)|[61st](https://zwiftpower\.com/events\.php?zid=162691)|[96th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|[113th](https://zwiftpower\.com/events\.php?zid=176148)
A|46|4865|[Maico Voets ✈](https://zwiftpower\.com/profile\.php?z=543349)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|—|—|—|[12th](https://zwiftpower\.com/events\.php?zid=162691)|[26th](https://zwiftpower\.com/events\.php?zid=166630)|[13th](https://zwiftpower\.com/events\.php?zid=171935)|[12th](https://zwiftpower\.com/events\.php?zid=173451)|[7th](https://zwiftpower\.com/events\.php?zid=176148)
A|47|4859|[Adam Zimmerman 🔰](https://zwiftpower\.com/profile\.php?z=7915)|[⚒](https://zwiftpower\.com/team\.php?id=3973)|[8th](https://zwiftpower\.com/events\.php?zid=152778)|—|[6th](https://zwiftpower\.com/events\.php?zid=159334)|—|[43rd](https://zwiftpower\.com/events\.php?zid=166630)|[15th](https://zwiftpower\.com/events\.php?zid=171935)|—|[🥇](https://zwiftpower\.com/events\.php?zid=176148)
A|47|4859|[Piotr Truszczyński \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=175948)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[14th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=166630)|[16th](https://zwiftpower\.com/events\.php?zid=171935)|[13th](https://zwiftpower\.com/events\.php?zid=173451)|[12th](https://zwiftpower\.com/events\.php?zid=176148)
A|48|4856|[Mark Gardiner](https://zwiftpower\.com/profile\.php?z=691846)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|[91st](https://zwiftpower\.com/events\.php?zid=155628)|[128th](https://zwiftpower\.com/events\.php?zid=159334)|[132nd](https://zwiftpower\.com/events\.php?zid=162691)|—|[62nd](https://zwiftpower\.com/events\.php?zid=171935)|[80th](https://zwiftpower\.com/events\.php?zid=173451)|[82nd](https://zwiftpower\.com/events\.php?zid=176148)
A|49|4825|[Søren Bay 🔰](https://zwiftpower\.com/profile\.php?z=255659)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[37th](https://zwiftpower\.com/events\.php?zid=152778)|[7th](https://zwiftpower\.com/events\.php?zid=155628)|[10th](https://zwiftpower\.com/events\.php?zid=159334)|[15th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=176148)
A|50|4782|[Bartosz Nowaczek](https://zwiftpower\.com/profile\.php?z=358286)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[115th](https://zwiftpower\.com/events\.php?zid=152778)|—|[117th](https://zwiftpower\.com/events\.php?zid=159334)|[113th](https://zwiftpower\.com/events\.php?zid=162691)|[108th](https://zwiftpower\.com/events\.php?zid=166630)|—|[81st](https://zwiftpower\.com/events\.php?zid=173451)|[78th](https://zwiftpower\.com/events\.php?zid=176148)
A|51|4723|[Oliver Dowd 🔰🐆](https://zwiftpower\.com/profile\.php?z=244714)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[32nd](https://zwiftpower\.com/events\.php?zid=152778)|[8th](https://zwiftpower\.com/events\.php?zid=155628)|[39th](https://zwiftpower\.com/events\.php?zid=159334)|[33rd](https://zwiftpower\.com/events\.php?zid=162691)|—|[29th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|52|4709|[Mon Ritch](https://zwiftpower\.com/profile\.php?z=135635)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|[34th](https://zwiftpower\.com/events\.php?zid=152778)|[4th](https://zwiftpower\.com/events\.php?zid=155628)|[40th](https://zwiftpower\.com/events\.php?zid=159334)|[16th](https://zwiftpower\.com/events\.php?zid=162691)|[54th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|53|4681|[Dan Fleeman \- DigDeepCoaching\.Com](https://zwiftpower\.com/profile\.php?z=49551)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[5th](https://zwiftpower\.com/events\.php?zid=152778)|[141st](https://zwiftpower\.com/events\.php?zid=155628)|[7th](https://zwiftpower\.com/events\.php?zid=159334)|—|[🥈](https://zwiftpower\.com/events\.php?zid=166630)|—|[7th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|54|4657|[Philipp Diegner 🔰](https://zwiftpower\.com/profile\.php?z=26829)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|[7th](https://zwiftpower\.com/events\.php?zid=152778)|[137th](https://zwiftpower\.com/events\.php?zid=155628)|[🥉](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[8th](https://zwiftpower\.com/events\.php?zid=171935)|—|[19th](https://zwiftpower\.com/events\.php?zid=176148)
A|55|4655|[Cameron Jeffers](https://zwiftpower\.com/profile\.php?z=9659)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|—|—|[33rd](https://zwiftpower\.com/events\.php?zid=159334)|[20th](https://zwiftpower\.com/events\.php?zid=162691)|[13th](https://zwiftpower\.com/events\.php?zid=166630)|[79th](https://zwiftpower\.com/events\.php?zid=171935)|—|[30th](https://zwiftpower\.com/events\.php?zid=176148)
A|56|4653|[Simon Merritt \(CRYO\-GEN\) \(GSV\)](https://zwiftpower\.com/profile\.php?z=38493)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[89th](https://zwiftpower\.com/events\.php?zid=152778)|—|[25th](https://zwiftpower\.com/events\.php?zid=159334)|[30th](https://zwiftpower\.com/events\.php?zid=162691)|[8th](https://zwiftpower\.com/events\.php?zid=166630)|—|[24th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|57|4595|[David Pacheco 🔰](https://zwiftpower\.com/profile\.php?z=57523)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|—|—|[70th](https://zwiftpower\.com/events\.php?zid=159334)|—|[53rd](https://zwiftpower\.com/events\.php?zid=166630)|[38th](https://zwiftpower\.com/events\.php?zid=171935)|[30th](https://zwiftpower\.com/events\.php?zid=173451)|[14th](https://zwiftpower\.com/events\.php?zid=176148)
A|58|4585|[Jarno Vader](https://zwiftpower\.com/profile\.php?z=61708)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|[27th](https://zwiftpower\.com/events\.php?zid=152778)|—|[75th](https://zwiftpower\.com/events\.php?zid=159334)|[27th](https://zwiftpower\.com/events\.php?zid=162691)|[33rd](https://zwiftpower\.com/events\.php?zid=166630)|[48th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|59|4569|[james gelsthorpe](https://zwiftpower\.com/profile\.php?z=965858)|[Velo One](https://zwiftpower\.com/team\.php?id=2078)|—|[40th](https://zwiftpower\.com/events\.php?zid=155628)|[77th](https://zwiftpower\.com/events\.php?zid=159334)|[57th](https://zwiftpower\.com/events\.php?zid=162691)|[17th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|[27th](https://zwiftpower\.com/events\.php?zid=176148)
A|60|4559|[Max Powersimon Ⓥ](https://zwiftpower\.com/profile\.php?z=19498)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|[67th](https://zwiftpower\.com/events\.php?zid=159334)|[41st](https://zwiftpower\.com/events\.php?zid=162691)|[48th](https://zwiftpower\.com/events\.php?zid=166630)|[24th](https://zwiftpower\.com/events\.php?zid=171935)|—|[43rd](https://zwiftpower\.com/events\.php?zid=176148)
A|61|4549|[Ryo Misaki](https://zwiftpower\.com/profile\.php?z=380080)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|[17th](https://zwiftpower\.com/events\.php?zid=152778)|[139th](https://zwiftpower\.com/events\.php?zid=155628)|[26th](https://zwiftpower\.com/events\.php?zid=159334)|[36th](https://zwiftpower\.com/events\.php?zid=162691)|[10th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|62|4461|[Kristian Nøhr 🔰](https://zwiftpower\.com/profile\.php?z=473748)|—|[52nd](https://zwiftpower\.com/events\.php?zid=152778)|[33rd](https://zwiftpower\.com/events\.php?zid=155628)|[50th](https://zwiftpower\.com/events\.php?zid=159334)|—|[87th](https://zwiftpower\.com/events\.php?zid=166630)|[50th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|63|4449|[Tom Gakes](https://zwiftpower\.com/profile\.php?z=387828)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|[30th](https://zwiftpower\.com/events\.php?zid=152778)|[80th](https://zwiftpower\.com/events\.php?zid=155628)|[53rd](https://zwiftpower\.com/events\.php?zid=159334)|[23rd](https://zwiftpower\.com/events\.php?zid=162691)|[92nd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|64|4435|[LUKASZ GRABARCZYK](https://zwiftpower\.com/profile\.php?z=506082)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[63rd](https://zwiftpower\.com/events\.php?zid=152778)|[63rd](https://zwiftpower\.com/events\.php?zid=155628)|[48th](https://zwiftpower\.com/events\.php?zid=159334)|[68th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|[43rd](https://zwiftpower\.com/events\.php?zid=173451)|—
A|65|4371|[Eric Borchert // Track Mob Berlin](https://zwiftpower\.com/profile\.php?z=408117)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|[71st](https://zwiftpower\.com/events\.php?zid=152778)|[38th](https://zwiftpower\.com/events\.php?zid=155628)|[76th](https://zwiftpower\.com/events\.php?zid=159334)|[75th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|[57th](https://zwiftpower\.com/events\.php?zid=176148)
A|66|4365|[Brage Bergeng \[V\]🐢](https://zwiftpower\.com/profile\.php?z=429749)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[79th](https://zwiftpower\.com/events\.php?zid=152778)|[146th](https://zwiftpower\.com/events\.php?zid=155628)|—|[40th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|[32nd](https://zwiftpower\.com/events\.php?zid=173451)|[23rd](https://zwiftpower\.com/events\.php?zid=176148)
A|67|4353|[Rusty Devlin🔰Charged Drinks](https://zwiftpower\.com/profile\.php?z=27060)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[65th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|[87th](https://zwiftpower\.com/events\.php?zid=162691)|[73rd](https://zwiftpower\.com/events\.php?zid=166630)|[52nd](https://zwiftpower\.com/events\.php?zid=171935)|—|[49th](https://zwiftpower\.com/events\.php?zid=176148)
A|68|4341|[Tomasz Ciepłucha \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=934433)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=159334)|—|[100th](https://zwiftpower\.com/events\.php?zid=166630)|[99th](https://zwiftpower\.com/events\.php?zid=171935)|[46th](https://zwiftpower\.com/events\.php?zid=173451)|[45th](https://zwiftpower\.com/events\.php?zid=176148)
A|69|4321|[Guus Bouman🦆](https://zwiftpower\.com/profile\.php?z=408391)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[40th](https://zwiftpower\.com/events\.php?zid=152778)|[93rd](https://zwiftpower\.com/events\.php?zid=155628)|—|[90th](https://zwiftpower\.com/events\.php?zid=162691)|[75th](https://zwiftpower\.com/events\.php?zid=166630)|[44th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|70|4301|[Ryan Gooderham](https://zwiftpower\.com/profile\.php?z=266973)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|—|[65th](https://zwiftpower\.com/events\.php?zid=155628)|[54th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[55th](https://zwiftpower\.com/events\.php?zid=171935)|[105th](https://zwiftpower\.com/events\.php?zid=173451)|[73rd](https://zwiftpower\.com/events\.php?zid=176148)
A|71|4177|[Jürgen Schäfer\(KC\) Elbspitze 19](https://zwiftpower\.com/profile\.php?z=466660)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[73rd](https://zwiftpower\.com/events\.php?zid=152778)|[68th](https://zwiftpower\.com/events\.php?zid=155628)|—|[141st](https://zwiftpower\.com/events\.php?zid=162691)|[81st](https://zwiftpower\.com/events\.php?zid=166630)|—|[51st](https://zwiftpower\.com/events\.php?zid=173451)|—
A|72|3967|[Jacek Tarach](https://zwiftpower\.com/profile\.php?z=62218)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|[110th](https://zwiftpower\.com/events\.php?zid=159334)|[111th](https://zwiftpower\.com/events\.php?zid=162691)|[127th](https://zwiftpower\.com/events\.php?zid=166630)|[84th](https://zwiftpower\.com/events\.php?zid=171935)|[87th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|73|3909|[Miha Kosmač](https://zwiftpower\.com/profile\.php?z=484492)|[Top\-Fit](https://zwiftpower\.com/team\.php?id=2019)|[107th](https://zwiftpower\.com/events\.php?zid=152778)|[112th](https://zwiftpower\.com/events\.php?zid=155628)|[135th](https://zwiftpower\.com/events\.php?zid=159334)|[99th](https://zwiftpower\.com/events\.php?zid=162691)|[95th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|74|3879|[Marcel Bischof \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=170911)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[134th](https://zwiftpower\.com/events\.php?zid=152778)|[98th](https://zwiftpower\.com/events\.php?zid=155628)|[104th](https://zwiftpower\.com/events\.php?zid=159334)|[106th](https://zwiftpower\.com/events\.php?zid=162691)|[121st](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|75|3875|[Shane Johnson CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=879319)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[128th](https://zwiftpower\.com/events\.php?zid=155628)|[133rd](https://zwiftpower\.com/events\.php?zid=159334)|[108th](https://zwiftpower\.com/events\.php?zid=162691)|[98th](https://zwiftpower\.com/events\.php?zid=166630)|—|[98th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|76|3872|[Wouter Sybrandy](https://zwiftpower\.com/profile\.php?z=2021)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|—|—|[17th](https://zwiftpower\.com/events\.php?zid=159334)|[25th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|[18th](https://zwiftpower\.com/events\.php?zid=173451)|[6th](https://zwiftpower\.com/events\.php?zid=176148)
A|77|3834|[Gavin Dempster🦕🔰](https://zwiftpower\.com/profile\.php?z=6499)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|[🥉](https://zwiftpower\.com/events\.php?zid=152778)|—|[15th](https://zwiftpower\.com/events\.php?zid=159334)|[58th](https://zwiftpower\.com/events\.php?zid=162691)|—|[9th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|78|3819|[Manuel Ruhland \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=1047299)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|—|[127th](https://zwiftpower\.com/events\.php?zid=155628)|[138th](https://zwiftpower\.com/events\.php?zid=159334)|—|[118th](https://zwiftpower\.com/events\.php?zid=166630)|[106th](https://zwiftpower\.com/events\.php?zid=171935)|[104th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|79|3798|[Daniel Pettinger](https://zwiftpower\.com/profile\.php?z=90946)|[Vision](https://zwiftpower\.com/team\.php?id=9)|—|—|—|—|[19th](https://zwiftpower\.com/events\.php?zid=166630)|[26th](https://zwiftpower\.com/events\.php?zid=171935)|[22nd](https://zwiftpower\.com/events\.php?zid=173451)|[36th](https://zwiftpower\.com/events\.php?zid=176148)
A|80|3768|[Brian Hodges](https://zwiftpower\.com/profile\.php?z=399270)|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=159334)|[45th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|[45th](https://zwiftpower\.com/events\.php?zid=173451)|[10th](https://zwiftpower\.com/events\.php?zid=176148)
A|81|3749|[Leon van Bon](https://zwiftpower\.com/profile\.php?z=96777)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|[147th](https://zwiftpower\.com/events\.php?zid=152778)|[100th](https://zwiftpower\.com/events\.php?zid=155628)|—|[117th](https://zwiftpower\.com/events\.php?zid=162691)|[154th](https://zwiftpower\.com/events\.php?zid=166630)|[110th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|82|3739|[Ingo Ruhmund \(BMPRO\)](https://zwiftpower\.com/profile\.php?z=452096)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|—|—|—|[145th](https://zwiftpower\.com/events\.php?zid=162691)|[157th](https://zwiftpower\.com/events\.php?zid=166630)|[108th](https://zwiftpower\.com/events\.php?zid=171935)|[108th](https://zwiftpower\.com/events\.php?zid=173451)|[115th](https://zwiftpower\.com/events\.php?zid=176148)
A|83|3676|[Reece Egner](https://zwiftpower\.com/profile\.php?z=243035)|[HUUB Cycling Pulse](https://zwiftpower\.com/team\.php?id=3911)|[41st](https://zwiftpower\.com/events\.php?zid=152778)|[44th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|[63rd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|[16th](https://zwiftpower\.com/events\.php?zid=176148)
A|84|3642|[gavin arber RPR](https://zwiftpower\.com/profile\.php?z=606)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[47th](https://zwiftpower\.com/events\.php?zid=152778)|[15th](https://zwiftpower\.com/events\.php?zid=155628)|—|[48th](https://zwiftpower\.com/events\.php?zid=162691)|[71st](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|85|3636|[Ed Laverack](https://zwiftpower\.com/profile\.php?z=703388)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[26th](https://zwiftpower\.com/events\.php?zid=152778)|—|[63rd](https://zwiftpower\.com/events\.php?zid=159334)|[84th](https://zwiftpower\.com/events\.php?zid=162691)|[11th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|86|3614|[Andreas Rois \(KC\)](https://zwiftpower\.com/profile\.php?z=991994)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[53rd](https://zwiftpower\.com/events\.php?zid=152778)|—|[74th](https://zwiftpower\.com/events\.php?zid=159334)|—|[22nd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|[46th](https://zwiftpower\.com/events\.php?zid=176148)
A|87|3600|[Ash Beech](https://zwiftpower\.com/profile\.php?z=644482)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[45th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|[34th](https://zwiftpower\.com/events\.php?zid=162691)|—|[32nd](https://zwiftpower\.com/events\.php?zid=171935)|[91st](https://zwiftpower\.com/events\.php?zid=173451)|—
A|88|3580|[Matt Parsons](https://zwiftpower\.com/profile\.php?z=845430)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[29th](https://zwiftpower\.com/events\.php?zid=155628)|—|[91st](https://zwiftpower\.com/events\.php?zid=162691)|[38th](https://zwiftpower\.com/events\.php?zid=166630)|[54th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|89|3512|[Evan Bennewies 🔰](https://zwiftpower\.com/profile\.php?z=360816)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[61st](https://zwiftpower\.com/events\.php?zid=152778)|[27th](https://zwiftpower\.com/events\.php?zid=155628)|[82nd](https://zwiftpower\.com/events\.php?zid=159334)|[76th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|90|3478|[Tim Sullivan 🔰](https://zwiftpower\.com/profile\.php?z=165253)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|—|[88th](https://zwiftpower\.com/events\.php?zid=155628)|—|[43rd](https://zwiftpower\.com/events\.php?zid=162691)|[72nd](https://zwiftpower\.com/events\.php?zid=166630)|[60th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|91|3446|[Espen Komissar Fossheim](https://zwiftpower\.com/profile\.php?z=54738)|[Vision](https://zwiftpower\.com/team\.php?id=9)|—|[62nd](https://zwiftpower\.com/events\.php?zid=155628)|[93rd](https://zwiftpower\.com/events\.php?zid=159334)|—|[56th](https://zwiftpower\.com/events\.php?zid=166630)|[68th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|92|3444|[Simon Harrison](https://zwiftpower\.com/profile\.php?z=524175)|—|—|—|—|—|[103rd](https://zwiftpower\.com/events\.php?zid=166630)|[56th](https://zwiftpower\.com/events\.php?zid=171935)|[47th](https://zwiftpower\.com/events\.php?zid=173451)|[74th](https://zwiftpower\.com/events\.php?zid=176148)
A|93|3432|[Peter Göransson](https://zwiftpower\.com/profile\.php?z=674292)|[Team 63\.8N](https://zwiftpower\.com/team\.php?id=2936)|[93rd](https://zwiftpower\.com/events\.php?zid=152778)|[66th](https://zwiftpower\.com/events\.php?zid=155628)|[66th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[61st](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|94|3426|[Klaudiusz Debski \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=58823)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[111th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|[105th](https://zwiftpower\.com/events\.php?zid=162691)|[30th](https://zwiftpower\.com/events\.php?zid=166630)|[43rd](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|95|3388|[Alex Sankovic](https://zwiftpower\.com/profile\.php?z=506515)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|[74th](https://zwiftpower\.com/events\.php?zid=162691)|[85th](https://zwiftpower\.com/events\.php?zid=166630)|[80th](https://zwiftpower\.com/events\.php?zid=171935)|—|[69th](https://zwiftpower\.com/events\.php?zid=176148)
A|96|3308|[Sam Mills 🍌](https://zwiftpower\.com/profile\.php?z=171558)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[135th](https://zwiftpower\.com/events\.php?zid=152778)|[69th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|[78th](https://zwiftpower\.com/events\.php?zid=173451)|[66th](https://zwiftpower\.com/events\.php?zid=176148)
A|97|3256|[Morten Jul Pedersen 🐌](https://zwiftpower\.com/profile\.php?z=162474)|[FUSION EVO](https://zwiftpower\.com/team\.php?id=3185)|—|[75th](https://zwiftpower\.com/events\.php?zid=155628)|[92nd](https://zwiftpower\.com/events\.php?zid=159334)|[96th](https://zwiftpower\.com/events\.php?zid=162691)|[111th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|98|3234|[Johannes Reining \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=629172)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|—|—|—|—|[126th](https://zwiftpower\.com/events\.php?zid=166630)|[86th](https://zwiftpower\.com/events\.php?zid=171935)|[88th](https://zwiftpower\.com/events\.php?zid=173451)|[85th](https://zwiftpower\.com/events\.php?zid=176148)
A|99|3194|[Thomas Jendrusch 🔰 \[R2\]](https://zwiftpower\.com/profile\.php?z=28466)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|—|—|[115th](https://zwiftpower\.com/events\.php?zid=159334)|[101st](https://zwiftpower\.com/events\.php?zid=162691)|[102nd](https://zwiftpower\.com/events\.php?zid=166630)|[87th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|100|3182|[Artūras K1212](https://zwiftpower\.com/profile\.php?z=900471)|[LTZ Racing Team](https://zwiftpower\.com/team\.php?id=3734)|[119th](https://zwiftpower\.com/events\.php?zid=152778)|[101st](https://zwiftpower\.com/events\.php?zid=155628)|[116th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[75th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|101|3150|[Richard Moorhouse](https://zwiftpower\.com/profile\.php?z=8903)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[130th](https://zwiftpower\.com/events\.php?zid=152778)|[105th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|[96th](https://zwiftpower\.com/events\.php?zid=171935)|[96th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|102|3108|[David Fidalgo](https://zwiftpower\.com/profile\.php?z=197561)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|[131st](https://zwiftpower\.com/events\.php?zid=162691)|—|[98th](https://zwiftpower\.com/events\.php?zid=171935)|[102nd](https://zwiftpower\.com/events\.php?zid=173451)|[117th](https://zwiftpower\.com/events\.php?zid=176148)
A|103|3102|[Pedro Errecart](https://zwiftpower\.com/profile\.php?z=745344)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[138th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|[69th](https://zwiftpower\.com/events\.php?zid=162691)|[151st](https://zwiftpower\.com/events\.php?zid=166630)|—|[93rd](https://zwiftpower\.com/events\.php?zid=173451)|—
A|104|3080|[Michał Dobrzyński\[Team Poland\]](https://zwiftpower\.com/profile\.php?z=1021580)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|[129th](https://zwiftpower\.com/events\.php?zid=162691)|[135th](https://zwiftpower\.com/events\.php?zid=166630)|[97th](https://zwiftpower\.com/events\.php?zid=171935)|[101st](https://zwiftpower\.com/events\.php?zid=173451)|—
A|105|3064|[Sven Gerhardt INC 👽](https://zwiftpower\.com/profile\.php?z=982806)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[146th](https://zwiftpower\.com/events\.php?zid=152778)|[129th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|[106th](https://zwiftpower\.com/events\.php?zid=173451)|[89th](https://zwiftpower\.com/events\.php?zid=176148)
A|106|2961|[Sam Lindsay](https://zwiftpower\.com/profile\.php?z=545920)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|[5th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[11th](https://zwiftpower\.com/events\.php?zid=171935)|[5th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|107|2946|[Luke GJ Potter](https://zwiftpower\.com/profile\.php?z=55379)|[IRL](https://zwiftpower\.com/team\.php?id=3369)|—|—|—|[147th](https://zwiftpower\.com/events\.php?zid=162691)|[159th](https://zwiftpower\.com/events\.php?zid=166630)|[112th](https://zwiftpower\.com/events\.php?zid=171935)|[111th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|108|2942|[norman henderson Cryo Gen](https://zwiftpower\.com/profile\.php?z=99549)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[135th](https://zwiftpower\.com/events\.php?zid=155628)|[140th](https://zwiftpower\.com/events\.php?zid=159334)|[124th](https://zwiftpower\.com/events\.php?zid=162691)|[132nd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|109|2935|[Gavin Hughes](https://zwiftpower\.com/profile\.php?z=54447)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=166630)|[10th](https://zwiftpower\.com/events\.php?zid=171935)|[8th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|110|2903|[Alan Macleod Ⓥ](https://zwiftpower\.com/profile\.php?z=429012)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|—|—|[12th](https://zwiftpower\.com/events\.php?zid=159334)|[24th](https://zwiftpower\.com/events\.php?zid=162691)|[14th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|111|2881|[Julian White](https://zwiftpower\.com/profile\.php?z=563359)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[22nd](https://zwiftpower\.com/events\.php?zid=155628)|[22nd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|[17th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|112|2863|[Tom Townsend](https://zwiftpower\.com/profile\.php?z=66453)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[28th](https://zwiftpower\.com/events\.php?zid=152778)|[11th](https://zwiftpower\.com/events\.php?zid=155628)|[31st](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|113|2861|[Mattias Lundqvist🔰 Aktivitus Testcenter](https://zwiftpower\.com/profile\.php?z=455884)|[Sz](https://zwiftpower\.com/team\.php?id=32)|—|[30th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=173451)|[25th](https://zwiftpower\.com/events\.php?zid=176148)
A|114|2813|[Łukasz Góralik](https://zwiftpower\.com/profile\.php?z=409658)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[31st](https://zwiftpower\.com/events\.php?zid=152778)|[12th](https://zwiftpower\.com/events\.php?zid=155628)|[52nd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|115|2809|[Aleksej Calmann 🔰](https://zwiftpower\.com/profile\.php?z=297914)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[50th](https://zwiftpower\.com/events\.php?zid=152778)|[9th](https://zwiftpower\.com/events\.php?zid=155628)|—|[38th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|116|2801|[Johan Noraker Nossen 🔰](https://zwiftpower\.com/profile\.php?z=135469)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|—|—|[28th](https://zwiftpower\.com/events\.php?zid=159334)|[29th](https://zwiftpower\.com/events\.php?zid=162691)|[44th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|117|2777|[Richard Unwin](https://zwiftpower\.com/profile\.php?z=49421)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|[47th](https://zwiftpower\.com/events\.php?zid=162691)|[27th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|[39th](https://zwiftpower\.com/events\.php?zid=176148)
A|118|2765|[jesse klassen5240](https://zwiftpower\.com/profile\.php?z=1001790)|—|—|—|[51st](https://zwiftpower\.com/events\.php?zid=159334)|[42nd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|[26th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|119|2763|[Jesper Attermann 🔰](https://zwiftpower\.com/profile\.php?z=67632)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[44th](https://zwiftpower\.com/events\.php?zid=152778)|[14th](https://zwiftpower\.com/events\.php?zid=155628)|[62nd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|119|2763|[Patrick Woelke \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=883585)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=162691)|[99th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|[8th](https://zwiftpower\.com/events\.php?zid=176148)
A|120|2749|[Sandy Taylor](https://zwiftpower\.com/profile\.php?z=212882)|[GGCC](https://zwiftpower\.com/team\.php?id=1611)|[49th](https://zwiftpower\.com/events\.php?zid=152778)|[47th](https://zwiftpower\.com/events\.php?zid=155628)|—|[31st](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|121|2725|[Sam Greenhalgh](https://zwiftpower\.com/profile\.php?z=143679)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|[43rd](https://zwiftpower\.com/events\.php?zid=159334)|[62nd](https://zwiftpower\.com/events\.php?zid=162691)|[34th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|122|2717|[Phil Whiteley](https://zwiftpower\.com/profile\.php?z=161329)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[56th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|[65th](https://zwiftpower\.com/events\.php?zid=176148)
A|123|2651|[Steve Lampier\- Saint Piran](https://zwiftpower\.com/profile\.php?z=332529)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|—|—|[44th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[88th](https://zwiftpower\.com/events\.php?zid=171935)|—|[44th](https://zwiftpower\.com/events\.php?zid=176148)
A|124|2643|[Darren Reddick](https://zwiftpower\.com/profile\.php?z=843142)|—|—|—|—|—|[51st](https://zwiftpower\.com/events\.php?zid=166630)|[70th](https://zwiftpower\.com/events\.php?zid=171935)|[59th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|125|2639|[Kristian Frøland INC](https://zwiftpower\.com/profile\.php?z=323707)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|[48th](https://zwiftpower\.com/events\.php?zid=155628)|[55th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|[79th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|126|2625|[Koen Van Geyt](https://zwiftpower\.com/profile\.php?z=115080)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|[51st](https://zwiftpower\.com/events\.php?zid=152778)|[24th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|[114th](https://zwiftpower\.com/events\.php?zid=176148)
A|127|2619|[Magnus Brinch Larsen 🔰](https://zwiftpower\.com/profile\.php?z=20421)|[FUSION EVO](https://zwiftpower\.com/team\.php?id=3185)|[68th](https://zwiftpower\.com/events\.php?zid=152778)|[89th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|[35th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|128|2597|[Guido De Vries](https://zwiftpower\.com/profile\.php?z=180278)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[95th](https://zwiftpower\.com/events\.php?zid=152778)|—|[84th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|[24th](https://zwiftpower\.com/events\.php?zid=176148)
A|128|2597|[Maciej Majewski](https://zwiftpower\.com/profile\.php?z=458986)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|[65th](https://zwiftpower\.com/events\.php?zid=159334)|[72nd](https://zwiftpower\.com/events\.php?zid=162691)|—|[66th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|129|2587|[Janek Czapliński](https://zwiftpower\.com/profile\.php?z=213569)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[81st](https://zwiftpower\.com/events\.php?zid=152778)|—|—|[66th](https://zwiftpower\.com/events\.php?zid=162691)|[61st](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|130|2573|[Jeroen Wellner](https://zwiftpower\.com/profile\.php?z=44582)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|—|—|—|[93rd](https://zwiftpower\.com/events\.php?zid=162691)|[65th](https://zwiftpower\.com/events\.php?zid=166630)|[57th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|130|2573|[Anders Jensen](https://zwiftpower\.com/profile\.php?z=863566)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[21st](https://zwiftpower\.com/events\.php?zid=152778)|[140th](https://zwiftpower\.com/events\.php?zid=155628)|—|[54th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|131|2567|[Mathieu Decock](https://zwiftpower\.com/profile\.php?z=509565)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|[46th](https://zwiftpower\.com/events\.php?zid=152778)|[124th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|[48th](https://zwiftpower\.com/events\.php?zid=176148)
A|132|2549|[Steven Pink \(FNet Velo\)](https://zwiftpower\.com/profile\.php?z=905145)|[FNet Velo](https://zwiftpower\.com/team\.php?id=3840)|—|—|[88th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[63rd](https://zwiftpower\.com/events\.php?zid=171935)|—|[76th](https://zwiftpower\.com/events\.php?zid=176148)
A|133|2543|[Birger Rye](https://zwiftpower\.com/profile\.php?z=446438)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|—|[59th](https://zwiftpower\.com/events\.php?zid=162691)|[105th](https://zwiftpower\.com/events\.php?zid=166630)|—|[66th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|134|2533|[Morten Holt](https://zwiftpower\.com/profile\.php?z=508874)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[84th](https://zwiftpower\.com/events\.php?zid=152778)|[70th](https://zwiftpower\.com/events\.php?zid=155628)|—|[81st](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|135|2527|[Jarryd Else](https://zwiftpower\.com/profile\.php?z=169015)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[67th](https://zwiftpower\.com/events\.php?zid=152778)|[50th](https://zwiftpower\.com/events\.php?zid=155628)|—|[121st](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|136|2513|[Jesper Olsen](https://zwiftpower\.com/profile\.php?z=41135)|—|—|—|—|—|[90th](https://zwiftpower\.com/events\.php?zid=166630)|—|[57th](https://zwiftpower\.com/events\.php?zid=173451)|[98th](https://zwiftpower\.com/events\.php?zid=176148)
A|137|2509|[Luke Dix](https://zwiftpower\.com/profile\.php?z=214249)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|—|[56th](https://zwiftpower\.com/events\.php?zid=155628)|[89th](https://zwiftpower\.com/events\.php?zid=159334)|[102nd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|138|2507|[Bas Seldenrijk 👌](https://zwiftpower\.com/profile\.php?z=274131)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[104th](https://zwiftpower\.com/events\.php?zid=152778)|[76th](https://zwiftpower\.com/events\.php?zid=155628)|[68th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|138|2507|[kris Spin King](https://zwiftpower\.com/profile\.php?z=292133)|—|—|—|—|—|[113th](https://zwiftpower\.com/events\.php?zid=166630)|—|[63rd](https://zwiftpower\.com/events\.php?zid=173451)|[72nd](https://zwiftpower\.com/events\.php?zid=176148)
A|139|2491|[Jason Hegarty](https://zwiftpower\.com/profile\.php?z=45648)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|[99th](https://zwiftpower\.com/events\.php?zid=159334)|—|[97th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|[60th](https://zwiftpower\.com/events\.php?zid=176148)
A|140|2447|[Quentijn Mulleneers 🍟](https://zwiftpower\.com/profile\.php?z=161017)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[125th](https://zwiftpower\.com/events\.php?zid=152778)|[51st](https://zwiftpower\.com/events\.php?zid=155628)|[102nd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|141|2445|[Bernard Opedal](https://zwiftpower\.com/profile\.php?z=586093)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[110th](https://zwiftpower\.com/events\.php?zid=152778)|[49th](https://zwiftpower\.com/events\.php?zid=155628)|—|[120th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|142|2415|[Paweł Kalita](https://zwiftpower\.com/profile\.php?z=64694)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|[103rd](https://zwiftpower\.com/events\.php?zid=162691)|[109th](https://zwiftpower\.com/events\.php?zid=166630)|[82nd](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|143|2377|[Ruben Larsen 🔰](https://zwiftpower\.com/profile\.php?z=32290)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|—|[113th](https://zwiftpower\.com/events\.php?zid=155628)|[86th](https://zwiftpower\.com/events\.php?zid=159334)|[114th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|144|2371|[gavin ash 🔰](https://zwiftpower\.com/profile\.php?z=157370)|[WECC](https://zwiftpower\.com/team\.php?id=1992)|[112th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|[110th](https://zwiftpower\.com/events\.php?zid=162691)|—|[94th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|145|2363|[Anton Alex](https://zwiftpower\.com/profile\.php?z=601528)|[Team 63\.8N](https://zwiftpower\.com/team\.php?id=2936)|[114th](https://zwiftpower\.com/events\.php?zid=152778)|[85th](https://zwiftpower\.com/events\.php?zid=155628)|[121st](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|146|2361|[Dustin Byers](https://zwiftpower\.com/profile\.php?z=399357)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|[130th](https://zwiftpower\.com/events\.php?zid=162691)|—|[85th](https://zwiftpower\.com/events\.php?zid=171935)|—|[106th](https://zwiftpower\.com/events\.php?zid=176148)
A|147|2357|[Russell Jolliffe](https://zwiftpower\.com/profile\.php?z=291602)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|[150th](https://zwiftpower\.com/events\.php?zid=166630)|—|[74th](https://zwiftpower\.com/events\.php?zid=173451)|[99th](https://zwiftpower\.com/events\.php?zid=176148)
A|148|2333|[J\. Williams 🐿](https://zwiftpower\.com/profile\.php?z=189343)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[136th](https://zwiftpower\.com/events\.php?zid=152778)|—|[111th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|[88th](https://zwiftpower\.com/events\.php?zid=176148)
A|149|2327|[Linas S\.](https://zwiftpower\.com/profile\.php?z=224318)|[LTZ Racing Team](https://zwiftpower\.com/team\.php?id=3734)|—|[132nd](https://zwiftpower\.com/events\.php?zid=155628)|[56th](https://zwiftpower\.com/events\.php?zid=159334)|[150th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|150|2297|[Jonas Salamonsen](https://zwiftpower\.com/profile\.php?z=441899)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[122nd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|[141st](https://zwiftpower\.com/events\.php?zid=166630)|—|—|[90th](https://zwiftpower\.com/events\.php?zid=176148)
A|151|2281|[Nuno Mendez](https://zwiftpower\.com/profile\.php?z=899397)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|[120th](https://zwiftpower\.com/events\.php?zid=155628)|—|[137th](https://zwiftpower\.com/events\.php?zid=162691)|[104th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|152|2259|[Luis GO Ⓥ เจ ☁️](https://zwiftpower\.com/profile\.php?z=270869)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|[134th](https://zwiftpower\.com/events\.php?zid=162691)|[131st](https://zwiftpower\.com/events\.php?zid=166630)|—|[107th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|153|2243|[Paul Hayward](https://zwiftpower\.com/profile\.php?z=6917)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|—|—|[122nd](https://zwiftpower\.com/events\.php?zid=159334)|[115th](https://zwiftpower\.com/events\.php?zid=162691)|[143rd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|154|2239|[Jan Jöckel \(KC\)](https://zwiftpower\.com/profile\.php?z=444506)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|[136th](https://zwiftpower\.com/events\.php?zid=159334)|[109th](https://zwiftpower\.com/events\.php?zid=162691)|[137th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|155|2219|[Grahame Mckenzie\[BRT\]](https://zwiftpower\.com/profile\.php?z=77990)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|[114th](https://zwiftpower\.com/events\.php?zid=155628)|[134th](https://zwiftpower\.com/events\.php?zid=159334)|—|[144th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|156|2203|[Maokoto Smz](https://zwiftpower\.com/profile\.php?z=303116)|—|—|[117th](https://zwiftpower\.com/events\.php?zid=155628)|[130th](https://zwiftpower\.com/events\.php?zid=159334)|—|[153rd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|157|2177|[makoto takahashi4774](https://zwiftpower\.com/profile\.php?z=979149)|—|—|[134th](https://zwiftpower\.com/events\.php?zid=155628)|[137th](https://zwiftpower\.com/events\.php?zid=159334)|[142nd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|158|1976|[Niki H 🔰](https://zwiftpower\.com/profile\.php?z=174313)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[4th](https://zwiftpower\.com/events\.php?zid=152778)|—|[9th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|159|1972|[Vidar Mehl 🐂](https://zwiftpower\.com/profile\.php?z=91105)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[12th](https://zwiftpower\.com/events\.php?zid=152778)|[🥉](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|160|1960|[Max Bike INC](https://zwiftpower\.com/profile\.php?z=505474)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|—|[12th](https://zwiftpower\.com/events\.php?zid=166630)|—|[9th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|161|1946|[Adam Pierzga](https://zwiftpower\.com/profile\.php?z=44142)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|[21st](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[7th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|162|1936|[Jo Karstensen 🔰](https://zwiftpower\.com/profile\.php?z=499890)|—|—|—|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=171935)|[15th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|163|1916|[Damir Ejup](https://zwiftpower\.com/profile\.php?z=299036)|—|—|—|[29th](https://zwiftpower\.com/events\.php?zid=159334)|[14th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|164|1896|[Jake Simons 🔰](https://zwiftpower\.com/profile\.php?z=305016)|[FINESSE](https://zwiftpower\.com/team\.php?id=2779)|—|—|[11th](https://zwiftpower\.com/events\.php?zid=159334)|—|[42nd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|165|1892|[Conor White](https://zwiftpower\.com/profile\.php?z=310524)|—|—|—|—|—|—|—|[38th](https://zwiftpower\.com/events\.php?zid=173451)|[17th](https://zwiftpower\.com/events\.php?zid=176148)
A|166|1890|[Igor Kopse](https://zwiftpower\.com/profile\.php?z=79239)|—|[36th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|[20th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|167|1880|[Martin Schei Rye XP](https://zwiftpower\.com/profile\.php?z=295831)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|—|—|—|[26th](https://zwiftpower\.com/events\.php?zid=162691)|—|[35th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|168|1878|[Yasuyuki Kobayashi](https://zwiftpower\.com/profile\.php?z=917068)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|—|—|—|—|[31st](https://zwiftpower\.com/events\.php?zid=166630)|—|[31st](https://zwiftpower\.com/events\.php?zid=173451)|—
A|169|1874|[Jokim Pluijmers](https://zwiftpower\.com/profile\.php?z=15354)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[43rd](https://zwiftpower\.com/events\.php?zid=152778)|[21st](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|170|1852|[Paul Cox](https://zwiftpower\.com/profile\.php?z=78847)|[HUUB Cycling Pulse](https://zwiftpower\.com/team\.php?id=3911)|[29th](https://zwiftpower\.com/events\.php?zid=152778)|[46th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|171|1850|[Justin Brusche](https://zwiftpower\.com/profile\.php?z=1024428)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|—|—|—|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=173451)|[55th](https://zwiftpower\.com/events\.php?zid=176148)
A|172|1846|[Sam Wadsley](https://zwiftpower\.com/profile\.php?z=237172)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|—|[31st](https://zwiftpower\.com/events\.php?zid=155628)|[47th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|173|1842|[Jan Vanaudenaerde](https://zwiftpower\.com/profile\.php?z=152302)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|—|—|—|[49th](https://zwiftpower\.com/events\.php?zid=162691)|—|[31st](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|174|1828|[Gavin Richardson](https://zwiftpower\.com/profile\.php?z=184210)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|—|—|—|[40th](https://zwiftpower\.com/events\.php?zid=173451)|[47th](https://zwiftpower\.com/events\.php?zid=176148)
A|175|1824|[Fordy💋](https://zwiftpower\.com/profile\.php?z=104271)|[Vision](https://zwiftpower\.com/team\.php?id=9)|—|—|—|[55th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|[34th](https://zwiftpower\.com/events\.php?zid=176148)
A|176|1816|[Mark Hansom](https://zwiftpower\.com/profile\.php?z=14136)|[AIMS](https://zwiftpower\.com/team\.php?id=4072)|—|[53rd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|[40th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|177|1796|[Matthieu Daudé](https://zwiftpower\.com/profile\.php?z=919908)|[TC15](https://zwiftpower\.com/team\.php?id=3041)|—|—|—|—|—|[49th](https://zwiftpower\.com/events\.php?zid=171935)|[54th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|178|1778|[Frømling 🔰](https://zwiftpower\.com/profile\.php?z=105270)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[66th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|[46th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|179|1770|[Nathan Killam 👨🏻 4 Hire](https://zwiftpower\.com/profile\.php?z=16044)|—|—|—|—|[79th](https://zwiftpower\.com/events\.php?zid=162691)|[37th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|179|1770|[Nicolmighty NSK](https://zwiftpower\.com/profile\.php?z=967384)|—|—|—|—|[60th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|[56th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|180|1766|[Simon Oxenham](https://zwiftpower\.com/profile\.php?z=1011)|[VC10](https://zwiftpower\.com/team\.php?id=2888)|—|[57th](https://zwiftpower\.com/events\.php?zid=155628)|[61st](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|181|1762|[Gavin Berry](https://zwiftpower\.com/profile\.php?z=153073)|—|[88th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|[32nd](https://zwiftpower\.com/events\.php?zid=176148)
A|182|1756|[James Vickers 🦜🔰](https://zwiftpower\.com/profile\.php?z=738971)|[HUUB Cycling Pulse](https://zwiftpower\.com/team\.php?id=3911)|[87th](https://zwiftpower\.com/events\.php?zid=152778)|—|[36th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|183|1752|[Third Wheel 🚳](https://zwiftpower\.com/profile\.php?z=48068)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[70th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|[55th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|184|1750|[John Blumenthal](https://zwiftpower\.com/profile\.php?z=10853)|[ZHR](https://zwiftpower\.com/team\.php?id=1979)|[76th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|[50th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|185|1738|[Joakim Lisson](https://zwiftpower\.com/profile\.php?z=113401)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[91st](https://zwiftpower\.com/events\.php?zid=152778)|[41st](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|185|1738|[Jörgen Johansson](https://zwiftpower\.com/profile\.php?z=388931)|—|—|—|—|—|[101st](https://zwiftpower\.com/events\.php?zid=166630)|—|—|[31st](https://zwiftpower\.com/events\.php?zid=176148)
A|186|1728|[Andrew Hale](https://zwiftpower\.com/profile\.php?z=571545)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|—|—|—|—|—|[64th](https://zwiftpower\.com/events\.php?zid=171935)|[73rd](https://zwiftpower\.com/events\.php?zid=173451)|—
A|187|1720|[Michael Larsen](https://zwiftpower\.com/profile\.php?z=104458)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|—|—|[80th](https://zwiftpower\.com/events\.php?zid=166630)|—|[61st](https://zwiftpower\.com/events\.php?zid=173451)|—
A|188|1718|[Artur Kub \[Grupetto W\-wa\]](https://zwiftpower\.com/profile\.php?z=498366)|[GrupettoWarszawa](https://zwiftpower\.com/team\.php?id=3022)|—|—|—|—|—|[71st](https://zwiftpower\.com/events\.php?zid=171935)|[71st](https://zwiftpower\.com/events\.php?zid=173451)|—
A|189|1716|[Ross Hetherington 🐌](https://zwiftpower\.com/profile\.php?z=129517)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|—|[64th](https://zwiftpower\.com/events\.php?zid=155628)|[79th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|190|1714|[Mick Kemper 🚀](https://zwiftpower\.com/profile\.php?z=20932)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|[42nd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|[102nd](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|191|1696|[Staffan Hopstadius](https://zwiftpower\.com/profile\.php?z=182715)|[Sz](https://zwiftpower\.com/team\.php?id=32)|—|—|[85th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|[68th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|192|1690|[Tom Fitzpatrick](https://zwiftpower\.com/profile\.php?z=60739)|—|—|—|—|—|[86th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|[70th](https://zwiftpower\.com/events\.php?zid=176148)
A|193|1688|[David Kim Amado 🥐 🍰 🥖](https://zwiftpower\.com/profile\.php?z=690464)|[Vision](https://zwiftpower\.com/team\.php?id=9)|—|—|[60th](https://zwiftpower\.com/events\.php?zid=159334)|[97th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|194|1676|[Gerrie Scholtes 🔰](https://zwiftpower\.com/profile\.php?z=22315)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|—|[67th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|[96th](https://zwiftpower\.com/events\.php?zid=176148)
A|194|1676|[Thomas Maaløe 🔰](https://zwiftpower\.com/profile\.php?z=265502)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[106th](https://zwiftpower\.com/events\.php?zid=152778)|—|[57th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|195|1666|[Ernie Austin](https://zwiftpower\.com/profile\.php?z=432019)|—|—|—|—|—|—|[73rd](https://zwiftpower\.com/events\.php?zid=171935)|[95th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|196|1664|[Yukio Bergholdt 🔰](https://zwiftpower\.com/profile\.php?z=49249)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|—|—|—|[80th](https://zwiftpower\.com/events\.php?zid=162691)|[89th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|197|1662|[Kaj\-André Døving \[X\]](https://zwiftpower\.com/profile\.php?z=69210)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[25th](https://zwiftpower\.com/events\.php?zid=152778)|[145th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|197|1662|[Goswinus Dijkstra 🔰](https://zwiftpower\.com/profile\.php?z=103446)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|[102nd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|[68th](https://zwiftpower\.com/events\.php?zid=176148)
A|198|1654|[Brennen Dwyer](https://zwiftpower\.com/profile\.php?z=13867)|—|—|—|—|—|[88th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|[86th](https://zwiftpower\.com/events\.php?zid=176148)
A|198|1654|[Ethan Browne](https://zwiftpower\.com/profile\.php?z=294744)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[85th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|[89th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|199|1646|[Martin Sundelin](https://zwiftpower\.com/profile\.php?z=325750)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[97th](https://zwiftpower\.com/events\.php?zid=152778)|[81st](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|200|1632|[Ben Oliver\-lewis7856](https://zwiftpower\.com/profile\.php?z=404182)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|—|—|[108th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|[77th](https://zwiftpower\.com/events\.php?zid=176148)
A|201|1610|[James Donaldson](https://zwiftpower\.com/profile\.php?z=5081)|[G\-T](https://zwiftpower\.com/team\.php?id=3790)|[105th](https://zwiftpower\.com/events\.php?zid=152778)|—|[91st](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|201|1610|[Remco Beckers](https://zwiftpower\.com/profile\.php?z=506582)|—|—|—|—|[127th](https://zwiftpower\.com/events\.php?zid=162691)|[69th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|202|1608|[The Spocker \[X\] 🔰 🖖](https://zwiftpower\.com/profile\.php?z=125855)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[139th](https://zwiftpower\.com/events\.php?zid=152778)|[58th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|203|1600|[Adam Egner](https://zwiftpower\.com/profile\.php?z=247562)|[NRT](https://zwiftpower\.com/team\.php?id=2389)|—|[94th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|[107th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|203|1600|[Andy Barrie](https://zwiftpower\.com/profile\.php?z=248557)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|—|—|—|[97th](https://zwiftpower\.com/events\.php?zid=173451)|[104th](https://zwiftpower\.com/events\.php?zid=176148)
A|204|1592|[Jan Fiebekorn \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=462183)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[123rd](https://zwiftpower\.com/events\.php?zid=152778)|[82nd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|205|1584|[John Bergenske⚡️BRT A2](https://zwiftpower\.com/profile\.php?z=180408)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[108th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|[101st](https://zwiftpower\.com/events\.php?zid=176148)
A|206|1582|[Paul\-Philippe Nelles 🔰](https://zwiftpower\.com/profile\.php?z=401617)|[Vision](https://zwiftpower\.com/team\.php?id=9)|—|[111th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|[99th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|207|1576|[Mark Scoot](https://zwiftpower\.com/profile\.php?z=62584)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|—|—|[103rd](https://zwiftpower\.com/events\.php?zid=171935)|—|[110th](https://zwiftpower\.com/events\.php?zid=176148)
A|207|1576|[Christoph Scheiber](https://zwiftpower\.com/profile\.php?z=457587)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[83rd](https://zwiftpower\.com/events\.php?zid=152778)|[130th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|208|1572|[Michał Woźniak](https://zwiftpower\.com/profile\.php?z=64147)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|—|[123rd](https://zwiftpower\.com/events\.php?zid=166630)|—|[92nd](https://zwiftpower\.com/events\.php?zid=173451)|—
A|209|1556|[Sonny Peterson](https://zwiftpower\.com/profile\.php?z=917866)|[Sz](https://zwiftpower\.com/team\.php?id=32)|—|—|—|[140th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|[83rd](https://zwiftpower\.com/events\.php?zid=173451)|—
A|210|1550|[Phil Donnelly \[BRT\]](https://zwiftpower\.com/profile\.php?z=514370)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|—|[142nd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|[84th](https://zwiftpower\.com/events\.php?zid=176148)
A|211|1548|[Sam Duffy](https://zwiftpower\.com/profile\.php?z=771005)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|[103rd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|[124th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|212|1546|[a\_TRI\_ called\_quest](https://zwiftpower\.com/profile\.php?z=265783)|—|—|[119th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|[109th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|213|1538|[simon lloyd](https://zwiftpower\.com/profile\.php?z=367040)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|[131st](https://zwiftpower\.com/events\.php?zid=155628)|[101st](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|214|1534|[michael Openshaw](https://zwiftpower\.com/profile\.php?z=1080848)|—|—|[118th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|[116th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|215|1494|[James Kiff](https://zwiftpower\.com/profile\.php?z=230612)|—|[143rd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|[111th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|216|1488|[MAS PPT](https://zwiftpower\.com/profile\.php?z=148298)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|—|[146th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|[111th](https://zwiftpower\.com/events\.php?zid=176148)
A|217|1484|[Bart Keizer](https://zwiftpower\.com/profile\.php?z=41959)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[150th](https://zwiftpower\.com/events\.php?zid=152778)|—|[109th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|218|1480|[Christoph Kroenert \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=70131)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|—|[123rd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|[138th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|219|1456|[Marc Pierschel \[VEGAN\]](https://zwiftpower\.com/profile\.php?z=918949)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[142nd](https://zwiftpower\.com/events\.php?zid=152778)|—|[131st](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|220|1452|[Dicky @ The Gateway Cycles](https://zwiftpower\.com/profile\.php?z=89796)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[132nd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|[143rd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|221|1448|[Bart Van den Berghe](https://zwiftpower\.com/profile\.php?z=699029)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|—|—|—|[138th](https://zwiftpower\.com/events\.php?zid=162691)|[139th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|222|1440|[Danijel Cancarevic](https://zwiftpower\.com/profile\.php?z=935351)|—|—|—|[129th](https://zwiftpower\.com/events\.php?zid=159334)|—|[152nd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|223|1438|[A\.Guilhoto<<GARBIKE>>](https://zwiftpower\.com/profile\.php?z=51911)|[PTz](https://zwiftpower\.com/team\.php?id=1)|—|—|—|[146th](https://zwiftpower\.com/events\.php?zid=162691)|[136th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|224|997|[Dan Bigham](https://zwiftpower\.com/profile\.php?z=548979)|—|[🥈](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|225|995|[Sean Vintin 🔰](https://zwiftpower\.com/profile\.php?z=137577)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|226|991|[M\.A\.](https://zwiftpower\.com/profile\.php?z=219167)|—|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|227|989|[SCOTTIE WEISS](https://zwiftpower\.com/profile\.php?z=745)|—|—|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|227|989|[Pierre Almeida](https://zwiftpower\.com/profile\.php?z=1028049)|—|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|228|983|[Aaron Coles🔰](https://zwiftpower\.com/profile\.php?z=428001)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|229|981|[Ollie Peckover 3R 🔰](https://zwiftpower\.com/profile\.php?z=382994)|—|[10th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|229|981|[Pär Bäcker🔰](https://zwiftpower\.com/profile\.php?z=504853)|[Superbonobo](https://zwiftpower\.com/team\.php?id=3631)|—|[10th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|230|975|[Luke Barfoot](https://zwiftpower\.com/profile\.php?z=30816)|—|—|—|—|—|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=176148)
A|231|971|[Nico Wüstenfeld 🔰 \[R2\]](https://zwiftpower\.com/profile\.php?z=1130)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|[15th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|232|969|[Jim McQuaide 🔰](https://zwiftpower\.com/profile\.php?z=7528)|[\[DRAFT\]](https://zwiftpower\.com/team\.php?id=1634)|—|[16th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|233|965|[Michael Madsen 🔰](https://zwiftpower\.com/profile\.php?z=104082)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|—|[18th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|233|965|[Ollie Jones 🔰🐝](https://zwiftpower\.com/profile\.php?z=137795)|—|[18th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|233|965|[Garry Beanland 7939](https://zwiftpower\.com/profile\.php?z=489072)|—|—|—|—|—|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=176148)
A|234|963|[Mariusz Cukierski \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=2984)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|[19th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|234|963|[Valentin Cheval](https://zwiftpower\.com/profile\.php?z=134586)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|—|—|[19th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|235|959|[Neil Richards ⚽](https://zwiftpower\.com/profile\.php?z=385769)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|—|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|235|959|[Kristian Oftedal](https://zwiftpower\.com/profile\.php?z=407425)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|236|953|[Ryan Bevis 🐉](https://zwiftpower\.com/profile\.php?z=291522)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[24th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|236|953|[Sean Pantellere 👖 🔰](https://zwiftpower\.com/profile\.php?z=486826)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|—|—|—|—|[24th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|237|951|[Adam Connellan](https://zwiftpower\.com/profile\.php?z=122246)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|—|—|[25th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|238|945|[Edwin Driessen](https://zwiftpower\.com/profile\.php?z=219489)|—|—|—|—|—|[28th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|239|941|[Ricardo Veras](https://zwiftpower\.com/profile\.php?z=790603)|—|—|—|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|240|937|[Per\-Johnny Doving \[X\]](https://zwiftpower\.com/profile\.php?z=46603)|[Team X](https://zwiftpower\.com/team\.php?id=2)|—|[32nd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|240|937|[Kasper Norden INC](https://zwiftpower\.com/profile\.php?z=48656)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|[32nd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|240|937|[Nick Glavac](https://zwiftpower\.com/profile\.php?z=602717)|—|—|—|—|[32nd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|241|935|[Patrick BLAIR \[AFC\] 🔰](https://zwiftpower\.com/profile\.php?z=8149)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|[33rd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|242|931|[Andy Sheret ØVB](https://zwiftpower\.com/profile\.php?z=174184)|[ØVB](https://zwiftpower\.com/team\.php?id=1576)|[35th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|243|929|[Ludovic Pollet](https://zwiftpower\.com/profile\.php?z=187160)|[Team X](https://zwiftpower\.com/team\.php?id=2)|—|[36th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|244|927|[Michael Podzaline](https://zwiftpower\.com/profile\.php?z=15522)|[Team X](https://zwiftpower\.com/team\.php?id=2)|—|—|—|[37th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|245|925|[Matt Sinclair](https://zwiftpower\.com/profile\.php?z=115138)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[38th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|246|923|[Rob Britt](https://zwiftpower\.com/profile\.php?z=307476)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|—|—|—|—|[39th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|246|923|[Bruce Zoom7214](https://zwiftpower\.com/profile\.php?z=1060589)|—|—|—|—|—|[39th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|247|919|[Jens Thoegersen](https://zwiftpower\.com/profile\.php?z=295550)|[DBR](https://zwiftpower\.com/team\.php?id=1986)|—|—|[41st](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|247|919|[Teppo Laurio](https://zwiftpower\.com/profile\.php?z=486471)|[FZR](https://zwiftpower\.com/team\.php?id=2834)|—|—|—|—|[41st](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|248|917|[Cam Napper](https://zwiftpower\.com/profile\.php?z=11806)|[Team AHDR](https://zwiftpower\.com/team\.php?id=629)|—|[42nd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|248|917|[Tadeusz Wlodarczyk \[Team PL\]](https://zwiftpower\.com/profile\.php?z=698392)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|—|—|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=176148)
A|249|913|[Quentin Goossens](https://zwiftpower\.com/profile\.php?z=246034)|—|—|—|—|[44th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|250|911|[Jens Frede Rasmussen](https://zwiftpower\.com/profile\.php?z=42327)|[Vision](https://zwiftpower\.com/team\.php?id=9)|—|[45th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|251|909|[Keith Gater WRT](https://zwiftpower\.com/profile\.php?z=288649)|—|—|—|—|—|—|[46th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|252|907|[Baptiste Boivin](https://zwiftpower\.com/profile\.php?z=87980)|[Foudre ⚡️](https://zwiftpower\.com/team\.php?id=2898)|—|—|—|—|[47th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|252|907|[Benjamin Werner \(KC\)](https://zwiftpower\.com/profile\.php?z=178127)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|—|—|[47th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|253|905|[Jeroen Blondeel](https://zwiftpower\.com/profile\.php?z=381944)|[1D](https://zwiftpower\.com/team\.php?id=3265)|[48th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|254|903|[Arvils Vikstrems Ⓥ](https://zwiftpower\.com/profile\.php?z=48835)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|[49th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|254|903|[Chris Gregory](https://zwiftpower\.com/profile\.php?z=149534)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|[49th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|255|901|[Jeffrey van Norden](https://zwiftpower\.com/profile\.php?z=620461)|[JDM](https://zwiftpower\.com/team\.php?id=2317)|—|—|—|—|—|—|—|[50th](https://zwiftpower\.com/events\.php?zid=176148)
A|256|899|[Bertus Turksma](https://zwiftpower\.com/profile\.php?z=197582)|[Dutch Racing](https://zwiftpower\.com/team\.php?id=3751)|—|—|—|—|—|—|—|[51st](https://zwiftpower\.com/events\.php?zid=176148)
A|257|897|[Tristan Guillemette](https://zwiftpower\.com/profile\.php?z=578461)|—|—|—|—|—|[52nd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|258|893|[Rune Jogert](https://zwiftpower\.com/profile\.php?z=271935)|—|—|—|—|—|—|—|—|[54th](https://zwiftpower\.com/events\.php?zid=176148)
A|258|893|[Michael Damm](https://zwiftpower\.com/profile\.php?z=402565)|—|[54th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|259|891|[Shawn Gray \[X\]](https://zwiftpower\.com/profile\.php?z=48456)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[55th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|259|891|[Roar Lindalen, Holand CK](https://zwiftpower\.com/profile\.php?z=212924)|[Holand SK](https://zwiftpower\.com/team\.php?id=293)|—|[55th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|260|889|[Espen Jogert 7498](https://zwiftpower\.com/profile\.php?z=541455)|—|—|—|—|—|—|—|—|[56th](https://zwiftpower\.com/events\.php?zid=176148)
A|261|885|[H Jonkers](https://zwiftpower\.com/profile\.php?z=1021960)|—|—|—|—|—|—|—|[58th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|262|881|[Brett Wakefield](https://zwiftpower\.com/profile\.php?z=4310)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[60th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|262|881|[Josh Taylor](https://zwiftpower\.com/profile\.php?z=720120)|—|—|[60th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|263|879|[Stuart Crawford 🚑 \[KRT\]](https://zwiftpower\.com/profile\.php?z=58435)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|[61st](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|264|877|[Eoin McGovern](https://zwiftpower\.com/profile\.php?z=109461)|[Vision](https://zwiftpower\.com/team\.php?id=9)|—|—|—|—|—|—|[62nd](https://zwiftpower\.com/events\.php?zid=173451)|—
A|265|875|[Patrick McCrann \(EnduranceNation\.us\)](https://zwiftpower\.com/profile\.php?z=328064)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|—|—|—|[63rd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|266|873|[Simon Nielsen 🔰](https://zwiftpower\.com/profile\.php?z=182972)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|—|—|—|[64th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|267|867|[Gunter Gellaerts](https://zwiftpower\.com/profile\.php?z=4109)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|—|—|—|—|—|—|—|[67th](https://zwiftpower\.com/events\.php?zid=176148)
A|267|867|[Henrik Öijer www\.henrikoijer\.com](https://zwiftpower\.com/profile\.php?z=484770)|[Sz](https://zwiftpower\.com/team\.php?id=32)|—|—|—|[67th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|268|863|[Tom Schyvens](https://zwiftpower\.com/profile\.php?z=426416)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|—|—|[69th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|268|863|[Tom Lowe](https://zwiftpower\.com/profile\.php?z=832843)|—|—|—|—|—|—|—|[69th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|269|861|[Fredrik Steen](https://zwiftpower\.com/profile\.php?z=180411)|[Vision](https://zwiftpower\.com/team\.php?id=9)|—|—|—|[70th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|270|857|[Hans Forhaug](https://zwiftpower\.com/profile\.php?z=982758)|—|—|—|—|—|—|—|[72nd](https://zwiftpower\.com/events\.php?zid=173451)|—
A|271|855|[James Ward](https://zwiftpower\.com/profile\.php?z=57004)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|[73rd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|272|853|[Odd Erlend Hansen Berg](https://zwiftpower\.com/profile\.php?z=183471)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|[74th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|273|851|[Niels Grote Beverborg](https://zwiftpower\.com/profile\.php?z=44114)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[75th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|273|851|[Lyndon Thompson \(Colten Care/TFC\) 🏡](https://zwiftpower\.com/profile\.php?z=297988)|[Colten Care](https://zwiftpower\.com/team\.php?id=4150)|—|—|—|—|—|—|—|[75th](https://zwiftpower\.com/events\.php?zid=176148)
A|274|849|[Richard Watson](https://zwiftpower\.com/profile\.php?z=265568)|[MOONGLU](https://zwiftpower\.com/team\.php?id=2863)|—|—|—|—|—|[76th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|275|847|[Jim O'Daniel](https://zwiftpower\.com/profile\.php?z=45462)|—|—|—|—|[77th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|275|847|[Darren Jones](https://zwiftpower\.com/profile\.php?z=76507)|—|—|[77th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|275|847|"[Matt Bruss \| Brazen Dropouts](https://zwiftpower\.com/profile\.php?z=420808)"|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|—|—|—|—|—|—|[77th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|276|845|[Joseph Bank 🔰](https://zwiftpower\.com/profile\.php?z=11837)|[AFC](https://zwiftpower\.com/team\.php?id=1405)|—|[78th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|276|845|[Chris Bartley](https://zwiftpower\.com/profile\.php?z=396062)|[Aerobyeq\-KRT](https://zwiftpower\.com/team\.php?id=3795)|—|—|[78th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|276|845|[Jorre Rentmeesters](https://zwiftpower\.com/profile\.php?z=596495)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|—|—|—|—|[78th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|277|843|[Richard Smith](https://zwiftpower\.com/profile\.php?z=111707)|—|—|[79th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|277|843|[Andrew Taylor](https://zwiftpower\.com/profile\.php?z=185391)|—|—|—|—|—|[79th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|278|841|[Stuart Mcgibbon](https://zwiftpower\.com/profile\.php?z=21336)|—|—|—|[80th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|278|841|[Etienne Mille](https://zwiftpower\.com/profile\.php?z=492621)|—|—|—|—|—|—|—|—|[80th](https://zwiftpower\.com/events\.php?zid=176148)
A|279|839|[Seth Stingley](https://zwiftpower\.com/profile\.php?z=942020)|[MarSting Story](https://zwiftpower\.com/team\.php?id=3253)|—|—|—|—|—|—|—|[81st](https://zwiftpower\.com/events\.php?zid=176148)
A|280|837|[Jorge Sousa](https://zwiftpower\.com/profile\.php?z=159892)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|—|—|—|—|—|[82nd](https://zwiftpower\.com/events\.php?zid=173451)|—
A|280|837|[Pieter Verhoeven](https://zwiftpower\.com/profile\.php?z=311684)|—|—|—|—|—|[82nd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|281|835|[il pinguino](https://zwiftpower\.com/profile\.php?z=53921)|—|—|—|—|[83rd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|281|835|[Ole\-Christian Ostreng](https://zwiftpower\.com/profile\.php?z=185955)|[Holand SK](https://zwiftpower\.com/team\.php?id=293)|—|—|—|—|—|[83rd](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|282|823|[Bram Maassen3474](https://zwiftpower\.com/profile\.php?z=405176)|—|—|—|—|—|—|[89th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|283|821|[Chris Smith ZHR A](https://zwiftpower\.com/profile\.php?z=70787)|—|—|—|—|—|—|—|[90th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|283|821|[Gaute Vikøren](https://zwiftpower\.com/profile\.php?z=115865)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|—|—|[90th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|283|821|[Martin van Laak](https://zwiftpower\.com/profile\.php?z=185716)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|[90th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|284|817|[Tom Shaw](https://zwiftpower\.com/profile\.php?z=230819)|—|—|—|—|—|—|—|—|[92nd](https://zwiftpower\.com/events\.php?zid=176148)
A|284|817|[Jesper Nielsen](https://zwiftpower\.com/profile\.php?z=256495)|[DBR](https://zwiftpower\.com/team\.php?id=1986)|—|[92nd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|284|817|[N\.Zaki](https://zwiftpower\.com/profile\.php?z=349154)|—|[92nd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|285|815|[Oliver W Bates](https://zwiftpower\.com/profile\.php?z=256868)|—|—|—|—|—|[93rd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|286|813|[Miles Billings](https://zwiftpower\.com/profile\.php?z=8901)|[545Velo](https://zwiftpower\.com/team\.php?id=341)|—|—|—|[94th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|286|813|[Don Fucs](https://zwiftpower\.com/profile\.php?z=47334)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|—|—|—|—|[94th](https://zwiftpower\.com/events\.php?zid=173451)|—
A|286|813|[Ben Skinner](https://zwiftpower\.com/profile\.php?z=852057)|—|—|—|—|—|[94th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|287|811|[Morten Austestad](https://zwiftpower\.com/profile\.php?z=59075)|[Holmenkollen CK](https://zwiftpower\.com/team\.php?id=3469)|—|—|—|[95th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|287|811|[Gijs Lummen](https://zwiftpower\.com/profile\.php?z=164835)|—|—|—|—|—|—|[95th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|287|811|[Ben Jones](https://zwiftpower\.com/profile\.php?z=537347)|[Cycle Specific](https://zwiftpower\.com/team\.php?id=4135)|—|—|—|—|—|—|—|[95th](https://zwiftpower\.com/events\.php?zid=176148)
A|288|809|[Piotr Kramarzyk \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=480688)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|[96th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|288|809|[Super\.Byke Viana](https://zwiftpower\.com/profile\.php?z=520670)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|[96th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|289|807|[Steve Fleetwood 🐀🔰](https://zwiftpower\.com/profile\.php?z=75421)|[Vision](https://zwiftpower\.com/team\.php?id=9)|—|—|[97th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|290|805|[Tobias Müller](https://zwiftpower\.com/profile\.php?z=119944)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|[98th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|291|801|[Bob McGlue🐉](https://zwiftpower\.com/profile\.php?z=129911)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|[100th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|292|799|[A Friend \(DT\)](https://zwiftpower\.com/profile\.php?z=6925)|—|—|—|—|—|—|[101st](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|293|797|[Christian Moxon \[BRT\]](https://zwiftpower\.com/profile\.php?z=35037)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|—|—|—|—|[102nd](https://zwiftpower\.com/events\.php?zid=176148)
A|293|797|[Joakim Parviainen🔰](https://zwiftpower\.com/profile\.php?z=828573)|[Superbonobo](https://zwiftpower\.com/team\.php?id=3631)|—|[102nd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|294|795|[Alf Erik Hagebakken](https://zwiftpower\.com/profile\.php?z=167476)|[GTD](https://zwiftpower\.com/team\.php?id=304)|—|—|[103rd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|294|795|[Troels Vest 🔰](https://zwiftpower\.com/profile\.php?z=402078)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|[103rd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|294|795|[Pawel Kochanowski](https://zwiftpower\.com/profile\.php?z=659309)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|—|—|—|[103rd](https://zwiftpower\.com/events\.php?zid=173451)|—
A|294|795|[Scott Main \[BRT\]](https://zwiftpower\.com/profile\.php?z=1002910)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|—|—|—|—|[103rd](https://zwiftpower\.com/events\.php?zid=176148)
A|295|793|[Matthew Hanford 2792](https://zwiftpower\.com/profile\.php?z=223820)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|—|—|—|—|—|[104th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|295|793|[Mark Dabell](https://zwiftpower\.com/profile\.php?z=297114)|—|—|[104th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|296|791|[Aaron Barry⚡️ \[BRT\]](https://zwiftpower\.com/profile\.php?z=743592)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|—|—|—|—|[105th](https://zwiftpower\.com/events\.php?zid=176148)
A|297|789|[Louis FL\- Team QC](https://zwiftpower\.com/profile\.php?z=5710)|[1R](https://zwiftpower\.com/team\.php?id=387)|—|—|[106th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|297|789|[Andreas Wolf](https://zwiftpower\.com/profile\.php?z=855340)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|[106th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|298|787|[Oliver Attinger CP](https://zwiftpower\.com/profile\.php?z=659834)|[HUUB Cycling Pulse](https://zwiftpower\.com/team\.php?id=3911)|—|[107th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|298|787|[Simon Spinczyk](https://zwiftpower\.com/profile\.php?z=991216)|[Tri Geckos](https://zwiftpower\.com/team\.php?id=4073)|—|—|—|—|—|—|—|[107th](https://zwiftpower\.com/events\.php?zid=176148)
A|299|785|[James Farren](https://zwiftpower\.com/profile\.php?z=63968)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|—|—|—|—|—|—|—|[108th](https://zwiftpower\.com/events\.php?zid=176148)
A|299|785|[Bret Wilks](https://zwiftpower\.com/profile\.php?z=328081)|[ICC](https://zwiftpower\.com/team\.php?id=2494)|—|[108th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|300|783|[Rich Wallwork](https://zwiftpower\.com/profile\.php?z=35468)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[109th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|300|783|[Lars K Granlund](https://zwiftpower\.com/profile\.php?z=162758)|[TTK](https://zwiftpower\.com/team\.php?id=2304)|—|—|—|—|—|—|—|[109th](https://zwiftpower\.com/events\.php?zid=176148)
A|300|783|[Kevin Colton4910](https://zwiftpower\.com/profile\.php?z=809355)|—|—|—|—|—|—|[109th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
A|301|777|[Mark Brock](https://zwiftpower\.com/profile\.php?z=148876)|[PTz](https://zwiftpower\.com/team\.php?id=1)|—|—|—|[112th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|302|775|[David Howell7106](https://zwiftpower\.com/profile\.php?z=231994)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|—|—|[113th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|303|773|[Jeremy Brunello](https://zwiftpower\.com/profile\.php?z=524642)|—|—|—|[114th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|303|773|[Alex Harris](https://zwiftpower\.com/profile\.php?z=561748)|—|—|—|—|—|[114th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|304|771|[Jason Siegle](https://zwiftpower\.com/profile\.php?z=33716)|—|—|[115th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|304|771|[James Corlett](https://zwiftpower\.com/profile\.php?z=190807)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|—|[115th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|305|769|[Jason Kenning](https://zwiftpower\.com/profile\.php?z=71129)|—|—|—|—|[116th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|305|769|[Romain Malbreil](https://zwiftpower\.com/profile\.php?z=259570)|—|—|[116th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|305|769|[Welsh Wizardd](https://zwiftpower\.com/profile\.php?z=642562)|—|—|—|—|—|—|—|—|[116th](https://zwiftpower\.com/events\.php?zid=176148)
A|305|769|[Franek GREGARIO](https://zwiftpower\.com/profile\.php?z=834315)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[116th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|306|767|[Poul Mærsk \- WKG](https://zwiftpower\.com/profile\.php?z=1015309)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|—|—|—|—|[117th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|307|763|[Will Duffy](https://zwiftpower\.com/profile\.php?z=820305)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|[119th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|308|761|[Andre Schmidt](https://zwiftpower\.com/profile\.php?z=126837)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|[120th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|308|761|[G Gauthier \# CLAN KNOX](https://zwiftpower\.com/profile\.php?z=214653)|[CLAN KNOX](https://zwiftpower\.com/team\.php?id=3073)|—|—|[120th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|308|761|[Henrik Johannesson](https://zwiftpower\.com/profile\.php?z=490481)|—|—|—|—|—|[120th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|309|759|[Daniel Colman](https://zwiftpower\.com/profile\.php?z=915207)|[P2R](https://zwiftpower\.com/team\.php?id=4025)|—|[121st](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|310|757|[Mark Warters](https://zwiftpower\.com/profile\.php?z=7164)|—|—|[122nd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|310|757|[David Billings](https://zwiftpower\.com/profile\.php?z=181148)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|[122nd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|310|757|[Jacek Rządca](https://zwiftpower\.com/profile\.php?z=472694)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|—|[122nd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|311|755|[Håvard Bjerke](https://zwiftpower\.com/profile\.php?z=668604)|—|—|—|[123rd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|312|753|[alm0st\.fast \(KC\)](https://zwiftpower\.com/profile\.php?z=458147)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[124th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|313|751|[Benjamin Klammt](https://zwiftpower\.com/profile\.php?z=37125)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|—|—|[125th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|313|751|[David Akesson](https://zwiftpower\.com/profile\.php?z=396670)|[Team 63\.8N](https://zwiftpower\.com/team\.php?id=2936)|—|[125th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|313|751|[Andrew Kirby](https://zwiftpower\.com/profile\.php?z=933861)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|[125th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|314|749|[Lloyd Wise](https://zwiftpower\.com/profile\.php?z=385139)|[Bikestrong\-KTM](https://zwiftpower\.com/team\.php?id=3878)|[126th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|314|749|[Jeroen Kniep](https://zwiftpower\.com/profile\.php?z=506438)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|—|—|—|[126th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|315|747|[Adam Bacon](https://zwiftpower\.com/profile\.php?z=841389)|—|[127th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|316|743|[Per\-Christian Barholm](https://zwiftpower\.com/profile\.php?z=64554)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|—|—|—|—|[129th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|316|743|[Anders Moustgaard 🔰](https://zwiftpower\.com/profile\.php?z=771995)|—|[129th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|317|741|[Tomasz Woźniak](https://zwiftpower\.com/profile\.php?z=66572)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|—|[130th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|318|739|[Alex McMillan KISS B](https://zwiftpower\.com/profile\.php?z=525583)|—|[131st](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|319|735|[Nathan Hey](https://zwiftpower\.com/profile\.php?z=20082)|—|—|[133rd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
A|319|735|[Tim Mason](https://zwiftpower\.com/profile\.php?z=132847)|[EBR](https://zwiftpower\.com/team\.php?id=3741)|—|—|—|[133rd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|319|735|[Ronny Kambach \(KC\)](https://zwiftpower\.com/profile\.php?z=985936)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[133rd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|320|733|[Sam Vijay 6303](https://zwiftpower\.com/profile\.php?z=707058)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|—|—|—|—|[134th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|321|723|[Niels Heldens](https://zwiftpower\.com/profile\.php?z=883266)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|—|—|—|[139th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|321|723|[Poisson Lune](https://zwiftpower\.com/profile\.php?z=1011634)|—|—|—|[139th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
A|322|721|"[Alexander Chatzakis \[BRT\]⚡️\| 🏊🏽‍🚴🏽‍🏃🏽70\.3\| 🇳🇱🇬🇷](https://zwiftpower\.com/profile\.php?z=496345)"|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|—|[140th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|323|719|[Craig Hodgson \(Muckle CC\)](https://zwiftpower\.com/profile\.php?z=1125198)|—|[141st](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|324|711|[J CCL](https://zwiftpower\.com/profile\.php?z=32116)|—|—|—|—|—|[145th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|324|711|[Karl Tiselius](https://zwiftpower\.com/profile\.php?z=181548)|[Sz](https://zwiftpower\.com/team\.php?id=32)|[145th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|325|705|[Kilian Hemart Ⓥ](https://zwiftpower\.com/profile\.php?z=488019)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[148th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|325|705|[Jonas Larsen](https://zwiftpower\.com/profile\.php?z=592962)|[FUSION EVO](https://zwiftpower\.com/team\.php?id=3185)|—|—|—|[148th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|326|703|[Tom Wassink](https://zwiftpower\.com/profile\.php?z=3351)|—|—|—|—|[149th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
A|326|703|[Scott Mitchell](https://zwiftpower\.com/profile\.php?z=144703)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[149th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|326|703|[Christiaan Stange](https://zwiftpower\.com/profile\.php?z=452001)|—|—|—|—|—|[149th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|327|697|[Tyler Pearce Ⓥ](https://zwiftpower\.com/profile\.php?z=290307)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[152nd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
A|328|691|[Matteo Casadei](https://zwiftpower\.com/profile\.php?z=199374)|[VelociT](https://zwiftpower\.com/team\.php?id=1071)|—|—|—|—|[155th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|329|689|[Matthew Goss](https://zwiftpower\.com/profile\.php?z=874407)|—|—|—|—|—|[156th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
A|330|685|[Paul vdG CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=329191)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|[158th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|1|7934|[Mark Worthington](https://zwiftpower\.com/profile\.php?z=71062)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[21st](https://zwiftpower\.com/events\.php?zid=152778)|[🥈](https://zwiftpower\.com/events\.php?zid=155628)|[🥇](https://zwiftpower\.com/events\.php?zid=159334)|[🥇](https://zwiftpower\.com/events\.php?zid=162691)|[🥈](https://zwiftpower\.com/events\.php?zid=166630)|[6th](https://zwiftpower\.com/events\.php?zid=171935)|[🥉](https://zwiftpower\.com/events\.php?zid=173451)|[🥇](https://zwiftpower\.com/events\.php?zid=176148)
B|2|7828|[Dave Brodmann](https://zwiftpower\.com/profile\.php?z=15543)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[22nd](https://zwiftpower\.com/events\.php?zid=152778)|[🥉](https://zwiftpower\.com/events\.php?zid=155628)|[7th](https://zwiftpower\.com/events\.php?zid=159334)|[20th](https://zwiftpower\.com/events\.php?zid=162691)|[7th](https://zwiftpower\.com/events\.php?zid=166630)|[16th](https://zwiftpower\.com/events\.php?zid=171935)|[9th](https://zwiftpower\.com/events\.php?zid=173451)|[6th](https://zwiftpower\.com/events\.php?zid=176148)
B|3|6793|[Andrew Robinson TFC Racing](https://zwiftpower\.com/profile\.php?z=100901)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|[32nd](https://zwiftpower\.com/events\.php?zid=155628)|[9th](https://zwiftpower\.com/events\.php?zid=159334)|[19th](https://zwiftpower\.com/events\.php?zid=162691)|[11th](https://zwiftpower\.com/events\.php?zid=166630)|[21st](https://zwiftpower\.com/events\.php?zid=171935)|[10th](https://zwiftpower\.com/events\.php?zid=173451)|[5th](https://zwiftpower\.com/events\.php?zid=176148)
B|4|6771|[Jonathan Davey](https://zwiftpower\.com/profile\.php?z=1474)|—|[17th](https://zwiftpower\.com/events\.php?zid=152778)|[19th](https://zwiftpower\.com/events\.php?zid=155628)|[4th](https://zwiftpower\.com/events\.php?zid=159334)|[32nd](https://zwiftpower\.com/events\.php?zid=162691)|—|[12th](https://zwiftpower\.com/events\.php?zid=171935)|[17th](https://zwiftpower\.com/events\.php?zid=173451)|[17th](https://zwiftpower\.com/events\.php?zid=176148)
B|5|6699|[Sonic The Hedgehog](https://zwiftpower\.com/profile\.php?z=49294)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[28th](https://zwiftpower\.com/events\.php?zid=152778)|[27th](https://zwiftpower\.com/events\.php?zid=155628)|[32nd](https://zwiftpower\.com/events\.php?zid=159334)|—|[13th](https://zwiftpower\.com/events\.php?zid=166630)|[22nd](https://zwiftpower\.com/events\.php?zid=171935)|[14th](https://zwiftpower\.com/events\.php?zid=173451)|[18th](https://zwiftpower\.com/events\.php?zid=176148)
B|6|6675|[James Ayton](https://zwiftpower\.com/profile\.php?z=164395)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|[29th](https://zwiftpower\.com/events\.php?zid=155628)|[23rd](https://zwiftpower\.com/events\.php?zid=159334)|[12th](https://zwiftpower\.com/events\.php?zid=162691)|[25th](https://zwiftpower\.com/events\.php?zid=166630)|[27th](https://zwiftpower\.com/events\.php?zid=171935)|[28th](https://zwiftpower\.com/events\.php?zid=173451)|[22nd](https://zwiftpower\.com/events\.php?zid=176148)
B|7|6645|[Peter Campbell](https://zwiftpower\.com/profile\.php?z=40108)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[33rd](https://zwiftpower\.com/events\.php?zid=152778)|[12th](https://zwiftpower\.com/events\.php?zid=155628)|[27th](https://zwiftpower\.com/events\.php?zid=159334)|—|[50th](https://zwiftpower\.com/events\.php?zid=166630)|[25th](https://zwiftpower\.com/events\.php?zid=171935)|[15th](https://zwiftpower\.com/events\.php?zid=173451)|[19th](https://zwiftpower\.com/events\.php?zid=176148)
B|8|6603|[Arthur Araújo](https://zwiftpower\.com/profile\.php?z=330442)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[56th](https://zwiftpower\.com/events\.php?zid=152778)|[36th](https://zwiftpower\.com/events\.php?zid=155628)|—|[33rd](https://zwiftpower\.com/events\.php?zid=162691)|[5th](https://zwiftpower\.com/events\.php?zid=166630)|[24th](https://zwiftpower\.com/events\.php?zid=171935)|[19th](https://zwiftpower\.com/events\.php?zid=173451)|[29th](https://zwiftpower\.com/events\.php?zid=176148)
B|9|6587|[Ole Bergtun \(BCK\)](https://zwiftpower\.com/profile\.php?z=67220)|—|[38th](https://zwiftpower\.com/events\.php?zid=152778)|[16th](https://zwiftpower\.com/events\.php?zid=155628)|[20th](https://zwiftpower\.com/events\.php?zid=159334)|[49th](https://zwiftpower\.com/events\.php?zid=162691)|[24th](https://zwiftpower\.com/events\.php?zid=166630)|[32nd](https://zwiftpower\.com/events\.php?zid=171935)|[31st](https://zwiftpower\.com/events\.php?zid=173451)|—
B|10|6555|[Stephan Maertens Ⓥ 🐇🥕](https://zwiftpower\.com/profile\.php?z=37710)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[41st](https://zwiftpower\.com/events\.php?zid=152778)|[18th](https://zwiftpower\.com/events\.php?zid=155628)|[36th](https://zwiftpower\.com/events\.php?zid=159334)|[41st](https://zwiftpower\.com/events\.php?zid=162691)|[17th](https://zwiftpower\.com/events\.php?zid=166630)|[45th](https://zwiftpower\.com/events\.php?zid=171935)|—|[28th](https://zwiftpower\.com/events\.php?zid=176148)
B|11|6497|[Jon Izagirre PTz](https://zwiftpower\.com/profile\.php?z=170606)|[PTz](https://zwiftpower\.com/team\.php?id=1)|—|[74th](https://zwiftpower\.com/events\.php?zid=155628)|[28th](https://zwiftpower\.com/events\.php?zid=159334)|[27th](https://zwiftpower\.com/events\.php?zid=162691)|[47th](https://zwiftpower\.com/events\.php?zid=166630)|[33rd](https://zwiftpower\.com/events\.php?zid=171935)|[21st](https://zwiftpower\.com/events\.php?zid=173451)|[25th](https://zwiftpower\.com/events\.php?zid=176148)
B|12|6249|[S Ssmith \(V\) Ⓥ ^\.\.^ 🐣🌱🐾](https://zwiftpower\.com/profile\.php?z=175002)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[66th](https://zwiftpower\.com/events\.php?zid=152778)|[67th](https://zwiftpower\.com/events\.php?zid=155628)|[68th](https://zwiftpower\.com/events\.php?zid=159334)|[76th](https://zwiftpower\.com/events\.php?zid=162691)|[45th](https://zwiftpower\.com/events\.php?zid=166630)|—|[25th](https://zwiftpower\.com/events\.php?zid=173451)|[32nd](https://zwiftpower\.com/events\.php?zid=176148)
B|13|5726|[lee saunders \(BRT\)](https://zwiftpower\.com/profile\.php?z=177623)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|[33rd](https://zwiftpower\.com/events\.php?zid=155628)|[26th](https://zwiftpower\.com/events\.php?zid=159334)|[29th](https://zwiftpower\.com/events\.php?zid=162691)|[23rd](https://zwiftpower\.com/events\.php?zid=166630)|—|[16th](https://zwiftpower\.com/events\.php?zid=173451)|[13th](https://zwiftpower\.com/events\.php?zid=176148)
B|14|5710|[Nick Nellis 🔰](https://zwiftpower\.com/profile\.php?z=460177)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|[53rd](https://zwiftpower\.com/events\.php?zid=155628)|[30th](https://zwiftpower\.com/events\.php?zid=159334)|[39th](https://zwiftpower\.com/events\.php?zid=162691)|[10th](https://zwiftpower\.com/events\.php?zid=166630)|—|[5th](https://zwiftpower\.com/events\.php?zid=173451)|[11th](https://zwiftpower\.com/events\.php?zid=176148)
B|15|5602|[Oliver Steffen](https://zwiftpower\.com/profile\.php?z=196547)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|[23rd](https://zwiftpower\.com/events\.php?zid=155628)|[37th](https://zwiftpower\.com/events\.php?zid=159334)|[42nd](https://zwiftpower\.com/events\.php?zid=162691)|[29th](https://zwiftpower\.com/events\.php?zid=166630)|—|[35th](https://zwiftpower\.com/events\.php?zid=173451)|[36th](https://zwiftpower\.com/events\.php?zid=176148)
B|16|5558|[Ricardo Gama 💣](https://zwiftpower\.com/profile\.php?z=108059)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|[40th](https://zwiftpower\.com/events\.php?zid=155628)|—|[47th](https://zwiftpower\.com/events\.php?zid=162691)|[39th](https://zwiftpower\.com/events\.php?zid=166630)|[26th](https://zwiftpower\.com/events\.php?zid=171935)|[27th](https://zwiftpower\.com/events\.php?zid=173451)|[45th](https://zwiftpower\.com/events\.php?zid=176148)
B|17|5498|[Daniel Revo \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=427562)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[45th](https://zwiftpower\.com/events\.php?zid=152778)|[51st](https://zwiftpower\.com/events\.php?zid=155628)|[31st](https://zwiftpower\.com/events\.php?zid=159334)|[57th](https://zwiftpower\.com/events\.php?zid=162691)|—|[36th](https://zwiftpower\.com/events\.php?zid=171935)|[34th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|18|5402|[Andrew Keedle](https://zwiftpower\.com/profile\.php?z=117018)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|[103rd](https://zwiftpower\.com/events\.php?zid=152778)|[52nd](https://zwiftpower\.com/events\.php?zid=155628)|[25th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[43rd](https://zwiftpower\.com/events\.php?zid=171935)|[38th](https://zwiftpower\.com/events\.php?zid=173451)|[41st](https://zwiftpower\.com/events\.php?zid=176148)
B|19|5324|[Tom Weiler \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=157775)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[75th](https://zwiftpower\.com/events\.php?zid=152778)|[64th](https://zwiftpower\.com/events\.php?zid=155628)|[61st](https://zwiftpower\.com/events\.php?zid=159334)|—|[52nd](https://zwiftpower\.com/events\.php?zid=166630)|[38th](https://zwiftpower\.com/events\.php?zid=171935)|—|[51st](https://zwiftpower\.com/events\.php?zid=176148)
B|20|4975|[Neil Hughes](https://zwiftpower\.com/profile\.php?z=160845)|[ZSUN](https://zwiftpower\.com/team\.php?id=161)|[8th](https://zwiftpower\.com/events\.php?zid=152778)|[🥇](https://zwiftpower\.com/events\.php?zid=155628)|[🥉](https://zwiftpower\.com/events\.php?zid=159334)|[🥈](https://zwiftpower\.com/events\.php?zid=162691)|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=173451)|—
B|21|4847|[M agnus \(KC\)](https://zwiftpower\.com/profile\.php?z=348200)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[29th](https://zwiftpower\.com/events\.php?zid=152778)|[14th](https://zwiftpower\.com/events\.php?zid=155628)|[10th](https://zwiftpower\.com/events\.php?zid=159334)|[8th](https://zwiftpower\.com/events\.php?zid=162691)|[18th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|22|4787|[Ned Bowen 🏹](https://zwiftpower\.com/profile\.php?z=833859)|[Flèche](https://zwiftpower\.com/team\.php?id=3497)|[27th](https://zwiftpower\.com/events\.php?zid=152778)|[25th](https://zwiftpower\.com/events\.php?zid=155628)|[11th](https://zwiftpower\.com/events\.php?zid=159334)|[11th](https://zwiftpower\.com/events\.php?zid=162691)|[35th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|23|4591|[Rob Bate \[TPH\]](https://zwiftpower\.com/profile\.php?z=73789)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|[62nd](https://zwiftpower\.com/events\.php?zid=152778)|—|[51st](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[41st](https://zwiftpower\.com/events\.php?zid=171935)|[29th](https://zwiftpower\.com/events\.php?zid=173451)|[24th](https://zwiftpower\.com/events\.php?zid=176148)
B|24|4499|[Quentin Lafaye \[Vision\]](https://zwiftpower\.com/profile\.php?z=96812)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[77th](https://zwiftpower\.com/events\.php?zid=152778)|[48th](https://zwiftpower\.com/events\.php?zid=155628)|[48th](https://zwiftpower\.com/events\.php?zid=159334)|[45th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|[35th](https://zwiftpower\.com/events\.php?zid=176148)
B|25|4493|[Andreas Kasperek \(KC\)](https://zwiftpower\.com/profile\.php?z=86467)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[52nd](https://zwiftpower\.com/events\.php?zid=152778)|—|[47th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[58th](https://zwiftpower\.com/events\.php?zid=171935)|[53rd](https://zwiftpower\.com/events\.php?zid=173451)|[46th](https://zwiftpower\.com/events\.php?zid=176148)
B|26|4457|[Trond Sannes Marthinsen](https://zwiftpower\.com/profile\.php?z=124057)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|—|—|[50th](https://zwiftpower\.com/events\.php?zid=159334)|[72nd](https://zwiftpower\.com/events\.php?zid=162691)|[54th](https://zwiftpower\.com/events\.php?zid=166630)|[50th](https://zwiftpower\.com/events\.php?zid=171935)|—|[48th](https://zwiftpower\.com/events\.php?zid=176148)
B|27|3966|[Chris Pritchard](https://zwiftpower\.com/profile\.php?z=30255)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|—|[6th](https://zwiftpower\.com/events\.php?zid=155628)|[🥈](https://zwiftpower\.com/events\.php?zid=159334)|[🥉](https://zwiftpower\.com/events\.php?zid=162691)|—|[8th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|28|3902|[Johannes Reining \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=629172)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[19th](https://zwiftpower\.com/events\.php?zid=152778)|[17th](https://zwiftpower\.com/events\.php?zid=155628)|[5th](https://zwiftpower\.com/events\.php?zid=159334)|[10th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|29|3878|[Nick Bunyard1980](https://zwiftpower\.com/profile\.php?z=612430)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=166630)|[23rd](https://zwiftpower\.com/events\.php?zid=171935)|[6th](https://zwiftpower\.com/events\.php?zid=173451)|[26th](https://zwiftpower\.com/events\.php?zid=176148)
B|30|3840|[Mateusz Kucharski \[TeamPL\] B1](https://zwiftpower\.com/profile\.php?z=95410)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[35th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|[5th](https://zwiftpower\.com/events\.php?zid=162691)|[22nd](https://zwiftpower\.com/events\.php?zid=166630)|—|[20th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|31|3776|[Dave Gould CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=600137)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[24th](https://zwiftpower\.com/events\.php?zid=155628)|[29th](https://zwiftpower\.com/events\.php?zid=159334)|[31st](https://zwiftpower\.com/events\.php?zid=162691)|[30th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|32|3724|[Michael Haar \(KC\)](https://zwiftpower\.com/profile\.php?z=955094)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[28th](https://zwiftpower\.com/events\.php?zid=171935)|[33rd](https://zwiftpower\.com/events\.php?zid=173451)|[37th](https://zwiftpower\.com/events\.php?zid=176148)
B|33|3720|[Mark Anthony Jones](https://zwiftpower\.com/profile\.php?z=75787)|[PTz](https://zwiftpower\.com/team\.php?id=1)|[15th](https://zwiftpower\.com/events\.php?zid=152778)|[15th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|[23rd](https://zwiftpower\.com/events\.php?zid=173451)|[89th](https://zwiftpower\.com/events\.php?zid=176148)
B|34|3690|[Thomas Fischer 🦁 \(KC\)](https://zwiftpower\.com/profile\.php?z=160790)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[48th](https://zwiftpower\.com/events\.php?zid=152778)|[26th](https://zwiftpower\.com/events\.php?zid=155628)|—|[44th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|[39th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|35|3636|[Daniel Hunt OCC](https://zwiftpower\.com/profile\.php?z=182368)|[OCC](https://zwiftpower\.com/team\.php?id=2620)|—|—|—|[63rd](https://zwiftpower\.com/events\.php?zid=162691)|—|[42nd](https://zwiftpower\.com/events\.php?zid=171935)|[32nd](https://zwiftpower\.com/events\.php?zid=173451)|[47th](https://zwiftpower\.com/events\.php?zid=176148)
B|36|3612|[Sarel Boshoff Ⓥ](https://zwiftpower\.com/profile\.php?z=1128952)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|[44th](https://zwiftpower\.com/events\.php?zid=159334)|[101st](https://zwiftpower\.com/events\.php?zid=162691)|[34th](https://zwiftpower\.com/events\.php?zid=166630)|[17th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|37|3600|[Stuart Lynne](https://zwiftpower\.com/profile\.php?z=78201)|[PTz](https://zwiftpower\.com/team\.php?id=1)|—|[34th](https://zwiftpower\.com/events\.php?zid=155628)|[45th](https://zwiftpower\.com/events\.php?zid=159334)|[43rd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|[80th](https://zwiftpower\.com/events\.php?zid=176148)
B|38|3554|[Rene Meijer](https://zwiftpower\.com/profile\.php?z=803509)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|—|[83rd](https://zwiftpower\.com/events\.php?zid=155628)|[63rd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=173451)|[49th](https://zwiftpower\.com/events\.php?zid=176148)
B|39|3542|[marinko barac\(KC\)](https://zwiftpower\.com/profile\.php?z=823018)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[57th](https://zwiftpower\.com/events\.php?zid=152778)|—|[66th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[56th](https://zwiftpower\.com/events\.php?zid=171935)|—|[52nd](https://zwiftpower\.com/events\.php?zid=176148)
B|40|3502|[Glebster Carr \[BRT\]](https://zwiftpower\.com/profile\.php?z=572357)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|[57th](https://zwiftpower\.com/events\.php?zid=159334)|[75th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|[59th](https://zwiftpower\.com/events\.php?zid=173451)|[60th](https://zwiftpower\.com/events\.php?zid=176148)
B|41|3488|[Pedro Correia](https://zwiftpower\.com/profile\.php?z=718875)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[93rd](https://zwiftpower\.com/events\.php?zid=152778)|[10th](https://zwiftpower\.com/events\.php?zid=155628)|[75th](https://zwiftpower\.com/events\.php?zid=159334)|[80th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|42|3462|[Ian Bottomore](https://zwiftpower\.com/profile\.php?z=163377)|—|—|—|—|[83rd](https://zwiftpower\.com/events\.php?zid=162691)|[85th](https://zwiftpower\.com/events\.php?zid=166630)|[55th](https://zwiftpower\.com/events\.php?zid=171935)|[48th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|43|3452|[Stu Gardiner ⓥ](https://zwiftpower\.com/profile\.php?z=446939)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|—|[74th](https://zwiftpower\.com/events\.php?zid=166630)|[69th](https://zwiftpower\.com/events\.php?zid=171935)|[56th](https://zwiftpower\.com/events\.php?zid=173451)|[77th](https://zwiftpower\.com/events\.php?zid=176148)
B|44|3390|[Florian Meyer \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=45416)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[78th](https://zwiftpower\.com/events\.php?zid=152778)|—|[82nd](https://zwiftpower\.com/events\.php?zid=159334)|—|[76th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|[71st](https://zwiftpower\.com/events\.php?zid=176148)
B|45|3384|[Christian Kumpfe \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=49743)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[67th](https://zwiftpower\.com/events\.php?zid=152778)|—|[72nd](https://zwiftpower\.com/events\.php?zid=159334)|[97th](https://zwiftpower\.com/events\.php?zid=162691)|—|[74th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|45|3384|[Ben Harris](https://zwiftpower\.com/profile\.php?z=774028)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|[59th](https://zwiftpower\.com/events\.php?zid=159334)|[98th](https://zwiftpower\.com/events\.php?zid=162691)|—|[65th](https://zwiftpower\.com/events\.php?zid=171935)|—|[88th](https://zwiftpower\.com/events\.php?zid=176148)
B|46|3382|[Vítor Santos](https://zwiftpower\.com/profile\.php?z=281363)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|—|[85th](https://zwiftpower\.com/events\.php?zid=159334)|—|[82nd](https://zwiftpower\.com/events\.php?zid=166630)|—|[68th](https://zwiftpower\.com/events\.php?zid=173451)|[76th](https://zwiftpower\.com/events\.php?zid=176148)
B|47|3326|[Sam Bird](https://zwiftpower\.com/profile\.php?z=447439)|—|—|—|—|[79th](https://zwiftpower\.com/events\.php?zid=162691)|[99th](https://zwiftpower\.com/events\.php?zid=166630)|—|[74th](https://zwiftpower\.com/events\.php?zid=173451)|[87th](https://zwiftpower\.com/events\.php?zid=176148)
B|48|3274|[Michael Birkenmaier\[BMPRO\]](https://zwiftpower\.com/profile\.php?z=470237)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[95th](https://zwiftpower\.com/events\.php?zid=152778)|[90th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|[101st](https://zwiftpower\.com/events\.php?zid=166630)|[79th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|49|2969|[Michi Seipold \(KC\)](https://zwiftpower\.com/profile\.php?z=415204)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=166630)|[10th](https://zwiftpower\.com/events\.php?zid=171935)|—|[4th](https://zwiftpower\.com/events\.php?zid=176148)
B|50|2933|[Dan Wong](https://zwiftpower\.com/profile\.php?z=541321)|—|—|—|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=171935)|[13th](https://zwiftpower\.com/events\.php?zid=173451)|[9th](https://zwiftpower\.com/events\.php?zid=176148)
B|51|2913|[MarcAndre Doucet](https://zwiftpower\.com/profile\.php?z=703114)|[Stallion](https://zwiftpower\.com/team\.php?id=4154)|—|—|—|[24th](https://zwiftpower\.com/events\.php?zid=162691)|—|[14th](https://zwiftpower\.com/events\.php?zid=171935)|[7th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|52|2871|[Mathias Kemnitz](https://zwiftpower\.com/profile\.php?z=116780)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|—|[37th](https://zwiftpower\.com/events\.php?zid=166630)|[18th](https://zwiftpower\.com/events\.php?zid=171935)|[11th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|53|2853|[Toshiya Kondo](https://zwiftpower\.com/profile\.php?z=376815)|—|[26th](https://zwiftpower\.com/events\.php?zid=152778)|—|[13th](https://zwiftpower\.com/events\.php?zid=159334)|[36th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|54|2847|[Bogdan SITO \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=55688)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[23rd](https://zwiftpower\.com/events\.php?zid=152778)|[20th](https://zwiftpower\.com/events\.php?zid=155628)|[35th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|55|2817|[Stuart Hardy⚡️\[BRT\]🔰](https://zwiftpower\.com/profile\.php?z=591535)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[39th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=171935)|[24th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|56|2805|[Paul Yoder](https://zwiftpower\.com/profile\.php?z=941353)|—|—|—|[58th](https://zwiftpower\.com/events\.php?zid=159334)|[26th](https://zwiftpower\.com/events\.php?zid=162691)|—|[15th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|57|2803|[Marcus Fellowes](https://zwiftpower\.com/profile\.php?z=754114)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[44th](https://zwiftpower\.com/events\.php?zid=152778)|—|[17th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|[39th](https://zwiftpower\.com/events\.php?zid=176148)
B|58|2759|[Darren Phillips](https://zwiftpower\.com/profile\.php?z=48345)|—|[43rd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|[58th](https://zwiftpower\.com/events\.php?zid=162691)|[21st](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|59|2741|[Espen Børsting](https://zwiftpower\.com/profile\.php?z=50855)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|—|—|—|—|[71st](https://zwiftpower\.com/events\.php?zid=166630)|—|[37th](https://zwiftpower\.com/events\.php?zid=173451)|[23rd](https://zwiftpower\.com/events\.php?zid=176148)
B|60|2733|[William price \-\-\-\- Chester tri 2087](https://zwiftpower\.com/profile\.php?z=320581)|—|—|—|[39th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=173451)|[78th](https://zwiftpower\.com/events\.php?zid=176148)
B|61|2729|[Sven Fessler \(KC\)](https://zwiftpower\.com/profile\.php?z=48283)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|[73rd](https://zwiftpower\.com/events\.php?zid=162691)|[20th](https://zwiftpower\.com/events\.php?zid=166630)|[44th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|62|2719|[James Fowler](https://zwiftpower\.com/profile\.php?z=237877)|[FinishStrong](https://zwiftpower\.com/team\.php?id=1740)|—|—|—|[53rd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|[46th](https://zwiftpower\.com/events\.php?zid=173451)|[43rd](https://zwiftpower\.com/events\.php?zid=176148)
B|63|2715|[Matt Hilbert](https://zwiftpower\.com/profile\.php?z=946001)|—|—|—|—|—|[66th](https://zwiftpower\.com/events\.php?zid=166630)|[40th](https://zwiftpower\.com/events\.php?zid=171935)|—|[38th](https://zwiftpower\.com/events\.php?zid=176148)
B|64|2705|[Oskar Thofte](https://zwiftpower\.com/profile\.php?z=334279)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|—|—|[24th](https://zwiftpower\.com/events\.php?zid=159334)|[55th](https://zwiftpower\.com/events\.php?zid=162691)|—|[70th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|65|2693|[Tom Box](https://zwiftpower\.com/profile\.php?z=84)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|—|—|—|[48th](https://zwiftpower\.com/events\.php?zid=162691)|[48th](https://zwiftpower\.com/events\.php?zid=166630)|[59th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|66|2687|[Daniel Potter ⚡](https://zwiftpower\.com/profile\.php?z=83111)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[71st](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|[54th](https://zwiftpower\.com/events\.php?zid=171935)|—|[33rd](https://zwiftpower\.com/events\.php?zid=176148)
B|67|2673|[D Todd CRYO\-GEN\(B2\)](https://zwiftpower\.com/profile\.php?z=595717)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|[72nd](https://zwiftpower\.com/events\.php?zid=166630)|—|[51st](https://zwiftpower\.com/events\.php?zid=173451)|[42nd](https://zwiftpower\.com/events\.php?zid=176148)
B|68|2659|[Dean Hardie](https://zwiftpower\.com/profile\.php?z=96086)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[54th](https://zwiftpower\.com/events\.php?zid=152778)|—|[64th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|[54th](https://zwiftpower\.com/events\.php?zid=176148)
B|69|2657|[Matt Hill](https://zwiftpower\.com/profile\.php?z=503800)|[ICC](https://zwiftpower\.com/team\.php?id=2494)|[63rd](https://zwiftpower\.com/events\.php?zid=152778)|[47th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|[63rd](https://zwiftpower\.com/events\.php?zid=176148)
B|70|2643|[Carl Foster](https://zwiftpower\.com/profile\.php?z=879014)|—|—|—|[80th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[51st](https://zwiftpower\.com/events\.php?zid=171935)|[49th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|71|2639|[Trevor King](https://zwiftpower\.com/profile\.php?z=908872)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|—|—|—|—|[78th](https://zwiftpower\.com/events\.php?zid=166630)|[63rd](https://zwiftpower\.com/events\.php?zid=171935)|[41st](https://zwiftpower\.com/events\.php?zid=173451)|—
B|72|2615|[ⓥ Tim Jones \(Cowley Road Condors CC\)](https://zwiftpower\.com/profile\.php?z=850532)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|[84th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[57th](https://zwiftpower\.com/events\.php?zid=171935)|—|[53rd](https://zwiftpower\.com/events\.php?zid=176148)
B|73|2601|[Henrik Andlem Borresen](https://zwiftpower\.com/profile\.php?z=653639)|[HSIL](https://zwiftpower\.com/team\.php?id=3049)|—|—|—|—|—|[64th](https://zwiftpower\.com/events\.php?zid=171935)|[67th](https://zwiftpower\.com/events\.php?zid=173451)|[70th](https://zwiftpower\.com/events\.php?zid=176148)
B|74|2591|[Basti Löcken \[R2\]](https://zwiftpower\.com/profile\.php?z=591300)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|—|—|—|[82nd](https://zwiftpower\.com/events\.php?zid=162691)|—|[52nd](https://zwiftpower\.com/events\.php?zid=171935)|[72nd](https://zwiftpower\.com/events\.php?zid=173451)|—
B|75|2585|[Richard Jackson](https://zwiftpower\.com/profile\.php?z=556224)|—|—|—|—|—|[79th](https://zwiftpower\.com/events\.php?zid=166630)|—|[58th](https://zwiftpower\.com/events\.php?zid=173451)|[72nd](https://zwiftpower\.com/events\.php?zid=176148)
B|76|2571|[Rodney Young](https://zwiftpower\.com/profile\.php?z=113175)|[PTz](https://zwiftpower\.com/team\.php?id=1)|[84th](https://zwiftpower\.com/events\.php?zid=152778)|—|[73rd](https://zwiftpower\.com/events\.php?zid=159334)|[59th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|77|2545|[Terje Dahl](https://zwiftpower\.com/profile\.php?z=282852)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[73rd](https://zwiftpower\.com/events\.php?zid=152778)|—|[67th](https://zwiftpower\.com/events\.php?zid=159334)|[89th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|78|2461|[Michael Placidi Vegan Italiano/Australiano7558](https://zwiftpower\.com/profile\.php?z=82858)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[100th](https://zwiftpower\.com/events\.php?zid=152778)|[87th](https://zwiftpower\.com/events\.php?zid=155628)|—|[84th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|79|2443|[Chris Forster](https://zwiftpower\.com/profile\.php?z=858816)|—|[99th](https://zwiftpower\.com/events\.php?zid=152778)|[92nd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|[89th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|80|1988|[Andy Barrie](https://zwiftpower\.com/profile\.php?z=248557)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=162691)|[🥇](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|81|1982|[Jon Arnar Sigurjonsson TFC](https://zwiftpower\.com/profile\.php?z=230021)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|[7th](https://zwiftpower\.com/events\.php?zid=162691)|—|[🥉](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|81|1982|[Paul Smith 1145](https://zwiftpower\.com/profile\.php?z=467100)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|—|[4th](https://zwiftpower\.com/events\.php?zid=155628)|[6th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|82|1980|[Michal Jablonka](https://zwiftpower\.com/profile\.php?z=44878)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=173451)|[🥉](https://zwiftpower\.com/events\.php?zid=176148)
B|82|1980|[Eddie Kelly](https://zwiftpower\.com/profile\.php?z=821829)|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=162691)|—|[🥈](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|83|1974|[Jan Thrun](https://zwiftpower\.com/profile\.php?z=249691)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|[9th](https://zwiftpower\.com/events\.php?zid=152778)|[5th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|84|1972|[Jacques La Cock](https://zwiftpower\.com/profile\.php?z=69795)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[7th](https://zwiftpower\.com/events\.php?zid=152778)|—|[8th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|85|1970|[Craig Nicholls](https://zwiftpower\.com/profile\.php?z=242281)|—|—|—|[15th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|86|1966|[Ruben Guis](https://zwiftpower\.com/profile\.php?z=52559)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|[11th](https://zwiftpower\.com/events\.php?zid=152778)|[7th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|87|1952|[Paul Hayward](https://zwiftpower\.com/profile\.php?z=6917)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[14th](https://zwiftpower\.com/events\.php?zid=152778)|[11th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|88|1944|[TCaplen \- Toby McTobeface](https://zwiftpower\.com/profile\.php?z=14123)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|—|[17th](https://zwiftpower\.com/events\.php?zid=162691)|[12th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|89|1936|[Przemo Wicio](https://zwiftpower\.com/profile\.php?z=403051)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|[14th](https://zwiftpower\.com/events\.php?zid=162691)|[19th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|90|1924|[Jonathan Reeves TFC Racing](https://zwiftpower\.com/profile\.php?z=72122)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|[12th](https://zwiftpower\.com/events\.php?zid=159334)|—|[27th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|91|1922|[Paul Lewis](https://zwiftpower\.com/profile\.php?z=544654)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|—|[21st](https://zwiftpower\.com/events\.php?zid=155628)|[19th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|92|1918|[corey ring Bourne Wheelers CC](https://zwiftpower\.com/profile\.php?z=242439)|—|—|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=166630)|—|[36th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|93|1908|"[Alexander Chatzakis \[BRT\]⚡️\| 🏊🏽‍🚴🏽‍🏃🏽70\.3\| 🇳🇱🇬🇷](https://zwiftpower\.com/profile\.php?z=496345)"|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=159334)|[25th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|94|1904|[Daniel Valverde](https://zwiftpower\.com/profile\.php?z=59136)|[Team Bergziegen](https://zwiftpower\.com/team\.php?id=3486)|—|—|[21st](https://zwiftpower\.com/events\.php?zid=159334)|[28th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|95|1892|[Mark Scoot](https://zwiftpower\.com/profile\.php?z=62584)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[24th](https://zwiftpower\.com/events\.php?zid=152778)|[31st](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|95|1892|[Jamie Woodward](https://zwiftpower\.com/profile\.php?z=685302)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|—|—|—|—|[15th](https://zwiftpower\.com/events\.php?zid=166630)|—|[40th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|96|1890|[Dan Fox \(Rowe & King\)](https://zwiftpower\.com/profile\.php?z=268249)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=162691)|[40th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|96|1890|[Trond G\. Amundsen](https://zwiftpower\.com/profile\.php?z=412981)|[Holmenkollen CK](https://zwiftpower\.com/team\.php?id=3469)|[30th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|[26th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|97|1868|[Anders Ueland Skeie](https://zwiftpower\.com/profile\.php?z=44666)|—|[36th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|[31st](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|98|1866|[Peter van Gellekom](https://zwiftpower\.com/profile\.php?z=152483)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|—|—|—|[34th](https://zwiftpower\.com/events\.php?zid=162691)|—|[34th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|99|1862|[Andrei Benea](https://zwiftpower\.com/profile\.php?z=218066)|[Team CLS](https://zwiftpower\.com/team\.php?id=1426)|—|[37th](https://zwiftpower\.com/events\.php?zid=155628)|[33rd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|100|1860|[Zuko Inc7](https://zwiftpower\.com/profile\.php?z=285309)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|[49th](https://zwiftpower\.com/events\.php?zid=155628)|—|[22nd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|101|1852|[Brian Hanson \(CRYO\-GEN\)](https://zwiftpower\.com/profile\.php?z=675708)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|[38th](https://zwiftpower\.com/events\.php?zid=159334)|[37th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|102|1850|[Jorge Sousa](https://zwiftpower\.com/profile\.php?z=159892)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|—|—|[60th](https://zwiftpower\.com/events\.php?zid=162691)|[16th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|103|1844|[John Morton](https://zwiftpower\.com/profile\.php?z=186724)|—|—|[38th](https://zwiftpower\.com/events\.php?zid=155628)|[41st](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|103|1844|[Bouke Lubach](https://zwiftpower\.com/profile\.php?z=851007)|—|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=162691)|[49th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|104|1830|[Ta abaluga1204](https://zwiftpower\.com/profile\.php?z=1216297)|—|—|—|—|—|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=173451)|[44th](https://zwiftpower\.com/events\.php?zid=176148)
B|105|1816|[Pedro Prego](https://zwiftpower\.com/profile\.php?z=867117)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|[60th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|[33rd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|106|1812|[Pete Tizzard](https://zwiftpower\.com/profile\.php?z=70038)|—|—|—|—|[81st](https://zwiftpower\.com/events\.php?zid=162691)|[14th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|106|1812|[Gerhard Reich \(KC\)](https://zwiftpower\.com/profile\.php?z=501754)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[60th](https://zwiftpower\.com/events\.php?zid=152778)|[35th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|107|1810|[Johan Öberg](https://zwiftpower\.com/profile\.php?z=96206)|[ART](https://zwiftpower\.com/team\.php?id=2271)|—|[43rd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|[53rd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|108|1808|[Mark Munro](https://zwiftpower\.com/profile\.php?z=133572)|—|—|—|—|[46th](https://zwiftpower\.com/events\.php?zid=162691)|[51st](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|109|1798|[Stephen Balls KISS RT](https://zwiftpower\.com/profile\.php?z=122150)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|[76th](https://zwiftpower\.com/events\.php?zid=159334)|—|[26th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|110|1792|[David Greenwood](https://zwiftpower\.com/profile\.php?z=60661)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|[65th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|[40th](https://zwiftpower\.com/events\.php?zid=176148)
B|111|1788|[Grega Gregoric](https://zwiftpower\.com/profile\.php?z=509176)|[Top\-Fit](https://zwiftpower\.com/team\.php?id=2019)|—|[76th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|[31st](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|112|1784|[Roar Mathisen](https://zwiftpower\.com/profile\.php?z=481344)|—|—|—|[55th](https://zwiftpower\.com/events\.php?zid=159334)|[54th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|113|1782|[Diederik Westerhof \[X\]](https://zwiftpower\.com/profile\.php?z=56853)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[65th](https://zwiftpower\.com/events\.php?zid=152778)|[45th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|114|1776|[Stefan Seebauer \(KC\)](https://zwiftpower\.com/profile\.php?z=200900)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|—|[68th](https://zwiftpower\.com/events\.php?zid=166630)|—|[45th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|115|1772|[Håvard Thomassen](https://zwiftpower\.com/profile\.php?z=46032)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[61st](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|[54th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|116|1762|[Guy Vdsmis TFC ZZRC](https://zwiftpower\.com/profile\.php?z=980133)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|—|—|[64th](https://zwiftpower\.com/events\.php?zid=173451)|[56th](https://zwiftpower\.com/events\.php?zid=176148)
B|117|1752|[Jorge Teixeira](https://zwiftpower\.com/profile\.php?z=296051)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|—|—|[61st](https://zwiftpower\.com/events\.php?zid=162691)|[64th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|118|1750|[Brandon Zuckerberg](https://zwiftpower\.com/profile\.php?z=445447)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|—|—|—|—|—|—|[57th](https://zwiftpower\.com/events\.php?zid=173451)|[69th](https://zwiftpower\.com/events\.php?zid=176148)
B|119|1748|[Daniel Gassmann](https://zwiftpower\.com/profile\.php?z=57897)|—|—|—|—|—|—|—|[63rd](https://zwiftpower\.com/events\.php?zid=173451)|[64th](https://zwiftpower\.com/events\.php?zid=176148)
B|120|1742|[Jonathan Graciano](https://zwiftpower\.com/profile\.php?z=57316)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|—|—|—|—|—|—|[75th](https://zwiftpower\.com/events\.php?zid=173451)|[55th](https://zwiftpower\.com/events\.php?zid=176148)
B|121|1738|[Don Morgan](https://zwiftpower\.com/profile\.php?z=16961)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|—|—|—|[70th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|[62nd](https://zwiftpower\.com/events\.php?zid=173451)|—
B|121|1738|[Kris De Swert](https://zwiftpower\.com/profile\.php?z=282903)|[\-AG\-](https://zwiftpower\.com/team\.php?id=2897)|—|—|—|—|[57th](https://zwiftpower\.com/events\.php?zid=166630)|[75th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|122|1736|[Ben Gillett](https://zwiftpower\.com/profile\.php?z=315040)|—|—|—|—|—|—|—|[71st](https://zwiftpower\.com/events\.php?zid=173451)|[62nd](https://zwiftpower\.com/events\.php?zid=176148)
B|122|1736|[Reino Kleband Ⓥ](https://zwiftpower\.com/profile\.php?z=803761)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|—|[73rd](https://zwiftpower\.com/events\.php?zid=166630)|[60th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|123|1734|[Paul Morrison](https://zwiftpower\.com/profile\.php?z=597478)|[G\-T](https://zwiftpower\.com/team\.php?id=3790)|[80th](https://zwiftpower\.com/events\.php?zid=152778)|[54th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|123|1734|[Stuart Weatherley \[TPH\]](https://zwiftpower\.com/profile\.php?z=966125)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|—|[82nd](https://zwiftpower\.com/events\.php?zid=155628)|[52nd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|124|1730|[Ger van Mossel](https://zwiftpower\.com/profile\.php?z=347435)|—|—|—|[56th](https://zwiftpower\.com/events\.php?zid=159334)|—|[80th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|125|1722|[GORDON JACKSON](https://zwiftpower\.com/profile\.php?z=19671)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|—|[69th](https://zwiftpower\.com/events\.php?zid=155628)|[71st](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|126|1720|[Theun Bouius](https://zwiftpower\.com/profile\.php?z=700655)|—|—|—|—|—|—|—|[66th](https://zwiftpower\.com/events\.php?zid=173451)|[75th](https://zwiftpower\.com/events\.php?zid=176148)
B|127|1712|[Magnus Andersson](https://zwiftpower\.com/profile\.php?z=891264)|[Team 63\.8N](https://zwiftpower\.com/team\.php?id=2936)|[90th](https://zwiftpower\.com/events\.php?zid=152778)|[55th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|128|1710|[Jens Kroepelien 3399](https://zwiftpower\.com/profile\.php?z=197170)|—|—|—|[83rd](https://zwiftpower\.com/events\.php?zid=159334)|—|[63rd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|129|1708|[John van der Griend](https://zwiftpower\.com/profile\.php?z=735211)|—|—|[70th](https://zwiftpower\.com/events\.php?zid=155628)|—|[77th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|130|1704|[James Wilkinson](https://zwiftpower\.com/profile\.php?z=448922)|[Ride WLF](https://zwiftpower\.com/team\.php?id=2257)|—|—|—|—|[88th](https://zwiftpower\.com/events\.php?zid=166630)|[61st](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|131|1694|[Chris Sanders](https://zwiftpower\.com/profile\.php?z=893943)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|[92nd](https://zwiftpower\.com/events\.php?zid=166630)|[62nd](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|132|1690|[Kenneth Nymoen 9631](https://zwiftpower\.com/profile\.php?z=611482)|—|[85th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|[71st](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|133|1688|[Ben Willson🦎\[Vision\]🔰](https://zwiftpower\.com/profile\.php?z=995424)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[76th](https://zwiftpower\.com/events\.php?zid=152778)|[81st](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|134|1678|[Markus Krupp \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=441576)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[70th](https://zwiftpower\.com/events\.php?zid=152778)|—|[92nd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|135|1676|[Alexandre Bernardo\[TugaZ\]](https://zwiftpower\.com/profile\.php?z=443530)|—|—|—|—|[86th](https://zwiftpower\.com/events\.php?zid=162691)|—|[77th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|136|1668|[Hauke Westenberg](https://zwiftpower\.com/profile\.php?z=449960)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|—|[95th](https://zwiftpower\.com/events\.php?zid=166630)|[72nd](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|137|1654|[Patrick Flickert \(KC\)](https://zwiftpower\.com/profile\.php?z=508975)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[86th](https://zwiftpower\.com/events\.php?zid=152778)|[88th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|138|1650|[Richard Skinstad\-Jansen](https://zwiftpower\.com/profile\.php?z=61248)|[Tunge Gir](https://zwiftpower\.com/team\.php?id=3418)|—|—|—|[85th](https://zwiftpower\.com/events\.php?zid=162691)|[91st](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|139|1636|[Scott Moore \(RCS\)](https://zwiftpower\.com/profile\.php?z=86866)|—|—|[91st](https://zwiftpower\.com/events\.php?zid=155628)|—|[92nd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|140|1632|[keith palmer](https://zwiftpower\.com/profile\.php?z=57284)|[ICC](https://zwiftpower\.com/team\.php?id=2494)|—|[96th](https://zwiftpower\.com/events\.php?zid=155628)|[89th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|141|999|[Paul\-Philippe Nelles 🔰](https://zwiftpower\.com/profile\.php?z=401617)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[🥇](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|142|997|[Simon Oxenham](https://zwiftpower\.com/profile\.php?z=1011)|[VC10](https://zwiftpower\.com/team\.php?id=2888)|[🥈](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|142|997|[Alex Gleim](https://zwiftpower\.com/profile\.php?z=4838)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=173451)|—
B|142|997|[Mikel Aristegi](https://zwiftpower\.com/profile\.php?z=274849)|—|—|—|—|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=176148)
B|143|995|[Mark Gardiner](https://zwiftpower\.com/profile\.php?z=691846)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[🥉](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|144|993|[Tom Hargreaves 🦆](https://zwiftpower\.com/profile\.php?z=26709)|[NUUN SIGMA](https://zwiftpower\.com/team\.php?id=2330)|—|—|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|144|993|[Gregg O'Malley](https://zwiftpower\.com/profile\.php?z=162054)|[FINESSE](https://zwiftpower\.com/team\.php?id=2779)|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|144|993|[Charles\-Antoine Marcotte](https://zwiftpower\.com/profile\.php?z=197250)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|144|993|[Stijn Derycke](https://zwiftpower\.com/profile\.php?z=335088)|[1D](https://zwiftpower\.com/team\.php?id=3265)|[4th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|144|993|[Marcin Dabrowski](https://zwiftpower\.com/profile\.php?z=949300)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|145|991|[Brian Kelly](https://zwiftpower\.com/profile\.php?z=5147)|—|—|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|145|991|[Hulk Smash](https://zwiftpower\.com/profile\.php?z=33385)|[SAS Cycle Club](https://zwiftpower\.com/team\.php?id=1348)|[5th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|146|989|[Dustin Byers](https://zwiftpower\.com/profile\.php?z=399357)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[6th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|147|987|[Richard Sparkes](https://zwiftpower\.com/profile\.php?z=50676)|—|—|—|—|—|—|[7th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|147|987|[Leon Bond](https://zwiftpower\.com/profile\.php?z=266642)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|—|—|—|—|[7th](https://zwiftpower\.com/events\.php?zid=176148)
B|148|985|[Rory McAdams](https://zwiftpower\.com/profile\.php?z=102983)|—|—|[8th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|148|985|[Rich B](https://zwiftpower\.com/profile\.php?z=247393)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|—|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=176148)
B|149|983|[Pawel Kochanowski](https://zwiftpower\.com/profile\.php?z=659309)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|[9th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|149|983|[Horst Grünbichler\(KC\),radrooTeam](https://zwiftpower\.com/profile\.php?z=842176)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|149|983|[Adam Cole5011](https://zwiftpower\.com/profile\.php?z=988367)|—|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|150|981|[Per\-Christian Barholm](https://zwiftpower\.com/profile\.php?z=64554)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[10th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|150|981|[Chris Whalley](https://zwiftpower\.com/profile\.php?z=73302)|—|—|—|—|—|—|—|—|[10th](https://zwiftpower\.com/events\.php?zid=176148)
B|151|979|[C Sampley\-DIRT](https://zwiftpower\.com/profile\.php?z=61402)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|152|977|[Andy Hadfield](https://zwiftpower\.com/profile\.php?z=309897)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|—|—|—|—|[12th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|152|977|[Jan Jöckel \(KC\)](https://zwiftpower\.com/profile\.php?z=444506)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[12th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|152|977|[Andre Bakken](https://zwiftpower\.com/profile\.php?z=474758)|—|—|—|—|—|—|—|—|[12th](https://zwiftpower\.com/events\.php?zid=176148)
B|153|975|[Louis FL\- Team QC](https://zwiftpower\.com/profile\.php?z=5710)|[1R](https://zwiftpower\.com/team\.php?id=387)|[13th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|153|975|[Alexander Koop](https://zwiftpower\.com/profile\.php?z=168192)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|—|[13th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|153|975|[Stian Gylder\-Haugland\[V\]](https://zwiftpower\.com/profile\.php?z=430101)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|154|973|[\. Sigurgrimur](https://zwiftpower\.com/profile\.php?z=82570)|[SCK](https://zwiftpower\.com/team\.php?id=3783)|—|—|—|—|—|—|—|[14th](https://zwiftpower\.com/events\.php?zid=176148)
B|154|973|[Anders Tveit](https://zwiftpower\.com/profile\.php?z=596576)|[TCK](https://zwiftpower\.com/team\.php?id=2422)|—|—|[14th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|155|971|[Anton Kriel](https://zwiftpower\.com/profile\.php?z=14216)|—|—|—|—|—|—|—|—|[15th](https://zwiftpower\.com/events\.php?zid=176148)
B|155|971|[Larry Parker🔰](https://zwiftpower\.com/profile\.php?z=229626)|[VR2209](https://zwiftpower\.com/team\.php?id=3121)|—|—|—|[15th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|156|969|[Andy Davies](https://zwiftpower\.com/profile\.php?z=6720)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[16th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|156|969|[Mark Tiffany ⚡️\[BRT\]](https://zwiftpower\.com/profile\.php?z=91996)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|—|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=176148)
B|156|969|[Tsuyoshi Shirasaki\(BALBA\-F\)](https://zwiftpower\.com/profile\.php?z=677784)|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|157|965|[Tomasz Skalski](https://zwiftpower\.com/profile\.php?z=197372)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|157|965|[David Fidalgo](https://zwiftpower\.com/profile\.php?z=197561)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|[18th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|157|965|[Oliver Attinger CP](https://zwiftpower\.com/profile\.php?z=659834)|[HUUB Cycling Pulse](https://zwiftpower\.com/team\.php?id=3911)|[18th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|158|963|[P K](https://zwiftpower\.com/profile\.php?z=484)|—|—|—|—|—|—|[19th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|159|961|[Arne Danielsen \[V\]](https://zwiftpower\.com/profile\.php?z=65053)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[20th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|159|961|[Tom Palfrey](https://zwiftpower\.com/profile\.php?z=876356)|—|—|—|—|—|—|—|—|[20th](https://zwiftpower\.com/events\.php?zid=176148)
B|159|961|[Lewis Wake](https://zwiftpower\.com/profile\.php?z=900521)|[KCRT](https://zwiftpower\.com/team\.php?id=4040)|—|—|—|—|—|[20th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|160|959|[Pete Hicks](https://zwiftpower\.com/profile\.php?z=62449)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|—|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=176148)
B|160|959|[Stephen Cann](https://zwiftpower\.com/profile\.php?z=128516)|—|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|161|957|[Ørjan K Sandnes](https://zwiftpower\.com/profile\.php?z=181173)|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|161|957|[Loris van de Kassteele \(ITA\)](https://zwiftpower\.com/profile\.php?z=810115)|[Team Italy](https://zwiftpower\.com/team\.php?id=588)|—|—|—|—|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=173451)|—
B|162|955|[B\. S Johansen](https://zwiftpower\.com/profile\.php?z=155898)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|—|—|—|[23rd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|163|951|[James Hay 👽](https://zwiftpower\.com/profile\.php?z=548008)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[25th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|164|947|[clement palozzi](https://zwiftpower\.com/profile\.php?z=88907)|—|—|—|—|—|—|—|—|[27th](https://zwiftpower\.com/events\.php?zid=176148)
B|165|945|[Adrian Zvara](https://zwiftpower\.com/profile\.php?z=453256)|—|—|[28th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|165|945|[Eric Larson](https://zwiftpower\.com/profile\.php?z=981211)|—|—|—|—|—|[28th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|166|943|[Pete Butler](https://zwiftpower\.com/profile\.php?z=11957)|—|—|—|—|—|—|[29th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|167|941|[Marcin P](https://zwiftpower\.com/profile\.php?z=50059)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|—|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=176148)
B|167|941|[Alex Solomides](https://zwiftpower\.com/profile\.php?z=56178)|[NGCA](https://zwiftpower\.com/team\.php?id=2023)|—|[30th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|168|939|[Jan Becher](https://zwiftpower\.com/profile\.php?z=19250)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|[31st](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|168|939|[David Cardis](https://zwiftpower\.com/profile\.php?z=878863)|[CyclingGames](https://zwiftpower\.com/team\.php?id=4088)|—|—|—|—|—|—|—|[31st](https://zwiftpower\.com/events\.php?zid=176148)
B|169|937|[Matthew Page \- Bikes Etc Magazine](https://zwiftpower\.com/profile\.php?z=472521)|—|[32nd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|169|937|[Marcus Geßner](https://zwiftpower\.com/profile\.php?z=948975)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|—|[32nd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|170|933|[Paul Skipper \[BRT\]](https://zwiftpower\.com/profile\.php?z=12026)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|—|—|—|—|[34th](https://zwiftpower\.com/events\.php?zid=176148)
B|170|933|[Jon Poole](https://zwiftpower\.com/profile\.php?z=513566)|—|—|—|[34th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|170|933|[konsuke kondo](https://zwiftpower\.com/profile\.php?z=954147)|—|[34th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|171|931|[Harald Sætermo](https://zwiftpower\.com/profile\.php?z=516214)|—|—|—|—|—|—|[35th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|171|931|[Conor Hanna](https://zwiftpower\.com/profile\.php?z=589521)|—|—|—|—|[35th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|172|929|[CELLE Cédric](https://zwiftpower\.com/profile\.php?z=925935)|—|—|—|—|—|[36th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|173|927|[Mikkel Staunsholm 👾](https://zwiftpower\.com/profile\.php?z=19963)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[37th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|173|927|[David Mulholland](https://zwiftpower\.com/profile\.php?z=571602)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|—|[37th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|174|925|[Heiko Witt](https://zwiftpower\.com/profile\.php?z=325159)|—|—|—|—|—|[38th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|174|925|[Andy Wylde](https://zwiftpower\.com/profile\.php?z=737856)|—|—|—|—|[38th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|175|923|[Brian Rhoden](https://zwiftpower\.com/profile\.php?z=53663)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|—|[39th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|175|923|[Jim Horstmann Ⓥ](https://zwiftpower\.com/profile\.php?z=182374)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|—|—|[39th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|176|921|[james barlow](https://zwiftpower\.com/profile\.php?z=168223)|—|—|—|[40th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|176|921|[Isaac Hulse](https://zwiftpower\.com/profile\.php?z=729330)|[CYH](https://zwiftpower\.com/team\.php?id=3812)|[40th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|176|921|[Fabian Frinzel \(KC\)](https://zwiftpower\.com/profile\.php?z=1075763)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|[40th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|177|919|[Dave Hyne\-Jones](https://zwiftpower\.com/profile\.php?z=78280)|—|—|[41st](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|177|919|[Chris Orgill CRC LFCC](https://zwiftpower\.com/profile\.php?z=181867)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|—|[41st](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|178|917|[Tom Wassink](https://zwiftpower\.com/profile\.php?z=3351)|—|—|—|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|178|917|[Juanjo Ferres](https://zwiftpower\.com/profile\.php?z=69023)|[ZESP](https://zwiftpower\.com/team\.php?id=2146)|—|[42nd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|178|917|[Paul Bridgman](https://zwiftpower\.com/profile\.php?z=330950)|[CycleChat](https://zwiftpower\.com/team\.php?id=1458)|[42nd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|179|915|[Magnus Strahlert](https://zwiftpower\.com/profile\.php?z=227517)|[FSK](https://zwiftpower\.com/team\.php?id=3382)|—|—|—|—|[43rd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|179|915|[Dominik Gielas \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=737147)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|[43rd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|179|915|[R Ansolo](https://zwiftpower\.com/profile\.php?z=920115)|—|—|—|—|—|—|—|[43rd](https://zwiftpower\.com/events\.php?zid=173451)|—
B|180|913|[David Szentesi6476](https://zwiftpower\.com/profile\.php?z=374304)|—|—|—|—|—|—|—|[44th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|180|913|[Daniel Hind9210](https://zwiftpower\.com/profile\.php?z=702156)|—|—|[44th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|180|913|[Henryk Tokarski](https://zwiftpower\.com/profile\.php?z=834023)|[PLN](https://zwiftpower\.com/team\.php?id=3124)|—|—|—|—|[44th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|181|909|[Martin Rezac](https://zwiftpower\.com/profile\.php?z=38051)|—|—|[46th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|181|909|[George Olive Ⓥ](https://zwiftpower\.com/profile\.php?z=80614)|[Team CLS](https://zwiftpower\.com/team\.php?id=1426)|[46th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|181|909|[Mark Howlett](https://zwiftpower\.com/profile\.php?z=172455)|[GGCC](https://zwiftpower\.com/team\.php?id=1611)|—|—|[46th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|181|909|[Bobby Riley](https://zwiftpower\.com/profile\.php?z=245435)|—|—|—|—|—|[46th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|181|909|[Brandon Reinholt](https://zwiftpower\.com/profile\.php?z=963378)|—|—|—|—|—|—|[46th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|182|907|[Marc Schulz\-Bacon \(KC\)](https://zwiftpower\.com/profile\.php?z=177541)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|—|—|[47th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|182|907|[Francis Lafleche](https://zwiftpower\.com/profile\.php?z=469512)|—|—|—|—|—|—|—|[47th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|182|907|[Kevin Vander Ende \[Splunk>\]](https://zwiftpower\.com/profile\.php?z=966423)|[Splunk>](https://zwiftpower\.com/team\.php?id=3933)|[47th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|183|905|[Paul Hornby](https://zwiftpower\.com/profile\.php?z=66046)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|—|—|[48th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|184|903|[Harry Roberts](https://zwiftpower\.com/profile\.php?z=320)|—|—|—|—|—|—|[49th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|184|903|[Alex Mitchell\-Touche](https://zwiftpower\.com/profile\.php?z=46172)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[49th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|184|903|[William Warthon](https://zwiftpower\.com/profile\.php?z=95164)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|[49th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|185|901|[Bob Chew](https://zwiftpower\.com/profile\.php?z=4859)|—|—|[50th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|185|901|[Dirk Jan Los](https://zwiftpower\.com/profile\.php?z=41646)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[50th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|185|901|[Alexis Lavoie](https://zwiftpower\.com/profile\.php?z=156078)|—|—|—|—|—|—|—|—|[50th](https://zwiftpower\.com/events\.php?zid=176148)
B|185|901|[Pawel Szczygielski](https://zwiftpower\.com/profile\.php?z=170523)|—|—|—|—|[50th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|185|901|[Shane O'Neill](https://zwiftpower\.com/profile\.php?z=956534)|—|—|—|—|—|—|—|[50th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|186|899|[James Bentley](https://zwiftpower\.com/profile\.php?z=64145)|—|—|—|—|[51st](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|186|899|[Blasius K\.](https://zwiftpower\.com/profile\.php?z=902957)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|[51st](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|187|897|[Brian Monaghan](https://zwiftpower\.com/profile\.php?z=308807)|—|—|—|—|[52nd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|187|897|[Stewart Box](https://zwiftpower\.com/profile\.php?z=342042)|—|—|—|—|—|—|—|[52nd](https://zwiftpower\.com/events\.php?zid=173451)|—
B|188|895|[James Gray](https://zwiftpower\.com/profile\.php?z=313395)|—|—|—|[53rd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|188|895|[Ole Koch](https://zwiftpower\.com/profile\.php?z=415815)|—|—|—|—|—|—|[53rd](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|188|895|[Maarten Clochet \(GuideOn Flanders\)](https://zwiftpower\.com/profile\.php?z=729901)|[1D](https://zwiftpower\.com/team\.php?id=3265)|[53rd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|189|893|[T obi\.Scheller 5944](https://zwiftpower\.com/profile\.php?z=257264)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|—|—|[54th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|190|891|[Augustin Bossart](https://zwiftpower\.com/profile\.php?z=68390)|—|—|—|—|—|—|—|[55th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|190|891|[Koen van Wieren](https://zwiftpower\.com/profile\.php?z=466388)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|[55th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|190|891|[giuseppe durante](https://zwiftpower\.com/profile\.php?z=904546)|[Team Italy](https://zwiftpower\.com/team\.php?id=588)|—|—|—|—|[55th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|191|889|[Matthew James](https://zwiftpower\.com/profile\.php?z=23924)|[RCC](https://zwiftpower\.com/team\.php?id=265)|—|—|—|[56th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|191|889|[gabriel montesi](https://zwiftpower\.com/profile\.php?z=118491)|—|—|[56th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|191|889|[Mighty Mariner \(06:30CC\)1897](https://zwiftpower\.com/profile\.php?z=876037)|—|—|—|—|—|[56th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|192|887|[Ricardo Alarcon](https://zwiftpower\.com/profile\.php?z=421155)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|—|—|—|—|[57th](https://zwiftpower\.com/events\.php?zid=176148)
B|192|887|[Peter Juel Jensen](https://zwiftpower\.com/profile\.php?z=1021739)|—|—|[57th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|193|885|[Joel hawks / Windsor Door Racing p/b Bicycles Plus](https://zwiftpower\.com/profile\.php?z=197)|[Windsor Door Rac](https://zwiftpower\.com/team\.php?id=2037)|[58th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|193|885|[chris cooksey](https://zwiftpower\.com/profile\.php?z=219752)|—|—|—|—|—|[58th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|193|885|[Roland Schleyer](https://zwiftpower\.com/profile\.php?z=421721)|[Ruhr\-Sued](https://zwiftpower\.com/team\.php?id=1912)|—|—|—|—|—|—|—|[58th](https://zwiftpower\.com/events\.php?zid=176148)
B|193|885|[André Kachlmeier \[Vegan\]](https://zwiftpower\.com/profile\.php?z=891003)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|[58th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|194|883|[Travis Pollock](https://zwiftpower\.com/profile\.php?z=64205)|—|—|—|—|—|[59th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|194|883|[Tristan Young](https://zwiftpower\.com/profile\.php?z=625599)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|—|—|—|—|—|—|—|[59th](https://zwiftpower\.com/events\.php?zid=176148)
B|194|883|[Dirk Koning](https://zwiftpower\.com/profile\.php?z=856651)|—|[59th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|194|883|[Simon Spinczyk](https://zwiftpower\.com/profile\.php?z=991216)|[Tri Geckos](https://zwiftpower\.com/team\.php?id=4073)|—|[59th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|195|881|[George York](https://zwiftpower\.com/profile\.php?z=98092)|[COS Racing](https://zwiftpower\.com/team\.php?id=86)|—|—|—|—|[60th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|195|881|[Craig McIntosh](https://zwiftpower\.com/profile\.php?z=210353)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|[60th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|195|881|[Hideaki Hara](https://zwiftpower\.com/profile\.php?z=404558)|—|—|—|—|—|—|—|[60th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|196|879|[James Richardson](https://zwiftpower\.com/profile\.php?z=441885)|[Vision](https://zwiftpower\.com/team\.php?id=9)|—|[61st](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|196|879|[Simeon Wishlade](https://zwiftpower\.com/profile\.php?z=521363)|—|—|—|—|—|—|—|—|[61st](https://zwiftpower\.com/events\.php?zid=176148)
B|196|879|[Alvin Martin1768](https://zwiftpower\.com/profile\.php?z=592927)|—|—|—|—|—|[61st](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|196|879|[Mark Boyle \(ECC\)](https://zwiftpower\.com/profile\.php?z=918500)|—|—|—|—|—|—|—|[61st](https://zwiftpower\.com/events\.php?zid=173451)|—
B|197|877|[Rob Lyons TFC B2](https://zwiftpower\.com/profile\.php?z=100630)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|[62nd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|197|877|[L\. Saga](https://zwiftpower\.com/profile\.php?z=388475)|—|—|—|—|[62nd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|197|877|[Arild Myklebust](https://zwiftpower\.com/profile\.php?z=478415)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|—|—|—|—|[62nd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|197|877|[Calle Mellergård](https://zwiftpower\.com/profile\.php?z=868092)|—|—|—|[62nd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|198|875|[Thomas Schröter \[R2\]](https://zwiftpower\.com/profile\.php?z=271773)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|—|[63rd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|199|873|[Derek Brauch](https://zwiftpower\.com/profile\.php?z=275844)|—|—|—|—|[64th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|199|873|[Scott Main \[BRT\]](https://zwiftpower\.com/profile\.php?z=1002910)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[64th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|200|871|[Derek Atkins](https://zwiftpower\.com/profile\.php?z=11615)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|[65th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|200|871|[Jonah Arnold](https://zwiftpower\.com/profile\.php?z=172379)|—|—|[65th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|200|871|[A\.sugi](https://zwiftpower\.com/profile\.php?z=278071)|—|—|—|—|—|—|—|[65th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|200|871|[Marc Rutar](https://zwiftpower\.com/profile\.php?z=420915)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|—|—|[65th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|200|871|[Nelson Guia](https://zwiftpower\.com/profile\.php?z=1091621)|—|—|—|—|—|—|—|—|[65th](https://zwiftpower\.com/events\.php?zid=176148)
B|201|869|[Sven De Schryver](https://zwiftpower\.com/profile\.php?z=177491)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|—|—|—|—|—|—|—|[66th](https://zwiftpower\.com/events\.php?zid=176148)
B|201|869|[Frank Wöhrleee](https://zwiftpower\.com/profile\.php?z=639232)|—|—|—|—|[66th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|201|869|[Trond Bjoerklund](https://zwiftpower\.com/profile\.php?z=763978)|—|—|—|—|—|—|[66th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|201|869|[John Davidson](https://zwiftpower\.com/profile\.php?z=878188)|—|—|[66th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|202|867|[Jon Carpenter](https://zwiftpower\.com/profile\.php?z=371910)|[OB's](https://zwiftpower\.com/team\.php?id=3104)|—|—|—|—|—|—|—|[67th](https://zwiftpower\.com/events\.php?zid=176148)
B|202|867|[David Santos](https://zwiftpower\.com/profile\.php?z=437626)|[TugaZ](https://zwiftpower\.com/team\.php?id=3652)|—|—|—|—|—|[67th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|202|867|[Christian Opedal](https://zwiftpower\.com/profile\.php?z=462694)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|—|—|—|[67th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|202|867|[Matt Kurton](https://zwiftpower\.com/profile\.php?z=1062872)|—|—|—|—|—|[67th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|203|865|[Simon Piniel](https://zwiftpower\.com/profile\.php?z=202301)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[68th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|203|865|[Stefan Ihlen](https://zwiftpower\.com/profile\.php?z=251644)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|—|—|—|—|—|—|—|[68th](https://zwiftpower\.com/events\.php?zid=176148)
B|203|865|[Matthias Dietrich](https://zwiftpower\.com/profile\.php?z=414645)|—|—|—|—|—|—|[68th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|203|865|[Jeff Nary](https://zwiftpower\.com/profile\.php?z=415598)|—|—|—|—|[68th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|203|865|[Jeff is a Banana 1193](https://zwiftpower\.com/profile\.php?z=1096837)|—|—|[68th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|204|863|[Jan\-Willem Scheerder](https://zwiftpower\.com/profile\.php?z=66397)|—|—|—|—|—|—|—|[69th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|204|863|[The Real Sam Woodhouse \- RideBiker/RNR/Velosport](https://zwiftpower\.com/profile\.php?z=117531)|—|—|—|[69th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|204|863|[Matthew Thomas](https://zwiftpower\.com/profile\.php?z=452396)|—|—|—|—|—|[69th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|204|863|[Lee Perkins](https://zwiftpower\.com/profile\.php?z=484667)|[ICC](https://zwiftpower\.com/team\.php?id=2494)|[69th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|204|863|[Paul Wisbey](https://zwiftpower\.com/profile\.php?z=542812)|—|—|—|—|[69th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|205|861|[Martin Leduc](https://zwiftpower\.com/profile\.php?z=4236)|—|—|—|—|—|[70th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|205|861|[Glenn Seljåsen](https://zwiftpower\.com/profile\.php?z=67228)|[Vision](https://zwiftpower\.com/team\.php?id=9)|—|—|[70th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|205|861|[Christoph Dahmen](https://zwiftpower\.com/profile\.php?z=701233)|—|—|—|—|—|—|—|[70th](https://zwiftpower\.com/events\.php?zid=173451)|—
B|206|859|[Craig Gardner](https://zwiftpower\.com/profile\.php?z=151938)|—|—|—|—|—|—|[71st](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|206|859|[Raitis Nackalns](https://zwiftpower\.com/profile\.php?z=964958)|—|—|[71st](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|207|857|[Nick Dickinson](https://zwiftpower\.com/profile\.php?z=161092)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[72nd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|207|857|[Christophe Moeri1173](https://zwiftpower\.com/profile\.php?z=1137726)|—|—|[72nd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|208|855|[huw Williams](https://zwiftpower\.com/profile\.php?z=350605)|[WECC](https://zwiftpower\.com/team\.php?id=1992)|—|[73rd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|208|855|[Julien Guyon](https://zwiftpower\.com/profile\.php?z=578354)|[Foudre ⚡️](https://zwiftpower\.com/team\.php?id=2898)|—|—|—|—|—|—|[73rd](https://zwiftpower\.com/events\.php?zid=173451)|—
B|208|855|[Leo Bandi](https://zwiftpower\.com/profile\.php?z=959474)|—|—|—|—|—|—|[73rd](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|208|855|[Nuno Felicio](https://zwiftpower\.com/profile\.php?z=1112258)|[TugaZ](https://zwiftpower\.com/team\.php?id=3652)|—|—|—|—|—|—|—|[73rd](https://zwiftpower\.com/events\.php?zid=176148)
B|209|853|[Tim Hoogstrate](https://zwiftpower\.com/profile\.php?z=68716)|[Digglers \(Doped\)](https://zwiftpower\.com/team\.php?id=4060)|[74th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|209|853|[Richard Lewey](https://zwiftpower\.com/profile\.php?z=122702)|—|—|—|—|—|—|—|—|[74th](https://zwiftpower\.com/events\.php?zid=176148)
B|209|853|[Nick Lloyd Kiss B](https://zwiftpower\.com/profile\.php?z=137357)|—|—|—|[74th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|209|853|[wim van bouwel](https://zwiftpower\.com/profile\.php?z=332102)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|—|—|—|[74th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|210|851|[Martin Diserens](https://zwiftpower\.com/profile\.php?z=116708)|[Team SUI](https://zwiftpower\.com/team\.php?id=391)|—|—|—|—|[75th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|210|851|[Felix Graf](https://zwiftpower\.com/profile\.php?z=977242)|—|—|[75th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|211|849|[Jim Downing6518](https://zwiftpower\.com/profile\.php?z=961226)|—|—|—|—|—|—|[76th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|212|847|[David Farrington](https://zwiftpower\.com/profile\.php?z=5534)|[CCC](https://zwiftpower\.com/team\.php?id=2930)|—|—|—|—|[77th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|212|847|[Andreas Spieß](https://zwiftpower\.com/profile\.php?z=371455)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|—|[77th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|212|847|[Tom Clough](https://zwiftpower\.com/profile\.php?z=394805)|—|—|—|[77th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|213|845|[Tako Bruinsma](https://zwiftpower\.com/profile\.php?z=21126)|—|—|—|—|—|—|[78th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|213|845|[Thorsten Neugebauer \[R2\]](https://zwiftpower\.com/profile\.php?z=172830)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|—|—|[78th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|213|845|[Marcin Kowalczyk](https://zwiftpower\.com/profile\.php?z=530478)|—|—|[78th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|213|845|[Jarosław Lapawa9777](https://zwiftpower\.com/profile\.php?z=908424)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|[78th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|214|843|[Ben Cox](https://zwiftpower\.com/profile\.php?z=15232)|—|—|—|—|—|—|—|—|[79th](https://zwiftpower\.com/events\.php?zid=176148)
B|214|843|[Robert Oroloko](https://zwiftpower\.com/profile\.php?z=211178)|[Oroloko Team](https://zwiftpower\.com/team\.php?id=1892)|—|—|[79th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|214|843|[Murray](https://zwiftpower\.com/profile\.php?z=487003)|—|—|[79th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|214|843|[Florian Felderer](https://zwiftpower\.com/profile\.php?z=912929)|—|[79th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|215|841|[Ryan Olszewski](https://zwiftpower\.com/profile\.php?z=95407)|—|—|—|—|—|—|[80th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
B|215|841|[Gary Fawcett](https://zwiftpower\.com/profile\.php?z=307228)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|[80th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|216|839|[Barry Dickson](https://zwiftpower\.com/profile\.php?z=3309)|—|[81st](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|216|839|[Jim Bullock](https://zwiftpower\.com/profile\.php?z=69883)|—|—|—|—|—|[81st](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|216|839|[Jon DeLacey](https://zwiftpower\.com/profile\.php?z=133933)|—|—|—|[81st](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|216|839|[Peer Schweder5020](https://zwiftpower\.com/profile\.php?z=911594)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|—|—|—|—|—|—|—|[81st](https://zwiftpower\.com/events\.php?zid=176148)
B|217|837|[Robbie Thorpe](https://zwiftpower\.com/profile\.php?z=543451)|—|—|—|—|—|—|—|—|[82nd](https://zwiftpower\.com/events\.php?zid=176148)
B|217|837|[Colin Sweeney](https://zwiftpower\.com/profile\.php?z=597696)|—|[82nd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|218|835|[Rob Keates TFC Racing](https://zwiftpower\.com/profile\.php?z=98197)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|[83rd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|218|835|[Oliver Peacock](https://zwiftpower\.com/profile\.php?z=404761)|—|—|—|—|—|—|—|—|[83rd](https://zwiftpower\.com/events\.php?zid=176148)
B|218|835|[Benedikt Mankel](https://zwiftpower\.com/profile\.php?z=968253)|—|[83rd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|219|833|[Tom Incaking](https://zwiftpower\.com/profile\.php?z=6386)|—|—|[84th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|219|833|[Lance Mitamura 5699](https://zwiftpower\.com/profile\.php?z=631643)|—|—|—|—|—|—|—|—|[84th](https://zwiftpower\.com/events\.php?zid=176148)
B|219|833|[Chris Kaye](https://zwiftpower\.com/profile\.php?z=1092286)|[ICC](https://zwiftpower\.com/team\.php?id=2494)|—|—|—|—|[84th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|220|831|[Dave Crowther](https://zwiftpower\.com/profile\.php?z=527640)|—|—|[85th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|220|831|[Marek Grzesiecki](https://zwiftpower\.com/profile\.php?z=616717)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|—|—|—|—|[85th](https://zwiftpower\.com/events\.php?zid=176148)
B|221|829|[Justin G 3905](https://zwiftpower\.com/profile\.php?z=531409)|[Team Italy](https://zwiftpower\.com/team\.php?id=588)|—|—|[86th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|221|829|[matthew lebelinski icc](https://zwiftpower\.com/profile\.php?z=791971)|[ICC](https://zwiftpower\.com/team\.php?id=2494)|—|[86th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|221|829|[Luca Cathomen1082105](https://zwiftpower\.com/profile\.php?z=1082105)|—|—|—|—|—|[86th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|221|829|[Miha Rojko \(Canyon\)](https://zwiftpower\.com/profile\.php?z=1129382)|—|—|—|—|—|—|—|—|[86th](https://zwiftpower\.com/events\.php?zid=176148)
B|222|827|[David Darley](https://zwiftpower\.com/profile\.php?z=25658)|—|—|—|—|[87th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|222|827|[Martin Browne](https://zwiftpower\.com/profile\.php?z=285984)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[87th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|222|827|[Sebastian Lopens](https://zwiftpower\.com/profile\.php?z=675173)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|—|—|[87th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|222|827|[Steve Tuck TFC](https://zwiftpower\.com/profile\.php?z=988894)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|[87th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|223|825|[DAVID PARRINGTON](https://zwiftpower\.com/profile\.php?z=95706)|—|—|—|—|[88th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|223|825|[j perry \(B\)](https://zwiftpower\.com/profile\.php?z=377081)|—|—|—|[88th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|223|825|[Max Olesen \[TMF\]](https://zwiftpower\.com/profile\.php?z=1126914)|[TMF](https://zwiftpower\.com/team\.php?id=3815)|[88th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|224|823|[Daren Chandisingh 🌱🔰](https://zwiftpower\.com/profile\.php?z=40771)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[89th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|224|823|[Stephen Williams](https://zwiftpower\.com/profile\.php?z=759123)|—|—|[89th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|225|821|[Chris Duffy](https://zwiftpower\.com/profile\.php?z=476846)|—|—|—|—|—|[90th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|225|821|[Rafał Karolewicz](https://zwiftpower\.com/profile\.php?z=1064292)|—|—|—|[90th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|225|821|[Mike T2015](https://zwiftpower\.com/profile\.php?z=1147478)|—|—|—|—|[90th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|226|819|[Yi Zhu](https://zwiftpower\.com/profile\.php?z=99960)|—|—|—|[91st](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|226|819|[Edwin Croucher TeamEN](https://zwiftpower\.com/profile\.php?z=348947)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[91st](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|226|819|[Vince Van Gestel](https://zwiftpower\.com/profile\.php?z=457942)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|—|—|—|[91st](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|227|817|[Lars Hoffmann \(KC\)](https://zwiftpower\.com/profile\.php?z=456254)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[92nd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|228|815|[Andreas Wolf](https://zwiftpower\.com/profile\.php?z=36808)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|[93rd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|228|815|[David Farrer](https://zwiftpower\.com/profile\.php?z=173551)|—|—|—|—|[93rd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|228|815|[Conor Dalby](https://zwiftpower\.com/profile\.php?z=396581)|—|—|—|—|—|[93rd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|228|815|[Macario Chen](https://zwiftpower\.com/profile\.php?z=830497)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|[93rd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
B|229|813|[Joe Allen \[BRT\]](https://zwiftpower\.com/profile\.php?z=162388)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[94th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|229|813|[Jon Steadman](https://zwiftpower\.com/profile\.php?z=366020)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|—|—|[94th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|229|813|[Adrian Kopf](https://zwiftpower\.com/profile\.php?z=559115)|—|—|[94th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|229|813|[Tim Kniep 8737](https://zwiftpower\.com/profile\.php?z=694829)|—|—|—|—|[94th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|230|811|[Simon Crerie TFC](https://zwiftpower\.com/profile\.php?z=137267)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|[95th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|230|811|[Step\_ Mey](https://zwiftpower\.com/profile\.php?z=518912)|[Foudre ⚡️](https://zwiftpower\.com/team\.php?id=2898)|—|—|—|[95th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|231|809|[Ace Thunder](https://zwiftpower\.com/profile\.php?z=56559)|—|—|—|—|—|[96th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|231|809|[C\.FightMS](https://zwiftpower\.com/profile\.php?z=58651)|[ZHR](https://zwiftpower\.com/team\.php?id=1979)|[96th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|231|809|[Robert Zatorski](https://zwiftpower\.com/profile\.php?z=593924)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|[96th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|232|807|[ben caine](https://zwiftpower\.com/profile\.php?z=44421)|—|—|—|—|—|[97th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|232|807|[Thomas Gerstmann \(KC\)](https://zwiftpower\.com/profile\.php?z=189206)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[97th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|232|807|[Steve Trout](https://zwiftpower\.com/profile\.php?z=231683)|[PTz](https://zwiftpower\.com/team\.php?id=1)|—|[97th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
B|233|805|[LOUIS\-PIERRE DUPUIS\(pjcgrch\)\(JZQc\) B 62 years](https://zwiftpower\.com/profile\.php?z=261617)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|[98th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|233|805|[Christian Lüddeckens](https://zwiftpower\.com/profile\.php?z=269482)|[338](https://zwiftpower\.com/team\.php?id=3784)|—|—|—|—|[98th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|234|803|[Mike Herle](https://zwiftpower\.com/profile\.php?z=292576)|—|—|—|—|[99th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|235|801|[Nuno Madeira](https://zwiftpower\.com/profile\.php?z=122420)|—|—|—|—|—|[100th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|235|801|[Jeff Allen](https://zwiftpower\.com/profile\.php?z=225142)|—|—|—|—|[100th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
B|236|799|[Anders Grauballe](https://zwiftpower\.com/profile\.php?z=1148907)|—|[101st](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|237|797|[Rüdiger Grosch](https://zwiftpower\.com/profile\.php?z=14064)|—|[102nd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
B|237|797|[Joost Rusche 3893](https://zwiftpower\.com/profile\.php?z=954385)|—|—|—|—|—|[102nd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
B|238|795|[Nathan Weselake](https://zwiftpower\.com/profile\.php?z=951720)|—|—|—|—|—|[103rd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|1|7446|[Igor Molino \-ITA\-\(C\)](https://zwiftpower\.com/profile\.php?z=35914)|[Team Italy](https://zwiftpower\.com/team\.php?id=588)|[36th](https://zwiftpower\.com/events\.php?zid=152778)|[52nd](https://zwiftpower\.com/events\.php?zid=155628)|[51st](https://zwiftpower\.com/events\.php?zid=159334)|[50th](https://zwiftpower\.com/events\.php?zid=162691)|[18th](https://zwiftpower\.com/events\.php?zid=166630)|[26th](https://zwiftpower\.com/events\.php?zid=171935)|[20th](https://zwiftpower\.com/events\.php?zid=173451)|[28th](https://zwiftpower\.com/events\.php?zid=176148)
C|2|6891|[Jorge Caldeira 🐹](https://zwiftpower\.com/profile\.php?z=181509)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|[🥉](https://zwiftpower\.com/events\.php?zid=155628)|[9th](https://zwiftpower\.com/events\.php?zid=159334)|[18th](https://zwiftpower\.com/events\.php?zid=162691)|[5th](https://zwiftpower\.com/events\.php?zid=166630)|[10th](https://zwiftpower\.com/events\.php?zid=171935)|[11th](https://zwiftpower\.com/events\.php?zid=173451)|[🥈](https://zwiftpower\.com/events\.php?zid=176148)
C|3|6777|[Dave Clarkson CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=393818)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[22nd](https://zwiftpower\.com/events\.php?zid=155628)|[12th](https://zwiftpower\.com/events\.php?zid=159334)|[36th](https://zwiftpower\.com/events\.php?zid=162691)|[6th](https://zwiftpower\.com/events\.php?zid=166630)|[24th](https://zwiftpower\.com/events\.php?zid=171935)|[6th](https://zwiftpower\.com/events\.php?zid=173451)|[9th](https://zwiftpower\.com/events\.php?zid=176148)
C|4|6741|[Victor Klinkvis \(vision C1\)](https://zwiftpower\.com/profile\.php?z=383195)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[11th](https://zwiftpower\.com/events\.php?zid=152778)|[16th](https://zwiftpower\.com/events\.php?zid=155628)|[26th](https://zwiftpower\.com/events\.php?zid=159334)|[16th](https://zwiftpower\.com/events\.php?zid=162691)|[26th](https://zwiftpower\.com/events\.php?zid=166630)|[16th](https://zwiftpower\.com/events\.php?zid=171935)|[22nd](https://zwiftpower\.com/events\.php?zid=173451)|—
C|5|6705|[Miguel Rola Costa](https://zwiftpower\.com/profile\.php?z=94972)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[7th](https://zwiftpower\.com/events\.php?zid=152778)|[🥈](https://zwiftpower\.com/events\.php?zid=155628)|[28th](https://zwiftpower\.com/events\.php?zid=159334)|[28th](https://zwiftpower\.com/events\.php?zid=162691)|[34th](https://zwiftpower\.com/events\.php?zid=166630)|[42nd](https://zwiftpower\.com/events\.php?zid=171935)|[10th](https://zwiftpower\.com/events\.php?zid=173451)|—
C|6|6459|[Peter Enserer\(KC\)](https://zwiftpower\.com/profile\.php?z=887372)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[33rd](https://zwiftpower\.com/events\.php?zid=152778)|[38th](https://zwiftpower\.com/events\.php?zid=155628)|[31st](https://zwiftpower\.com/events\.php?zid=159334)|[63rd](https://zwiftpower\.com/events\.php?zid=162691)|[49th](https://zwiftpower\.com/events\.php?zid=166630)|[33rd](https://zwiftpower\.com/events\.php?zid=171935)|[27th](https://zwiftpower\.com/events\.php?zid=173451)|—
C|7|6431|[Paulo Monteiro](https://zwiftpower\.com/profile\.php?z=631207)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|[45th](https://zwiftpower\.com/events\.php?zid=155628)|[49th](https://zwiftpower\.com/events\.php?zid=159334)|[52nd](https://zwiftpower\.com/events\.php?zid=162691)|[37th](https://zwiftpower\.com/events\.php?zid=166630)|[32nd](https://zwiftpower\.com/events\.php?zid=171935)|[34th](https://zwiftpower\.com/events\.php?zid=173451)|[39th](https://zwiftpower\.com/events\.php?zid=176148)
C|8|5890|[Janosch Zbick](https://zwiftpower\.com/profile\.php?z=180178)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[19th](https://zwiftpower\.com/events\.php?zid=152778)|[10th](https://zwiftpower\.com/events\.php?zid=155628)|[6th](https://zwiftpower\.com/events\.php?zid=159334)|—|[14th](https://zwiftpower\.com/events\.php?zid=166630)|[6th](https://zwiftpower\.com/events\.php?zid=171935)|—|[🥉](https://zwiftpower\.com/events\.php?zid=176148)
C|9|5836|[Robin Strong](https://zwiftpower\.com/profile\.php?z=174273)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[5th](https://zwiftpower\.com/events\.php?zid=152778)|—|[14th](https://zwiftpower\.com/events\.php?zid=159334)|[30th](https://zwiftpower\.com/events\.php?zid=162691)|[20th](https://zwiftpower\.com/events\.php?zid=166630)|—|[8th](https://zwiftpower\.com/events\.php?zid=173451)|[8th](https://zwiftpower\.com/events\.php?zid=176148)
C|10|5764|[Simon Crerie TFC](https://zwiftpower\.com/profile\.php?z=137267)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|[30th](https://zwiftpower\.com/events\.php?zid=159334)|[11th](https://zwiftpower\.com/events\.php?zid=162691)|[32nd](https://zwiftpower\.com/events\.php?zid=166630)|[13th](https://zwiftpower\.com/events\.php?zid=171935)|[13th](https://zwiftpower\.com/events\.php?zid=173451)|[22nd](https://zwiftpower\.com/events\.php?zid=176148)
C|11|5664|[Frank Zitzelsberger \(KC\)](https://zwiftpower\.com/profile\.php?z=466517)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[50th](https://zwiftpower\.com/events\.php?zid=152778)|[60th](https://zwiftpower\.com/events\.php?zid=155628)|[23rd](https://zwiftpower\.com/events\.php?zid=159334)|[23rd](https://zwiftpower\.com/events\.php?zid=162691)|[10th](https://zwiftpower\.com/events\.php?zid=166630)|—|[5th](https://zwiftpower\.com/events\.php?zid=173451)|—
C|12|5640|[Peter O'Donoghue 5083](https://zwiftpower\.com/profile\.php?z=107547)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[31st](https://zwiftpower\.com/events\.php?zid=152778)|[47th](https://zwiftpower\.com/events\.php?zid=155628)|[41st](https://zwiftpower\.com/events\.php?zid=159334)|—|[23rd](https://zwiftpower\.com/events\.php?zid=166630)|—|[12th](https://zwiftpower\.com/events\.php?zid=173451)|[29th](https://zwiftpower\.com/events\.php?zid=176148)
C|13|5450|[Sandy McMillan](https://zwiftpower\.com/profile\.php?z=987913)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[46th](https://zwiftpower\.com/events\.php?zid=155628)|[46th](https://zwiftpower\.com/events\.php?zid=159334)|[59th](https://zwiftpower\.com/events\.php?zid=162691)|—|[50th](https://zwiftpower\.com/events\.php?zid=171935)|[44th](https://zwiftpower\.com/events\.php?zid=173451)|[33rd](https://zwiftpower\.com/events\.php?zid=176148)
C|14|4817|[Mark Mortimer](https://zwiftpower\.com/profile\.php?z=73851)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|[21st](https://zwiftpower\.com/events\.php?zid=159334)|[24th](https://zwiftpower\.com/events\.php?zid=162691)|[30th](https://zwiftpower\.com/events\.php?zid=166630)|—|[7th](https://zwiftpower\.com/events\.php?zid=173451)|[12th](https://zwiftpower\.com/events\.php?zid=176148)
C|15|4787|[Jarek Slezak](https://zwiftpower\.com/profile\.php?z=466194)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|[37th](https://zwiftpower\.com/events\.php?zid=162691)|[21st](https://zwiftpower\.com/events\.php?zid=166630)|[17th](https://zwiftpower\.com/events\.php?zid=171935)|[18th](https://zwiftpower\.com/events\.php?zid=173451)|[16th](https://zwiftpower\.com/events\.php?zid=176148)
C|16|4627|[Joe Reynolds](https://zwiftpower\.com/profile\.php?z=273556)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[35th](https://zwiftpower\.com/events\.php?zid=152778)|[31st](https://zwiftpower\.com/events\.php?zid=155628)|[50th](https://zwiftpower\.com/events\.php?zid=159334)|[48th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|[25th](https://zwiftpower\.com/events\.php?zid=176148)
C|17|4613|[Shug Wilson Ⓥ\[VEGAN\]](https://zwiftpower\.com/profile\.php?z=541203)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|[35th](https://zwiftpower\.com/events\.php?zid=155628)|[47th](https://zwiftpower\.com/events\.php?zid=159334)|—|[39th](https://zwiftpower\.com/events\.php?zid=166630)|[39th](https://zwiftpower\.com/events\.php?zid=171935)|[36th](https://zwiftpower\.com/events\.php?zid=173451)|—
C|18|4463|[Dave Kerr TFC](https://zwiftpower\.com/profile\.php?z=617959)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[41st](https://zwiftpower\.com/events\.php?zid=152778)|[58th](https://zwiftpower\.com/events\.php?zid=155628)|—|[60th](https://zwiftpower\.com/events\.php?zid=162691)|[64th](https://zwiftpower\.com/events\.php?zid=166630)|—|[48th](https://zwiftpower\.com/events\.php?zid=173451)|—
C|19|3880|[Steve Fry](https://zwiftpower\.com/profile\.php?z=159322)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[22nd](https://zwiftpower\.com/events\.php?zid=152778)|—|[13th](https://zwiftpower\.com/events\.php?zid=159334)|[12th](https://zwiftpower\.com/events\.php?zid=162691)|[15th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|20|3834|[Akers Harake](https://zwiftpower\.com/profile\.php?z=585048)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=162691)|—|[22nd](https://zwiftpower\.com/events\.php?zid=171935)|[15th](https://zwiftpower\.com/events\.php?zid=173451)|[6th](https://zwiftpower\.com/events\.php?zid=176148)
C|21|3824|[Andrew Lea Watkins](https://zwiftpower\.com/profile\.php?z=255851)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|[34th](https://zwiftpower\.com/events\.php?zid=159334)|[22nd](https://zwiftpower\.com/events\.php?zid=162691)|—|[14th](https://zwiftpower\.com/events\.php?zid=171935)|—|[20th](https://zwiftpower\.com/events\.php?zid=176148)
C|22|3768|[Jamie Fox](https://zwiftpower\.com/profile\.php?z=454725)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[32nd](https://zwiftpower\.com/events\.php?zid=152778)|[30th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|[32nd](https://zwiftpower\.com/events\.php?zid=173451)|[24th](https://zwiftpower\.com/events\.php?zid=176148)
C|23|3728|[paul williams](https://zwiftpower\.com/profile\.php?z=122512)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|[44th](https://zwiftpower\.com/events\.php?zid=162691)|[43rd](https://zwiftpower\.com/events\.php?zid=166630)|—|[24th](https://zwiftpower\.com/events\.php?zid=173451)|[27th](https://zwiftpower\.com/events\.php?zid=176148)
C|24|3678|[Ole Petter Lødøen](https://zwiftpower\.com/profile\.php?z=554963)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[26th](https://zwiftpower\.com/events\.php?zid=152778)|—|[55th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[35th](https://zwiftpower\.com/events\.php?zid=171935)|[47th](https://zwiftpower\.com/events\.php?zid=173451)|—
C|25|3612|[Fabian Rothmeier](https://zwiftpower\.com/profile\.php?z=887482)|—|—|—|[61st](https://zwiftpower\.com/events\.php?zid=159334)|—|[58th](https://zwiftpower\.com/events\.php?zid=166630)|[36th](https://zwiftpower\.com/events\.php?zid=171935)|—|[41st](https://zwiftpower\.com/events\.php?zid=176148)
C|26|3588|[Tam Burns CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=17442)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|[45th](https://zwiftpower\.com/events\.php?zid=159334)|[57th](https://zwiftpower\.com/events\.php?zid=162691)|[61st](https://zwiftpower\.com/events\.php?zid=166630)|—|[45th](https://zwiftpower\.com/events\.php?zid=173451)|—
C|27|2975|[Ian Bottomore](https://zwiftpower\.com/profile\.php?z=163377)|—|[6th](https://zwiftpower\.com/events\.php?zid=152778)|[5th](https://zwiftpower\.com/events\.php?zid=155628)|[🥉](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|28|2933|[Tulipunane Radiaator](https://zwiftpower\.com/profile\.php?z=984823)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|—|—|—|[31st](https://zwiftpower\.com/events\.php?zid=162691)|[🥇](https://zwiftpower\.com/events\.php?zid=166630)|[🥉](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|29|2929|[Michael Willer](https://zwiftpower\.com/profile\.php?z=436965)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=166630)|[11th](https://zwiftpower\.com/events\.php?zid=171935)|—|[4th](https://zwiftpower\.com/events\.php?zid=176148)
C|30|2921|[Theun Bouius](https://zwiftpower\.com/profile\.php?z=700655)|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=162691)|[25th](https://zwiftpower\.com/events\.php?zid=166630)|[7th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|31|2911|[Martin Ashton CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=319164)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[17th](https://zwiftpower\.com/events\.php?zid=155628)|[4th](https://zwiftpower\.com/events\.php?zid=159334)|[25th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|32|2903|[Tristan Young](https://zwiftpower\.com/profile\.php?z=625599)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|[24th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|[23rd](https://zwiftpower\.com/events\.php?zid=171935)|[🥉](https://zwiftpower\.com/events\.php?zid=173451)|—
C|33|2897|[Matt Hilbert](https://zwiftpower\.com/profile\.php?z=946001)|—|—|[21st](https://zwiftpower\.com/events\.php?zid=155628)|[18th](https://zwiftpower\.com/events\.php?zid=159334)|[14th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|34|2895|[Matty Hermann](https://zwiftpower\.com/profile\.php?z=246215)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|[30th](https://zwiftpower\.com/events\.php?zid=173451)|[19th](https://zwiftpower\.com/events\.php?zid=176148)
C|35|2885|[Ben Gillett](https://zwiftpower\.com/profile\.php?z=315040)|—|—|[11th](https://zwiftpower\.com/events\.php?zid=155628)|[35th](https://zwiftpower\.com/events\.php?zid=159334)|[13th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|36|2873|[Sven Fahrenwald](https://zwiftpower\.com/profile\.php?z=434649)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|—|—|—|—|—|[28th](https://zwiftpower\.com/events\.php?zid=171935)|[23rd](https://zwiftpower\.com/events\.php?zid=173451)|[14th](https://zwiftpower\.com/events\.php?zid=176148)
C|37|2835|[Jurian Onderdijk](https://zwiftpower\.com/profile\.php?z=679250)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[49th](https://zwiftpower\.com/events\.php?zid=155628)|[16th](https://zwiftpower\.com/events\.php?zid=159334)|[19th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|38|2807|[Paulo Gago](https://zwiftpower\.com/profile\.php?z=418284)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|—|—|[27th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|[39th](https://zwiftpower\.com/events\.php?zid=173451)|[32nd](https://zwiftpower\.com/events\.php?zid=176148)
C|39|2797|[Rene Fuchs \(KC\)](https://zwiftpower\.com/profile\.php?z=220518)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[23rd](https://zwiftpower\.com/events\.php?zid=152778)|[37th](https://zwiftpower\.com/events\.php?zid=155628)|[43rd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|39|2797|[Greg Agran 5235](https://zwiftpower\.com/profile\.php?z=466810)|—|—|[32nd](https://zwiftpower\.com/events\.php?zid=155628)|[44th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[27th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|40|2795|[Lars Schrödel\(KC\)](https://zwiftpower\.com/profile\.php?z=946113)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|—|—|[47th](https://zwiftpower\.com/events\.php?zid=171935)|[26th](https://zwiftpower\.com/events\.php?zid=173451)|[31st](https://zwiftpower\.com/events\.php?zid=176148)
C|41|2745|[Bill Manard](https://zwiftpower\.com/profile\.php?z=7678)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[38th](https://zwiftpower\.com/events\.php?zid=152778)|—|[37th](https://zwiftpower\.com/events\.php?zid=159334)|—|[54th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|42|2727|[Eric P\. Flanderhijn](https://zwiftpower\.com/profile\.php?z=723344)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|[43rd](https://zwiftpower\.com/events\.php?zid=152778)|[54th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|[41st](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|43|1994|[Lar Hayden TFC \(C\)](https://zwiftpower\.com/profile\.php?z=763657)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[🥉](https://zwiftpower\.com/events\.php?zid=152778)|[🥇](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|44|1992|[Ryan DeadPool Miller TFC \(C\)](https://zwiftpower\.com/profile\.php?z=331384)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=176148)
C|45|1980|[Chris Moore](https://zwiftpower\.com/profile\.php?z=51627)|—|—|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=166630)|[8th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|46|1974|[Tom Lincoln](https://zwiftpower\.com/profile\.php?z=665383)|—|[12th](https://zwiftpower\.com/events\.php?zid=152778)|—|[🥈](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|47|1968|[Mark Leeming \[KRT\]](https://zwiftpower\.com/profile\.php?z=325878)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|[12th](https://zwiftpower\.com/events\.php?zid=155628)|[5th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|48|1966|[Kamil Szelag](https://zwiftpower\.com/profile\.php?z=610990)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|[11th](https://zwiftpower\.com/events\.php?zid=159334)|[7th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|49|1962|[Oliver Ennig \(KC\)](https://zwiftpower\.com/profile\.php?z=419731)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[13th](https://zwiftpower\.com/events\.php?zid=152778)|[7th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|50|1960|[Hauke Westenberg](https://zwiftpower\.com/profile\.php?z=449960)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=173451)|[17th](https://zwiftpower\.com/events\.php?zid=176148)
C|50|1960|[Micha Roth \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=800842)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[14th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|[7th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|51|1956|[Gav Holzman](https://zwiftpower\.com/profile\.php?z=767825)|—|—|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|[7th](https://zwiftpower\.com/events\.php?zid=176148)
C|52|1952|[René Rolighed](https://zwiftpower\.com/profile\.php?z=42595)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[21st](https://zwiftpower\.com/events\.php?zid=152778)|[4th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|53|1946|[Brendan Worley CRYO\-GEN C](https://zwiftpower\.com/profile\.php?z=300554)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|[15th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=176148)
C|53|1946|"[Ralph Hoffman \| Gruppo C](https://zwiftpower\.com/profile\.php?z=967255)"|—|—|—|—|[26th](https://zwiftpower\.com/events\.php?zid=162691)|[🥈](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|54|1944|[Harald Saurer \(KC\)](https://zwiftpower\.com/profile\.php?z=1048852)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[25th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|55|1942|[F\. Afonso](https://zwiftpower\.com/profile\.php?z=125584)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|—|—|[15th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|[15th](https://zwiftpower\.com/events\.php?zid=176148)
C|56|1938|[Andrew Block Ⓥ](https://zwiftpower\.com/profile\.php?z=19300)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[17th](https://zwiftpower\.com/events\.php?zid=152778)|[15th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|56|1938|[Fabrix Greg](https://zwiftpower\.com/profile\.php?z=821450)|[Team Italy](https://zwiftpower\.com/team\.php?id=588)|—|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=166630)|[19th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|57|1928|[Fabrice Lccq Ⓥ](https://zwiftpower\.com/profile\.php?z=650081)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|—|—|—|[14th](https://zwiftpower\.com/events\.php?zid=173451)|[23rd](https://zwiftpower\.com/events\.php?zid=176148)
C|58|1926|[Darren Green \(BRT\)](https://zwiftpower\.com/profile\.php?z=30856)|[BRT](https://zwiftpower\.com/team\.php?id=4)|[20th](https://zwiftpower\.com/events\.php?zid=152778)|[18th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|59|1924|[Paulo Próspero](https://zwiftpower\.com/profile\.php?z=91765)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|—|[27th](https://zwiftpower\.com/events\.php?zid=159334)|—|[12th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|60|1922|[GedR](https://zwiftpower\.com/profile\.php?z=298972)|[PTT](https://zwiftpower\.com/team\.php?id=3757)|—|—|—|[32nd](https://zwiftpower\.com/events\.php?zid=162691)|[8th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|60|1922|[Jon Wakefield](https://zwiftpower\.com/profile\.php?z=703866)|—|[18th](https://zwiftpower\.com/events\.php?zid=152778)|—|[22nd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|60|1922|[Tobias Hindemitt](https://zwiftpower\.com/profile\.php?z=1066527)|—|—|—|[19th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=173451)|—
C|61|1894|[Nuno Felicio](https://zwiftpower\.com/profile\.php?z=1112258)|[TugaZ](https://zwiftpower\.com/team\.php?id=3652)|—|—|—|—|[45th](https://zwiftpower\.com/events\.php?zid=166630)|—|[9th](https://zwiftpower\.com/events\.php?zid=173451)|—
C|62|1892|[Thomas Engel \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=520425)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|—|[26th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|[29th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|63|1878|[Waldemar Kacprzyk](https://zwiftpower\.com/profile\.php?z=169976)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|[36th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|[26th](https://zwiftpower\.com/events\.php?zid=176148)
C|64|1876|[Gonçalo Afonso](https://zwiftpower\.com/profile\.php?z=355653)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|—|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=171935)|[33rd](https://zwiftpower\.com/events\.php?zid=173451)|—
C|65|1874|[John Tetreault5620](https://zwiftpower\.com/profile\.php?z=999091)|—|[29th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|[35th](https://zwiftpower\.com/events\.php?zid=176148)
C|66|1854|[Mick Fisher](https://zwiftpower\.com/profile\.php?z=38412)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|—|—|[37th](https://zwiftpower\.com/events\.php?zid=173451)|[37th](https://zwiftpower\.com/events\.php?zid=176148)
C|66|1854|[Tyler Bradshaw2470](https://zwiftpower\.com/profile\.php?z=491267)|—|—|—|—|[54th](https://zwiftpower\.com/events\.php?zid=162691)|—|[20th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|67|1848|[Tomek Spyra](https://zwiftpower\.com/profile\.php?z=954752)|—|[39th](https://zwiftpower\.com/events\.php?zid=152778)|—|[38th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|68|1846|[shii tod](https://zwiftpower\.com/profile\.php?z=365733)|—|—|—|—|[47th](https://zwiftpower\.com/events\.php?zid=162691)|[31st](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|69|1844|[Philippe Galland](https://zwiftpower\.com/profile\.php?z=65991)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|[38th](https://zwiftpower\.com/events\.php?zid=162691)|[41st](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|70|1842|[Darren Ridley](https://zwiftpower\.com/profile\.php?z=178403)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|[51st](https://zwiftpower\.com/events\.php?zid=166630)|—|[29th](https://zwiftpower\.com/events\.php?zid=173451)|—
C|71|1838|[Jamie Baker](https://zwiftpower\.com/profile\.php?z=43788)|[None](https://zwiftpower\.com/team\.php?id=30)|—|—|—|—|—|[46th](https://zwiftpower\.com/events\.php?zid=171935)|—|[36th](https://zwiftpower\.com/events\.php?zid=176148)
C|72|1834|[Alexander Jehle \(KC\)](https://zwiftpower\.com/profile\.php?z=1093931)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|—|—|—|[46th](https://zwiftpower\.com/events\.php?zid=173451)|[38th](https://zwiftpower\.com/events\.php?zid=176148)
C|73|1826|[Jon Gaudio](https://zwiftpower\.com/profile\.php?z=153683)|[GGCC](https://zwiftpower\.com/team\.php?id=1611)|—|—|[39th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|[49th](https://zwiftpower\.com/events\.php?zid=173451)|—
C|74|1824|[Jon Hume](https://zwiftpower\.com/profile\.php?z=465110)|—|—|[40th](https://zwiftpower\.com/events\.php?zid=155628)|—|[49th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|75|1820|[Zorza Von K](https://zwiftpower\.com/profile\.php?z=231903)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|[51st](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|[40th](https://zwiftpower\.com/events\.php?zid=173451)|—
C|76|1802|[T schuckNorris](https://zwiftpower\.com/profile\.php?z=967999)|—|—|[43rd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|[57th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|77|1792|[Ann\-Katrin Kuzmenko \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=871472)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[49th](https://zwiftpower\.com/events\.php?zid=152778)|[56th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|78|1790|[josef haider \( KC \)](https://zwiftpower\.com/profile\.php?z=1094195)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|[53rd](https://zwiftpower\.com/events\.php?zid=155628)|[53rd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|79|1786|[Tom Scott](https://zwiftpower\.com/profile\.php?z=36868)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|[53rd](https://zwiftpower\.com/events\.php?zid=162691)|[55th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|80|999|[Dan Lucey TFC \(C1\)](https://zwiftpower\.com/profile\.php?z=14693)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|80|999|[Simon Russell](https://zwiftpower\.com/profile\.php?z=68077)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|80|999|[Dennis Braun](https://zwiftpower\.com/profile\.php?z=220647)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[🥇](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|80|999|[Matt Anderson](https://zwiftpower\.com/profile\.php?z=478039)|—|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|80|999|[Chris Morton](https://zwiftpower\.com/profile\.php?z=1046991)|—|—|—|—|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=173451)|—
C|81|997|[Kenny Peterson](https://zwiftpower\.com/profile\.php?z=40492)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[🥈](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|81|997|[Jimmy Tee](https://zwiftpower\.com/profile\.php?z=59364)|—|—|—|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=173451)|—
C|81|997|[jody wainwright](https://zwiftpower\.com/profile\.php?z=480927)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|81|997|[Todd Hoover](https://zwiftpower\.com/profile\.php?z=496158)|[SWOZR](https://zwiftpower\.com/team\.php?id=2971)|—|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|82|995|[Henrik Andlem Borresen](https://zwiftpower\.com/profile\.php?z=653639)|[HSIL](https://zwiftpower\.com/team\.php?id=3049)|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|83|993|[Daniel Mould](https://zwiftpower\.com/profile\.php?z=885901)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|83|993|[Allen Jones](https://zwiftpower\.com/profile\.php?z=962108)|—|[4th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|84|991|[Reg Bestwick TFC](https://zwiftpower\.com/profile\.php?z=45213)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=176148)
C|84|991|[Charlie Ryan NTR](https://zwiftpower\.com/profile\.php?z=185957)|[NTR](https://zwiftpower\.com/team\.php?id=2860)|—|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|85|989|[Olav Harkjerr Owesen](https://zwiftpower\.com/profile\.php?z=69270)|[Tunge Gir](https://zwiftpower\.com/team\.php?id=3418)|—|[6th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|85|989|[Robert Dodds CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=378471)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|86|987|[Alasdair MacMillan](https://zwiftpower\.com/profile\.php?z=522143)|—|—|—|[7th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|87|985|[Stian Horne](https://zwiftpower\.com/profile\.php?z=5322)|[RCC](https://zwiftpower\.com/team\.php?id=265)|[8th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|87|985|[philip jackson](https://zwiftpower\.com/profile\.php?z=133743)|—|—|[8th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|87|985|[Brandon Zuckerberg](https://zwiftpower\.com/profile\.php?z=445447)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|87|985|[David Cooper](https://zwiftpower\.com/profile\.php?z=449415)|[GNF](https://zwiftpower\.com/team\.php?id=2324)|—|—|[8th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|88|983|[keith palmer](https://zwiftpower\.com/profile\.php?z=57284)|[ICC](https://zwiftpower\.com/team\.php?id=2494)|[9th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|88|983|[John Spanner](https://zwiftpower\.com/profile\.php?z=227827)|[ICC](https://zwiftpower\.com/team\.php?id=2494)|—|[9th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|88|983|[Marcus Fellowes](https://zwiftpower\.com/profile\.php?z=754114)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|88|983|[Udo Möller \(KC\)](https://zwiftpower\.com/profile\.php?z=903326)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|89|981|[Pål Eddy Stokvik](https://zwiftpower\.com/profile\.php?z=45387)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|[10th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|89|981|[Aaron Cupp](https://zwiftpower\.com/profile\.php?z=402580)|—|—|—|[10th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|89|981|[J T](https://zwiftpower\.com/profile\.php?z=909782)|—|[10th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|89|981|[Sarel Boshoff Ⓥ](https://zwiftpower\.com/profile\.php?z=1128952)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|—|—|—|—|[10th](https://zwiftpower\.com/events\.php?zid=176148)
C|90|979|[kenneth skoglund](https://zwiftpower\.com/profile\.php?z=174307)|—|—|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|90|979|[Brant Law3193](https://zwiftpower\.com/profile\.php?z=897994)|[Team AHDR](https://zwiftpower\.com/team\.php?id=629)|—|—|—|—|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=176148)
C|91|977|[Thomas Reinecke](https://zwiftpower\.com/profile\.php?z=209977)|[RuhrRiders](https://zwiftpower\.com/team\.php?id=41)|—|—|—|—|—|[12th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|92|975|[Ted Kupsick6136](https://zwiftpower\.com/profile\.php?z=214086)|[USMES](https://zwiftpower\.com/team\.php?id=79)|—|[13th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|93|973|[Stuart Davidson](https://zwiftpower\.com/profile\.php?z=197270)|[Myerscough](https://zwiftpower\.com/team\.php?id=2401)|—|[14th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|94|971|[Iain Morrison](https://zwiftpower\.com/profile\.php?z=320606)|—|—|—|—|—|—|[15th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|94|971|[Michael Fischbach \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=859636)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|[15th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|95|969|[Andreas Phillipsen](https://zwiftpower\.com/profile\.php?z=151939)|—|[16th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|95|969|[83Graus](https://zwiftpower\.com/profile\.php?z=1117159)|—|—|—|—|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=173451)|—
C|96|967|[Daniel Gassmann](https://zwiftpower\.com/profile\.php?z=57897)|—|—|—|[17th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|96|967|[Rui Inácio](https://zwiftpower\.com/profile\.php?z=196516)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|—|—|—|—|—|[17th](https://zwiftpower\.com/events\.php?zid=173451)|—
C|96|967|[Ben Hrycej](https://zwiftpower\.com/profile\.php?z=578319)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|—|[17th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|96|967|[Luke Huebschen 2173](https://zwiftpower\.com/profile\.php?z=1033903)|—|—|—|—|[17th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|97|965|[Chris Johnson Mapperley CC](https://zwiftpower\.com/profile\.php?z=35931)|—|—|—|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|97|965|[Vitor Pires \(Carecas Do Pedal/ATX CYCLINGSTORE\)](https://zwiftpower\.com/profile\.php?z=76624)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|—|—|—|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=176148)
C|98|963|[Laurens Van Meeteren](https://zwiftpower\.com/profile\.php?z=298701)|[HIK](https://zwiftpower\.com/team\.php?id=719)|—|—|—|—|—|—|[19th](https://zwiftpower\.com/events\.php?zid=173451)|—
C|98|963|[Lee Millward](https://zwiftpower\.com/profile\.php?z=299923)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|[19th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|98|963|[Simon Morgan](https://zwiftpower\.com/profile\.php?z=865457)|—|—|[19th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|99|961|[Skyler Street](https://zwiftpower\.com/profile\.php?z=120373)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|[20th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|99|961|[Ove Wadman SZR](https://zwiftpower\.com/profile\.php?z=840538)|—|—|—|[20th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|99|961|[Stuart Martin](https://zwiftpower\.com/profile\.php?z=1046694)|—|—|—|—|[20th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|100|959|[Ken Olav Jenssen](https://zwiftpower\.com/profile\.php?z=87153)|—|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|100|959|[Fredrik Larsen](https://zwiftpower\.com/profile\.php?z=253013)|—|—|—|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|100|959|[Michiel Kersten](https://zwiftpower\.com/profile\.php?z=425370)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=176148)
C|101|955|[Mathew Jones](https://zwiftpower\.com/profile\.php?z=635949)|—|—|[23rd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|102|953|[James Fowler](https://zwiftpower\.com/profile\.php?z=237877)|[FinishStrong](https://zwiftpower\.com/team\.php?id=1740)|—|—|[24th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|102|953|[Sebastian Slotwinski \[DIRT\]](https://zwiftpower\.com/profile\.php?z=251729)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|[24th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|102|953|[Nathan Beauchamp1422](https://zwiftpower\.com/profile\.php?z=1105977)|—|—|—|—|—|[24th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|103|951|[Mike Taylor](https://zwiftpower\.com/profile\.php?z=369058)|—|—|—|—|—|—|[25th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|103|951|[Ryan Hawkins](https://zwiftpower\.com/profile\.php?z=581316)|—|—|[25th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|103|951|[Anders Nilsen](https://zwiftpower\.com/profile\.php?z=836490)|—|—|—|—|—|—|—|[25th](https://zwiftpower\.com/events\.php?zid=173451)|—
C|103|951|[Michael walsh](https://zwiftpower\.com/profile\.php?z=1063692)|—|—|—|[25th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|104|947|[David George](https://zwiftpower\.com/profile\.php?z=215470)|[Slipstreamers](https://zwiftpower\.com/team\.php?id=2360)|—|—|—|—|[27th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|104|947|[Ben Pentland](https://zwiftpower\.com/profile\.php?z=216403)|—|—|[27th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|104|947|[Reiner Heming ZRG\-R](https://zwiftpower\.com/profile\.php?z=800245)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|[27th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|105|945|[Edwin M](https://zwiftpower\.com/profile\.php?z=254175)|—|—|[28th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|105|945|[Marc\-Olivier Jacques](https://zwiftpower\.com/profile\.php?z=450022)|—|—|—|—|—|—|—|[28th](https://zwiftpower\.com/events\.php?zid=173451)|—
C|105|945|[Anthony Walstow](https://zwiftpower\.com/profile\.php?z=464979)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|[28th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|105|945|[William Turner7233](https://zwiftpower\.com/profile\.php?z=727174)|—|[28th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|106|943|[Alan Clarke Kiss C](https://zwiftpower\.com/profile\.php?z=77260)|—|—|—|—|[29th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|106|943|[Jim Rauser TFC \(C\)](https://zwiftpower\.com/profile\.php?z=533103)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|[29th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|106|943|[Andy Webb 5012](https://zwiftpower\.com/profile\.php?z=849616)|—|—|—|—|—|[29th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|106|943|[Piotr Kaliski](https://zwiftpower\.com/profile\.php?z=850113)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|[29th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|107|941|[New Scot](https://zwiftpower\.com/profile\.php?z=126700)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[30th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|107|941|[pato briggs](https://zwiftpower\.com/profile\.php?z=360770)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|—|—|—|—|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=176148)
C|108|939|[James Wyatt](https://zwiftpower\.com/profile\.php?z=525033)|—|—|—|—|—|—|—|[31st](https://zwiftpower\.com/events\.php?zid=173451)|—
C|108|939|[Aart Valkhof](https://zwiftpower\.com/profile\.php?z=1063943)|[SAS Cycle Club](https://zwiftpower\.com/team\.php?id=1348)|—|—|—|—|—|[31st](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|109|937|[mark stringfellow](https://zwiftpower\.com/profile\.php?z=232215)|—|—|—|[32nd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|110|935|[Perrie Higgs](https://zwiftpower\.com/profile\.php?z=43047)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|—|—|—|—|[33rd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|110|935|[David 8656](https://zwiftpower\.com/profile\.php?z=86146)|[ICC](https://zwiftpower\.com/team\.php?id=2494)|—|[33rd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|110|935|[ben brook](https://zwiftpower\.com/profile\.php?z=412366)|—|—|—|[33rd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|110|935|[Martijn Visser](https://zwiftpower\.com/profile\.php?z=947859)|—|—|—|—|[33rd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|111|933|[Nuno Madeira](https://zwiftpower\.com/profile\.php?z=122420)|—|—|—|—|—|—|—|—|[34th](https://zwiftpower\.com/events\.php?zid=176148)
C|111|933|[Roy Kämpe 7552](https://zwiftpower\.com/profile\.php?z=175788)|—|[34th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|111|933|[Jason Thompson](https://zwiftpower\.com/profile\.php?z=838875)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|[34th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|111|933|[Steve Tuck TFC](https://zwiftpower\.com/profile\.php?z=988894)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|[34th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|111|933|[Alan Aslin](https://zwiftpower\.com/profile\.php?z=1105559)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|—|[34th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|112|931|[Ro Ka](https://zwiftpower\.com/profile\.php?z=74037)|[G\-T](https://zwiftpower\.com/team\.php?id=3790)|—|—|—|—|[35th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|112|931|[Paul Paetsch](https://zwiftpower\.com/profile\.php?z=911884)|—|—|—|—|—|—|—|[35th](https://zwiftpower\.com/events\.php?zid=173451)|—
C|112|931|[Ricardo Prazeres](https://zwiftpower\.com/profile\.php?z=1094852)|—|—|—|—|[35th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|113|929|[Jørn Peder Jørgensen DBR](https://zwiftpower\.com/profile\.php?z=59677)|[DBR](https://zwiftpower\.com/team\.php?id=1986)|—|[36th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|113|929|[M Barker](https://zwiftpower\.com/profile\.php?z=1144733)|[Ride WLF](https://zwiftpower\.com/team\.php?id=2257)|—|—|—|—|[36th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|114|927|[Manuel Rocha](https://zwiftpower\.com/profile\.php?z=70647)|[eRoberto](https://zwiftpower\.com/team\.php?id=3470)|[37th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|114|927|[Maj C](https://zwiftpower\.com/profile\.php?z=918062)|[TRON](https://zwiftpower\.com/team\.php?id=3861)|—|—|—|—|—|[37th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|115|925|[Jim Nelson 9806](https://zwiftpower\.com/profile\.php?z=372064)|—|—|—|—|—|—|—|[38th](https://zwiftpower\.com/events\.php?zid=173451)|—
C|115|925|[Josselin Girault](https://zwiftpower\.com/profile\.php?z=378566)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|[38th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|115|925|[Victor Moldoveanu](https://zwiftpower\.com/profile\.php?z=824632)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|—|—|—|—|—|[38th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|116|923|[Oliver Loehlein](https://zwiftpower\.com/profile\.php?z=185005)|[MTCT](https://zwiftpower\.com/team\.php?id=120)|—|—|—|[39th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|116|923|[BJ Alcantara](https://zwiftpower\.com/profile\.php?z=458847)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|[39th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|117|921|"[Erik van Kempen \| Trailrunners Brabant](https://zwiftpower\.com/profile\.php?z=829565)"|—|—|—|—|—|—|—|—|[40th](https://zwiftpower\.com/events\.php?zid=176148)
C|117|921|[carrie hennessy](https://zwiftpower\.com/profile\.php?z=947459)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|[40th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|117|921|[Lars Magnussen](https://zwiftpower\.com/profile\.php?z=976559)|—|—|—|[40th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|117|921|[Ian Jervis5254](https://zwiftpower\.com/profile\.php?z=980837)|—|—|—|—|—|—|[40th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|117|921|[Joel Honeycutt](https://zwiftpower\.com/profile\.php?z=1042921)|—|[40th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|117|921|[William Kletzien](https://zwiftpower\.com/profile\.php?z=1055895)|—|—|—|—|—|[40th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|118|919|[Simon Jackson](https://zwiftpower\.com/profile\.php?z=11130)|—|—|[41st](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|118|919|[David Robinson](https://zwiftpower\.com/profile\.php?z=230951)|—|—|—|—|[41st](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|118|919|[Mr Marcin](https://zwiftpower\.com/profile\.php?z=378083)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|—|—|—|[41st](https://zwiftpower\.com/events\.php?zid=173451)|—
C|119|917|[Suvir Venkataraman](https://zwiftpower\.com/profile\.php?z=487232)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|—|—|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|119|917|[Jonathan Wyszo](https://zwiftpower\.com/profile\.php?z=496663)|—|—|—|—|—|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=173451)|—
C|119|917|[Ben DeCoste](https://zwiftpower\.com/profile\.php?z=644101)|—|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|119|917|[Krzysztof Kalek](https://zwiftpower\.com/profile\.php?z=894350)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[42nd](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|119|917|[Joey McGuire](https://zwiftpower\.com/profile\.php?z=980538)|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|119|917|[Eknab Rabbit](https://zwiftpower\.com/profile\.php?z=1148807)|—|—|—|—|—|—|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=176148)
C|120|915|[Ryan Blackmon](https://zwiftpower\.com/profile\.php?z=140272)|—|—|—|—|—|—|—|[43rd](https://zwiftpower\.com/events\.php?zid=173451)|—
C|120|915|[Christian Pirker \(KC\)](https://zwiftpower\.com/profile\.php?z=304918)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|[43rd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|120|915|[Andreas Eriksson](https://zwiftpower\.com/profile\.php?z=874457)|—|—|—|—|—|—|[43rd](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|120|915|[Tomasz Chojnowski](https://zwiftpower\.com/profile\.php?z=933278)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|—|—|—|—|[43rd](https://zwiftpower\.com/events\.php?zid=176148)
C|121|913|[Alex Martin\-Verdinos](https://zwiftpower\.com/profile\.php?z=86125)|—|—|—|—|—|[44th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|121|913|[David Ashton \(Rowe & King\)](https://zwiftpower\.com/profile\.php?z=446846)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|—|—|—|—|—|—|—|[44th](https://zwiftpower\.com/events\.php?zid=176148)
C|121|913|[Dominic Pridham](https://zwiftpower\.com/profile\.php?z=855322)|—|[44th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|121|913|[Petr Proschl4603](https://zwiftpower\.com/profile\.php?z=975076)|—|—|[44th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|121|913|[Nic Laughton](https://zwiftpower\.com/profile\.php?z=1038438)|—|—|—|—|—|—|[44th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|122|911|[Dominique Louis G\-T](https://zwiftpower\.com/profile\.php?z=15213)|[G\-T](https://zwiftpower\.com/team\.php?id=3790)|[45th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|122|911|[Steve Holmes 2202](https://zwiftpower\.com/profile\.php?z=101524)|—|—|—|—|—|—|—|—|[45th](https://zwiftpower\.com/events\.php?zid=176148)
C|122|911|[Nate Leader](https://zwiftpower\.com/profile\.php?z=836977)|—|—|—|—|[45th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|122|911|[Lars Amelang](https://zwiftpower\.com/profile\.php?z=1080643)|—|—|—|—|—|—|[45th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|123|909|[B Graham](https://zwiftpower\.com/profile\.php?z=58369)|—|—|—|—|—|—|—|—|[46th](https://zwiftpower\.com/events\.php?zid=176148)
C|123|909|[Scott Fenwick \[G\_T\] KISS \(C\)](https://zwiftpower\.com/profile\.php?z=777834)|[G\-T](https://zwiftpower\.com/team\.php?id=3790)|[46th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|123|909|[Richard Ham](https://zwiftpower\.com/profile\.php?z=948394)|—|—|—|—|[46th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|123|909|[Barry Mark](https://zwiftpower\.com/profile\.php?z=971776)|—|—|—|—|—|[46th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|124|907|[Andrew BREEZE \(C\)](https://zwiftpower\.com/profile\.php?z=549813)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|—|—|—|[47th](https://zwiftpower\.com/events\.php?zid=176148)
C|124|907|[John Swaine](https://zwiftpower\.com/profile\.php?z=556486)|[MRSA](https://zwiftpower\.com/team\.php?id=2204)|—|—|—|—|[47th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|124|907|[Steve Stahl](https://zwiftpower\.com/profile\.php?z=729848)|—|[47th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|125|905|[Ditlev Friis](https://zwiftpower\.com/profile\.php?z=12422)|—|—|—|—|—|—|[48th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|125|905|[Joao Cravo](https://zwiftpower\.com/profile\.php?z=160528)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|—|—|—|—|—|—|[48th](https://zwiftpower\.com/events\.php?zid=176148)
C|125|905|[Adam Lewis \(C\) 5075](https://zwiftpower\.com/profile\.php?z=288749)|—|[48th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
C|125|905|[Keith Mitchell](https://zwiftpower\.com/profile\.php?z=414864)|[The Saints](https://zwiftpower\.com/team\.php?id=3086)|—|—|[48th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|125|905|[Matt Packer](https://zwiftpower\.com/profile\.php?z=471752)|—|—|—|—|—|[48th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|125|905|[Brooke Gibby](https://zwiftpower\.com/profile\.php?z=494306)|—|—|[48th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|126|903|[Erik van Beusekom HIK](https://zwiftpower\.com/profile\.php?z=254265)|[HIK](https://zwiftpower\.com/team\.php?id=719)|—|—|—|—|—|—|—|[49th](https://zwiftpower\.com/events\.php?zid=176148)
C|126|903|[victor choi](https://zwiftpower\.com/profile\.php?z=1102973)|—|—|—|—|—|—|[49th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
C|127|901|[Maria Gydesen](https://zwiftpower\.com/profile\.php?z=176130)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|—|—|—|—|—|—|[50th](https://zwiftpower\.com/events\.php?zid=173451)|—
C|127|901|[Nicky Moir9054](https://zwiftpower\.com/profile\.php?z=896045)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|—|—|—|—|[50th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|127|901|[Achim Neumann \(KC\)](https://zwiftpower\.com/profile\.php?z=1055544)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|[50th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|128|899|[Yves Gagnon 8604](https://zwiftpower\.com/profile\.php?z=468035)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|[51st](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|129|897|[Jezz Ellis](https://zwiftpower\.com/profile\.php?z=10560)|—|—|—|—|—|[52nd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|129|897|[Bruno Gonz Jones](https://zwiftpower\.com/profile\.php?z=1056617)|—|—|—|[52nd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|130|895|[Matt Deans](https://zwiftpower\.com/profile\.php?z=507330)|—|—|—|—|—|[53rd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|131|893|[Daniel Qr](https://zwiftpower\.com/profile\.php?z=984997)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|[54th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|132|891|[Ronald Couture](https://zwiftpower\.com/profile\.php?z=932591)|—|—|[55th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|132|891|[Tom Lishman\-Walker \[DONNYONTWITCH\]](https://zwiftpower\.com/profile\.php?z=1167472)|—|—|—|—|[55th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|133|889|[john evans](https://zwiftpower\.com/profile\.php?z=243273)|[Team CLS](https://zwiftpower\.com/team\.php?id=1426)|—|—|[56th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|133|889|[Ruben Rocha](https://zwiftpower\.com/profile\.php?z=325023)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|—|—|—|[56th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|133|889|[Tom Freitag](https://zwiftpower\.com/profile\.php?z=425174)|—|—|—|—|[56th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|134|887|[Farrell Kilbane \(GKV\)](https://zwiftpower\.com/profile\.php?z=275412)|[WATTBIKE](https://zwiftpower\.com/team\.php?id=2738)|—|—|[57th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|134|887|[Brendan Howard](https://zwiftpower\.com/profile\.php?z=972696)|—|—|[57th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|135|885|[Vin L ⚡](https://zwiftpower\.com/profile\.php?z=868022)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|[58th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|135|885|[David Macpherson](https://zwiftpower\.com/profile\.php?z=1161611)|—|—|—|[58th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|136|883|[Wade Wilson](https://zwiftpower\.com/profile\.php?z=84455)|—|—|[59th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
C|136|883|[Lukasz Zielinski](https://zwiftpower\.com/profile\.php?z=86725)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|[59th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|136|883|[David LE GONIDEC](https://zwiftpower\.com/profile\.php?z=807025)|[Team France](https://zwiftpower\.com/team\.php?id=574)|—|—|—|—|[59th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|137|881|[Francois Sammut](https://zwiftpower\.com/profile\.php?z=108090)|—|—|—|[60th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|137|881|[raspin raspin](https://zwiftpower\.com/profile\.php?z=879744)|—|—|—|—|—|[60th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|138|879|[Fredrik Söderbäck](https://zwiftpower\.com/profile\.php?z=1127377)|—|—|—|—|[61st](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|139|877|[Oliver Kraft \(ZRG\)](https://zwiftpower\.com/profile\.php?z=593461)|—|—|—|—|[62nd](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|139|877|[Krzysiek Cierniak](https://zwiftpower\.com/profile\.php?z=627395)|—|—|—|[62nd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|139|877|[Joshua Gross](https://zwiftpower\.com/profile\.php?z=1107507)|—|—|—|—|—|[62nd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|140|875|[Paige Haydon Lutz \(CRYO\-GEN\)](https://zwiftpower\.com/profile\.php?z=311565)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|[63rd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
C|140|875|[Frank Schaudig](https://zwiftpower\.com/profile\.php?z=447596)|—|—|—|[63rd](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|141|873|[Ralf D\.](https://zwiftpower\.com/profile\.php?z=315260)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|—|—|[64th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
C|141|873|[Frank Göldner](https://zwiftpower\.com/profile\.php?z=535460)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|—|—|—|[64th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
C|142|871|[Hans VEGAN](https://zwiftpower\.com/profile\.php?z=422779)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|[65th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
D|1|6949|[Marcus Pfurtscheller](https://zwiftpower\.com/profile\.php?z=1156098)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|[🥈](https://zwiftpower\.com/events\.php?zid=155628)|[5th](https://zwiftpower\.com/events\.php?zid=159334)|[🥇](https://zwiftpower\.com/events\.php?zid=162691)|[9th](https://zwiftpower\.com/events\.php?zid=166630)|[🥉](https://zwiftpower\.com/events\.php?zid=171935)|[6th](https://zwiftpower\.com/events\.php?zid=173451)|[🥉](https://zwiftpower\.com/events\.php?zid=176148)
D|2|4923|[Jim Peyton](https://zwiftpower\.com/profile\.php?z=485205)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|[7th](https://zwiftpower\.com/events\.php?zid=162691)|[14th](https://zwiftpower\.com/events\.php?zid=166630)|[4th](https://zwiftpower\.com/events\.php?zid=171935)|[8th](https://zwiftpower\.com/events\.php?zid=173451)|[8th](https://zwiftpower\.com/events\.php?zid=176148)
D|3|3938|[David Ashton \(Rowe & King\)](https://zwiftpower\.com/profile\.php?z=446846)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|—|[6th](https://zwiftpower\.com/events\.php?zid=155628)|—|[10th](https://zwiftpower\.com/events\.php?zid=162691)|—|[6th](https://zwiftpower\.com/events\.php?zid=171935)|[11th](https://zwiftpower\.com/events\.php?zid=173451)|—
D|4|3934|[🛴BJ AFONSO🚲](https://zwiftpower\.com/profile\.php?z=11775)|[PTz](https://zwiftpower\.com/team\.php?id=1)|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=162691)|[15th](https://zwiftpower\.com/events\.php?zid=166630)|—|[10th](https://zwiftpower\.com/events\.php?zid=173451)|[4th](https://zwiftpower\.com/events\.php?zid=176148)
D|5|3912|[Alex Cutmore \[TPH\]](https://zwiftpower\.com/profile\.php?z=1155359)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|[9th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|[12th](https://zwiftpower\.com/events\.php?zid=162691)|—|[15th](https://zwiftpower\.com/events\.php?zid=171935)|—|[10th](https://zwiftpower\.com/events\.php?zid=176148)
D|6|2993|[Victor Moldoveanu](https://zwiftpower\.com/profile\.php?z=824632)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|—|[🥇](https://zwiftpower\.com/events\.php?zid=155628)|[🥈](https://zwiftpower\.com/events\.php?zid=159334)|[🥈](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
D|7|2985|[Joao Cravo](https://zwiftpower\.com/profile\.php?z=160528)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|[🥉](https://zwiftpower\.com/events\.php?zid=155628)|—|—|[5th](https://zwiftpower\.com/events\.php?zid=166630)|—|[🥇](https://zwiftpower\.com/events\.php?zid=173451)|—
D|8|2965|[Oldstuntmonkey MTB Some Thieving Bastards Stole My Bikes](https://zwiftpower\.com/profile\.php?z=292646)|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|[11th](https://zwiftpower\.com/events\.php?zid=171935)|[4th](https://zwiftpower\.com/events\.php?zid=173451)|—
D|9|2905|[Alex Diaz](https://zwiftpower\.com/profile\.php?z=408631)|—|—|—|[10th](https://zwiftpower\.com/events\.php?zid=159334)|—|[23rd](https://zwiftpower\.com/events\.php?zid=166630)|[16th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
D|10|1996|[Andy Beveridge TFC](https://zwiftpower\.com/profile\.php?z=51283)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=173451)|[🥇](https://zwiftpower\.com/events\.php?zid=176148)
D|11|1990|[Gonçalo Afonso](https://zwiftpower\.com/profile\.php?z=355653)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=162691)|[🥈](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
D|12|1986|[dave boudreau](https://zwiftpower\.com/profile\.php?z=14667)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=173451)|—
D|13|1984|[Oli Masters](https://zwiftpower\.com/profile\.php?z=948112)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|[7th](https://zwiftpower\.com/events\.php?zid=166630)|[🥈](https://zwiftpower\.com/events\.php?zid=171935)|—|—
D|14|1980|[Jeff\.Edwards\.](https://zwiftpower\.com/profile\.php?z=1450)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[5th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
D|15|1976|[Lee Barber](https://zwiftpower\.com/profile\.php?z=250163)|—|[6th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|[7th](https://zwiftpower\.com/events\.php?zid=176148)
D|15|1976|[Mike Louie](https://zwiftpower\.com/profile\.php?z=261230)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|—|[5th](https://zwiftpower\.com/events\.php?zid=155628)|[8th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
D|16|1972|[Rob Jessen CRYO\-GEN \(D1\)](https://zwiftpower\.com/profile\.php?z=332069)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=166630)|—|[7th](https://zwiftpower\.com/events\.php?zid=173451)|—
D|17|1970|[Al Mac GTC](https://zwiftpower\.com/profile\.php?z=973293)|[GTC](https://zwiftpower\.com/team\.php?id=3736)|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=162691)|—|[7th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
D|18|1964|[Damon McKay](https://zwiftpower\.com/profile\.php?z=3508)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=166630)|[🥇](https://zwiftpower\.com/events\.php?zid=171935)|—|—
D|19|1962|[Ann\-Katrin Kuzmenko \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=871472)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|—|—|—|—|[12th](https://zwiftpower\.com/events\.php?zid=166630)|[8th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
D|20|1948|[Rich Collins](https://zwiftpower\.com/profile\.php?z=558894)|—|[7th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|[20th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
D|21|1946|[Wolfgang Becker KC](https://zwiftpower\.com/profile\.php?z=379505)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=162691)|[17th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
D|22|999|[Zorza Von K](https://zwiftpower\.com/profile\.php?z=231903)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
D|22|999|[Paulo Monteiro](https://zwiftpower\.com/profile\.php?z=631207)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|[🥇](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
D|22|999|[Stephen Hunt](https://zwiftpower\.com/profile\.php?z=863229)|—|—|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
D|23|997|[Lee Robinson TFC](https://zwiftpower\.com/profile\.php?z=130504)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=176148)
D|23|997|[Petr Proschl4603](https://zwiftpower\.com/profile\.php?z=975076)|—|[🥈](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
D|24|995|[Martin Björgvik 🐌](https://zwiftpower\.com/profile\.php?z=15197)|[Sz](https://zwiftpower\.com/team\.php?id=32)|—|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
D|24|995|[Tom Atkinson](https://zwiftpower\.com/profile\.php?z=337225)|—|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
D|24|995|[Ole\-Martin Andersen](https://zwiftpower\.com/profile\.php?z=560581)|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
D|24|995|[Marek Milkamanowicz](https://zwiftpower\.com/profile\.php?z=889003)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[🥉](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
D|25|993|[Ralf D\.](https://zwiftpower\.com/profile\.php?z=315260)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|—|[4th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
D|25|993|[Achim Neumann \(KC\)](https://zwiftpower\.com/profile\.php?z=1055544)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|[4th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
D|25|993|[Jonas Bo Madsen \[DBR\]](https://zwiftpower\.com/profile\.php?z=1109805)|[DBR](https://zwiftpower\.com/team\.php?id=1986)|—|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
D|26|991|[Peter Walsh](https://zwiftpower\.com/profile\.php?z=136151)|—|—|—|—|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=176148)
D|26|991|[David Jones](https://zwiftpower\.com/profile\.php?z=858171)|—|—|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
D|26|991|[Marty Anfinsen](https://zwiftpower\.com/profile\.php?z=1137448)|—|—|—|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=173451)|—
D|27|989|[Richard McCulloch](https://zwiftpower\.com/profile\.php?z=50300)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=176148)
D|27|989|[$nail W](https://zwiftpower\.com/profile\.php?z=250028)|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
D|28|987|[Aaron Burns](https://zwiftpower\.com/profile\.php?z=738877)|[The Saints](https://zwiftpower\.com/team\.php?id=3086)|—|—|[7th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
D|28|987|[Eric Nadin](https://zwiftpower\.com/profile\.php?z=928144)|—|—|[7th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
D|29|985|[Xavier Laroche](https://zwiftpower\.com/profile\.php?z=427658)|[Equipe JZQc](https://zwiftpower\.com/team\.php?id=2571)|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
D|29|985|[Chris Blount](https://zwiftpower\.com/profile\.php?z=583514)|—|[8th](https://zwiftpower\.com/events\.php?zid=152778)|—|—|—|—|—|—|—
D|29|985|[CJ Spencer 2511](https://zwiftpower\.com/profile\.php?z=886960)|—|—|[8th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
D|30|983|[David Lopez](https://zwiftpower\.com/profile\.php?z=402705)|—|—|—|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=173451)|—
D|30|983|[Magnus O AHLSEN Ahlsé](https://zwiftpower\.com/profile\.php?z=417135)|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
D|30|983|[Joseph Padilla \[HERD\]](https://zwiftpower\.com/profile\.php?z=923771)|[HERD](https://zwiftpower\.com/team\.php?id=2639)|—|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
D|30|983|[John Brice](https://zwiftpower\.com/profile\.php?z=995905)|[PACK](https://zwiftpower\.com/team\.php?id=2928)|—|—|—|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=176148)
D|30|983|[Marlene Ott \[BMPRO\]](https://zwiftpower\.com/profile\.php?z=1006971)|[BMPRO](https://zwiftpower\.com/team\.php?id=3343)|—|[9th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
D|31|981|[Chris Taylor](https://zwiftpower\.com/profile\.php?z=407450)|—|—|[10th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
D|31|981|[Nigel Savage](https://zwiftpower\.com/profile\.php?z=953423)|[SAS Cycle Club](https://zwiftpower\.com/team\.php?id=1348)|—|—|—|—|[10th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
D|31|981|[L Young](https://zwiftpower\.com/profile\.php?z=973891)|—|—|—|—|—|—|[10th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
D|32|979|[Zoltán Fazekas](https://zwiftpower\.com/profile\.php?z=450622)|—|—|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
D|32|979|[Anthony Darragh](https://zwiftpower\.com/profile\.php?z=933793)|—|—|—|—|—|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=176148)
D|32|979|[John Qertz 7842](https://zwiftpower\.com/profile\.php?z=1023771)|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=159334)|—|—|—|—|—
D|32|979|[Adam Herring](https://zwiftpower\.com/profile\.php?z=1095145)|—|—|[11th](https://zwiftpower\.com/events\.php?zid=155628)|—|—|—|—|—|—
D|33|977|[steve davis 8027](https://zwiftpower\.com/profile\.php?z=237249)|[SGCC](https://zwiftpower\.com/team\.php?id=3149)|—|—|—|—|—|[12th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
D|33|977|[Blair Cottreau](https://zwiftpower\.com/profile\.php?z=1044214)|—|—|—|—|—|—|—|[12th](https://zwiftpower\.com/events\.php?zid=173451)|—
D|34|975|[Adam Schwartz2600](https://zwiftpower\.com/profile\.php?z=2398)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=162691)|—|—|—|—
D|34|975|[Dário Ferreira \[TugaZ\]](https://zwiftpower\.com/profile\.php?z=910139)|—|—|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
D|34|975|[Marty Doscher](https://zwiftpower\.com/profile\.php?z=927833)|—|—|—|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
D|35|973|[Achim Butschek \(KC\)](https://zwiftpower\.com/profile\.php?z=80942)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|—|—|[14th](https://zwiftpower\.com/events\.php?zid=171935)|—|—
D|36|969|[Hein Wick](https://zwiftpower\.com/profile\.php?z=51696)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|—|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
D|37|963|[David Lutz \(CRYO\-GEN\)](https://zwiftpower\.com/profile\.php?z=410620)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|[19th](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
D|38|959|[Miguel Remízio \[TugaZ\]](https://zwiftpower\.com/profile\.php?z=1119738)|[TugaZRT](https://zwiftpower\.com/team\.php?id=2466)|—|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—
D|39|957|[Patrick D\.](https://zwiftpower\.com/profile\.php?z=372550)|—|—|—|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=166630)|—|—|—



## KISS Community League — Europe Women

Category|Rank|Total Points|Name|Team|1|2|3|4|5|6|7|8
---|---|---|---|---|---|---|---|---|---|---|---|---
A|1|7968|[Cecilia Hansen👽🔰](https://zwiftpower\.com/profile\.php?z=6306)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[🥉](https://zwiftpower\.com/events\.php?zid=154308)|[🥈](https://zwiftpower\.com/events\.php?zid=155629)|[🥉](https://zwiftpower\.com/events\.php?zid=159335)|[🥇](https://zwiftpower\.com/events\.php?zid=162692)|[🥉](https://zwiftpower\.com/events\.php?zid=166629)|[🥈](https://zwiftpower\.com/events\.php?zid=171936)|[4th](https://zwiftpower\.com/events\.php?zid=173447)|[🥈](https://zwiftpower\.com/events\.php?zid=176149)
A|2|7918|[Kristin Falck](https://zwiftpower\.com/profile\.php?z=56138)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|[7th](https://zwiftpower\.com/events\.php?zid=154308)|[4th](https://zwiftpower\.com/events\.php?zid=155629)|[7th](https://zwiftpower\.com/events\.php?zid=159335)|[5th](https://zwiftpower\.com/events\.php?zid=162692)|[8th](https://zwiftpower\.com/events\.php?zid=166629)|[4th](https://zwiftpower\.com/events\.php?zid=171936)|[5th](https://zwiftpower\.com/events\.php?zid=173447)|[5th](https://zwiftpower\.com/events\.php?zid=176149)
A|3|6961|[Vicki Whitelaw](https://zwiftpower\.com/profile\.php?z=372588)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[🥈](https://zwiftpower\.com/events\.php?zid=154308)|[5th](https://zwiftpower\.com/events\.php?zid=155629)|[🥈](https://zwiftpower\.com/events\.php?zid=159335)|[6th](https://zwiftpower\.com/events\.php?zid=162692)|[🥇](https://zwiftpower\.com/events\.php?zid=166629)|[🥉](https://zwiftpower\.com/events\.php?zid=171936)|—|[4th](https://zwiftpower\.com/events\.php?zid=176149)
A|4|6809|[Lois Brewer](https://zwiftpower\.com/profile\.php?z=317074)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[17th](https://zwiftpower\.com/events\.php?zid=154308)|[20th](https://zwiftpower\.com/events\.php?zid=155629)|[14th](https://zwiftpower\.com/events\.php?zid=159335)|[13th](https://zwiftpower\.com/events\.php?zid=162692)|[10th](https://zwiftpower\.com/events\.php?zid=166629)|[11th](https://zwiftpower\.com/events\.php?zid=171936)|—|[14th](https://zwiftpower\.com/events\.php?zid=176149)
A|5|5886|[Laura van Regenmortel](https://zwiftpower\.com/profile\.php?z=376739)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|[10th](https://zwiftpower\.com/events\.php?zid=154308)|[13th](https://zwiftpower\.com/events\.php?zid=155629)|[10th](https://zwiftpower\.com/events\.php?zid=159335)|—|—|[9th](https://zwiftpower\.com/events\.php?zid=171936)|[7th](https://zwiftpower\.com/events\.php?zid=173447)|[11th](https://zwiftpower\.com/events\.php?zid=176149)
A|6|5876|[Siri Hildonen \[CANYON\]](https://zwiftpower\.com/profile\.php?z=12501)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|[11th](https://zwiftpower\.com/events\.php?zid=154308)|—|[16th](https://zwiftpower\.com/events\.php?zid=159335)|—|[15th](https://zwiftpower\.com/events\.php?zid=166629)|[6th](https://zwiftpower\.com/events\.php?zid=171936)|[8th](https://zwiftpower\.com/events\.php?zid=173447)|[9th](https://zwiftpower\.com/events\.php?zid=176149)
A|7|5866|[Lea Milleu](https://zwiftpower\.com/profile\.php?z=470626)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|—|[19th](https://zwiftpower\.com/events\.php?zid=155629)|—|[7th](https://zwiftpower\.com/events\.php?zid=162692)|[13th](https://zwiftpower\.com/events\.php?zid=166629)|[13th](https://zwiftpower\.com/events\.php?zid=171936)|[10th](https://zwiftpower\.com/events\.php?zid=173447)|[8th](https://zwiftpower\.com/events\.php?zid=176149)
A|8|5856|[Leah Brewer](https://zwiftpower\.com/profile\.php?z=317804)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[15th](https://zwiftpower\.com/events\.php?zid=154308)|[14th](https://zwiftpower\.com/events\.php?zid=155629)|—|[9th](https://zwiftpower\.com/events\.php?zid=162692)|[12th](https://zwiftpower\.com/events\.php?zid=166629)|[10th](https://zwiftpower\.com/events\.php?zid=171936)|—|[15th](https://zwiftpower\.com/events\.php?zid=176149)
A|9|4991|[Louise Houbak](https://zwiftpower\.com/profile\.php?z=625825)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[🥇](https://zwiftpower\.com/events\.php?zid=154308)|[🥇](https://zwiftpower\.com/events\.php?zid=155629)|[🥇](https://zwiftpower\.com/events\.php?zid=159335)|[🥉](https://zwiftpower\.com/events\.php?zid=162692)|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=173447)|—
A|10|4959|[Nathalie Eklund](https://zwiftpower\.com/profile\.php?z=185030)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|[5th](https://zwiftpower\.com/events\.php?zid=154308)|[6th](https://zwiftpower\.com/events\.php?zid=155629)|[4th](https://zwiftpower\.com/events\.php?zid=159335)|[🥈](https://zwiftpower\.com/events\.php?zid=162692)|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=176149)
A|11|4919|[Polly Mason](https://zwiftpower\.com/profile\.php?z=136220)|—|—|—|—|[12th](https://zwiftpower\.com/events\.php?zid=162692)|[7th](https://zwiftpower\.com/events\.php?zid=166629)|[5th](https://zwiftpower\.com/events\.php?zid=171936)|[9th](https://zwiftpower\.com/events\.php?zid=173447)|[10th](https://zwiftpower\.com/events\.php?zid=176149)
A|12|4881|[Clare Rhead](https://zwiftpower\.com/profile\.php?z=301757)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[9th](https://zwiftpower\.com/events\.php?zid=154308)|[16th](https://zwiftpower\.com/events\.php?zid=155629)|[13th](https://zwiftpower\.com/events\.php?zid=159335)|[10th](https://zwiftpower\.com/events\.php?zid=162692)|—|[14th](https://zwiftpower\.com/events\.php?zid=171936)|—|—
A|13|4847|[Caroline Selin](https://zwiftpower\.com/profile\.php?z=166443)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=162692)|[17th](https://zwiftpower\.com/events\.php?zid=166629)|[15th](https://zwiftpower\.com/events\.php?zid=171936)|[13th](https://zwiftpower\.com/events\.php?zid=173447)|[18th](https://zwiftpower\.com/events\.php?zid=176149)
A|14|3952|[Moin Moin](https://zwiftpower\.com/profile\.php?z=417656)|—|[6th](https://zwiftpower\.com/events\.php?zid=154308)|[10th](https://zwiftpower\.com/events\.php?zid=155629)|[6th](https://zwiftpower\.com/events\.php?zid=159335)|[4th](https://zwiftpower\.com/events\.php?zid=162692)|—|—|—|—
A|15|3934|[Jennifer Real](https://zwiftpower\.com/profile\.php?z=360293)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[8th](https://zwiftpower\.com/events\.php?zid=154308)|[8th](https://zwiftpower\.com/events\.php?zid=155629)|—|—|—|—|[12th](https://zwiftpower\.com/events\.php?zid=173447)|[7th](https://zwiftpower\.com/events\.php?zid=176149)
A|16|3902|[Catherine Allen](https://zwiftpower\.com/profile\.php?z=281872)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=166629)|[16th](https://zwiftpower\.com/events\.php?zid=171936)|[14th](https://zwiftpower\.com/events\.php?zid=173447)|[12th](https://zwiftpower\.com/events\.php?zid=176149)
A|17|3880|[Inge Jansen](https://zwiftpower\.com/profile\.php?z=126110)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[16th](https://zwiftpower\.com/events\.php?zid=154308)|—|[15th](https://zwiftpower\.com/events\.php?zid=159335)|[15th](https://zwiftpower\.com/events\.php?zid=162692)|[16th](https://zwiftpower\.com/events\.php?zid=166629)|—|—|—
A|18|2995|[D aisy B](https://zwiftpower\.com/profile\.php?z=1105834)|—|—|—|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=171936)|[🥈](https://zwiftpower\.com/events\.php?zid=173447)|[🥇](https://zwiftpower\.com/events\.php?zid=176149)
A|19|2947|[Eva\-Cathérine Buchholz 🦅](https://zwiftpower\.com/profile\.php?z=438555)|[CANYON](https://zwiftpower\.com/team\.php?id=2144)|—|—|[9th](https://zwiftpower\.com/events\.php?zid=159335)|—|—|[8th](https://zwiftpower\.com/events\.php?zid=171936)|[11th](https://zwiftpower\.com/events\.php?zid=173447)|—
A|20|2939|[Kjersti Hoel](https://zwiftpower\.com/profile\.php?z=169175)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[13th](https://zwiftpower\.com/events\.php?zid=154308)|—|[11th](https://zwiftpower\.com/events\.php?zid=159335)|[8th](https://zwiftpower\.com/events\.php?zid=162692)|—|—|—|—
A|21|2935|[Monica Dew \[WAHOOCH\]](https://zwiftpower\.com/profile\.php?z=1180352)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|—|—|—|—|[14th](https://zwiftpower\.com/events\.php?zid=166629)|[7th](https://zwiftpower\.com/events\.php?zid=171936)|—|[13th](https://zwiftpower\.com/events\.php?zid=176149)
A|22|2923|[Kerstin Pöhl](https://zwiftpower\.com/profile\.php?z=458121)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|[17th](https://zwiftpower\.com/events\.php?zid=155629)|[12th](https://zwiftpower\.com/events\.php?zid=159335)|[11th](https://zwiftpower\.com/events\.php?zid=162692)|—|—|—|—
A|23|2905|[Katie Kookaburra](https://zwiftpower\.com/profile\.php?z=405642)|—|—|—|—|—|—|[17th](https://zwiftpower\.com/events\.php?zid=171936)|[16th](https://zwiftpower\.com/events\.php?zid=173447)|[16th](https://zwiftpower\.com/events\.php?zid=176149)
A|24|2891|[Haddi Conant](https://zwiftpower\.com/profile\.php?z=1141836)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|—|—|—|—|[19th](https://zwiftpower\.com/events\.php?zid=166629)|[18th](https://zwiftpower\.com/events\.php?zid=171936)|—|[19th](https://zwiftpower\.com/events\.php?zid=176149)
A|25|1984|[Istrate Daniela](https://zwiftpower\.com/profile\.php?z=1056287)|—|—|—|—|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=173447)|[🥉](https://zwiftpower\.com/events\.php?zid=176149)
A|26|1982|[Cassie Baldi](https://zwiftpower\.com/profile\.php?z=13266)|[ZHR](https://zwiftpower\.com/team\.php?id=1979)|—|[7th](https://zwiftpower\.com/events\.php?zid=155629)|—|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=173447)|—
A|27|1980|[sylvie boermans \[X\]](https://zwiftpower\.com/profile\.php?z=100763)|[Team X](https://zwiftpower\.com/team\.php?id=2)|—|[9th](https://zwiftpower\.com/events\.php?zid=155629)|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=166629)|—|—|—
A|28|1978|[Rachael Elliott](https://zwiftpower\.com/profile\.php?z=80103)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[4th](https://zwiftpower\.com/events\.php?zid=154308)|—|[8th](https://zwiftpower\.com/events\.php?zid=159335)|—|—|—|—|—
A|29|1956|[Superwomen Sonia](https://zwiftpower\.com/profile\.php?z=626757)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|—|[12th](https://zwiftpower\.com/events\.php?zid=155629)|—|—|[11th](https://zwiftpower\.com/events\.php?zid=166629)|—|—|—
A|30|1936|[Emily Slavin](https://zwiftpower\.com/profile\.php?z=385268)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=162692)|—|—|[15th](https://zwiftpower\.com/events\.php?zid=173447)|—
A|31|1930|[Maddy Lee\-Smith](https://zwiftpower\.com/profile\.php?z=609300)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[14th](https://zwiftpower\.com/events\.php?zid=154308)|[22nd](https://zwiftpower\.com/events\.php?zid=155629)|—|—|—|—|—|—
A|32|1924|[Kimberly Hoekstra](https://zwiftpower\.com/profile\.php?z=1122394)|—|[18th](https://zwiftpower\.com/events\.php?zid=154308)|[21st](https://zwiftpower\.com/events\.php?zid=155629)|—|—|—|—|—|—
A|33|995|[Mary Wilkinson](https://zwiftpower\.com/profile\.php?z=150208)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|[🥉](https://zwiftpower\.com/events\.php?zid=155629)|—|—|—|—|—|—
A|34|993|[Kerry Speers](https://zwiftpower\.com/profile\.php?z=20533)|—|—|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=166629)|—|—|—
A|35|991|[Claudia Behring 🐻🔰](https://zwiftpower\.com/profile\.php?z=251988)|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=159335)|—|—|—|—|—
A|35|991|[Caroline Murray](https://zwiftpower\.com/profile\.php?z=566158)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=166629)|—|—|—
A|36|989|[Leah Dixon](https://zwiftpower\.com/profile\.php?z=686696)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=166629)|—|—|—
A|37|979|[Illi Gardner](https://zwiftpower\.com/profile\.php?z=427078)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|[11th](https://zwiftpower\.com/events\.php?zid=155629)|—|—|—|—|—|—
A|38|977|[Aysegul Gokalp \[FEAR<\]](https://zwiftpower\.com/profile\.php?z=489939)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|[12th](https://zwiftpower\.com/events\.php?zid=154308)|—|—|—|—|—|—|—
A|38|977|[Maria Ossiansson](https://zwiftpower\.com/profile\.php?z=1046923)|—|—|—|—|—|—|[12th](https://zwiftpower\.com/events\.php?zid=171936)|—|—
A|39|973|[Penny Priest ☁](https://zwiftpower\.com/profile\.php?z=199305)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|—|[14th](https://zwiftpower\.com/events\.php?zid=162692)|—|—|—|—
A|40|971|[Emma Cockcroft](https://zwiftpower\.com/profile\.php?z=49469)|—|—|[15th](https://zwiftpower\.com/events\.php?zid=155629)|—|—|—|—|—|—
A|41|967|[Manon Lannergan](https://zwiftpower\.com/profile\.php?z=70362)|—|—|—|—|[17th](https://zwiftpower\.com/events\.php?zid=162692)|—|—|—|—
A|41|967|[Laura Epelbaum](https://zwiftpower\.com/profile\.php?z=377170)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|—|—|—|—|—|—|—|[17th](https://zwiftpower\.com/events\.php?zid=176149)
A|42|965|[Nina Gilljam](https://zwiftpower\.com/profile\.php?z=189774)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=166629)|—|—|—
A|42|965|[Stephanie Ossenbrink](https://zwiftpower\.com/profile\.php?z=479580)|[We Showed Up](https://zwiftpower\.com/team\.php?id=2216)|—|[18th](https://zwiftpower\.com/events\.php?zid=155629)|—|—|—|—|—|—
A|43|963|[Sil Cal \[REVO\]](https://zwiftpower\.com/profile\.php?z=447517)|[REVO](https://zwiftpower\.com/team\.php?id=2672)|—|—|—|—|—|[19th](https://zwiftpower\.com/events\.php?zid=171936)|—|—
B|1|6941|[Frederique Hate](https://zwiftpower\.com/profile\.php?z=94641)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[4th](https://zwiftpower\.com/events\.php?zid=154308)|[🥉](https://zwiftpower\.com/events\.php?zid=155629)|[8th](https://zwiftpower\.com/events\.php?zid=159335)|[11th](https://zwiftpower\.com/events\.php?zid=162692)|[🥈](https://zwiftpower\.com/events\.php?zid=166629)|[🥈](https://zwiftpower\.com/events\.php?zid=171936)|—|[🥉](https://zwiftpower\.com/events\.php?zid=176149)
B|2|5968|[Debbie Stockley \(Race3R\)](https://zwiftpower\.com/profile\.php?z=320052)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|—|—|[6th](https://zwiftpower\.com/events\.php?zid=159335)|[4th](https://zwiftpower\.com/events\.php?zid=162692)|[🥉](https://zwiftpower\.com/events\.php?zid=166629)|[🥇](https://zwiftpower\.com/events\.php?zid=171936)|[🥉](https://zwiftpower\.com/events\.php?zid=173447)|[🥈](https://zwiftpower\.com/events\.php?zid=176149)
B|3|5964|[sonia insley](https://zwiftpower\.com/profile\.php?z=70830)|[PTz](https://zwiftpower\.com/team\.php?id=1)|[🥉](https://zwiftpower\.com/events\.php?zid=154308)|[🥈](https://zwiftpower\.com/events\.php?zid=155629)|—|[7th](https://zwiftpower\.com/events\.php?zid=162692)|[5th](https://zwiftpower\.com/events\.php?zid=166629)|[🥉](https://zwiftpower\.com/events\.php?zid=171936)|[🥇](https://zwiftpower\.com/events\.php?zid=173447)|—
B|4|5954|[C Scott](https://zwiftpower\.com/profile\.php?z=340007)|[ZSUN](https://zwiftpower\.com/team\.php?id=161)|[🥇](https://zwiftpower\.com/events\.php?zid=154308)|—|[5th](https://zwiftpower\.com/events\.php?zid=159335)|[🥉](https://zwiftpower\.com/events\.php?zid=162692)|[6th](https://zwiftpower\.com/events\.php?zid=166629)|—|[4th](https://zwiftpower\.com/events\.php?zid=173447)|[7th](https://zwiftpower\.com/events\.php?zid=176149)
B|5|5922|[Cheryl Muldoon 🥦](https://zwiftpower\.com/profile\.php?z=113057)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[7th](https://zwiftpower\.com/events\.php?zid=154308)|[8th](https://zwiftpower\.com/events\.php?zid=155629)|[9th](https://zwiftpower\.com/events\.php?zid=159335)|[6th](https://zwiftpower\.com/events\.php?zid=162692)|[7th](https://zwiftpower\.com/events\.php?zid=166629)|—|[5th](https://zwiftpower\.com/events\.php?zid=173447)|—
B|6|4917|[Martha Kuntz](https://zwiftpower\.com/profile\.php?z=448610)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|—|[7th](https://zwiftpower\.com/events\.php?zid=155629)|[12th](https://zwiftpower\.com/events\.php?zid=159335)|[12th](https://zwiftpower\.com/events\.php?zid=162692)|—|[6th](https://zwiftpower\.com/events\.php?zid=171936)|[7th](https://zwiftpower\.com/events\.php?zid=173447)|—
B|7|3968|[Haddi Conant](https://zwiftpower\.com/profile\.php?z=1141836)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|[5th](https://zwiftpower\.com/events\.php?zid=154308)|[5th](https://zwiftpower\.com/events\.php?zid=155629)|[🥉](https://zwiftpower\.com/events\.php?zid=159335)|[5th](https://zwiftpower\.com/events\.php?zid=162692)|—|—|—|—
B|8|3966|[Siv Engmo](https://zwiftpower\.com/profile\.php?z=217993)|[PTz](https://zwiftpower\.com/team\.php?id=1)|[🥈](https://zwiftpower\.com/events\.php?zid=154308)|—|[7th](https://zwiftpower\.com/events\.php?zid=159335)|[8th](https://zwiftpower\.com/events\.php?zid=162692)|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=173447)|—
B|9|2991|[Line Heerup Madsen](https://zwiftpower\.com/profile\.php?z=927830)|—|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=162692)|[4th](https://zwiftpower\.com/events\.php?zid=166629)|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=176149)
B|10|2989|[C\. Harris Momsen](https://zwiftpower\.com/profile\.php?z=719263)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|[4th](https://zwiftpower\.com/events\.php?zid=159335)|[🥈](https://zwiftpower\.com/events\.php?zid=162692)|[🥇](https://zwiftpower\.com/events\.php?zid=166629)|—|—|—
B|11|2965|[Charlotte Watts](https://zwiftpower\.com/profile\.php?z=617677)|—|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=166629)|[4th](https://zwiftpower\.com/events\.php?zid=171936)|[6th](https://zwiftpower\.com/events\.php?zid=173447)|—
B|12|1984|[Fran Bambust](https://zwiftpower\.com/profile\.php?z=427114)|[Team AHDR](https://zwiftpower\.com/team\.php?id=629)|—|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=171936)|—|[4th](https://zwiftpower\.com/events\.php?zid=176149)
B|13|1974|[Silje Holmsen \[REVO\] B](https://zwiftpower\.com/profile\.php?z=188434)|[REVO](https://zwiftpower\.com/team\.php?id=2672)|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=166629)|—|—|[6th](https://zwiftpower\.com/events\.php?zid=176149)
B|14|1970|[Janis Denning KoS](https://zwiftpower\.com/profile\.php?z=152749)|[Vision](https://zwiftpower\.com/team\.php?id=9)|—|[6th](https://zwiftpower\.com/events\.php?zid=155629)|[10th](https://zwiftpower\.com/events\.php?zid=159335)|—|—|—|—|—
B|15|1968|[Sylvie Holmes ZSUN](https://zwiftpower\.com/profile\.php?z=87284)|[ZSUN](https://zwiftpower\.com/team\.php?id=161)|[6th](https://zwiftpower\.com/events\.php?zid=154308)|—|[11th](https://zwiftpower\.com/events\.php?zid=159335)|—|—|—|—|—
B|16|1956|[Tina Berger\-Schauer](https://zwiftpower\.com/profile\.php?z=471560)|[CyclingHub](https://zwiftpower\.com/team\.php?id=2635)|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=162692)|[10th](https://zwiftpower\.com/events\.php?zid=166629)|—|—|—
B|17|999|[Anja Leuendorff](https://zwiftpower\.com/profile\.php?z=170027)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|[🥇](https://zwiftpower\.com/events\.php?zid=155629)|—|—|—|—|—|—
B|17|999|[Jessica Hamilton](https://zwiftpower\.com/profile\.php?z=653395)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=159335)|—|—|—|—|—
B|18|997|[Leah Brewer](https://zwiftpower\.com/profile\.php?z=317804)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=159335)|—|—|—|—|—
B|19|993|[Stephanie Wendel \(KC\)](https://zwiftpower\.com/profile\.php?z=189986)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|[4th](https://zwiftpower\.com/events\.php?zid=155629)|—|—|—|—|—|—
B|20|991|[Liwi Feuerzahn](https://zwiftpower\.com/profile\.php?z=1212166)|—|—|—|—|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=176149)
B|21|985|[Nicola Hogan](https://zwiftpower\.com/profile\.php?z=635868)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[8th](https://zwiftpower\.com/events\.php?zid=154308)|—|—|—|—|—|—|—
B|22|983|[Gillian Leech](https://zwiftpower\.com/profile\.php?z=59724)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=162692)|—|—|—|—
B|22|983|[Becky Loberg Youngberg](https://zwiftpower\.com/profile\.php?z=421396)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|[9th](https://zwiftpower\.com/events\.php?zid=155629)|—|—|—|—|—|—
B|23|981|[Katarina Augustin INC](https://zwiftpower\.com/profile\.php?z=383329)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|[10th](https://zwiftpower\.com/events\.php?zid=162692)|—|—|—|—
C|1|5990|[Karissa Minn](https://zwiftpower\.com/profile\.php?z=39406)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[🥇](https://zwiftpower\.com/events\.php?zid=154308)|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=162692)|[🥈](https://zwiftpower\.com/events\.php?zid=166629)|[🥇](https://zwiftpower\.com/events\.php?zid=171936)|[🥇](https://zwiftpower\.com/events\.php?zid=173447)|[🥈](https://zwiftpower\.com/events\.php?zid=176149)
C|2|5972|[Tina Boine\-Frankenheim](https://zwiftpower\.com/profile\.php?z=107059)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|[🥈](https://zwiftpower\.com/events\.php?zid=155629)|[🥈](https://zwiftpower\.com/events\.php?zid=159335)|—|[4th](https://zwiftpower\.com/events\.php?zid=166629)|[🥈](https://zwiftpower\.com/events\.php?zid=171936)|[🥉](https://zwiftpower\.com/events\.php?zid=173447)|[4th](https://zwiftpower\.com/events\.php?zid=176149)
C|3|4963|[Kimberley Gray](https://zwiftpower\.com/profile\.php?z=327961)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|[🥉](https://zwiftpower\.com/events\.php?zid=155629)|[🥇](https://zwiftpower\.com/events\.php?zid=159335)|[4th](https://zwiftpower\.com/events\.php?zid=162692)|—|[6th](https://zwiftpower\.com/events\.php?zid=171936)|—|[7th](https://zwiftpower\.com/events\.php?zid=176149)
C|4|3988|[Vicky Steadman](https://zwiftpower\.com/profile\.php?z=506303)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=162692)|[🥉](https://zwiftpower\.com/events\.php?zid=166629)|—|[🥈](https://zwiftpower\.com/events\.php?zid=173447)|[🥇](https://zwiftpower\.com/events\.php?zid=176149)
C|5|3974|[Lucy Jay \[TPH\]](https://zwiftpower\.com/profile\.php?z=986803)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=162692)|—|[🥉](https://zwiftpower\.com/events\.php?zid=171936)|[4th](https://zwiftpower\.com/events\.php?zid=173447)|[🥉](https://zwiftpower\.com/events\.php?zid=176149)
C|6|3970|[Julia Heckmann \(KC & tritime women\)](https://zwiftpower\.com/profile\.php?z=917510)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|[4th](https://zwiftpower\.com/events\.php?zid=155629)|[🥉](https://zwiftpower\.com/events\.php?zid=159335)|—|[5th](https://zwiftpower\.com/events\.php?zid=166629)|—|—|[5th](https://zwiftpower\.com/events\.php?zid=176149)
C|7|2977|[Ruth Stapleton SNCC](https://zwiftpower\.com/profile\.php?z=572213)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[4th](https://zwiftpower\.com/events\.php?zid=154308)|[5th](https://zwiftpower\.com/events\.php?zid=155629)|[4th](https://zwiftpower\.com/events\.php?zid=159335)|—|—|—|—|—
C|8|2969|[Mandy McCoy](https://zwiftpower\.com/profile\.php?z=334424)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[5th](https://zwiftpower\.com/events\.php?zid=154308)|—|—|[6th](https://zwiftpower\.com/events\.php?zid=162692)|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=176149)
C|9|999|[tracking tracey](https://zwiftpower\.com/profile\.php?z=79911)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|[🥇](https://zwiftpower\.com/events\.php?zid=155629)|—|—|—|—|—|—
C|9|999|[Nicole Ternel](https://zwiftpower\.com/profile\.php?z=135833)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=166629)|—|—|—
C|10|997|[Gunhild Westergaard](https://zwiftpower\.com/profile\.php?z=1120212)|—|[🥈](https://zwiftpower\.com/events\.php?zid=154308)|—|—|—|—|—|—|—
C|11|995|[Ann\-Kathrin Bach \(ZRG\)](https://zwiftpower\.com/profile\.php?z=944139)|[ZRG](https://zwiftpower\.com/team\.php?id=1942)|[🥉](https://zwiftpower\.com/events\.php?zid=154308)|—|—|—|—|—|—|—
C|11|995|[Anne Loh](https://zwiftpower\.com/profile\.php?z=944358)|—|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=162692)|—|—|—|—
C|12|993|[a kent](https://zwiftpower\.com/profile\.php?z=693532)|—|—|—|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=171936)|—|—
C|13|991|[I Hall](https://zwiftpower\.com/profile\.php?z=944133)|—|—|—|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=173447)|—
C|13|991|[Heather Walke7128](https://zwiftpower\.com/profile\.php?z=1080715)|—|—|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=171936)|—|—
C|14|989|[A ZSUN Goat](https://zwiftpower\.com/profile\.php?z=346294)|[ZSUN](https://zwiftpower\.com/team\.php?id=161)|—|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=166629)|—|—|—
C|15|987|[Mariet Feltkamp](https://zwiftpower\.com/profile\.php?z=565539)|—|—|—|—|—|[7th](https://zwiftpower\.com/events\.php?zid=166629)|—|—|—
C|16|985|[Kate Cronin](https://zwiftpower\.com/profile\.php?z=340509)|—|—|—|—|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=176149)
C|16|985|[jude woodcock BRT Hellcatz](https://zwiftpower\.com/profile\.php?z=659665)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=166629)|—|—|—
D|1|4991|[Emma Lambert](https://zwiftpower\.com/profile\.php?z=19596)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=162692)|[🥈](https://zwiftpower\.com/events\.php?zid=166629)|[🥇](https://zwiftpower\.com/events\.php?zid=171936)|[🥈](https://zwiftpower\.com/events\.php?zid=173447)|[🥇](https://zwiftpower\.com/events\.php?zid=176149)
D|2|2997|[Lucy Jay \[TPH\]](https://zwiftpower\.com/profile\.php?z=986803)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|[🥇](https://zwiftpower\.com/events\.php?zid=154308)|[🥇](https://zwiftpower\.com/events\.php?zid=155629)|[🥇](https://zwiftpower\.com/events\.php?zid=159335)|—|—|—|—|—
D|3|2995|[Molly Cutmore \(TPH\) 6698](https://zwiftpower\.com/profile\.php?z=1164747)|[TPH Racing](https://zwiftpower\.com/team\.php?id=207)|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=159335)|—|[🥇](https://zwiftpower\.com/events\.php?zid=166629)|—|[🥇](https://zwiftpower\.com/events\.php?zid=173447)|—
D|4|2987|[Stefanie Held](https://zwiftpower\.com/profile\.php?z=536821)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=162692)|[4th](https://zwiftpower\.com/events\.php?zid=166629)|[🥈](https://zwiftpower\.com/events\.php?zid=171936)|—|—
D|5|997|[Jo Brown CRYO\-GEN\(D\)](https://zwiftpower\.com/profile\.php?z=439963)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|[🥈](https://zwiftpower\.com/events\.php?zid=155629)|—|—|—|—|—|—
D|6|995|[Sarah McNair](https://zwiftpower\.com/profile\.php?z=1032310)|—|—|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=166629)|—|—|—



## KISS Community League — Pacific-Asia

Category|Rank|Total Points|Name|Team|1|2|3|4|5|6|7|8
---|---|---|---|---|---|---|---|---|---|---|---|---
A|1|7758|[Chris Beck 🔰](https://zwiftpower\.com/profile\.php?z=177885)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|[11th](https://zwiftpower\.com/events\.php?zid=151390)|[19th](https://zwiftpower\.com/events\.php?zid=155565)|[24th](https://zwiftpower\.com/events\.php?zid=159271)|[38th](https://zwiftpower\.com/events\.php?zid=162625)|[4th](https://zwiftpower\.com/events\.php?zid=166628)|[8th](https://zwiftpower\.com/events\.php?zid=171934)|[7th](https://zwiftpower\.com/events\.php?zid=173434)|[14th](https://zwiftpower\.com/events\.php?zid=176147)
A|2|7584|[Adam Boyko](https://zwiftpower\.com/profile\.php?z=91519)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[26th](https://zwiftpower\.com/events\.php?zid=151390)|[30th](https://zwiftpower\.com/events\.php?zid=155565)|[21st](https://zwiftpower\.com/events\.php?zid=159271)|[28th](https://zwiftpower\.com/events\.php?zid=162625)|[48th](https://zwiftpower\.com/events\.php?zid=166628)|[19th](https://zwiftpower\.com/events\.php?zid=171934)|[24th](https://zwiftpower\.com/events\.php?zid=173434)|[16th](https://zwiftpower\.com/events\.php?zid=176147)
A|3|7402|[Donnacha Holmes 🦄](https://zwiftpower\.com/profile\.php?z=64274)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[28th](https://zwiftpower\.com/events\.php?zid=151390)|[38th](https://zwiftpower\.com/events\.php?zid=155565)|[37th](https://zwiftpower\.com/events\.php?zid=159271)|[46th](https://zwiftpower\.com/events\.php?zid=162625)|[49th](https://zwiftpower\.com/events\.php?zid=166628)|[34th](https://zwiftpower\.com/events\.php?zid=171934)|[40th](https://zwiftpower\.com/events\.php?zid=173434)|[31st](https://zwiftpower\.com/events\.php?zid=176147)
A|4|6891|[Martin Firestone III](https://zwiftpower\.com/profile\.php?z=99475)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[🥈](https://zwiftpower\.com/events\.php?zid=151390)|[4th](https://zwiftpower\.com/events\.php?zid=155565)|—|[13th](https://zwiftpower\.com/events\.php?zid=162625)|[22nd](https://zwiftpower\.com/events\.php?zid=166628)|[🥈](https://zwiftpower\.com/events\.php?zid=171934)|[5th](https://zwiftpower\.com/events\.php?zid=173434)|[10th](https://zwiftpower\.com/events\.php?zid=176147)
A|5|6855|[Matt Johnson](https://zwiftpower\.com/profile\.php?z=16607)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[7th](https://zwiftpower\.com/events\.php?zid=151390)|[25th](https://zwiftpower\.com/events\.php?zid=155565)|[6th](https://zwiftpower\.com/events\.php?zid=159271)|[12th](https://zwiftpower\.com/events\.php?zid=162625)|[9th](https://zwiftpower\.com/events\.php?zid=166628)|[4th](https://zwiftpower\.com/events\.php?zid=171934)|[13th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|6|6767|[Robin Zünd 🍺🔰](https://zwiftpower\.com/profile\.php?z=31080)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[17th](https://zwiftpower\.com/events\.php?zid=151390)|[26th](https://zwiftpower\.com/events\.php?zid=155565)|[7th](https://zwiftpower\.com/events\.php?zid=159271)|[7th](https://zwiftpower\.com/events\.php?zid=162625)|[21st](https://zwiftpower\.com/events\.php?zid=166628)|[17th](https://zwiftpower\.com/events\.php?zid=171934)|[25th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|7|6751|[Jerome Mariaud 🐌](https://zwiftpower\.com/profile\.php?z=173884)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[13th](https://zwiftpower\.com/events\.php?zid=151390)|[24th](https://zwiftpower\.com/events\.php?zid=155565)|[13th](https://zwiftpower\.com/events\.php?zid=159271)|[10th](https://zwiftpower\.com/events\.php?zid=162625)|[27th](https://zwiftpower\.com/events\.php?zid=166628)|—|[16th](https://zwiftpower\.com/events\.php?zid=173434)|[25th](https://zwiftpower\.com/events\.php?zid=176147)
A|8|6655|[Patrick McCrann \(EnduranceNation\.us\)](https://zwiftpower\.com/profile\.php?z=328064)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[21st](https://zwiftpower\.com/events\.php?zid=151390)|—|[20th](https://zwiftpower\.com/events\.php?zid=159271)|[29th](https://zwiftpower\.com/events\.php?zid=162625)|[32nd](https://zwiftpower\.com/events\.php?zid=166628)|[23rd](https://zwiftpower\.com/events\.php?zid=171934)|[32nd](https://zwiftpower\.com/events\.php?zid=173434)|[19th](https://zwiftpower\.com/events\.php?zid=176147)
A|9|6653|[Tim Sullivan 🔰](https://zwiftpower\.com/profile\.php?z=165253)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[19th](https://zwiftpower\.com/events\.php?zid=151390)|[31st](https://zwiftpower\.com/events\.php?zid=155565)|[29th](https://zwiftpower\.com/events\.php?zid=159271)|[23rd](https://zwiftpower\.com/events\.php?zid=162625)|—|[15th](https://zwiftpower\.com/events\.php?zid=171934)|[36th](https://zwiftpower\.com/events\.php?zid=173434)|[24th](https://zwiftpower\.com/events\.php?zid=176147)
A|10|6651|[Lukman CyclingTraining\.cc](https://zwiftpower\.com/profile\.php?z=12512)|[Cramp Champs](https://zwiftpower\.com/team\.php?id=2739)|[23rd](https://zwiftpower\.com/events\.php?zid=151390)|[23rd](https://zwiftpower\.com/events\.php?zid=155565)|[28th](https://zwiftpower\.com/events\.php?zid=159271)|—|[46th](https://zwiftpower\.com/events\.php?zid=166628)|[26th](https://zwiftpower\.com/events\.php?zid=171934)|[17th](https://zwiftpower\.com/events\.php?zid=173434)|[15th](https://zwiftpower\.com/events\.php?zid=176147)
A|11|6577|[Yuri Onofrei](https://zwiftpower\.com/profile\.php?z=194825)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|[33rd](https://zwiftpower\.com/events\.php?zid=151390)|[45th](https://zwiftpower\.com/events\.php?zid=155565)|[30th](https://zwiftpower\.com/events\.php?zid=159271)|[34th](https://zwiftpower\.com/events\.php?zid=162625)|[30th](https://zwiftpower\.com/events\.php?zid=166628)|[21st](https://zwiftpower\.com/events\.php?zid=171934)|[22nd](https://zwiftpower\.com/events\.php?zid=173434)|—
A|12|5952|[IAM Faraone🍦](https://zwiftpower\.com/profile\.php?z=464375)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|—|[🥈](https://zwiftpower\.com/events\.php?zid=155565)|[🥉](https://zwiftpower\.com/events\.php?zid=159271)|[8th](https://zwiftpower\.com/events\.php?zid=162625)|[🥉](https://zwiftpower\.com/events\.php?zid=166628)|[5th](https://zwiftpower\.com/events\.php?zid=171934)|[6th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|13|5930|[Holden Comeau 🔰](https://zwiftpower\.com/profile\.php?z=103766)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|—|[6th](https://zwiftpower\.com/events\.php?zid=155565)|[12th](https://zwiftpower\.com/events\.php?zid=159271)|[🥇](https://zwiftpower\.com/events\.php?zid=162625)|[12th](https://zwiftpower\.com/events\.php?zid=166628)|[6th](https://zwiftpower\.com/events\.php?zid=171934)|—|[🥇](https://zwiftpower\.com/events\.php?zid=176147)
A|14|5928|[Robert Lyons 🛴](https://zwiftpower\.com/profile\.php?z=248)|[Splunk](https://zwiftpower\.com/team\.php?id=3855)|[5th](https://zwiftpower\.com/events\.php?zid=151390)|[7th](https://zwiftpower\.com/events\.php?zid=155565)|[4th](https://zwiftpower\.com/events\.php?zid=159271)|[4th](https://zwiftpower\.com/events\.php?zid=162625)|[10th](https://zwiftpower\.com/events\.php?zid=166628)|[9th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
A|15|5872|[Matt Gardiner 🍩 🔰](https://zwiftpower\.com/profile\.php?z=58638)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|[🥉](https://zwiftpower\.com/events\.php?zid=151390)|[16th](https://zwiftpower\.com/events\.php?zid=155565)|[17th](https://zwiftpower\.com/events\.php?zid=159271)|—|[14th](https://zwiftpower\.com/events\.php?zid=166628)|[14th](https://zwiftpower\.com/events\.php?zid=171934)|—|[🥉](https://zwiftpower\.com/events\.php?zid=176147)
A|16|5860|[Justin Wagner](https://zwiftpower\.com/profile\.php?z=49001)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[8th](https://zwiftpower\.com/events\.php?zid=151390)|[12th](https://zwiftpower\.com/events\.php?zid=155565)|[14th](https://zwiftpower\.com/events\.php?zid=159271)|—|[11th](https://zwiftpower\.com/events\.php?zid=166628)|[13th](https://zwiftpower\.com/events\.php?zid=171934)|[15th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|17|5728|[Mash Potato](https://zwiftpower\.com/profile\.php?z=309860)|[Cramp Champs](https://zwiftpower\.com/team\.php?id=2739)|[22nd](https://zwiftpower\.com/events\.php?zid=151390)|—|[39th](https://zwiftpower\.com/events\.php?zid=159271)|[11th](https://zwiftpower\.com/events\.php?zid=162625)|—|[20th](https://zwiftpower\.com/events\.php?zid=171934)|[30th](https://zwiftpower\.com/events\.php?zid=173434)|[17th](https://zwiftpower\.com/events\.php?zid=176147)
A|18|5630|[Nick Bevilacqua](https://zwiftpower\.com/profile\.php?z=9876)|—|—|[33rd](https://zwiftpower\.com/events\.php?zid=155565)|—|[32nd](https://zwiftpower\.com/events\.php?zid=162625)|[40th](https://zwiftpower\.com/events\.php?zid=166628)|[27th](https://zwiftpower\.com/events\.php?zid=171934)|[33rd](https://zwiftpower\.com/events\.php?zid=173434)|[23rd](https://zwiftpower\.com/events\.php?zid=176147)
A|19|5480|[John Withrow \(EnduranceNation\.us\)](https://zwiftpower\.com/profile\.php?z=498960)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|—|[48th](https://zwiftpower\.com/events\.php?zid=155565)|[40th](https://zwiftpower\.com/events\.php?zid=159271)|[48th](https://zwiftpower\.com/events\.php?zid=162625)|[51st](https://zwiftpower\.com/events\.php?zid=166628)|[46th](https://zwiftpower\.com/events\.php?zid=171934)|—|[30th](https://zwiftpower\.com/events\.php?zid=176147)
A|20|4869|[Tyler West 🔰 ☕️🍦](https://zwiftpower\.com/profile\.php?z=162405)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=159271)|[17th](https://zwiftpower\.com/events\.php?zid=162625)|[7th](https://zwiftpower\.com/events\.php?zid=166628)|—|[10th](https://zwiftpower\.com/events\.php?zid=173434)|[12th](https://zwiftpower\.com/events\.php?zid=176147)
A|21|4787|[Matthew Yankow](https://zwiftpower\.com/profile\.php?z=159402)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[12th](https://zwiftpower\.com/events\.php?zid=151390)|[17th](https://zwiftpower\.com/events\.php?zid=155565)|[18th](https://zwiftpower\.com/events\.php?zid=159271)|[25th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|[37th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|22|4777|[D CHEAMDW KISS\(A\)](https://zwiftpower\.com/profile\.php?z=44184)|[Cramp Champs](https://zwiftpower\.com/team\.php?id=2739)|[20th](https://zwiftpower\.com/events\.php?zid=151390)|[28th](https://zwiftpower\.com/events\.php?zid=155565)|[19th](https://zwiftpower\.com/events\.php?zid=159271)|[19th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|[28th](https://zwiftpower\.com/events\.php?zid=176147)
A|23|4727|[James Hodges](https://zwiftpower\.com/profile\.php?z=16708)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|[34th](https://zwiftpower\.com/events\.php?zid=159271)|[35th](https://zwiftpower\.com/events\.php?zid=162625)|[24th](https://zwiftpower\.com/events\.php?zid=166628)|[25th](https://zwiftpower\.com/events\.php?zid=171934)|—|[21st](https://zwiftpower\.com/events\.php?zid=176147)
A|24|4679|[George Thivaios](https://zwiftpower\.com/profile\.php?z=250316)|[Splunk](https://zwiftpower\.com/team\.php?id=3855)|[30th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|[37th](https://zwiftpower\.com/events\.php?zid=162625)|[36th](https://zwiftpower\.com/events\.php?zid=166628)|[29th](https://zwiftpower\.com/events\.php?zid=171934)|[31st](https://zwiftpower\.com/events\.php?zid=173434)|—
A|25|3980|[Huang Xiao Yu🔰](https://zwiftpower\.com/profile\.php?z=75898)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|[6th](https://zwiftpower\.com/events\.php?zid=151390)|[🥇](https://zwiftpower\.com/events\.php?zid=155565)|[🥈](https://zwiftpower\.com/events\.php?zid=159271)|[🥉](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|26|3954|[Bernie Lacourciere](https://zwiftpower\.com/profile\.php?z=220957)|[GET\-SET](https://zwiftpower\.com/team\.php?id=2644)|[4th](https://zwiftpower\.com/events\.php?zid=151390)|—|[5th](https://zwiftpower\.com/events\.php?zid=159271)|[9th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|[7th](https://zwiftpower\.com/events\.php?zid=176147)
A|27|3938|[H\-G Becker \(V\)](https://zwiftpower\.com/profile\.php?z=1765)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|[10th](https://zwiftpower\.com/events\.php?zid=155565)|—|[14th](https://zwiftpower\.com/events\.php?zid=162625)|[6th](https://zwiftpower\.com/events\.php?zid=166628)|—|[🥉](https://zwiftpower\.com/events\.php?zid=173434)|—
A|28|3936|[Ulf Bäcker 🔰](https://zwiftpower\.com/profile\.php?z=884451)|[Superbonobo](https://zwiftpower\.com/team\.php?id=3631)|—|[9th](https://zwiftpower\.com/events\.php?zid=155565)|—|[5th](https://zwiftpower\.com/events\.php?zid=162625)|—|[16th](https://zwiftpower\.com/events\.php?zid=171934)|—|[4th](https://zwiftpower\.com/events\.php?zid=176147)
A|29|3904|[Sticky 🤡](https://zwiftpower\.com/profile\.php?z=196246)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|[22nd](https://zwiftpower\.com/events\.php?zid=155565)|[8th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|[11th](https://zwiftpower\.com/events\.php?zid=171934)|—|[9th](https://zwiftpower\.com/events\.php?zid=176147)
A|30|3882|[Matt Brandt🔰🐺](https://zwiftpower\.com/profile\.php?z=78451)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|—|[8th](https://zwiftpower\.com/events\.php?zid=155565)|[31st](https://zwiftpower\.com/events\.php?zid=159271)|—|[16th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|[6th](https://zwiftpower\.com/events\.php?zid=176147)
A|31|3876|[FinK Andi](https://zwiftpower\.com/profile\.php?z=495)|[Splunk](https://zwiftpower\.com/team\.php?id=3855)|[9th](https://zwiftpower\.com/events\.php?zid=151390)|[13th](https://zwiftpower\.com/events\.php?zid=155565)|[16th](https://zwiftpower\.com/events\.php?zid=159271)|[26th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|32|3814|[Colin Mitchell](https://zwiftpower\.com/profile\.php?z=55166)|[Cramp Champs](https://zwiftpower\.com/team\.php?id=2739)|[18th](https://zwiftpower\.com/events\.php?zid=151390)|[21st](https://zwiftpower\.com/events\.php?zid=155565)|[32nd](https://zwiftpower\.com/events\.php?zid=159271)|—|—|[24th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
A|33|3764|"[Matt Bruss \| Brazen Dropouts](https://zwiftpower\.com/profile\.php?z=420808)"|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|—|[39th](https://zwiftpower\.com/events\.php?zid=155565)|[33rd](https://zwiftpower\.com/events\.php?zid=159271)|—|—|[28th](https://zwiftpower\.com/events\.php?zid=171934)|—|[20th](https://zwiftpower\.com/events\.php?zid=176147)
A|34|3744|[THE Nighttrain ODZ](https://zwiftpower\.com/profile\.php?z=7134)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|[36th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|[33rd](https://zwiftpower\.com/events\.php?zid=171934)|[35th](https://zwiftpower\.com/events\.php?zid=173434)|[26th](https://zwiftpower\.com/events\.php?zid=176147)
A|35|3708|[Mark Scoot](https://zwiftpower\.com/profile\.php?z=62584)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|—|[39th](https://zwiftpower\.com/events\.php?zid=166628)|[40th](https://zwiftpower\.com/events\.php?zid=171934)|[42nd](https://zwiftpower\.com/events\.php?zid=173434)|[27th](https://zwiftpower\.com/events\.php?zid=176147)
A|36|3656|[Marc Pierschel \[VEGAN\]](https://zwiftpower\.com/profile\.php?z=918949)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|[47th](https://zwiftpower\.com/events\.php?zid=155565)|—|[43rd](https://zwiftpower\.com/events\.php?zid=162625)|[43rd](https://zwiftpower\.com/events\.php?zid=166628)|[41st](https://zwiftpower\.com/events\.php?zid=171934)|—|—
A|37|2989|[Aidar Zakarin 🔰](https://zwiftpower\.com/profile\.php?z=623306)|[Vision](https://zwiftpower\.com/team\.php?id=9)|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=166628)|[🥇](https://zwiftpower\.com/events\.php?zid=171934)|[🥇](https://zwiftpower\.com/events\.php?zid=173434)|—
A|38|2949|[LaowaiのQindie👨🏻](https://zwiftpower\.com/profile\.php?z=467523)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|[🥇](https://zwiftpower\.com/events\.php?zid=151390)|[🥉](https://zwiftpower\.com/events\.php?zid=155565)|[23rd](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
A|39|2905|[Luiz Lanfredi](https://zwiftpower\.com/profile\.php?z=8481)|[Splunk](https://zwiftpower\.com/team\.php?id=3855)|—|[18th](https://zwiftpower\.com/events\.php?zid=155565)|[9th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=171934)|—|—
A|40|2871|[Richard Port](https://zwiftpower\.com/profile\.php?z=324062)|—|—|—|[27th](https://zwiftpower\.com/events\.php?zid=159271)|[20th](https://zwiftpower\.com/events\.php?zid=162625)|[19th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|41|2861|[Watt Monkey 🐵🔰](https://zwiftpower\.com/profile\.php?z=44150)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[15th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|[29th](https://zwiftpower\.com/events\.php?zid=166628)|—|[27th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|42|2845|[Karl\-Axel Zander Persson](https://zwiftpower\.com/profile\.php?z=5994)|[Sz](https://zwiftpower\.com/team\.php?id=32)|[24th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|[24th](https://zwiftpower\.com/events\.php?zid=162625)|—|[31st](https://zwiftpower\.com/events\.php?zid=171934)|—|—
A|43|2833|[Russell Crowder 🔰](https://zwiftpower\.com/profile\.php?z=22662)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|[29th](https://zwiftpower\.com/events\.php?zid=155565)|[25th](https://zwiftpower\.com/events\.php?zid=159271)|[31st](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|44|2801|[Michael Danish](https://zwiftpower\.com/profile\.php?z=257206)|—|—|[40th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|[33rd](https://zwiftpower\.com/events\.php?zid=166628)|—|[28th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|44|2801|"[Chris ""Awesomely Average"" Wilson](https://zwiftpower\.com/profile\.php?z=794637)"|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|[34th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|[31st](https://zwiftpower\.com/events\.php?zid=166628)|[36th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
A|45|2783|[Jerome Ferson](https://zwiftpower\.com/profile\.php?z=213439)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|—|[33rd](https://zwiftpower\.com/events\.php?zid=162625)|—|[38th](https://zwiftpower\.com/events\.php?zid=171934)|[39th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|46|1998|[Samuel Brännlund INC🔰](https://zwiftpower\.com/profile\.php?z=660836)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=159271)|—|[🥇](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|47|1978|[Mon Ritch](https://zwiftpower\.com/profile\.php?z=135635)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|—|—|—|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=173434)|[8th](https://zwiftpower\.com/events\.php?zid=176147)
A|48|1960|[Abeki Kanji \[ZWC\]](https://zwiftpower\.com/profile\.php?z=335605)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|—|[11th](https://zwiftpower\.com/events\.php?zid=155565)|[10th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
A|48|1960|[Pär Bäcker🔰](https://zwiftpower\.com/profile\.php?z=504853)|[Superbonobo](https://zwiftpower\.com/team\.php?id=3631)|—|—|[11th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|[10th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
A|49|1948|[Andreas Rois \(KC\)](https://zwiftpower\.com/profile\.php?z=991994)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|[9th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|50|1912|[Third Wheel 🚳](https://zwiftpower\.com/profile\.php?z=48068)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|[27th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
A|51|1894|[Andy Jenner 🦑](https://zwiftpower\.com/profile\.php?z=24974)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|[32nd](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=176147)
A|52|1890|[Glen Gollrad 🦊 \[VFOX\]](https://zwiftpower\.com/profile\.php?z=495351)|[Vixen/Fox](https://zwiftpower\.com/team\.php?id=1940)|—|[14th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=171934)|—|—
A|53|1878|[Daan Laros](https://zwiftpower\.com/profile\.php?z=332952)|—|—|—|—|—|[28th](https://zwiftpower\.com/events\.php?zid=166628)|—|[34th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|54|1844|[M Ssmith \(V\)](https://zwiftpower\.com/profile\.php?z=321324)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|[45th](https://zwiftpower\.com/events\.php?zid=162625)|[34th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|55|1842|[rob brunken](https://zwiftpower\.com/profile\.php?z=137488)|—|—|—|[35th](https://zwiftpower\.com/events\.php?zid=159271)|—|[45th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|56|1836|[Anatoly Solodov](https://zwiftpower\.com/profile\.php?z=1057970)|—|[34th](https://zwiftpower\.com/events\.php?zid=151390)|[49th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
A|57|1832|[Adam Chevin Ⓥ🌱](https://zwiftpower\.com/profile\.php?z=70177)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|—|—|[44th](https://zwiftpower\.com/events\.php?zid=171934)|[41st](https://zwiftpower\.com/events\.php?zid=173434)|—
A|58|1830|[Matthew Crawford 🌵](https://zwiftpower\.com/profile\.php?z=11265)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|—|[40th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|[46th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|59|1828|[Jean\-Yan Perron](https://zwiftpower\.com/profile\.php?z=390308)|[Laferte bicycle](https://zwiftpower\.com/team\.php?id=4054)|—|[42nd](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|[45th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|59|1828|[Poisson Lune](https://zwiftpower\.com/profile\.php?z=1011634)|—|—|—|—|—|[44th](https://zwiftpower\.com/events\.php?zid=166628)|—|[43rd](https://zwiftpower\.com/events\.php?zid=173434)|—
A|60|1816|[yu guantou](https://zwiftpower\.com/profile\.php?z=640047)|—|—|[43rd](https://zwiftpower\.com/events\.php?zid=155565)|—|[50th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|61|997|[Alessandro Barra](https://zwiftpower\.com/profile\.php?z=12495)|[Team Italy](https://zwiftpower\.com/team\.php?id=588)|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|61|997|[Andreas Serop Larsen](https://zwiftpower\.com/profile\.php?z=20358)|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|61|997|[Yuki Tanaka](https://zwiftpower\.com/profile\.php?z=35498)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|—|—|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=173434)|—
A|61|997|[Niki H 🔰](https://zwiftpower\.com/profile\.php?z=174313)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=176147)
A|62|995|[Mattias Lundqvist🔰 Aktivitus Testcenter](https://zwiftpower\.com/profile\.php?z=455884)|[Sz](https://zwiftpower\.com/team\.php?id=32)|—|—|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=171934)|—|—
A|63|991|[Yunfei Wu \+1S🍓](https://zwiftpower\.com/profile\.php?z=22572)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|—|[5th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
A|63|991|[Simon Nielsen 🔰](https://zwiftpower\.com/profile\.php?z=182972)|[FUSION ECT](https://zwiftpower\.com/team\.php?id=2854)|—|—|—|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=176147)
A|64|989|[Hefin Evans](https://zwiftpower\.com/profile\.php?z=1123557)|—|—|—|—|[6th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|65|987|[FUZHOU HANCHEN \+1S A](https://zwiftpower\.com/profile\.php?z=388541)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|—|—|—|—|—|[7th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
A|66|985|[Chris Smith \[BRT\]](https://zwiftpower\.com/profile\.php?z=15080)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|66|985|[Bas Pater](https://zwiftpower\.com/profile\.php?z=859868)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|—|—|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|67|981|[David Talbott](https://zwiftpower\.com/profile\.php?z=270280)|[Splunk](https://zwiftpower\.com/team\.php?id=3855)|[10th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
A|68|979|[Kim Little \[X\]](https://zwiftpower\.com/profile\.php?z=6724)|[Zwift All Stars](https://zwiftpower\.com/team\.php?id=3702)|—|—|—|—|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=176147)
A|68|979|[Ryan Mckay](https://zwiftpower\.com/profile\.php?z=245426)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|69|977|[Matt Curbeau 🔰](https://zwiftpower\.com/profile\.php?z=11677)|[Velocio](https://zwiftpower\.com/team\.php?id=2582)|—|—|—|—|—|[12th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
A|69|977|[Norm Hughes](https://zwiftpower\.com/profile\.php?z=190665)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|—|—|—|—|[12th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|70|975|[Timmy Bauer 🔰](https://zwiftpower\.com/profile\.php?z=861542)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|—|—|—|—|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=176147)
A|70|975|[Kirill Krasavin 🔰](https://zwiftpower\.com/profile\.php?z=960217)|[KLD](https://zwiftpower\.com/team\.php?id=2163)|—|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|71|973|[Rob Wood](https://zwiftpower\.com/profile\.php?z=95589)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|[14th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
A|71|973|[Kappa Ebisen](https://zwiftpower\.com/profile\.php?z=368295)|—|—|—|—|—|—|—|[14th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|72|971|[Daniel Pettinger](https://zwiftpower\.com/profile\.php?z=90946)|[Vision](https://zwiftpower\.com/team\.php?id=9)|—|[15th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
A|72|971|[Chris Sherriffs](https://zwiftpower\.com/profile\.php?z=95241)|[Ribble Pro Cycling](https://zwiftpower\.com/team\.php?id=3686)|—|—|—|[15th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|72|971|[Dafydd Williams](https://zwiftpower\.com/profile\.php?z=123836)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|[15th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
A|72|971|[Mathieu Decock](https://zwiftpower\.com/profile\.php?z=509565)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|—|—|—|—|[15th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|73|969|[Dave Reed \(Bioracer Moriarty Bikes\)](https://zwiftpower\.com/profile\.php?z=558707)|[BMB](https://zwiftpower\.com/team\.php?id=2385)|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|73|969|[K KONNO](https://zwiftpower\.com/profile\.php?z=563135)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|[16th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
A|74|967|[Koen Van Geyt](https://zwiftpower\.com/profile\.php?z=115080)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|—|—|—|—|[17th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|75|965|[Ludovic Pollet](https://zwiftpower\.com/profile\.php?z=187160)|[Team X](https://zwiftpower\.com/team\.php?id=2)|—|—|—|—|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=176147)
A|75|965|[Dariusz Zaremba \[TeamPL\]](https://zwiftpower\.com/profile\.php?z=564515)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|75|965|[Maxim Golev \[V\]](https://zwiftpower\.com/profile\.php?z=811341)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|—|—|—|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|76|963|[A\-Ruei ☢️](https://zwiftpower\.com/profile\.php?z=364142)|[TTR](https://zwiftpower\.com/team\.php?id=3750)|—|—|—|—|—|—|[19th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|77|961|[Justin Purificati](https://zwiftpower\.com/profile\.php?z=52048)|[Zwift All Stars](https://zwiftpower\.com/team\.php?id=3702)|—|[20th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
A|77|961|[Tomoaki Takahashi 🍦🍨🍰🎂](https://zwiftpower\.com/profile\.php?z=76844)|[ZJRT](https://zwiftpower\.com/team\.php?id=2054)|—|—|—|—|[20th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|77|961|[Sir Wang \(SYB\) 💩](https://zwiftpower\.com/profile\.php?z=698447)|[TTR](https://zwiftpower\.com/team\.php?id=3750)|—|—|—|—|—|—|[20th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|78|959|[Alex Sankovic](https://zwiftpower\.com/profile\.php?z=506515)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=173434)|—
A|78|959|[Peter Göransson](https://zwiftpower\.com/profile\.php?z=674292)|[Team 63\.8N](https://zwiftpower\.com/team\.php?id=2936)|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|79|957|[Adrian Timmis 🔰 CadencESport](https://zwiftpower\.com/profile\.php?z=64680)|[CadencESport](https://zwiftpower\.com/team\.php?id=2839)|—|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|80|955|[Matt Sinclair](https://zwiftpower\.com/profile\.php?z=115138)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|—|—|[23rd](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|80|955|[Miguel Indoorain \[P//A\]](https://zwiftpower\.com/profile\.php?z=325557)|[P//A](https://zwiftpower\.com/team\.php?id=3898)|—|—|—|—|—|—|[23rd](https://zwiftpower\.com/events\.php?zid=173434)|—
A|81|951|[Rod Leach](https://zwiftpower\.com/profile\.php?z=44795)|[Team X](https://zwiftpower\.com/team\.php?id=2)|[25th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
A|81|951|[Michaël Duyck](https://zwiftpower\.com/profile\.php?z=274955)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|—|—|—|—|[25th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|82|949|[Mathieu Hould](https://zwiftpower\.com/profile\.php?z=71850)|—|—|—|—|—|—|—|[26th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|82|949|[Tom Schyvens](https://zwiftpower\.com/profile\.php?z=426416)|[BZR cycling team](https://zwiftpower\.com/team\.php?id=35)|—|—|—|—|[26th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|82|949|[Liu ZL \+1S](https://zwiftpower\.com/profile\.php?z=1155785)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|—|—|[26th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
A|83|947|[Daniel Sutherland](https://zwiftpower\.com/profile\.php?z=54066)|[Team BMTR](https://zwiftpower\.com/team\.php?id=2873)|[27th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
A|83|947|[Brian H](https://zwiftpower\.com/profile\.php?z=128548)|[Splunk](https://zwiftpower\.com/team\.php?id=3855)|—|—|—|[27th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|84|943|[Johnathan Freter 🚂 🔰](https://zwiftpower\.com/profile\.php?z=77064)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|—|—|—|—|—|—|—|[29th](https://zwiftpower\.com/events\.php?zid=176147)
A|84|943|[Jone Gravdal 🔰](https://zwiftpower\.com/profile\.php?z=121873)|[IndoorSpecialist\.com](https://zwiftpower\.com/team\.php?id=3964)|[29th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
A|84|943|[Michael Totz](https://zwiftpower\.com/profile\.php?z=250857)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|—|—|—|[29th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|85|941|[Derek Sawyer \- Rippers](https://zwiftpower\.com/profile\.php?z=168787)|—|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|85|941|[Jon Duke \[DIRT\]](https://zwiftpower\.com/profile\.php?z=462877)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
A|86|939|[Okanomizu Masaru Oka](https://zwiftpower\.com/profile\.php?z=42834)|[Cramp Champs](https://zwiftpower\.com/team\.php?id=2739)|[31st](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
A|87|937|[Il Pirata](https://zwiftpower\.com/profile\.php?z=29496)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|—|—|[32nd](https://zwiftpower\.com/events\.php?zid=171934)|—|—
A|87|937|[Syo Yone\(FRAT\-GSJ\)](https://zwiftpower\.com/profile\.php?z=31146)|—|[32nd](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
A|88|931|[Jarod Garrington](https://zwiftpower\.com/profile\.php?z=63002)|—|—|—|—|—|—|[35th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
A|88|931|[Goran Salavec Race3R](https://zwiftpower\.com/profile\.php?z=92474)|[Race3R](https://zwiftpower\.com/team\.php?id=791)|—|—|—|—|[35th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|88|931|[Kilian Hemart Ⓥ](https://zwiftpower\.com/profile\.php?z=488019)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|[35th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
A|89|929|[Dave Higham](https://zwiftpower\.com/profile\.php?z=113857)|—|—|[36th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
A|89|929|[Anton Alex](https://zwiftpower\.com/profile\.php?z=601528)|[Team 63\.8N](https://zwiftpower\.com/team\.php?id=2936)|—|—|—|[36th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|90|927|[Mathieu Drujon 🎩🔰](https://zwiftpower\.com/profile\.php?z=75218)|[Zwift All Stars](https://zwiftpower\.com/team\.php?id=3702)|—|[37th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
A|90|927|[David Fidalgo](https://zwiftpower\.com/profile\.php?z=197561)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|—|[37th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|90|927|[David Pell](https://zwiftpower\.com/profile\.php?z=772768)|—|—|—|—|—|—|[37th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
A|91|925|[Samuel Leguevaques](https://zwiftpower\.com/profile\.php?z=44125)|[Vision](https://zwiftpower\.com/team\.php?id=9)|—|—|[38th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
A|91|925|[Jakob Grønbæk, LAMONT](https://zwiftpower\.com/profile\.php?z=79657)|[LAMONT](https://zwiftpower\.com/team\.php?id=302)|—|—|—|—|[38th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|91|925|[takaaki kawai \(a\)8839](https://zwiftpower\.com/profile\.php?z=166760)|—|—|—|—|—|—|—|[38th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|92|923|[T Sato](https://zwiftpower\.com/profile\.php?z=136374)|—|—|—|—|[39th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|92|923|[Edwin Driessen](https://zwiftpower\.com/profile\.php?z=219489)|—|—|—|—|—|—|[39th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
A|93|919|[Lucas Van Drunen](https://zwiftpower\.com/profile\.php?z=6184)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|—|—|—|—|[41st](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|93|919|[Dan Swenson9257](https://zwiftpower\.com/profile\.php?z=89267)|—|—|[41st](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
A|93|919|[Steve Chadburn \[VFOX\]](https://zwiftpower\.com/profile\.php?z=822255)|[Vixen/Fox](https://zwiftpower\.com/team\.php?id=1940)|—|—|—|[41st](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|94|917|[Marjan Dolamic Jr\.](https://zwiftpower\.com/profile\.php?z=69480)|[RT WINAX\-food artists](https://zwiftpower\.com/team\.php?id=3928)|—|—|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|94|917|[Hiroshi Kikuchi](https://zwiftpower\.com/profile\.php?z=155607)|—|—|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|95|915|[Sven Gerhardt INC 👽](https://zwiftpower\.com/profile\.php?z=982806)|[Innovation](https://zwiftpower\.com/team\.php?id=975)|—|—|—|—|—|[43rd](https://zwiftpower\.com/events\.php?zid=171934)|—|—
A|96|913|[yuta otani](https://zwiftpower\.com/profile\.php?z=247629)|—|—|—|—|[44th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|96|913|[Yasuyuki Kobayashi](https://zwiftpower\.com/profile\.php?z=917068)|[Team ZWC](https://zwiftpower\.com/team\.php?id=1958)|—|—|—|—|—|—|[44th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|96|913|[Jerry Olsson](https://zwiftpower\.com/profile\.php?z=996335)|—|—|[44th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
A|97|911|[Jonathan Wass](https://zwiftpower\.com/profile\.php?z=158486)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|—|—|—|[45th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
A|98|909|[Elizabeth Ferguson 8217](https://zwiftpower\.com/profile\.php?z=991475)|—|—|[46th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
A|99|907|[Ghisallo Totsuka](https://zwiftpower\.com/profile\.php?z=90444)|—|—|—|—|—|—|—|[47th](https://zwiftpower\.com/events\.php?zid=173434)|—
A|99|907|[Michi Kangaroo](https://zwiftpower\.com/profile\.php?z=218825)|—|—|—|—|[47th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|99|907|[Nicky Manson](https://zwiftpower\.com/profile\.php?z=237508)|[Vision](https://zwiftpower\.com/team\.php?id=9)|—|—|—|—|[47th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|100|903|[alm0st\.fast \(KC\)](https://zwiftpower\.com/profile\.php?z=458147)|[KirchmairCycling](https://zwiftpower\.com/team\.php?id=2991)|—|—|—|[49th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|101|901|[Dave Shacks](https://zwiftpower\.com/profile\.php?z=47745)|[Hale Velo](https://zwiftpower\.com/team\.php?id=1981)|—|—|—|—|[50th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
A|102|899|[HIPSTER p�p�9113�](https://zwiftpower\.com/profile\.php?z=460423)|[CYC](https://zwiftpower\.com/team\.php?id=2957)|—|—|—|[51st](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
A|103|897|[Dennis Lock](https://zwiftpower\.com/profile\.php?z=979320)|[DBR](https://zwiftpower\.com/team\.php?id=1986)|—|—|—|—|[52nd](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
B|1|7948|[Rick Barbera](https://zwiftpower\.com/profile\.php?z=2946)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[7th](https://zwiftpower\.com/events\.php?zid=151390)|[6th](https://zwiftpower\.com/events\.php?zid=155565)|[🥉](https://zwiftpower\.com/events\.php?zid=159271)|[🥇](https://zwiftpower\.com/events\.php?zid=162625)|[6th](https://zwiftpower\.com/events\.php?zid=166628)|[5th](https://zwiftpower\.com/events\.php?zid=171934)|[🥇](https://zwiftpower\.com/events\.php?zid=173434)|[🥇](https://zwiftpower\.com/events\.php?zid=176147)
B|2|7852|[Florin Vasilescu](https://zwiftpower\.com/profile\.php?z=167)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[19th](https://zwiftpower\.com/events\.php?zid=151390)|[11th](https://zwiftpower\.com/events\.php?zid=155565)|[6th](https://zwiftpower\.com/events\.php?zid=159271)|[14th](https://zwiftpower\.com/events\.php?zid=162625)|[4th](https://zwiftpower\.com/events\.php?zid=166628)|[17th](https://zwiftpower\.com/events\.php?zid=171934)|[5th](https://zwiftpower\.com/events\.php?zid=173434)|[🥈](https://zwiftpower\.com/events\.php?zid=176147)
B|2|7852|[Matt Montgomery](https://zwiftpower\.com/profile\.php?z=240716)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[6th](https://zwiftpower\.com/events\.php?zid=151390)|[13th](https://zwiftpower\.com/events\.php?zid=155565)|[5th](https://zwiftpower\.com/events\.php?zid=159271)|[17th](https://zwiftpower\.com/events\.php?zid=162625)|[13th](https://zwiftpower\.com/events\.php?zid=166628)|[6th](https://zwiftpower\.com/events\.php?zid=171934)|[12th](https://zwiftpower\.com/events\.php?zid=173434)|[6th](https://zwiftpower\.com/events\.php?zid=176147)
B|3|7812|[Bryan Montgomery](https://zwiftpower\.com/profile\.php?z=199436)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[12th](https://zwiftpower\.com/events\.php?zid=151390)|[16th](https://zwiftpower\.com/events\.php?zid=155565)|[4th](https://zwiftpower\.com/events\.php?zid=159271)|[12th](https://zwiftpower\.com/events\.php?zid=162625)|[20th](https://zwiftpower\.com/events\.php?zid=166628)|[13th](https://zwiftpower\.com/events\.php?zid=171934)|[18th](https://zwiftpower\.com/events\.php?zid=173434)|[🥉](https://zwiftpower\.com/events\.php?zid=176147)
B|4|7792|[Jenn Edwards\(EnduranceNation\.us\)](https://zwiftpower\.com/profile\.php?z=183549)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[16th](https://zwiftpower\.com/events\.php?zid=151390)|[24th](https://zwiftpower\.com/events\.php?zid=155565)|[14th](https://zwiftpower\.com/events\.php?zid=159271)|[10th](https://zwiftpower\.com/events\.php?zid=162625)|[🥉](https://zwiftpower\.com/events\.php?zid=166628)|[7th](https://zwiftpower\.com/events\.php?zid=171934)|[21st](https://zwiftpower\.com/events\.php?zid=173434)|[13th](https://zwiftpower\.com/events\.php?zid=176147)
B|5|7740|[Nick Ashworth](https://zwiftpower\.com/profile\.php?z=260349)|[Summit](https://zwiftpower\.com/team\.php?id=3919)|[20th](https://zwiftpower\.com/events\.php?zid=151390)|[31st](https://zwiftpower\.com/events\.php?zid=155565)|[9th](https://zwiftpower\.com/events\.php?zid=159271)|[9th](https://zwiftpower\.com/events\.php?zid=162625)|[22nd](https://zwiftpower\.com/events\.php?zid=166628)|[8th](https://zwiftpower\.com/events\.php?zid=171934)|[24th](https://zwiftpower\.com/events\.php?zid=173434)|[11th](https://zwiftpower\.com/events\.php?zid=176147)
B|6|6919|[Just A Rollerblader Ⓥ 🌱](https://zwiftpower\.com/profile\.php?z=114584)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥉](https://zwiftpower\.com/events\.php?zid=151390)|[🥈](https://zwiftpower\.com/events\.php?zid=155565)|[🥈](https://zwiftpower\.com/events\.php?zid=159271)|[13th](https://zwiftpower\.com/events\.php?zid=162625)|[7th](https://zwiftpower\.com/events\.php?zid=166628)|[11th](https://zwiftpower\.com/events\.php?zid=171934)|[6th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|7|6669|[Mark Stahlkopf](https://zwiftpower\.com/profile\.php?z=329928)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[21st](https://zwiftpower\.com/events\.php?zid=151390)|[32nd](https://zwiftpower\.com/events\.php?zid=155565)|[22nd](https://zwiftpower\.com/events\.php?zid=159271)|[21st](https://zwiftpower\.com/events\.php?zid=162625)|[24th](https://zwiftpower\.com/events\.php?zid=166628)|—|[32nd](https://zwiftpower\.com/events\.php?zid=173434)|[17th](https://zwiftpower\.com/events\.php?zid=176147)
B|8|5696|[Brian Hagan](https://zwiftpower\.com/profile\.php?z=102945)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[23rd](https://zwiftpower\.com/events\.php?zid=151390)|[35th](https://zwiftpower\.com/events\.php?zid=155565)|—|[20th](https://zwiftpower\.com/events\.php?zid=162625)|[27th](https://zwiftpower\.com/events\.php?zid=166628)|[23rd](https://zwiftpower\.com/events\.php?zid=171934)|[27th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|9|5664|[Scott Giljum \(endurancenation\.us\)](https://zwiftpower\.com/profile\.php?z=341113)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[22nd](https://zwiftpower\.com/events\.php?zid=151390)|[34th](https://zwiftpower\.com/events\.php?zid=155565)|[24th](https://zwiftpower\.com/events\.php?zid=159271)|—|[32nd](https://zwiftpower\.com/events\.php?zid=166628)|[25th](https://zwiftpower\.com/events\.php?zid=171934)|[34th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|10|4937|[Timothy Busick](https://zwiftpower\.com/profile\.php?z=6158)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[8th](https://zwiftpower\.com/events\.php?zid=151390)|[14th](https://zwiftpower\.com/events\.php?zid=155565)|—|[4th](https://zwiftpower\.com/events\.php?zid=162625)|[5th](https://zwiftpower\.com/events\.php?zid=166628)|—|[🥉](https://zwiftpower\.com/events\.php?zid=173434)|—
B|11|4889|[JD Reich](https://zwiftpower\.com/profile\.php?z=832405)|[Summit](https://zwiftpower\.com/team\.php?id=3919)|[10th](https://zwiftpower\.com/events\.php?zid=151390)|[10th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=171934)|[17th](https://zwiftpower\.com/events\.php?zid=173434)|[5th](https://zwiftpower\.com/events\.php?zid=176147)
B|12|4839|[Shaun Solhjou](https://zwiftpower\.com/profile\.php?z=213130)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|[23rd](https://zwiftpower\.com/events\.php?zid=155565)|—|[19th](https://zwiftpower\.com/events\.php?zid=162625)|[21st](https://zwiftpower\.com/events\.php?zid=166628)|—|[16th](https://zwiftpower\.com/events\.php?zid=173434)|[4th](https://zwiftpower\.com/events\.php?zid=176147)
B|13|3924|[Mark Scoot](https://zwiftpower\.com/profile\.php?z=62584)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|[5th](https://zwiftpower\.com/events\.php?zid=151390)|[12th](https://zwiftpower\.com/events\.php?zid=155565)|[15th](https://zwiftpower\.com/events\.php?zid=159271)|[8th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
B|14|3884|[Tex Divine](https://zwiftpower\.com/profile\.php?z=82152)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|[🥉](https://zwiftpower\.com/events\.php?zid=155565)|—|[🥈](https://zwiftpower\.com/events\.php?zid=162625)|—|—|[37th](https://zwiftpower\.com/events\.php?zid=173434)|[18th](https://zwiftpower\.com/events\.php?zid=176147)
B|15|3846|[S Ssmith \(V\) Ⓥ ^\.\.^ 🐣🌱🐾](https://zwiftpower\.com/profile\.php?z=175002)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=162625)|[12th](https://zwiftpower\.com/events\.php?zid=166628)|[20th](https://zwiftpower\.com/events\.php?zid=171934)|[25th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|16|2955|[scott prosser 4206](https://zwiftpower\.com/profile\.php?z=119089)|[Team BMTR](https://zwiftpower\.com/team\.php?id=2873)|—|—|[10th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=171934)|[11th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|16|2955|[Jason Sterny](https://zwiftpower\.com/profile\.php?z=413819)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=162625)|[11th](https://zwiftpower\.com/events\.php?zid=166628)|[10th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
B|17|2921|[Shigeru Maehara](https://zwiftpower\.com/profile\.php?z=193217)|[TECHNOPOLIS](https://zwiftpower\.com/team\.php?id=3906)|—|—|[12th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=171934)|—|[7th](https://zwiftpower\.com/events\.php?zid=176147)
B|18|2907|[Graham Partain](https://zwiftpower\.com/profile\.php?z=64048)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|[15th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|[14th](https://zwiftpower\.com/events\.php?zid=171934)|[19th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|19|2893|[Jouni Malmivaara](https://zwiftpower\.com/profile\.php?z=176062)|[Summit](https://zwiftpower\.com/team\.php?id=3919)|—|—|[19th](https://zwiftpower\.com/events\.php?zid=159271)|[18th](https://zwiftpower\.com/events\.php?zid=162625)|—|[18th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
B|20|2877|[jakob wilson](https://zwiftpower\.com/profile\.php?z=64351)|[OMAHA VELO](https://zwiftpower\.com/team\.php?id=3984)|—|[38th](https://zwiftpower\.com/events\.php?zid=155565)|[11th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|[14th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|21|2841|[Ian Murray](https://zwiftpower\.com/profile\.php?z=20)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|—|[23rd](https://zwiftpower\.com/events\.php?zid=162625)|—|[28th](https://zwiftpower\.com/events\.php?zid=171934)|[30th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|22|1986|[Franco Battiato](https://zwiftpower\.com/profile\.php?z=14632)|[Team Z TAZ](https://zwiftpower\.com/team\.php?id=496)|[🥈](https://zwiftpower\.com/events\.php?zid=151390)|—|—|[6th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
B|23|1984|[Danny Baker Morvelo Basso RT](https://zwiftpower\.com/profile\.php?z=341967)|—|—|[7th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=171934)|—|—
B|24|1980|[Aleksandr Simashin 🐻](https://zwiftpower\.com/profile\.php?z=45225)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=171934)|[🥈](https://zwiftpower\.com/events\.php?zid=173434)|—
B|24|1980|[Ryan Kettleson](https://zwiftpower\.com/profile\.php?z=188883)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[4th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|[7th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
B|25|1962|[jhonny luke](https://zwiftpower\.com/profile\.php?z=388191)|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|[15th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|26|1960|[kosuke matsuura](https://zwiftpower\.com/profile\.php?z=1015507)|—|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|[12th](https://zwiftpower\.com/events\.php?zid=176147)
B|27|1958|[Donald Ritger](https://zwiftpower\.com/profile\.php?z=228063)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|[9th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|28|1952|[Rob Peters \(Team EN\)](https://zwiftpower\.com/profile\.php?z=536679)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|—|[17th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|29|1946|[simone sighinolfi Ⓥ](https://zwiftpower\.com/profile\.php?z=1085650)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|—|—|—|[20th](https://zwiftpower\.com/events\.php?zid=173434)|[8th](https://zwiftpower\.com/events\.php?zid=176147)
B|30|1936|[Wataru Yamada](https://zwiftpower\.com/profile\.php?z=626778)|—|—|—|[17th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=176147)
B|31|1934|[Francesco Roberto Bell 🔔](https://zwiftpower\.com/profile\.php?z=328472)|[Vision](https://zwiftpower\.com/team\.php?id=9)|[13th](https://zwiftpower\.com/events\.php?zid=151390)|[21st](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|32|1910|[Rene Meijer](https://zwiftpower\.com/profile\.php?z=803509)|[⚠️ Dropouts](https://zwiftpower\.com/team\.php?id=3711)|[17th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|[29th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
B|33|1906|[Martin Heintzelman](https://zwiftpower\.com/profile\.php?z=236040)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|—|—|[25th](https://zwiftpower\.com/events\.php?zid=166628)|—|[23rd](https://zwiftpower\.com/events\.php?zid=173434)|—
B|34|1902|[Greg Low](https://zwiftpower\.com/profile\.php?z=162837)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|—|—|—|—|[29th](https://zwiftpower\.com/events\.php?zid=173434)|[21st](https://zwiftpower\.com/events\.php?zid=176147)
B|35|1894|[Tatsuya Kitamura](https://zwiftpower\.com/profile\.php?z=178483)|—|—|[30th](https://zwiftpower\.com/events\.php?zid=155565)|—|[24th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
B|36|1890|[kainan](https://zwiftpower\.com/profile\.php?z=368108)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|—|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=166628)|[26th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
B|36|1890|[Yannick Lagadec](https://zwiftpower\.com/profile\.php?z=885274)|—|—|—|—|[27th](https://zwiftpower\.com/events\.php?zid=162625)|[29th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
B|37|1882|[Kristi Mccracken](https://zwiftpower\.com/profile\.php?z=195887)|[LPC](https://zwiftpower\.com/team\.php?id=2089)|—|—|—|—|—|—|[41st](https://zwiftpower\.com/events\.php?zid=173434)|[19th](https://zwiftpower\.com/events\.php?zid=176147)
B|38|1868|[Matt Partridge \[DIRT\]](https://zwiftpower\.com/profile\.php?z=458388)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|[43rd](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|[24th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
B|39|1854|[Bret Elder](https://zwiftpower\.com/profile\.php?z=128929)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|—|—|[36th](https://zwiftpower\.com/events\.php?zid=166628)|—|[38th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|40|999|[Matthew Crawford 🌵](https://zwiftpower\.com/profile\.php?z=11265)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[🥇](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
B|40|999|[adrian vizzari](https://zwiftpower\.com/profile\.php?z=17189)|—|—|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
B|40|999|[Peter Mosbaek 🔰](https://zwiftpower\.com/profile\.php?z=43225)|[Vision](https://zwiftpower\.com/team\.php?id=9)|—|[🥇](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|40|999|[Gregg O'Malley](https://zwiftpower\.com/profile\.php?z=162054)|[FINESSE](https://zwiftpower\.com/team\.php?id=2779)|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
B|40|999|[Joergen Yksnoey Rakvaag](https://zwiftpower\.com/profile\.php?z=448140)|[DBR](https://zwiftpower\.com/team\.php?id=1986)|—|—|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=171934)|—|—
B|41|997|[Rik van IJzendoorn](https://zwiftpower\.com/profile\.php?z=34365)|—|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
B|42|993|[Maxime F](https://zwiftpower\.com/profile\.php?z=177040)|—|—|—|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
B|42|993|[Thomas Stemmet](https://zwiftpower\.com/profile\.php?z=732876)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|42|993|[jongmoon chae5931](https://zwiftpower\.com/profile\.php?z=1021389)|—|—|[4th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|43|991|[Clive King](https://zwiftpower\.com/profile\.php?z=1978)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|[5th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|44|987|[Marlene Bjärehed \[VVixen\] 🔰](https://zwiftpower\.com/profile\.php?z=289822)|[Vixen/Fox](https://zwiftpower\.com/team\.php?id=1940)|—|—|—|—|—|—|[7th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|44|987|[Nathan Kershner 🅥](https://zwiftpower\.com/profile\.php?z=920279)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|[7th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
B|45|985|[Adam Chevin Ⓥ🌱](https://zwiftpower\.com/profile\.php?z=70177)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
B|45|985|[So Gyuseok/Team Trek\-Hwashin](https://zwiftpower\.com/profile\.php?z=178293)|[SilentK](https://zwiftpower\.com/team\.php?id=4129)|—|[8th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|45|985|[Janne Jakobsson](https://zwiftpower\.com/profile\.php?z=331176)|[Sz](https://zwiftpower\.com/team\.php?id=32)|—|—|[8th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
B|46|983|[Stephan Maertens Ⓥ 🐇🥕](https://zwiftpower\.com/profile\.php?z=37710)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|46|983|[M BeenSlacking](https://zwiftpower\.com/profile\.php?z=88308)|—|—|—|—|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=176147)
B|46|983|[Pierre Boorg](https://zwiftpower\.com/profile\.php?z=311040)|—|[9th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
B|47|981|[Ben Koh](https://zwiftpower\.com/profile\.php?z=16733)|[\[AsC\]](https://zwiftpower\.com/team\.php?id=3249)|—|—|—|—|—|—|—|[10th](https://zwiftpower\.com/events\.php?zid=176147)
B|47|981|[Laura Epelbaum](https://zwiftpower\.com/profile\.php?z=377170)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|—|—|—|—|[10th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
B|47|981|[Craig Wood](https://zwiftpower\.com/profile\.php?z=1115894)|[WATTBIKE](https://zwiftpower\.com/team\.php?id=2738)|—|—|—|—|—|—|[10th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|48|979|[Ian Anderson 🍊🚅](https://zwiftpower\.com/profile\.php?z=46121)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
B|48|979|[cyril TOURMENTE](https://zwiftpower\.com/profile\.php?z=855845)|[Foudre ⚡️](https://zwiftpower\.com/team\.php?id=2898)|[11th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
B|49|977|[Jasper Vinten \[DBR\]\-RB](https://zwiftpower\.com/profile\.php?z=417880)|[DBR](https://zwiftpower\.com/team\.php?id=1986)|—|—|—|—|—|[12th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
B|50|975|[John Shin Ⓥ](https://zwiftpower\.com/profile\.php?z=24810)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|[13th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
B|51|973|[Jono Finan \(IRL\)](https://zwiftpower\.com/profile\.php?z=210648)|[IRL](https://zwiftpower\.com/team\.php?id=3369)|—|—|—|—|[14th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
B|51|973|[Kazuki Nishimori](https://zwiftpower\.com/profile\.php?z=220282)|—|[14th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
B|51|973|[David Szentesi6476](https://zwiftpower\.com/profile\.php?z=374304)|—|—|—|—|—|—|—|—|[14th](https://zwiftpower\.com/events\.php?zid=176147)
B|52|971|[Korea Tan](https://zwiftpower\.com/profile\.php?z=18034)|[P2K](https://zwiftpower\.com/team\.php?id=1366)|—|—|—|—|—|—|—|[15th](https://zwiftpower\.com/events\.php?zid=176147)
B|52|971|[Piotr Ciurzyński](https://zwiftpower\.com/profile\.php?z=161070)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|[15th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
B|52|971|[Sergio Palacio](https://zwiftpower\.com/profile\.php?z=394487)|—|—|—|—|—|—|[15th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
B|52|971|[Sveinung Pedersen \[V\]](https://zwiftpower\.com/profile\.php?z=461978)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|—|—|—|[15th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
B|52|971|[Joe Randall](https://zwiftpower\.com/profile\.php?z=887096)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|—|—|[15th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
B|53|969|[Sakawa Agano](https://zwiftpower\.com/profile\.php?z=206174)|—|—|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
B|53|969|[Mike Lister \(DACE\)](https://zwiftpower\.com/profile\.php?z=368678)|[DACE](https://zwiftpower\.com/team\.php?id=2177)|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
B|53|969|[Igor Balakirev](https://zwiftpower\.com/profile\.php?z=441927)|[KLD](https://zwiftpower\.com/team\.php?id=2163)|—|—|[16th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
B|54|967|[Maxwell Smith](https://zwiftpower\.com/profile\.php?z=860816)|—|—|—|—|—|[17th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
B|55|965|[Dean Hardie](https://zwiftpower\.com/profile\.php?z=96086)|[TFC](https://zwiftpower\.com/team\.php?id=10)|[18th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
B|55|965|[TAKASHI MAYAMA](https://zwiftpower\.com/profile\.php?z=156051)|—|—|[18th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|55|965|[Eldon Pyburn](https://zwiftpower\.com/profile\.php?z=250404)|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
B|55|965|[Tsutomu Ohno](https://zwiftpower\.com/profile\.php?z=1157477)|—|—|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
B|56|963|[Erick Kercheval](https://zwiftpower\.com/profile\.php?z=234962)|[NLC](https://zwiftpower\.com/team\.php?id=1957)|—|[19th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|56|963|[Heung Jini \[MAFIA\]](https://zwiftpower\.com/profile\.php?z=501891)|—|—|—|—|—|[19th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
B|56|963|[Trevor Waggoner7729](https://zwiftpower\.com/profile\.php?z=922739)|—|—|—|—|—|—|[19th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
B|57|961|[Dave Mehl TeamEN \(C\) 7824](https://zwiftpower\.com/profile\.php?z=8539)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|—|—|—|—|—|—|—|[20th](https://zwiftpower\.com/events\.php?zid=176147)
B|57|961|[Alex Hatziyannis](https://zwiftpower\.com/profile\.php?z=103835)|[CRUZBIKE](https://zwiftpower\.com/team\.php?id=1922)|—|[20th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|57|961|[Christophe Moeri1173](https://zwiftpower\.com/profile\.php?z=1137726)|—|—|—|[20th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
B|58|959|[생수 \[PipeLine CyClub\]9039](https://zwiftpower\.com/profile\.php?z=220373)|—|—|—|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=171934)|—|—
B|58|959|[Terenzo Downey](https://zwiftpower\.com/profile\.php?z=400481)|[Team Italy](https://zwiftpower\.com/team\.php?id=588)|—|—|[21st](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
B|59|957|[Daniel Hunt OCC](https://zwiftpower\.com/profile\.php?z=182368)|[OCC](https://zwiftpower\.com/team\.php?id=2620)|—|[22nd](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|59|957|[ni qb](https://zwiftpower\.com/profile\.php?z=993849)|—|—|—|—|—|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=173434)|—
B|60|955|[Masayasu Saitou\(panda\)](https://zwiftpower\.com/profile\.php?z=96502)|—|—|—|—|—|[23rd](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
B|60|955|[Lewis Wake](https://zwiftpower\.com/profile\.php?z=900521)|[KCRT](https://zwiftpower\.com/team\.php?id=4040)|—|—|[23rd](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
B|61|951|[Fred Grafe](https://zwiftpower\.com/profile\.php?z=7074)|—|—|—|—|[25th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
B|61|951|[Claude Richard](https://zwiftpower\.com/profile\.php?z=63288)|—|—|[25th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|62|949|[Scott Adams](https://zwiftpower\.com/profile\.php?z=395445)|—|—|—|—|[26th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
B|62|949|[Akio Yamauchi \#EN](https://zwiftpower\.com/profile\.php?z=458632)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|—|—|—|—|—|—|[26th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|62|949|[Claus D Christiansen \[DBR\]](https://zwiftpower\.com/profile\.php?z=833347)|[DBR](https://zwiftpower\.com/team\.php?id=1986)|—|[26th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|62|949|[Sjors Jasperse](https://zwiftpower\.com/profile\.php?z=1026776)|—|—|—|—|—|[26th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
B|63|947|[Jennifer Crehan\. \(CLS\)](https://zwiftpower\.com/profile\.php?z=452471)|—|—|—|—|—|—|[27th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
B|63|947|[MHA\_ KOH\!\! 9595](https://zwiftpower\.com/profile\.php?z=587530)|—|—|[27th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|64|945|[Armando Mastracci XERT](https://zwiftpower\.com/profile\.php?z=93176)|—|—|[28th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|64|945|[Mattias Persson](https://zwiftpower\.com/profile\.php?z=172970)|—|—|—|—|[28th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
B|64|945|[Conor Dalby](https://zwiftpower\.com/profile\.php?z=396581)|—|—|—|—|—|—|—|[28th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|64|945|[Benedict Read](https://zwiftpower\.com/profile\.php?z=758452)|—|—|—|—|—|[28th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
B|65|943|[Scott Olsen](https://zwiftpower\.com/profile\.php?z=5027)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|[29th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|65|943|[박 성필4699](https://zwiftpower\.com/profile\.php?z=528613)|[CaM](https://zwiftpower\.com/team\.php?id=4120)|—|—|—|[29th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
B|66|941|[Jonas Spinoy Ⓥ 🇧🇪](https://zwiftpower\.com/profile\.php?z=347291)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
B|66|941|[Christoph Schmidt](https://zwiftpower\.com/profile\.php?z=1101955)|—|—|—|—|—|—|[30th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
B|67|939|[Adam Blake \[DIRT\]](https://zwiftpower\.com/profile\.php?z=344281)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|—|—|—|—|[31st](https://zwiftpower\.com/events\.php?zid=173434)|—
B|67|939|[Emily Slavin](https://zwiftpower\.com/profile\.php?z=385268)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|—|—|—|—|[31st](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
B|67|939|[ALEXANDER IVAKHIN](https://zwiftpower\.com/profile\.php?z=928303)|—|—|—|—|—|—|[31st](https://zwiftpower\.com/events\.php?zid=171934)|—|—
B|68|935|[Mark Creaby](https://zwiftpower\.com/profile\.php?z=55551)|—|—|—|—|—|—|—|[33rd](https://zwiftpower\.com/events\.php?zid=173434)|—
B|68|935|[Brian Monaghan](https://zwiftpower\.com/profile\.php?z=308807)|—|—|[33rd](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|68|935|[David Santos](https://zwiftpower\.com/profile\.php?z=437626)|[TugaZ](https://zwiftpower\.com/team\.php?id=3652)|—|—|—|—|[33rd](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
B|69|933|[Jesper Alberts 3760](https://zwiftpower\.com/profile\.php?z=413981)|—|—|—|—|—|[34th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
B|70|931|[Steven Rittenberg](https://zwiftpower\.com/profile\.php?z=558133)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|—|—|—|—|[35th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|70|931|[Les \- carlos](https://zwiftpower\.com/profile\.php?z=1057711)|—|—|—|—|—|[35th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
B|71|929|[FM 2k7](https://zwiftpower\.com/profile\.php?z=555098)|—|—|—|—|—|—|—|[36th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|71|929|[Paul Brehm](https://zwiftpower\.com/profile\.php?z=668328)|—|—|[36th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|72|927|[Vince Rowbottom](https://zwiftpower\.com/profile\.php?z=135040)|[WPTAZ](https://zwiftpower\.com/team\.php?id=23)|—|[37th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|73|923|[Richard Borget 6722](https://zwiftpower\.com/profile\.php?z=241408)|[SAS Cycle Club](https://zwiftpower\.com/team\.php?id=1348)|—|[39th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|73|923|[Don Bly](https://zwiftpower\.com/profile\.php?z=1134071)|—|—|—|—|—|—|—|[39th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|74|921|[Brian Martin](https://zwiftpower\.com/profile\.php?z=326628)|—|—|[40th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|74|921|[Rashidah](https://zwiftpower\.com/profile\.php?z=912236)|—|—|—|—|—|—|—|[40th](https://zwiftpower\.com/events\.php?zid=173434)|—
B|75|919|[Tom Haug](https://zwiftpower\.com/profile\.php?z=967068)|—|—|[41st](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|76|917|[Minero Ⓥ](https://zwiftpower\.com/profile\.php?z=144549)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|[42nd](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
B|76|917|[Cofitan Yeondoo/WATS](https://zwiftpower\.com/profile\.php?z=444658)|—|—|—|—|—|—|—|[42nd](https://zwiftpower\.com/events\.php?zid=173434)|—
B|77|915|[Kevin Robson5647](https://zwiftpower\.com/profile\.php?z=1099532)|[STOLEN GOAT](https://zwiftpower\.com/team\.php?id=2451)|—|—|—|—|—|—|[43rd](https://zwiftpower\.com/events\.php?zid=173434)|—
B|78|913|[Min Hong](https://zwiftpower\.com/profile\.php?z=68539)|[CaM](https://zwiftpower\.com/team\.php?id=4120)|—|[44th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
C|1|7944|[Tim dODZ](https://zwiftpower\.com/profile\.php?z=68374)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|[4th](https://zwiftpower\.com/events\.php?zid=151390)|[🥈](https://zwiftpower\.com/events\.php?zid=155565)|[8th](https://zwiftpower\.com/events\.php?zid=159271)|[7th](https://zwiftpower\.com/events\.php?zid=162625)|[🥉](https://zwiftpower\.com/events\.php?zid=166628)|[🥉](https://zwiftpower\.com/events\.php?zid=171934)|[🥈](https://zwiftpower\.com/events\.php?zid=173434)|[🥉](https://zwiftpower\.com/events\.php?zid=176147)
C|2|6851|[Gerrie Delport](https://zwiftpower\.com/profile\.php?z=23452)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|[8th](https://zwiftpower\.com/events\.php?zid=155565)|[14th](https://zwiftpower\.com/events\.php?zid=159271)|[15th](https://zwiftpower\.com/events\.php?zid=162625)|[17th](https://zwiftpower\.com/events\.php?zid=166628)|[14th](https://zwiftpower\.com/events\.php?zid=171934)|[🥉](https://zwiftpower\.com/events\.php?zid=173434)|[7th](https://zwiftpower\.com/events\.php?zid=176147)
C|3|5936|[Eric Olds \[BIKEMAN\.COM\]](https://zwiftpower\.com/profile\.php?z=873324)|[Bikeman\.com](https://zwiftpower\.com/team\.php?id=3969)|—|—|[6th](https://zwiftpower\.com/events\.php?zid=159271)|[8th](https://zwiftpower\.com/events\.php?zid=162625)|[6th](https://zwiftpower\.com/events\.php?zid=166628)|[4th](https://zwiftpower\.com/events\.php?zid=171934)|[6th](https://zwiftpower\.com/events\.php?zid=173434)|[5th](https://zwiftpower\.com/events\.php?zid=176147)
C|4|5890|[Morten Wernersen](https://zwiftpower\.com/profile\.php?z=727275)|[Vikings](https://zwiftpower\.com/team\.php?id=90)|[5th](https://zwiftpower\.com/events\.php?zid=151390)|[🥉](https://zwiftpower\.com/events\.php?zid=155565)|[12th](https://zwiftpower\.com/events\.php?zid=159271)|[24th](https://zwiftpower\.com/events\.php?zid=162625)|[🥇](https://zwiftpower\.com/events\.php?zid=166628)|[13th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
C|5|5844|[Ralph Moore \(endurancenation\.us\)](https://zwiftpower\.com/profile\.php?z=119074)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|—|[5th](https://zwiftpower\.com/events\.php?zid=155565)|[13th](https://zwiftpower\.com/events\.php?zid=159271)|[17th](https://zwiftpower\.com/events\.php?zid=162625)|[18th](https://zwiftpower\.com/events\.php?zid=166628)|[16th](https://zwiftpower\.com/events\.php?zid=171934)|—|[12th](https://zwiftpower\.com/events\.php?zid=176147)
C|6|4977|[Bastian Zwack](https://zwiftpower\.com/profile\.php?z=116546)|[Summit](https://zwiftpower\.com/team\.php?id=3919)|[🥇](https://zwiftpower\.com/events\.php?zid=151390)|[🥇](https://zwiftpower\.com/events\.php?zid=155565)|—|[9th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=173434)|[🥈](https://zwiftpower\.com/events\.php?zid=176147)
C|7|4931|[Mitch Dangremond \(Trisuppo\)](https://zwiftpower\.com/profile\.php?z=15790)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|[7th](https://zwiftpower\.com/events\.php?zid=159271)|[6th](https://zwiftpower\.com/events\.php?zid=162625)|[10th](https://zwiftpower\.com/events\.php?zid=166628)|[7th](https://zwiftpower\.com/events\.php?zid=171934)|[7th](https://zwiftpower\.com/events\.php?zid=173434)|—
C|8|4871|[R oryG \#TeamEN](https://zwiftpower\.com/profile\.php?z=391588)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|—|[10th](https://zwiftpower\.com/events\.php?zid=155565)|—|[14th](https://zwiftpower\.com/events\.php?zid=162625)|—|[19th](https://zwiftpower\.com/events\.php?zid=171934)|[16th](https://zwiftpower\.com/events\.php?zid=173434)|[8th](https://zwiftpower\.com/events\.php?zid=176147)
C|9|3972|[Sabra Gonzalez Team EN](https://zwiftpower\.com/profile\.php?z=329464)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=159271)|[5th](https://zwiftpower\.com/events\.php?zid=162625)|—|[5th](https://zwiftpower\.com/events\.php?zid=171934)|[5th](https://zwiftpower\.com/events\.php?zid=173434)|—
C|10|3920|[Derrek Sanks TeamEN](https://zwiftpower\.com/profile\.php?z=11111)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[6th](https://zwiftpower\.com/events\.php?zid=151390)|—|[11th](https://zwiftpower\.com/events\.php?zid=159271)|—|[14th](https://zwiftpower\.com/events\.php?zid=166628)|—|[11th](https://zwiftpower\.com/events\.php?zid=173434)|—
C|11|2963|[Skyler Street](https://zwiftpower\.com/profile\.php?z=120373)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|[14th](https://zwiftpower\.com/events\.php?zid=151390)|—|[🥉](https://zwiftpower\.com/events\.php?zid=159271)|[🥉](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
C|12|2957|[David Bussey](https://zwiftpower\.com/profile\.php?z=4725)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=171934)|[10th](https://zwiftpower\.com/events\.php?zid=173434)|[4th](https://zwiftpower\.com/events\.php?zid=176147)
C|13|2955|[Rich Stanbaugh \#EN](https://zwiftpower\.com/profile\.php?z=416912)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|—|—|—|—|[12th](https://zwiftpower\.com/events\.php?zid=166628)|[6th](https://zwiftpower\.com/events\.php?zid=171934)|—|[6th](https://zwiftpower\.com/events\.php?zid=176147)
C|14|2951|[Ben Greenhalgh Cryogen C](https://zwiftpower\.com/profile\.php?z=145025)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|[5th](https://zwiftpower\.com/events\.php?zid=159271)|[12th](https://zwiftpower\.com/events\.php?zid=162625)|[9th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
C|14|2951|[Brad Wagz ODZ](https://zwiftpower\.com/profile\.php?z=1114373)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=162625)|—|[8th](https://zwiftpower\.com/events\.php?zid=171934)|[14th](https://zwiftpower\.com/events\.php?zid=173434)|—
C|15|2923|[Hans VEGAN](https://zwiftpower\.com/profile\.php?z=422779)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|[9th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|[16th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|[15th](https://zwiftpower\.com/events\.php?zid=176147)
C|16|1998|[Anina Blankenship 5603](https://zwiftpower\.com/profile\.php?z=29249)|[Team Fearless](https://zwiftpower\.com/team\.php?id=1677)|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=162625)|—|[🥇](https://zwiftpower\.com/events\.php?zid=171934)|—|—
C|17|1994|[Greg Low](https://zwiftpower\.com/profile\.php?z=162837)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=166628)|[🥈](https://zwiftpower\.com/events\.php?zid=171934)|—|—
C|18|1964|[Kamil Szelag](https://zwiftpower\.com/profile\.php?z=610990)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=166628)|[15th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
C|19|1962|[Megan Herzog](https://zwiftpower\.com/profile\.php?z=120109)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=171934)|[9th](https://zwiftpower\.com/events\.php?zid=173434)|—
C|20|1944|[Joseph Pittman](https://zwiftpower\.com/profile\.php?z=522522)|—|[12th](https://zwiftpower\.com/events\.php?zid=151390)|—|[17th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
C|21|1942|[Donavon Beals4723](https://zwiftpower\.com/profile\.php?z=1123750)|—|—|[14th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=176147)
C|22|1936|[George Rodrigues](https://zwiftpower\.com/profile\.php?z=277743)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=162625)|—|[20th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
C|23|1932|[LeRoy Nahay](https://zwiftpower\.com/profile\.php?z=286502)|—|—|—|—|—|[15th](https://zwiftpower\.com/events\.php?zid=166628)|—|[20th](https://zwiftpower\.com/events\.php?zid=173434)|—
C|24|1930|[Scott Phillips](https://zwiftpower\.com/profile\.php?z=83123)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|—|[19th](https://zwiftpower\.com/events\.php?zid=162625)|—|[17th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
C|25|1900|[ØYVIND Terjesen](https://zwiftpower\.com/profile\.php?z=867771)|—|—|—|—|[29th](https://zwiftpower\.com/events\.php?zid=162625)|[22nd](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
C|26|999|[Gaetan Duperron6179](https://zwiftpower\.com/profile\.php?z=513196)|—|—|—|—|—|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=176147)
C|27|997|[Alex Callejo](https://zwiftpower\.com/profile\.php?z=55568)|[KISS RT](https://zwiftpower\.com/team\.php?id=36)|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
C|27|997|[Kerry Smith](https://zwiftpower\.com/profile\.php?z=164504)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[🥈](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
C|27|997|[Jim Williams ZZRC TIS](https://zwiftpower\.com/profile\.php?z=894423)|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
C|28|995|[Matt Partridge \[DIRT\]](https://zwiftpower\.com/profile\.php?z=458388)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|[🥉](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
C|29|993|[Roger Rider](https://zwiftpower\.com/profile\.php?z=20137)|—|—|—|—|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=173434)|—
C|29|993|[Ian Bundy](https://zwiftpower\.com/profile\.php?z=318161)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|[4th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
C|29|993|[Alasdair MacMillan](https://zwiftpower\.com/profile\.php?z=522143)|—|—|[4th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
C|30|991|[Jonas Spinoy Ⓥ 🇧🇪](https://zwiftpower\.com/profile\.php?z=347291)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|—|—|—|[5th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
C|31|989|[天蝎座 GUO🦂️⛵️](https://zwiftpower\.com/profile\.php?z=290402)|[Team \+1S](https://zwiftpower\.com/team\.php?id=1414)|—|[6th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
C|32|987|[Dave Mehl TeamEN \(C\) 7824](https://zwiftpower\.com/profile\.php?z=8539)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|—|—|—|—|[7th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
C|32|987|[Dawson Hardiman 4923](https://zwiftpower\.com/profile\.php?z=53909)|—|[7th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
C|32|987|[simone sighinolfi Ⓥ](https://zwiftpower\.com/profile\.php?z=1085650)|[VEGAN](https://zwiftpower\.com/team\.php?id=1845)|—|[7th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
C|33|985|[Sjoerd Keizers](https://zwiftpower\.com/profile\.php?z=233688)|[TeamNL](https://zwiftpower\.com/team\.php?id=2281)|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
C|33|985|[Al Price \(C\)](https://zwiftpower\.com/profile\.php?z=336603)|—|—|—|—|—|—|—|[8th](https://zwiftpower\.com/events\.php?zid=173434)|—
C|33|985|[Sulize Kriel](https://zwiftpower\.com/profile\.php?z=429670)|—|[8th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
C|34|983|[Danny Noble](https://zwiftpower\.com/profile\.php?z=92884)|—|—|—|—|—|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=176147)
C|34|983|[TITO](https://zwiftpower\.com/profile\.php?z=177990)|—|—|—|[9th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
C|34|983|[Elise Yanover](https://zwiftpower\.com/profile\.php?z=195469)|—|[9th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
C|35|981|[Mark Middleton \(Kos\)](https://zwiftpower\.com/profile\.php?z=5666)|[DIRT](https://zwiftpower\.com/team\.php?id=2707)|—|—|—|—|—|[10th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
C|35|981|[Bruno Barbosa](https://zwiftpower\.com/profile\.php?z=104049)|[PTz](https://zwiftpower\.com/team\.php?id=1)|—|—|—|—|—|—|—|[10th](https://zwiftpower\.com/events\.php?zid=176147)
C|35|981|[Jennifer Crehan\. \(CLS\)](https://zwiftpower\.com/profile\.php?z=452471)|—|—|—|—|[10th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
C|35|981|[Coffee Umanetz](https://zwiftpower\.com/profile\.php?z=945838)|—|[10th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
C|35|981|[John Maranto \(Riding realtor\) 7355 Dirt](https://zwiftpower\.com/profile\.php?z=965481)|—|—|—|[10th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
C|36|979|[Tim Parker](https://zwiftpower\.com/profile\.php?z=1496)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
C|36|979|[Gary Morris](https://zwiftpower\.com/profile\.php?z=198799)|[MGCC](https://zwiftpower\.com/team\.php?id=114)|—|[11th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
C|36|979|[David Ayton](https://zwiftpower\.com/profile\.php?z=225652)|—|—|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
C|36|979|[Jay Chapin](https://zwiftpower\.com/profile\.php?z=830907)|—|[11th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
C|36|979|[Brandon Walls](https://zwiftpower\.com/profile\.php?z=1153231)|—|—|—|—|—|—|—|—|[11th](https://zwiftpower\.com/events\.php?zid=176147)
C|37|977|[Mark Mortimer](https://zwiftpower\.com/profile\.php?z=73851)|[TFC](https://zwiftpower\.com/team\.php?id=10)|—|—|—|—|—|[12th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
C|37|977|[Claude Anderson](https://zwiftpower\.com/profile\.php?z=179098)|—|—|—|—|—|—|—|[12th](https://zwiftpower\.com/events\.php?zid=173434)|—
C|37|977|[Allan Howard 5857](https://zwiftpower\.com/profile\.php?z=987925)|—|—|[12th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
C|38|975|[Alex Bennett](https://zwiftpower\.com/profile\.php?z=14715)|—|—|[13th](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
C|38|975|[Steve RAWLINGS 6491](https://zwiftpower\.com/profile\.php?z=238826)|[teamWKG](https://zwiftpower\.com/team\.php?id=1330)|—|—|—|—|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=176147)
C|38|975|[Jason Strandberg](https://zwiftpower\.com/profile\.php?z=597788)|[Vixen/Fox](https://zwiftpower\.com/team\.php?id=1940)|[13th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
C|38|975|[Matt Wilson](https://zwiftpower\.com/profile\.php?z=611819)|—|—|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
C|38|975|[Mariusz Lembowicz](https://zwiftpower\.com/profile\.php?z=904184)|[Team Poland](https://zwiftpower\.com/team\.php?id=25)|—|—|—|—|—|—|[13th](https://zwiftpower\.com/events\.php?zid=173434)|—
C|39|973|[Alfred Lazelberger](https://zwiftpower\.com/profile\.php?z=935777)|—|—|—|—|—|—|—|—|[14th](https://zwiftpower\.com/events\.php?zid=176147)
C|40|971|[Jon Hume](https://zwiftpower\.com/profile\.php?z=465110)|—|—|—|[15th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
C|40|971|[P i\.G\.e\. Wang](https://zwiftpower\.com/profile\.php?z=1072304)|—|[15th](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
C|40|971|[mark aylett](https://zwiftpower\.com/profile\.php?z=1125451)|—|—|—|—|—|—|—|[15th](https://zwiftpower\.com/events\.php?zid=173434)|—
C|41|969|[Sean Michael Barnardo](https://zwiftpower\.com/profile\.php?z=12290)|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
C|41|969|[Frank Soltiz](https://zwiftpower\.com/profile\.php?z=64140)|[TeamODZ](https://zwiftpower\.com/team\.php?id=7)|—|—|—|[16th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
C|42|967|[Jeppe Sørensen](https://zwiftpower\.com/profile\.php?z=410451)|—|—|—|—|—|—|—|[17th](https://zwiftpower\.com/events\.php?zid=173434)|—
C|43|965|[Josh Roark](https://zwiftpower\.com/profile\.php?z=14493)|—|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
C|43|965|[Tam Burns CRYO\-GEN](https://zwiftpower\.com/profile\.php?z=17442)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=171934)|—|—
C|43|965|[G Pollack \#EN 9591](https://zwiftpower\.com/profile\.php?z=454924)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|—|—|—|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=173434)|—
C|43|965|[Les \- carlos](https://zwiftpower\.com/profile\.php?z=1057711)|—|—|—|[18th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
C|44|963|[Kim O'Neill](https://zwiftpower\.com/profile\.php?z=83416)|[BRT](https://zwiftpower\.com/team\.php?id=4)|—|—|—|—|[19th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
C|44|963|[Paul Wakefield](https://zwiftpower\.com/profile\.php?z=714712)|—|—|—|[19th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
C|44|963|[Shinichiro Murakami](https://zwiftpower\.com/profile\.php?z=1160795)|—|—|—|—|—|—|—|[19th](https://zwiftpower\.com/events\.php?zid=173434)|—
C|45|961|[Ricardo Moreno](https://zwiftpower\.com/profile\.php?z=93510)|—|—|—|—|[20th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
C|45|961|[John Frame KISS \(C\)](https://zwiftpower\.com/profile\.php?z=446376)|—|—|—|—|—|[20th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
C|45|961|[Zaid Siddiqui](https://zwiftpower\.com/profile\.php?z=1073243)|—|—|—|[20th](https://zwiftpower\.com/events\.php?zid=159271)|—|—|—|—|—
C|46|959|[David Creelman](https://zwiftpower\.com/profile\.php?z=403843)|—|—|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
C|46|959|"[Laurent ""Froome"" France](https://zwiftpower\.com/profile\.php?z=604431)"|—|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
C|46|959|[Mike Graffeo TeamEN](https://zwiftpower\.com/profile\.php?z=811941)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|—|—|—|—|—|—|[21st](https://zwiftpower\.com/events\.php?zid=173434)|—
C|47|957|[Li Kevin](https://zwiftpower\.com/profile\.php?z=24862)|—|—|—|—|[22nd](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
C|48|955|[y terra](https://zwiftpower\.com/profile\.php?z=102591)|—|—|—|—|[23rd](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
C|48|955|[shirley partridge](https://zwiftpower\.com/profile\.php?z=465464)|—|—|—|—|—|[23rd](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
C|49|953|[Dicky P9NRY 9117](https://zwiftpower\.com/profile\.php?z=1132959)|—|—|—|—|—|[24th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
C|50|951|[Clare Pollard](https://zwiftpower\.com/profile\.php?z=115478)|—|—|—|—|—|[25th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
C|50|951|[Katy Toms Cryo\-gen\(C\)](https://zwiftpower\.com/profile\.php?z=197276)|[Team CRYO\-GEN](https://zwiftpower\.com/team\.php?id=2740)|—|—|—|[25th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
C|51|949|[Fook AuYeung](https://zwiftpower\.com/profile\.php?z=325897)|—|—|—|—|[26th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
C|51|949|[Robert Aston](https://zwiftpower\.com/profile\.php?z=1073407)|—|—|—|—|—|[26th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
C|52|947|[john Melia 3188](https://zwiftpower\.com/profile\.php?z=403960)|—|—|—|—|[27th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
C|53|945|[heaven KIM&LEE](https://zwiftpower\.com/profile\.php?z=893693)|—|—|—|—|[28th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
D|1|5978|[Mark Edwards \(EnduranceNation\.us\)](https://zwiftpower\.com/profile\.php?z=387173)|[TeamEN](https://zwiftpower\.com/team\.php?id=1384)|[4th](https://zwiftpower\.com/events\.php?zid=151390)|[🥈](https://zwiftpower\.com/events\.php?zid=155565)|[🥇](https://zwiftpower\.com/events\.php?zid=159271)|[🥈](https://zwiftpower\.com/events\.php?zid=162625)|—|[🥉](https://zwiftpower\.com/events\.php?zid=171934)|[🥈](https://zwiftpower\.com/events\.php?zid=173434)|—
D|2|1998|[Alfred Lazelberger](https://zwiftpower\.com/profile\.php?z=935777)|—|—|—|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=171934)|[🥇](https://zwiftpower\.com/events\.php?zid=173434)|—
D|3|999|[Jonathan Bridge](https://zwiftpower\.com/profile\.php?z=126867)|—|—|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
D|3|999|[Jason Jones](https://zwiftpower\.com/profile\.php?z=443113)|—|—|—|—|[🥇](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
D|3|999|[Gareth Walsh 6151](https://zwiftpower\.com/profile\.php?z=532857)|—|[🥇](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
D|3|999|[Erik Dickes](https://zwiftpower\.com/profile\.php?z=1112157)|[OMAHA VELO](https://zwiftpower\.com/team\.php?id=3984)|—|[🥇](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
D|4|997|[Allan Card](https://zwiftpower\.com/profile\.php?z=88370)|—|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
D|4|997|[IM IVO \[TugaZ\]](https://zwiftpower\.com/profile\.php?z=630117)|—|—|—|—|—|—|[🥈](https://zwiftpower\.com/events\.php?zid=171934)|—|—
D|4|997|[Stewart Latham](https://zwiftpower\.com/profile\.php?z=852002)|—|[🥈](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
D|5|995|[Ricardo Moreno](https://zwiftpower\.com/profile\.php?z=93510)|—|[🥉](https://zwiftpower\.com/events\.php?zid=151390)|—|—|—|—|—|—|—
D|5|995|[Andrea Thackeray](https://zwiftpower\.com/profile\.php?z=322738)|—|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
D|5|995|[Rebecca Bourque BSR](https://zwiftpower\.com/profile\.php?z=906199)|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=155565)|—|—|—|—|—|—
D|5|995|[\. Popey\.](https://zwiftpower\.com/profile\.php?z=1198279)|—|—|—|—|—|[🥉](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—
D|6|993|[Nicky Moir9054](https://zwiftpower\.com/profile\.php?z=896045)|[Rowe & King](https://zwiftpower\.com/team\.php?id=2298)|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=162625)|—|—|—|—
D|6|993|[berit hougaard jensen](https://zwiftpower\.com/profile\.php?z=1147442)|—|—|—|—|—|[4th](https://zwiftpower\.com/events\.php?zid=166628)|—|—|—

